# Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, timedelta
from openpyxl import Workbook, load_workbook

# Cargar bases de datos 

In [2]:
# Configuración de pandas para mostrar más dígitos en la salida
pd.set_option('display.float_format', '{:.22f}'.format)

In [3]:
# Imprime el directorio actual
print("Directorio inicial:", os.getcwd())

# Cambiar al directorio 'FORMATO_290'
os.chdir('..')

# Confirma el cambio de directorio
print("Directorio después de moverse hacia arriba:", os.getcwd())

Directorio inicial: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290\Codigo
Directorio después de moverse hacia arriba: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290


In [4]:
ruta = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COGN.xlsx"

In [5]:
# Leer hoja específica y normalizar columnas
AÑO_ACTUAL = pd.read_excel(ruta, sheet_name="AÑO ACTUAL", dtype=str)
AÑO_ACTUAL.columns = [x.upper() for x in AÑO_ACTUAL.columns]

In [6]:
# Cargar hoja "RAMOS_COGN" como texto y estandarizar nombres de columnas
RAMOS_COGN = pd.read_excel(ruta, sheet_name="RAMOS_COGN", dtype=str)
RAMOS_COGN.columns = [x.upper() for x in RAMOS_COGN.columns]
RAMOS_COGN = RAMOS_COGN[['SOCIEDAD', 'LINE OF BUSS', 'COD_RAMO', 'RAMO']].drop_duplicates()

In [7]:
# Leer hoja específica y normalizar columnas
CUENTAS = pd.read_excel(ruta, sheet_name="CUENTAS", dtype=str)
CUENTAS.columns = [x.upper() for x in CUENTAS.columns]
CUENTAS = CUENTAS[['SOCIEDAD', 'CUENTA', 'CONCEPTO', 'ALTERNATIVA 6']].drop_duplicates()

In [8]:
UNIDAD_DE_CAPTURA = pd.read_excel(ruta, sheet_name="UNIDAD_DE_CAPTURA", dtype=str)
UNIDAD_DE_CAPTURA.columns = [x.upper() for x in UNIDAD_DE_CAPTURA.columns]

In [9]:
# Leer la hoja POLIZAS desde el archivo principal
POLIZAS = pd.read_excel(ruta, sheet_name="POLIZAS", dtype=str)
POLIZAS.columns = [x.upper() for x in POLIZAS.columns]

# Convertir PERIODO solo a fecha con formato yyyy-mm-dd
POLIZAS['PERIODO'] = pd.to_datetime(POLIZAS['PERIODO']).dt.strftime("%Y-%m-%d")

# Separar en AÑO, MES y DÍA como texto
POLIZAS[['AÑO', 'MES', 'DÍA']] = POLIZAS['PERIODO'].str.split("-", expand=True)

In [10]:
BALANCE_WP = pd.read_excel(ruta,sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)

# Renombrar columnas para que coincidan con la estructura anterior
BALANCE_WP.columns = ["ALTERNATIVA 6", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]

In [11]:
# Leer hoja BD_Local2 desde el archivo principal
Base = pd.read_excel(ruta, sheet_name="BD_Local2", dtype=str)
Base.columns = [x.upper() for x in Base.columns]
Base = Base.apply(lambda col: col.str.strip().str.replace(r'\s+', ' ', regex=True) if col.dtype == 'object' else col)

In [12]:
# Combinar datos
bd = AÑO_ACTUAL.merge(CUENTAS, on=['CUENTA', 'SOCIEDAD'], how='left')
bd = bd.merge(RAMOS_COGN, on=['LINE OF BUSS', 'SOCIEDAD'], how='left')
bd = bd.merge(UNIDAD_DE_CAPTURA, on='CONCEPTO', how='left').drop_duplicates()

variables = [
    "SALDO PERIODO 01", "SALDO PERIODO 02", "SALDO PERIODO 03", "SALDO PERIODO 04", 
    "SALDO PERIODO 05", "SALDO PERIODO 06", "SALDO PERIODO 07", "SALDO PERIODO 08", 
    "SALDO PERIODO 09", "SALDO PERIODO 10", "SALDO PERIODO 11", "SALDO PERIODO 12", 
    "SALDO PERIODO 13"
]

for variable in variables:
    if variable not in bd.columns:
        bd[variable] = np.nan

columnas_a_convertir = [
    "TOTAL CUENTA", "ARRASTRE SALDOS", "SALDO PERIODO 01", "SALDO PERIODO 02", "SALDO PERIODO 03",
    "SALDO PERIODO 04", "SALDO PERIODO 05", "SALDO PERIODO 06", "SALDO PERIODO 07",
    "SALDO PERIODO 08", "SALDO PERIODO 09", "SALDO PERIODO 10", "SALDO PERIODO 11",
    "SALDO PERIODO 12", "SALDO PERIODO 13","COD_RAMO"
]

for columna in columnas_a_convertir:
    bd[columna] = pd.to_numeric(bd[columna], errors='coerce')

    
bd['TOTAL'] = bd[['ARRASTRE SALDOS'] + variables].sum(axis=1, skipna=True)

In [13]:
ruta = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290\Bases\Plantilla_290_COGN.xlsx"
wb = openpyxl.load_workbook(ruta)
ws = wb['Hoja1']

# UNIDAD DE CAPTURA 1


## CONSULTA DE TOTALES PRIMAS RETENIDAS UNIDAD 1    

In [14]:
unidad1_total = bd[bd['UNIDAD'] == 1].groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad1_total['TOTAL'] = unidad1_total['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

def write_to_excel(data_frame, concept, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=3).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad1_total, "Primas Emitidas Directas", 4)
write_to_excel(unidad1_total, "Primas aceptadas en coaseguro", 5)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6)
write_to_excel(unidad1_total, "Aceptadas Reaseguro interior", 7)
write_to_excel(unidad1_total, "Aceptadas Reaseguro exterior", 8)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9)
write_to_excel(unidad1_total, "Cedidas Reaseguro interior (resta)", 10)
write_to_excel(unidad1_total, "Cedidas Reaseguro exterior (resta)", 11)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 12)
write_to_excel(unidad1_total, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)

def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
c4_005 = get_value(unidad1_total, "Primas Emitidas Directas")
c5_010 = get_value(unidad1_total, "Primas aceptadas en coaseguro")
c6_015 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
c7_020 = get_value(unidad1_total, "Aceptadas Reaseguro interior")
c8_025 = get_value(unidad1_total, "Aceptadas Reaseguro exterior")
c9_030 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
c10_035 = get_value(unidad1_total, "Cedidas Reaseguro interior (resta)")
c11_040 = get_value(unidad1_total, "Cedidas Reaseguro exterior (resta)")
c12_045 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
c13_050 = get_value(unidad1_total, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Calcular el total general según la fórmula dada
c14_999 = (c4_005 + c5_010 - c6_015 + c7_020 + c8_025 - c9_030 - c10_035 - c11_040 + c12_045 + c13_050)

# Escribir el total general en Excel
ws = wb['Hoja1']
ws.cell(row=14, column=3).value = c14_999


##  SUBTOTAL - RAMOS UNIDAD 1      

In [15]:
unidad1_SUBTOTAL = bd[bd['UNIDAD'] == 1].groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad1_SUBTOTAL['TOTAL'] = unidad1_SUBTOTAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Llamadas a la función para escribir en Excel
concepts = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

for concept, row_num in concepts:
    write_to_excel(unidad1_SUBTOTAL, concept, 4, row_num)


def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
d4_005 = get_value(unidad1_SUBTOTAL, "Primas Emitidas Directas")
d5_010 = get_value(unidad1_SUBTOTAL, "Primas aceptadas en coaseguro")
d6_015 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
d7_020 = get_value(unidad1_SUBTOTAL, "Aceptadas Reaseguro interior")
d8_025 = get_value(unidad1_SUBTOTAL, "Aceptadas Reaseguro exterior")
d9_030 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
d10_035 = get_value(unidad1_SUBTOTAL, "Cedidas Reaseguro interior (resta)")
d11_040 = get_value(unidad1_SUBTOTAL, "Cedidas Reaseguro exterior (resta)")
d12_045 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
d13_050 = get_value(unidad1_SUBTOTAL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular d14_999
d14_999 = (d4_005 + d5_010 - d6_015 + d7_020 + d8_025 - d9_030 - d10_035 - d11_040 + d12_045 + d13_050)

# Escribir el total general en Excel
ws = wb['Hoja1']
if d14_999 == 0:  # Verificar si el resultado es 0, lo que implica que no hubo datos para calcular
    d14_999 = {'Total': 0}  # Crear un diccionario para simular un DataFrame con un solo valor 'Total' = 0
    ws.cell(row=14, column=4).value = d14_999['Total']
else:
    ws.cell(row=14, column=4).value = d14_999


##  3   AUTOMOVILES  UNIDAD 1   

In [16]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 3
unidad1_AUTOMOVILES = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_AUTOMOVILES['TOTAL'] = unidad1_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 3, 4),
    ("Primas aceptadas en coaseguro", 3, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 3, 6),
    ("Aceptadas Reaseguro interior", 3, 7),
    ("Aceptadas Reaseguro exterior", 3, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 3, 9),
    ("Cedidas Reaseguro interior (resta)", 3, 10),
    ("Cedidas Reaseguro exterior (resta)", 3, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 3, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 3, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e4_005 = get_value(unidad1_AUTOMOVILES, "Primas Emitidas Directas", 3)
e5_010 = get_value(unidad1_AUTOMOVILES, "Primas aceptadas en coaseguro", 3)
e6_015 = get_value(unidad1_AUTOMOVILES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 3)
e7_020 = get_value(unidad1_AUTOMOVILES, "Aceptadas Reaseguro interior", 3)
e8_025 = get_value(unidad1_AUTOMOVILES, "Aceptadas Reaseguro exterior", 3)
e9_030 = get_value(unidad1_AUTOMOVILES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 3)
e10_035 = get_value(unidad1_AUTOMOVILES, "Cedidas Reaseguro interior (resta)", 3)
e11_040 = get_value(unidad1_AUTOMOVILES, "Cedidas Reaseguro exterior (resta)", 3)
e12_045 = get_value(unidad1_AUTOMOVILES, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 3)
e13_050 = get_value(unidad1_AUTOMOVILES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 3)

# Aplicar la fórmula específica para calcular ab14_999
e14_999 = (e4_005 + e5_010 - e6_015 + e7_020 + e8_025 - e9_030 - e10_035 - e11_040 + e12_045 + e13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=5, value=e14_999)


<Cell 'Hoja1'.E14>

##  4   SOAT UNIDAD 1 

In [17]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 4
unidad1_SOAT = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_SOAT['TOTAL'] = unidad1_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_SOAT, concept, 6, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f4_005 = get_value(unidad1_SOAT, "Primas Emitidas Directas")
f5_010 = get_value(unidad1_SOAT, "Primas aceptadas en coaseguro")
f6_015 = get_value(unidad1_SOAT, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
f7_020 = get_value(unidad1_SOAT, "Aceptadas Reaseguro interior")
f8_025 = get_value(unidad1_SOAT, "Aceptadas Reaseguro exterior")
f9_030 = get_value(unidad1_SOAT, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
f10_035 = get_value(unidad1_SOAT, "Cedidas Reaseguro interior (resta)")
f11_040 = get_value(unidad1_SOAT, "Cedidas Reaseguro exterior (resta)")
f12_045 = get_value(unidad1_SOAT, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
f13_050 = get_value(unidad1_SOAT, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular aC14_999
f14_999 = (f4_005 + f5_010 - f6_015 + f7_020 + f8_025 - f9_030 - f10_035 - f11_040 + f12_045 + f13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=6, value=f14_999)

<Cell 'Hoja1'.F14>

##    5   CUMPLIMIENTO UNIDAD 1  

In [18]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTO['TOTAL'] = unidad1_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 5, 4),
    ("Primas aceptadas en coaseguro", 5, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 5, 6),
    ("Aceptadas Reaseguro interior", 5, 7),
    ("Aceptadas Reaseguro exterior", 5, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 5, 9),
    ("Cedidas Reaseguro interior (resta)", 5, 10),
    ("Cedidas Reaseguro exterior (resta)", 5, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 5, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 5, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g4_005 = get_value(unidad1_CUMPLIMIENTO, "Primas Emitidas Directas", 5)
g5_010 = get_value(unidad1_CUMPLIMIENTO, "Primas aceptadas en coaseguro", 5)
g6_015 = get_value(unidad1_CUMPLIMIENTO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 5)
g7_020 = get_value(unidad1_CUMPLIMIENTO, "Aceptadas Reaseguro interior", 5)
g8_025 = get_value(unidad1_CUMPLIMIENTO, "Aceptadas Reaseguro exterior", 5)
g9_030 = get_value(unidad1_CUMPLIMIENTO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 5)
g10_035 = get_value(unidad1_CUMPLIMIENTO, "Cedidas Reaseguro interior (resta)", 5)
g11_040 = get_value(unidad1_CUMPLIMIENTO, "Cedidas Reaseguro exterior (resta)", 5)
g12_045 = get_value(unidad1_CUMPLIMIENTO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 5)
g13_050 = get_value(unidad1_CUMPLIMIENTO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 5)

# Aplicar la fórmula específica para calcular ad14_999
g14_999 = (g4_005 + g5_010 - g6_015 + g7_020 + g8_025 - g9_030 - g10_035 - g11_040 + g12_045 + g13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=7, value=g14_999)


<Cell 'Hoja1'.G14>

##  6   RESPONSABILIDAD CIVIL  UNIDAD 1 

In [19]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_RESPONSABILIDADCI = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_RESPONSABILIDADCI['TOTAL'] = unidad1_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 6, 4),
    ("Primas aceptadas en coaseguro", 6, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6, 6),
    ("Aceptadas Reaseguro interior", 6, 7),
    ("Aceptadas Reaseguro exterior", 6, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 6, 9),
    ("Cedidas Reaseguro interior (resta)", 6, 10),
    ("Cedidas Reaseguro exterior (resta)", 6, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 6, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 6, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h4_005 = get_value(unidad1_RESPONSABILIDADCI, "Primas Emitidas Directas", 6)
h5_010 = get_value(unidad1_RESPONSABILIDADCI, "Primas aceptadas en coaseguro", 6)
h6_015 = get_value(unidad1_RESPONSABILIDADCI, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6)
h7_020 = get_value(unidad1_RESPONSABILIDADCI, "Aceptadas Reaseguro interior", 6)
h8_025 = get_value(unidad1_RESPONSABILIDADCI, "Aceptadas Reaseguro exterior", 6)
h9_030 = get_value(unidad1_RESPONSABILIDADCI, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 6)
h10_035 = get_value(unidad1_RESPONSABILIDADCI, "Cedidas Reaseguro interior (resta)", 6)
h11_040 = get_value(unidad1_RESPONSABILIDADCI, "Cedidas Reaseguro exterior (resta)", 6)
h12_045 = get_value(unidad1_RESPONSABILIDADCI, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 6)
h13_050 = get_value(unidad1_RESPONSABILIDADCI, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 6)

# Aplicar la fórmula específica para calcular ae14_999
h14_999 = (h4_005 + h5_010 - h6_015 + h7_020 + h8_025 - h9_030 - h10_035 - h11_040 + h12_045 + h13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=8, value=h14_999)


<Cell 'Hoja1'.H14>

##  7   INCENDIO  UNIDAD 1

In [20]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_INCENDIO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_INCENDIO['TOTAL'] = unidad1_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 7, 4),
    ("Primas aceptadas en coaseguro", 7, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 7, 6),
    ("Aceptadas Reaseguro interior", 7, 7),
    ("Aceptadas Reaseguro exterior", 7, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 7, 9),
    ("Cedidas Reaseguro interior (resta)", 7, 10),
    ("Cedidas Reaseguro exterior (resta)", 7, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 7, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 7, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i4_005 = get_value(unidad1_INCENDIO, "Primas Emitidas Directas", 7)
i5_010 = get_value(unidad1_INCENDIO, "Primas aceptadas en coaseguro", 7)
i6_015 = get_value(unidad1_INCENDIO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 7)
i7_020 = get_value(unidad1_INCENDIO, "Aceptadas Reaseguro interior", 7)
i8_025 = get_value(unidad1_INCENDIO, "Aceptadas Reaseguro exterior", 7)
i9_030 = get_value(unidad1_INCENDIO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 7)
i10_035 = get_value(unidad1_INCENDIO, "Cedidas Reaseguro interior (resta)", 7)
i11_040 = get_value(unidad1_INCENDIO, "Cedidas Reaseguro exterior (resta)", 7)
i12_045 = get_value(unidad1_INCENDIO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 7)
i13_050 = get_value(unidad1_INCENDIO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 7)

# Aplicar la fórmula específica para calcular ae14_999
i14_999 = (i4_005 + i5_010 - i6_015 + i7_020 + i8_025 - i9_030 - i10_035 - i11_040 + i12_045 + i13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=9, value=i14_999)

<Cell 'Hoja1'.I14>

##   8   TERREMOTO 1         

In [21]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_TERREMOTO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_TERREMOTO['TOTAL'] = unidad1_TERREMOTO['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 8, 4),
    ("Primas aceptadas en coaseguro", 8, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 8, 6),
    ("Aceptadas Reaseguro interior", 8, 7),
    ("Aceptadas Reaseguro exterior", 8, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 8, 9),
    ("Cedidas Reaseguro interior (resta)", 8, 10),
    ("Cedidas Reaseguro exterior (resta)", 8, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 8, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 8, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_RESPONSABILIDADCI, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j4_005 = get_value(unidad1_TERREMOTO, "Primas Emitidas Directas", 8)
j5_010 = get_value(unidad1_TERREMOTO, "Primas aceptadas en coaseguro", 8)
j6_015 = get_value(unidad1_TERREMOTO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 8)
j7_020 = get_value(unidad1_TERREMOTO, "Aceptadas Reaseguro interior", 8)
j8_025 = get_value(unidad1_TERREMOTO, "Aceptadas Reaseguro exterior", 8)
j9_030 = get_value(unidad1_TERREMOTO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 8)
j10_035 = get_value(unidad1_TERREMOTO, "Cedidas Reaseguro interior (resta)", 8)
j11_040 = get_value(unidad1_TERREMOTO, "Cedidas Reaseguro exterior (resta)", 8)
j12_045 = get_value(unidad1_TERREMOTO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 8)
j13_050 = get_value(unidad1_TERREMOTO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 8)

# Aplicar la fórmula específica para calcular ae14_999
j14_999 = (j4_005 + j5_010 - j6_015 + j7_020 + j8_025 - j9_030 - j10_035 - j11_040 + j12_045 + j13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=10, value=j14_999)

<Cell 'Hoja1'.J14>

##  9   SUSTRACCION UNIDAD 1 

In [22]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_SUSTRACCION = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_SUSTRACCION['TOTAL'] = unidad1_SUSTRACCION['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 9, 4),
    ("Primas aceptadas en coaseguro", 9, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 9, 6),
    ("Aceptadas Reaseguro interior", 9, 7),
    ("Aceptadas Reaseguro exterior", 9, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9, 9),
    ("Cedidas Reaseguro interior (resta)", 9, 10),
    ("Cedidas Reaseguro exterior (resta)", 9, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 9, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 9, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k4_005 = get_value(unidad1_SUSTRACCION, "Primas Emitidas Directas", 9)
k5_010 = get_value(unidad1_SUSTRACCION, "Primas aceptadas en coaseguro", 9)
k6_015 = get_value(unidad1_SUSTRACCION, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 9)
k7_020 = get_value(unidad1_SUSTRACCION, "Aceptadas Reaseguro interior", 9)
k8_025 = get_value(unidad1_SUSTRACCION, "Aceptadas Reaseguro exterior", 9)
k9_030 = get_value(unidad1_SUSTRACCION, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9)
k10_035 = get_value(unidad1_SUSTRACCION, "Cedidas Reaseguro interior (resta)", 9)
k11_040 = get_value(unidad1_SUSTRACCION, "Cedidas Reaseguro exterior (resta)", 9)
k12_045 = get_value(unidad1_SUSTRACCION, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 9)
k13_050 = get_value(unidad1_SUSTRACCION, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 9)

# Aplicar la fórmula específica para calcular ae14_999
k14_999 = (k4_005 + k5_010 - k6_015 + k7_020 + k8_025 - k9_030 - k10_035 - k11_040 + k12_045 + k13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=11, value=k14_999)

<Cell 'Hoja1'.K14>

##  10  TRANSPORTE UNIDAD 1 

In [23]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_TRANSPORTE = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_TRANSPORTE['TOTAL'] = unidad1_TRANSPORTE['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 10, 4),
    ("Primas aceptadas en coaseguro", 10, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 10, 6),
    ("Aceptadas Reaseguro interior", 10, 7),
    ("Aceptadas Reaseguro exterior", 10, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 10, 9),
    ("Cedidas Reaseguro interior (resta)", 10, 10),
    ("Cedidas Reaseguro exterior (resta)", 10, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 10, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 10, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l4_005 = get_value(unidad1_TRANSPORTE, "Primas Emitidas Directas", 10)
l5_010 = get_value(unidad1_TRANSPORTE, "Primas aceptadas en coaseguro", 10)
l6_015 = get_value(unidad1_TRANSPORTE, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 10)
l7_020 = get_value(unidad1_TRANSPORTE, "Aceptadas Reaseguro interior", 10)
l8_025 = get_value(unidad1_TRANSPORTE, "Aceptadas Reaseguro exterior", 10)
l9_030 = get_value(unidad1_TRANSPORTE, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 10)
l10_035 = get_value(unidad1_TRANSPORTE, "Cedidas Reaseguro interior (resta)", 10)
l11_040 = get_value(unidad1_TRANSPORTE, "Cedidas Reaseguro exterior (resta)", 10)
l12_045 = get_value(unidad1_TRANSPORTE, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 10)
l13_050 = get_value(unidad1_TRANSPORTE, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 10)

# Aplicar la fórmula específica para calcular ae14_999
l14_999 = (l4_005 + l5_010 - l6_015 + l7_020 + l8_025 - l9_030 - l10_035 - l11_040 + l12_045 + l13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=12, value=l14_999)

<Cell 'Hoja1'.L14>

##  11  CORRIENTE DÉBIL UNIDAD 1 

In [24]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CORRIENDEBIL = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CORRIENDEBIL['TOTAL'] = unidad1_CORRIENDEBIL['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 11, 4),
    ("Primas aceptadas en coaseguro", 11, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 11, 6),
    ("Aceptadas Reaseguro interior", 11, 7),
    ("Aceptadas Reaseguro exterior", 11, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 11, 9),
    ("Cedidas Reaseguro interior (resta)", 11, 10),
    ("Cedidas Reaseguro exterior (resta)", 11, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 11, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 11, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m4_005 = get_value(unidad1_CORRIENDEBIL, "Primas Emitidas Directas", 11)
m5_010 = get_value(unidad1_CORRIENDEBIL, "Primas aceptadas en coaseguro", 11)
m6_015 = get_value(unidad1_CORRIENDEBIL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 11)
m7_020 = get_value(unidad1_CORRIENDEBIL, "Aceptadas Reaseguro interior", 11)
m8_025 = get_value(unidad1_CORRIENDEBIL, "Aceptadas Reaseguro exterior", 11)
m9_030 = get_value(unidad1_CORRIENDEBIL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 11)
m10_035 = get_value(unidad1_CORRIENDEBIL, "Cedidas Reaseguro interior (resta)", 11)
m11_040 = get_value(unidad1_CORRIENDEBIL, "Cedidas Reaseguro exterior (resta)", 11)
m12_045 = get_value(unidad1_CORRIENDEBIL, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 11)
m13_050 = get_value(unidad1_CORRIENDEBIL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 11)

# Aplicar la fórmula específica para calcular ae14_999
m14_999 = (m4_005 + m5_010 - m6_015 + m7_020 + m8_025 - m9_030 - m10_035 - m11_040 + m12_045 + m13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=13, value=m14_999)

<Cell 'Hoja1'.M14>

##  12  TODO RIESGO CONTRATISTA UNIDAD 1 

In [25]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_TODORIESGOCONTRATISTA = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_TODORIESGOCONTRATISTA['TOTAL'] = unidad1_TODORIESGOCONTRATISTA['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 12, 4),
    ("Primas aceptadas en coaseguro", 12, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 12, 6),
    ("Aceptadas Reaseguro interior", 12, 7),
    ("Aceptadas Reaseguro exterior", 12, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 12, 9),
    ("Cedidas Reaseguro interior (resta)", 12, 10),
    ("Cedidas Reaseguro exterior (resta)", 12, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 12, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n4_005 = get_value(unidad1_TODORIESGOCONTRATISTA, "Primas Emitidas Directas", 12)
n5_010 = get_value(unidad1_TODORIESGOCONTRATISTA, "Primas aceptadas en coaseguro", 12)
n6_015 = get_value(unidad1_TODORIESGOCONTRATISTA, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 12)
n7_020 = get_value(unidad1_TODORIESGOCONTRATISTA, "Aceptadas Reaseguro interior", 12)
n8_025 = get_value(unidad1_TODORIESGOCONTRATISTA, "Aceptadas Reaseguro exterior", 12)
n9_030 = get_value(unidad1_TODORIESGOCONTRATISTA, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 12)
n10_035 = get_value(unidad1_TODORIESGOCONTRATISTA, "Cedidas Reaseguro interior (resta)", 12)
n11_040 = get_value(unidad1_TODORIESGOCONTRATISTA, "Cedidas Reaseguro exterior (resta)", 12)
n12_045 = get_value(unidad1_TODORIESGOCONTRATISTA, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 12)
n13_050 = get_value(unidad1_TODORIESGOCONTRATISTA, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 12)

# Aplicar la fórmula específica para calcular ae14_999
n14_999 = (n4_005 + n5_010 - n6_015 + n7_020 + n8_025 - n9_030 - n10_035 - n11_040 + n12_045 + n13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=14, value=n14_999)

<Cell 'Hoja1'.N14>

##  13  MANEJO UNIDAD 1 

In [26]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_MANEJO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_MANEJO['TOTAL'] = unidad1_MANEJO['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 13, 4),
    ("Primas aceptadas en coaseguro", 13, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 13, 6),
    ("Aceptadas Reaseguro interior", 13, 7),
    ("Aceptadas Reaseguro exterior", 13, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 13, 9),
    ("Cedidas Reaseguro interior (resta)", 13, 10),
    ("Cedidas Reaseguro exterior (resta)", 13, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 13, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o4_005 = get_value(unidad1_MANEJO, "Primas Emitidas Directas", 13)
o5_010 = get_value(unidad1_MANEJO, "Primas aceptadas en coaseguro", 13)
o6_015 = get_value(unidad1_MANEJO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 13)
o7_020 = get_value(unidad1_MANEJO, "Aceptadas Reaseguro interior", 13)
o8_025 = get_value(unidad1_MANEJO, "Aceptadas Reaseguro exterior", 13)
o9_030 = get_value(unidad1_MANEJO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 13)
o10_035 = get_value(unidad1_MANEJO, "Cedidas Reaseguro interior (resta)", 13)
o11_040 = get_value(unidad1_MANEJO, "Cedidas Reaseguro exterior (resta)", 13)
o12_045 = get_value(unidad1_MANEJO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 13)
o13_050 = get_value(unidad1_MANEJO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)

# Aplicar la fórmula específica para calcular ae14_999
o14_999 = (o4_005 + o5_010 - o6_015 + o7_020 + o8_025 - o9_030 - o10_035 - o11_040 + o12_045 + o13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=15, value=o14_999)

<Cell 'Hoja1'.O14>

##  14  LUCRO CESANTE UNIDAD 1


In [27]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_LUCROCESANTE = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_LUCROCESANTE['TOTAL'] = unidad1_LUCROCESANTE['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 14, 4),
    ("Primas aceptadas en coaseguro", 14, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 14, 6),
    ("Aceptadas Reaseguro interior", 14, 7),
    ("Aceptadas Reaseguro exterior", 14, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 14, 9),
    ("Cedidas Reaseguro interior (resta)", 14, 10),
    ("Cedidas Reaseguro exterior (resta)", 14, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 14, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 14, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p4_005 = get_value(unidad1_LUCROCESANTE, "Primas Emitidas Directas", 14)
p5_010 = get_value(unidad1_LUCROCESANTE, "Primas aceptadas en coaseguro", 14)
p6_015 = get_value(unidad1_LUCROCESANTE, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 14)
p7_020 = get_value(unidad1_LUCROCESANTE, "Aceptadas Reaseguro interior", 14)
p8_025 = get_value(unidad1_LUCROCESANTE, "Aceptadas Reaseguro exterior", 14)
p9_030 = get_value(unidad1_LUCROCESANTE, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 14)
p10_035 = get_value(unidad1_LUCROCESANTE, "Cedidas Reaseguro interior (resta)", 14)
p11_040 = get_value(unidad1_LUCROCESANTE, "Cedidas Reaseguro exterior (resta)", 14)
p12_045 = get_value(unidad1_LUCROCESANTE, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 14)
p13_050 = get_value(unidad1_LUCROCESANTE, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 14)

# Aplicar la fórmula específica para calcular ae14_999
p14_999 = (p4_005 + p5_010 - p6_015 + p7_020 + p8_025 - p9_030 - p10_035 - p11_040 + p12_045 + p13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=16, value=p14_999)

<Cell 'Hoja1'.P14>

##  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 1


In [28]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_MONTAJEYROTURA = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_MONTAJEYROTURA['TOTAL'] = unidad1_MONTAJEYROTURA['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 15, 4),
    ("Primas aceptadas en coaseguro", 15, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 15, 6),
    ("Aceptadas Reaseguro interior", 15, 7),
    ("Aceptadas Reaseguro exterior", 15, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 15, 9),
    ("Cedidas Reaseguro interior (resta)", 15, 10),
    ("Cedidas Reaseguro exterior (resta)", 15, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 15, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 15, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q4_005 = get_value(unidad1_MONTAJEYROTURA, "Primas Emitidas Directas", 15)
q5_010 = get_value(unidad1_MONTAJEYROTURA, "Primas aceptadas en coaseguro", 15)
q6_015 = get_value(unidad1_MONTAJEYROTURA, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 15)
q7_020 = get_value(unidad1_MONTAJEYROTURA, "Aceptadas Reaseguro interior", 15)
q8_025 = get_value(unidad1_MONTAJEYROTURA, "Aceptadas Reaseguro exterior", 15)
q9_030 = get_value(unidad1_MONTAJEYROTURA, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 15)
q10_035 = get_value(unidad1_MONTAJEYROTURA, "Cedidas Reaseguro interior (resta)", 15)
q11_040 = get_value(unidad1_MONTAJEYROTURA, "Cedidas Reaseguro exterior (resta)", 15)
q12_045 = get_value(unidad1_MONTAJEYROTURA, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 15)
q13_050 = get_value(unidad1_MONTAJEYROTURA, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 15)

# Aplicar la fórmula específica para calcular ae14_999
q14_999 = (q4_005 + q5_010 - q6_015 + q7_020 + q8_025 - q9_030 - q10_035 - q11_040 + q12_045 + q13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=17, value=q14_999)

<Cell 'Hoja1'.Q14>

##  16  AVIACIÓN UNIDAD 1


In [29]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_AVIACION = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_AVIACION['TOTAL'] = unidad1_AVIACION['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 16, 4),
    ("Primas aceptadas en coaseguro", 16, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 16, 6),
    ("Aceptadas Reaseguro interior", 16, 7),
    ("Aceptadas Reaseguro exterior", 16, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 15, 9),
    ("Cedidas Reaseguro interior (resta)", 16, 10),
    ("Cedidas Reaseguro exterior (resta)", 16, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 16, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 16, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r4_005 = get_value(unidad1_AVIACION, "Primas Emitidas Directas", 16)
r5_010 = get_value(unidad1_AVIACION, "Primas aceptadas en coaseguro", 16)
r6_015 = get_value(unidad1_AVIACION, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 16)
r7_020 = get_value(unidad1_AVIACION, "Aceptadas Reaseguro interior", 16)
r8_025 = get_value(unidad1_AVIACION, "Aceptadas Reaseguro exterior", 16)
r9_030 = get_value(unidad1_AVIACION, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 16)
r10_035 = get_value(unidad1_AVIACION, "Cedidas Reaseguro interior (resta)", 16)
r11_040 = get_value(unidad1_AVIACION, "Cedidas Reaseguro exterior (resta)", 16)
r12_045 = get_value(unidad1_AVIACION, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 16)
r13_050 = get_value(unidad1_AVIACION, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 16)

# Aplicar la fórmula específica para calcular ae14_999
r14_999 = (r4_005 + r5_010 - r6_015 + r7_020 + r8_025 - r9_030 - r10_035 - r11_040 + r12_045 + r13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=18, value=r14_999)

<Cell 'Hoja1'.R14>

##  17  NAVEGACIÓN Y CASCO UNIDAD 1


In [30]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_NAVEGACION = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_NAVEGACION['TOTAL'] = unidad1_NAVEGACION['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 17, 4),
    ("Primas aceptadas en coaseguro", 17, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 17, 6),
    ("Aceptadas Reaseguro interior", 17, 7),
    ("Aceptadas Reaseguro exterior", 17, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 17, 9),
    ("Cedidas Reaseguro interior (resta)", 17, 10),
    ("Cedidas Reaseguro exterior (resta)", 17, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 17, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 17, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s4_005 = get_value(unidad1_NAVEGACION, "Primas Emitidas Directas", 17)
s5_010 = get_value(unidad1_NAVEGACION, "Primas aceptadas en coaseguro", 17)
s6_015 = get_value(unidad1_NAVEGACION, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 17)
s7_020 = get_value(unidad1_NAVEGACION, "Aceptadas Reaseguro interior", 17)
s8_025 = get_value(unidad1_NAVEGACION, "Aceptadas Reaseguro exterior", 17)
s9_030 = get_value(unidad1_NAVEGACION, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 17)
s10_035 = get_value(unidad1_NAVEGACION, "Cedidas Reaseguro interior (resta)", 17)
s11_040 = get_value(unidad1_NAVEGACION, "Cedidas Reaseguro exterior (resta)", 17)
s12_045 = get_value(unidad1_NAVEGACION, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 17)
s13_050 = get_value(unidad1_NAVEGACION, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 17)

# Aplicar la fórmula específica para calcular ae14_999
s14_999 = (s4_005 + s5_010 - s6_015 + s7_020 + s8_025 - s9_030 - s10_035 - s11_040 + s12_045 + s13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=19, value=s14_999)

<Cell 'Hoja1'.S14>

##  18  MINAS Y PETRÓLEOS UNIDAD 1


In [31]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_MINAS = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_MINAS['TOTAL'] = unidad1_MINAS['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 18, 4),
    ("Primas aceptadas en coaseguro", 18, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 18, 6),
    ("Aceptadas Reaseguro interior", 18, 7),
    ("Aceptadas Reaseguro exterior", 18, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 18, 9),
    ("Cedidas Reaseguro interior (resta)", 18, 10),
    ("Cedidas Reaseguro exterior (resta)", 18, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 18, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 18, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t4_005 = get_value(unidad1_MINAS, "Primas Emitidas Directas", 18)
t5_010 = get_value(unidad1_MINAS, "Primas aceptadas en coaseguro", 18)
t6_015 = get_value(unidad1_MINAS, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 18)
t7_020 = get_value(unidad1_MINAS, "Aceptadas Reaseguro interior", 18)
t8_025 = get_value(unidad1_MINAS, "Aceptadas Reaseguro exterior", 18)
t9_030 = get_value(unidad1_MINAS, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 18)
t10_035 = get_value(unidad1_MINAS, "Cedidas Reaseguro interior (resta)", 18)
t11_040 = get_value(unidad1_MINAS, "Cedidas Reaseguro exterior (resta)", 18)
t12_045 = get_value(unidad1_MINAS, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 18)
t13_050 = get_value(unidad1_MINAS, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 18)

# Aplicar la fórmula específica para calcular ae14_999
t14_999 = (t4_005 + t5_010 - t6_015 + t7_020 + t8_025 - t9_030 - t10_035 - t11_040 + t12_045 + t13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=20, value=t14_999)

<Cell 'Hoja1'.T14>

##  19  VIDRIOS UNIDAD 1


In [32]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_VIDRIOS = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_VIDRIOS['TOTAL'] = unidad1_VIDRIOS['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 19, 4),
    ("Primas aceptadas en coaseguro", 19, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 19, 6),
    ("Aceptadas Reaseguro interior", 19, 7),
    ("Aceptadas Reaseguro exterior", 19, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 19, 9),
    ("Cedidas Reaseguro interior (resta)", 19, 10),
    ("Cedidas Reaseguro exterior (resta)", 19, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 19, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 19, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u4_005 = get_value(unidad1_VIDRIOS, "Primas Emitidas Directas", 19)
u5_010 = get_value(unidad1_VIDRIOS, "Primas aceptadas en coaseguro", 19)
u6_015 = get_value(unidad1_VIDRIOS, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 19)
u7_020 = get_value(unidad1_VIDRIOS, "Aceptadas Reaseguro interior", 19)
u8_025 = get_value(unidad1_VIDRIOS, "Aceptadas Reaseguro exterior", 19)
u9_030 = get_value(unidad1_VIDRIOS, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 19)
u10_035 = get_value(unidad1_VIDRIOS, "Cedidas Reaseguro interior (resta)", 19)
u11_040 = get_value(unidad1_VIDRIOS, "Cedidas Reaseguro exterior (resta)", 19)
u12_045 = get_value(unidad1_VIDRIOS, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 19)
u13_050 = get_value(unidad1_VIDRIOS, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 19)

# Aplicar la fórmula específica para calcular ae14_999
u14_999 = (u4_005 + u5_010 - u6_015 + u7_020 + u8_025 - u9_030 - u10_035 - u11_040 + u12_045 + u13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=21, value=u14_999)

<Cell 'Hoja1'.U14>

##  20  CRÉDITO COMERCIAL UNIDAD 1


In [33]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CREDITOCOMERCIAL = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CREDITOCOMERCIAL['TOTAL'] = unidad1_CREDITOCOMERCIAL['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 20, 4),
    ("Primas aceptadas en coaseguro", 20, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 20, 6),
    ("Aceptadas Reaseguro interior", 20, 7),
    ("Aceptadas Reaseguro exterior", 20, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 20, 9),
    ("Cedidas Reaseguro interior (resta)", 20, 10),
    ("Cedidas Reaseguro exterior (resta)", 20, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 20, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 20, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v4_005 = get_value(unidad1_CREDITOCOMERCIAL, "Primas Emitidas Directas", 20)
v5_010 = get_value(unidad1_CREDITOCOMERCIAL, "Primas aceptadas en coaseguro", 20)
v6_015 = get_value(unidad1_CREDITOCOMERCIAL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 20)
v7_020 = get_value(unidad1_CREDITOCOMERCIAL, "Aceptadas Reaseguro interior", 20)
v8_025 = get_value(unidad1_CREDITOCOMERCIAL, "Aceptadas Reaseguro exterior", 20)
v9_030 = get_value(unidad1_CREDITOCOMERCIAL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 20)
v10_035 = get_value(unidad1_CREDITOCOMERCIAL, "Cedidas Reaseguro interior (resta)", 20)
v11_040 = get_value(unidad1_CREDITOCOMERCIAL, "Cedidas Reaseguro exterior (resta)", 20)
v12_045 = get_value(unidad1_CREDITOCOMERCIAL, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 20)
v13_050 = get_value(unidad1_CREDITOCOMERCIAL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 20)

# Aplicar la fórmula específica para calcular ae14_999
v14_999 = (v4_005 + v5_010 - v6_015 + v7_020 + v8_025 - v9_030 - v10_035 - v11_040 + v12_045 + v13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=22, value=v14_999)

<Cell 'Hoja1'.V14>

##  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 1


In [34]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CREDITOEXPORTACION = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CREDITOEXPORTACION['TOTAL'] = unidad1_CREDITOEXPORTACION['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 21, 4),
    ("Primas aceptadas en coaseguro", 21, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 21, 6),
    ("Aceptadas Reaseguro interior", 21, 7),
    ("Aceptadas Reaseguro exterior", 21, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 21, 9),
    ("Cedidas Reaseguro interior (resta)", 21, 10),
    ("Cedidas Reaseguro exterior (resta)", 21, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 21, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 21, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w4_005 = get_value(unidad1_CREDITOEXPORTACION, "Primas Emitidas Directas", 21)
w5_010 = get_value(unidad1_CREDITOEXPORTACION, "Primas aceptadas en coaseguro", 21)
w6_015 = get_value(unidad1_CREDITOEXPORTACION, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 21)
w7_020 = get_value(unidad1_CREDITOEXPORTACION, "Aceptadas Reaseguro interior", 21)
w8_025 = get_value(unidad1_CREDITOEXPORTACION, "Aceptadas Reaseguro exterior", 21)
w9_030 = get_value(unidad1_CREDITOEXPORTACION, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 21)
w10_035 = get_value(unidad1_CREDITOEXPORTACION, "Cedidas Reaseguro interior (resta)", 21)
w11_040 = get_value(unidad1_CREDITOEXPORTACION, "Cedidas Reaseguro exterior (resta)", 21)
w12_045 = get_value(unidad1_CREDITOEXPORTACION, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 21)
w13_050 = get_value(unidad1_CREDITOEXPORTACION, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 21)

# Aplicar la fórmula específica para calcular ae14_999
w14_999 = (w4_005 + w5_010 - w6_015 + w7_020 + w8_025 - w9_030 - w10_035 - w11_040 + w12_045 + w13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=23, value=w14_999)

<Cell 'Hoja1'.W14>

##  24  DESEMPLEO UNIDAD 1


In [35]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_DESEMPLEO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_DESEMPLEO['TOTAL'] = unidad1_DESEMPLEO['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 24, 4),
    ("Primas aceptadas en coaseguro", 24, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 24, 6),
    ("Aceptadas Reaseguro interior", 24, 7),
    ("Aceptadas Reaseguro exterior", 24, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 24, 9),
    ("Cedidas Reaseguro interior (resta)", 24, 10),
    ("Cedidas Reaseguro exterior (resta)", 24, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 24, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 24, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z4_005 = get_value(unidad1_DESEMPLEO, "Primas Emitidas Directas", 24)
z5_010 = get_value(unidad1_DESEMPLEO, "Primas aceptadas en coaseguro", 24)
z6_015 = get_value(unidad1_DESEMPLEO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 24)
z7_020 = get_value(unidad1_DESEMPLEO, "Aceptadas Reaseguro interior", 24)
z8_025 = get_value(unidad1_DESEMPLEO, "Aceptadas Reaseguro exterior", 24)
z9_030 = get_value(unidad1_DESEMPLEO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 24)
z10_035 = get_value(unidad1_DESEMPLEO, "Cedidas Reaseguro interior (resta)", 24)
z11_040 = get_value(unidad1_DESEMPLEO, "Cedidas Reaseguro exterior (resta)", 24)
z12_045 = get_value(unidad1_DESEMPLEO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 24)
z13_050 = get_value(unidad1_DESEMPLEO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 24)

# Aplicar la fórmula específica para calcular ae14_999
z14_999 = (z4_005 + z5_010 - z6_015 + z7_020 + z8_025 - z9_030 - z10_035 - z11_040 + z12_045 + z13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=26, value=z14_999)

<Cell 'Hoja1'.Z14>

##  25  HOGAR UNIDAD 1


In [36]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_HOGAR = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_HOGAR['TOTAL'] = unidad1_HOGAR['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 25, 4),
    ("Primas aceptadas en coaseguro", 25, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 25, 6),
    ("Aceptadas Reaseguro interior", 25, 7),
    ("Aceptadas Reaseguro exterior", 25, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 25, 9),
    ("Cedidas Reaseguro interior (resta)", 25, 10),
    ("Cedidas Reaseguro exterior (resta)", 25, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 25, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 25, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa4_005 = get_value(unidad1_HOGAR, "Primas Emitidas Directas", 25)
aa5_010 = get_value(unidad1_HOGAR, "Primas aceptadas en coaseguro", 25)
aa6_015 = get_value(unidad1_HOGAR, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 25)
aa7_020 = get_value(unidad1_HOGAR, "Aceptadas Reaseguro interior", 25)
aa8_025 = get_value(unidad1_HOGAR, "Aceptadas Reaseguro exterior", 25)
aa9_030 = get_value(unidad1_HOGAR, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 25)
aa10_035 = get_value(unidad1_HOGAR, "Cedidas Reaseguro interior (resta)", 25)
aa11_040 = get_value(unidad1_HOGAR, "Cedidas Reaseguro exterior (resta)", 25)
aa12_045 = get_value(unidad1_HOGAR, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 25)
aa13_050 = get_value(unidad1_HOGAR, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 25)

# Aplicar la fórmula específica para calcular ae14_999
aa14_999 = (aa4_005 + aa5_010 - aa6_015 + aa7_020 + aa8_025 - aa9_030 - aa10_035 - aa11_040 + aa12_045 + aa13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=27, value=aa14_999)

<Cell 'Hoja1'.AA14>

## 42 AGROPECUARIO UNIDAD 1


In [37]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_AGROPECUARIO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_AGROPECUARIO['TOTAL'] = unidad1_AGROPECUARIO['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 42, 4),
    ("Primas aceptadas en coaseguro", 42, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",42, 6),
    ("Aceptadas Reaseguro interior", 42, 7),
    ("Aceptadas Reaseguro exterior", 42, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 42, 9),
    ("Cedidas Reaseguro interior (resta)", 42, 10),
    ("Cedidas Reaseguro exterior (resta)", 42, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 42, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 42, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar4_005 = get_value(unidad1_AGROPECUARIO, "Primas Emitidas Directas", 42)
ar5_010 = get_value(unidad1_AGROPECUARIO, "Primas aceptadas en coaseguro", 42)
ar6_015 = get_value(unidad1_AGROPECUARIO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 42)
ar7_020 = get_value(unidad1_AGROPECUARIO, "Aceptadas Reaseguro interior", 42)
ar8_025 = get_value(unidad1_AGROPECUARIO, "Aceptadas Reaseguro exterior", 42)
ar9_030 = get_value(unidad1_AGROPECUARIO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 42)
ar10_035 = get_value(unidad1_AGROPECUARIO, "Cedidas Reaseguro interior (resta)", 42)
ar11_040 = get_value(unidad1_AGROPECUARIO, "Cedidas Reaseguro exterior (resta)", 42)
ar12_045 = get_value(unidad1_AGROPECUARIO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 42)
ar13_050 = get_value(unidad1_AGROPECUARIO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 42)

# Aplicar la fórmula específica para calcular ae14_999
ar14_999 = (ar4_005 + ar5_010 - ar6_015 + ar7_020 + ar8_025 - ar9_030 - ar10_035 - ar11_040 + ar12_045 + ar13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=44, value=ar14_999)

<Cell 'Hoja1'.AR14>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 1


In [38]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_OPERNOASIGNABLES = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_OPERNOASIGNABLES['TOTAL'] = unidad1_OPERNOASIGNABLES['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 43, 4),
    ("Primas aceptadas en coaseguro", 43, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",43, 6),
    ("Aceptadas Reaseguro interior", 43, 7),
    ("Aceptadas Reaseguro exterior", 43, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 43, 9),
    ("Cedidas Reaseguro interior (resta)", 43, 10),
    ("Cedidas Reaseguro exterior (resta)", 43, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 43, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 43, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as4_005 = get_value(unidad1_OPERNOASIGNABLES, "Primas Emitidas Directas", 43)
as5_010 = get_value(unidad1_OPERNOASIGNABLES, "Primas aceptadas en coaseguro", 43)
as6_015 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 43)
as7_020 = get_value(unidad1_OPERNOASIGNABLES, "Aceptadas Reaseguro interior", 43)
as8_025 = get_value(unidad1_OPERNOASIGNABLES, "Aceptadas Reaseguro exterior", 43)
as9_030 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 43)
as10_035 = get_value(unidad1_OPERNOASIGNABLES, "Cedidas Reaseguro interior (resta)", 43)
as11_040 = get_value(unidad1_OPERNOASIGNABLES, "Cedidas Reaseguro exterior (resta)", 43)
as12_045 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 43)
as13_050 = get_value(unidad1_OPERNOASIGNABLES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 43)

# Aplicar la fórmula específica para calcular ae14_999
as14_999 = (as4_005 + as5_010 - as6_015 + as7_020 + as8_025 - as9_030 - as10_035 - as11_040 + as12_045 + as13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=45, value=as14_999)

<Cell 'Hoja1'.AS14>

## 44 DECENAL UNIDAD 1


In [39]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_DECENAL = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_DECENAL['TOTAL'] = unidad1_DECENAL['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 44, 4),
    ("Primas aceptadas en coaseguro", 44, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",44, 6),
    ("Aceptadas Reaseguro interior", 44, 7),
    ("Aceptadas Reaseguro exterior", 44, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 44, 9),
    ("Cedidas Reaseguro interior (resta)", 44, 10),
    ("Cedidas Reaseguro exterior (resta)", 44, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 44, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 44, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at4_005 = get_value(unidad1_DECENAL, "Primas Emitidas Directas", 44)
at5_010 = get_value(unidad1_DECENAL, "Primas aceptadas en coaseguro", 44)
at6_015 = get_value(unidad1_DECENAL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 44)
at7_020 = get_value(unidad1_DECENAL, "Aceptadas Reaseguro interior", 44)
at8_025 = get_value(unidad1_DECENAL, "Aceptadas Reaseguro exterior", 44)
at9_030 = get_value(unidad1_DECENAL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 44)
at10_035 = get_value(unidad1_DECENAL, "Cedidas Reaseguro interior (resta)", 44)
at11_040 = get_value(unidad1_DECENAL, "Cedidas Reaseguro exterior (resta)", 44)
at12_045 = get_value(unidad1_DECENAL, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 44)
at13_050 = get_value(unidad1_DECENAL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 44)

# Aplicar la fórmula específica para calcular ae14_999
at14_999 = (at4_005 + at5_010 - at6_015 + at7_020 + at8_025 - at9_030 - at10_035 - at11_040 + at12_045 + at13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=46, value=at14_999)

<Cell 'Hoja1'.AT14>

## 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 1


In [40]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOESTATALES = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOESTATALES['TOTAL'] = unidad1_CUMPLIMIENTOESTATALES['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 45, 4),
    ("Primas aceptadas en coaseguro", 45, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",45, 6),
    ("Aceptadas Reaseguro interior", 45, 7),
    ("Aceptadas Reaseguro exterior", 45, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 45, 9),
    ("Cedidas Reaseguro interior (resta)", 45, 10),
    ("Cedidas Reaseguro exterior (resta)", 45, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 45, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 45, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOESTATALES, concept, ramo, 47, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
au4_005 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Primas Emitidas Directas", 45)
au5_010 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Primas aceptadas en coaseguro", 45)
au6_015 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 45)
au7_020 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Aceptadas Reaseguro interior", 45)
au8_025 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Aceptadas Reaseguro exterior", 45)
au9_030 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 45)
au10_035 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Cedidas Reaseguro interior (resta)", 45)
au11_040 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Cedidas Reaseguro exterior (resta)", 45)
au12_045 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 45)
au13_050 = get_value(unidad1_CUMPLIMIENTOESTATALES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 45)

# Aplicar la fórmula específica para calcular ae14_999
au14_999 = (au4_005 + au5_010 - au6_015 + au7_020 + au8_025 - au9_030 - au10_035 - au11_040 + au12_045 + au13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=47, value=au14_999)

<Cell 'Hoja1'.AU14>

## 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 1


In [41]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOEMPRESASSERV = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOEMPRESASSERV['TOTAL'] = unidad1_CUMPLIMIENTOEMPRESASSERV['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 46, 4),
    ("Primas aceptadas en coaseguro", 46, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",46, 6),
    ("Aceptadas Reaseguro interior", 46, 7),
    ("Aceptadas Reaseguro exterior", 46, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 46, 9),
    ("Cedidas Reaseguro interior (resta)", 46, 10),
    ("Cedidas Reaseguro exterior (resta)", 46, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 46, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 46, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOEMPRESASSERV, concept, ramo, 48, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
av4_005 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Primas Emitidas Directas", 46)
av5_010 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Primas aceptadas en coaseguro", 46)
av6_015 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 46)
av7_020 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Aceptadas Reaseguro interior", 46)
av8_025 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Aceptadas Reaseguro exterior", 46)
av9_030 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 46)
av10_035 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Cedidas Reaseguro interior (resta)", 46)
av11_040 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Cedidas Reaseguro exterior (resta)", 46)
av12_045 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 46)
av13_050 = get_value(unidad1_CUMPLIMIENTOEMPRESASSERV, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 46)

# Aplicar la fórmula específica para calcular ae14_999
av14_999 = (av4_005 + av5_010 - av6_015 + av7_020 + av8_025 - av9_030 - av10_035 - av11_040 + av12_045 + av13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=48, value=av14_999)

<Cell 'Hoja1'.AV14>

## 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 1


In [42]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOEMPRESASINDUSTRI = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOEMPRESASINDUSTRI['TOTAL'] = unidad1_CUMPLIMIENTOEMPRESASINDUSTRI['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 47, 4),
    ("Primas aceptadas en coaseguro", 47, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",47, 6),
    ("Aceptadas Reaseguro interior", 47, 7),
    ("Aceptadas Reaseguro exterior", 47, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 47, 9),
    ("Cedidas Reaseguro interior (resta)", 47, 10),
    ("Cedidas Reaseguro exterior (resta)", 47, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 47, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 47, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, concept, ramo, 49, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aw4_005 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Primas Emitidas Directas", 47)
aw5_010 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Primas aceptadas en coaseguro", 47)
aw6_015 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 47)
aw7_020 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Aceptadas Reaseguro interior", 47)
aw8_025 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Aceptadas Reaseguro exterior", 47)
aw9_030 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 47)
aw10_035 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Cedidas Reaseguro interior (resta)", 47)
aw11_040 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Cedidas Reaseguro exterior (resta)", 47)
aw12_045 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 47)
aw13_050 = get_value(unidad1_CUMPLIMIENTOEMPRESASINDUSTRI, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 47)

# Aplicar la fórmula específica para calcular ae14_999
aw14_999 = (aw4_005 + aw5_010 - aw6_015 + aw7_020 + aw8_025 - aw9_030 - aw10_035 - aw11_040 + aw12_045 + aw13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=49, value=aw14_999)

<Cell 'Hoja1'.AW14>

## 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 1


In [43]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOEMPRESASDISPOLEG = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOEMPRESASDISPOLEG['TOTAL'] = unidad1_CUMPLIMIENTOEMPRESASDISPOLEG['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 48, 4),
    ("Primas aceptadas en coaseguro", 48, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",48, 6),
    ("Aceptadas Reaseguro interior", 48, 7),
    ("Aceptadas Reaseguro exterior", 48, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 48, 9),
    ("Cedidas Reaseguro interior (resta)", 48, 10),
    ("Cedidas Reaseguro exterior (resta)", 48, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 48, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 48, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, concept, ramo, 50, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ax4_005 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Primas Emitidas Directas", 48)
ax5_010 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Primas aceptadas en coaseguro", 48)
ax6_015 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 48)
ax7_020 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Aceptadas Reaseguro interior", 48)
ax8_025 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Aceptadas Reaseguro exterior", 48)
ax9_030 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 48)
ax10_035 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Cedidas Reaseguro interior (resta)", 48)
ax11_040 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Cedidas Reaseguro exterior (resta)", 48)
ax12_045 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 48)
ax13_050 = get_value(unidad1_CUMPLIMIENTOEMPRESASDISPOLEG, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 48)

# Aplicar la fórmula específica para calcular ae14_999
ax14_999 = (ax4_005 + ax5_010 - ax6_015 + ax7_020 + ax8_025 - ax9_030 - ax10_035 - ax11_040 + ax12_045 + ax13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=50, value=ax14_999)

<Cell 'Hoja1'.AX14>

## 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 1


In [44]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOCAUCIONES = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOCAUCIONES['TOTAL'] = unidad1_CUMPLIMIENTOCAUCIONES['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 49, 4),
    ("Primas aceptadas en coaseguro", 49, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",49, 6),
    ("Aceptadas Reaseguro interior", 49, 7),
    ("Aceptadas Reaseguro exterior", 49, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 49, 9),
    ("Cedidas Reaseguro interior (resta)", 49, 10),
    ("Cedidas Reaseguro exterior (resta)", 49, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 49, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 49, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOCAUCIONES, concept, ramo, 51, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ay4_005 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Primas Emitidas Directas", 49)
ay5_010 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Primas aceptadas en coaseguro", 49)
ay6_015 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 49)
ay7_020 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Aceptadas Reaseguro interior", 49)
ay8_025 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Aceptadas Reaseguro exterior", 49)
ay9_030 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 49)
ay10_035 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Cedidas Reaseguro interior (resta)", 49)
ay11_040 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Cedidas Reaseguro exterior (resta)", 49)
ay12_045 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 49)
ay13_050 = get_value(unidad1_CUMPLIMIENTOCAUCIONES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 49)

# Aplicar la fórmula específica para calcular ae14_999
ay14_999 = (ay4_005 + ay5_010 - ay6_015 + ay7_020 + ay8_025 - ay9_030 - ay10_035 - ay11_040 + ay12_045 + ay13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=51, value=ay14_999)

<Cell 'Hoja1'.AY14>

## 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 1


In [45]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOARRENDAMIENTO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOARRENDAMIENTO['TOTAL'] = unidad1_CUMPLIMIENTOARRENDAMIENTO['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 50, 4),
    ("Primas aceptadas en coaseguro", 50, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",50, 6),
    ("Aceptadas Reaseguro interior", 50, 7),
    ("Aceptadas Reaseguro exterior", 50, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 50, 9),
    ("Cedidas Reaseguro interior (resta)", 50, 10),
    ("Cedidas Reaseguro exterior (resta)", 50, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 50, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 50, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOARRENDAMIENTO, concept, ramo, 52, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
az4_005 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Primas Emitidas Directas", 50)
az5_010 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Primas aceptadas en coaseguro", 50)
az6_015 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 50)
az7_020 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Aceptadas Reaseguro interior", 50)
az8_025 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Aceptadas Reaseguro exterior", 50)
az9_030 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 50)
az10_035 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Cedidas Reaseguro interior (resta)", 50)
az11_040 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Cedidas Reaseguro exterior (resta)", 50)
az12_045 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 50)
az13_050 = get_value(unidad1_CUMPLIMIENTOARRENDAMIENTO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 50)

# Aplicar la fórmula específica para calcular ae14_999
az14_999 = (az4_005 + az5_010 - az6_015 + az7_020 + az8_025 - az9_030 - az10_035 - az11_040 + az12_045 + az13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=52, value=az14_999)

<Cell 'Hoja1'.AZ14>

## 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 1


In [46]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_CUMPLIMIENTOPARTICULARES = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_CUMPLIMIENTOPARTICULARES['TOTAL'] = unidad1_CUMPLIMIENTOPARTICULARES['TOTAL'] .abs() # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 51, 4),
    ("Primas aceptadas en coaseguro", 51, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)",51, 6),
    ("Aceptadas Reaseguro interior", 51, 7),
    ("Aceptadas Reaseguro exterior", 51, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 51, 9),
    ("Cedidas Reaseguro interior (resta)", 51, 10),
    ("Cedidas Reaseguro exterior (resta)", 51, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 51, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 51, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_CUMPLIMIENTOPARTICULARES, concept, ramo, 53, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ba4_005 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Primas Emitidas Directas", 51)
ba5_010 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Primas aceptadas en coaseguro", 51)
ba6_015 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 51)
ba7_020 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Aceptadas Reaseguro interior", 51)
ba8_025 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Aceptadas Reaseguro exterior", 51)
ba9_030 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 51)
ba10_035 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Cedidas Reaseguro interior (resta)", 51)
ba11_040 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Cedidas Reaseguro exterior (resta)", 51)
ba12_045 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 51)
ba13_050 = get_value(unidad1_CUMPLIMIENTOPARTICULARES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 51)

# Aplicar la fórmula específica para calcular ae14_999
ba14_999 = (ba4_005 + ba5_010 - ba6_015 + ba7_020 + ba8_025 - ba9_030 - ba10_035 - ba11_040 + ba12_045 + ba13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=53, value=ba14_999)

<Cell 'Hoja1'.BA14>

# UNIDAD DE CAPTURA 2   

##    3   AUTOMOVILES UNIDAD 2 

In [47]:
# Filtrar y agrupar los datos
unidad2_AUTOMOVILES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_AUTOMOVILES['TOTAL'] = unidad2_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 3, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 3, 18),
    ("Liberación Reserva Matemática", 3, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 3, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 3, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 3, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 3, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 3, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 3, 26),
    ("Constitución Reserva matemática (resta)", 3, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 3, 28),
    ("Constitución Reserva matemática BEPs", 3, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 3, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 3, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e17_006 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Técnica de prima no devengada", 3)
e18_007 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Técnica por insuficiencia de prima", 3)
e19_010 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Matemática", 3)
e20_015 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Matemática Riesgos Laborales (resta)", 3)
e21_017 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Matemática BEPs (resta)", 3)
e22_020 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 3)
e23_022 = get_value(unidad2_AUTOMOVILES, "Liberación Reserva de Insuficiencia de Activos", 3)
e25_026 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva Técnica de prima no devengada (resta)", 3)
e26_027 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 3)
e27_030 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva matemática (resta)", 3)
e28_035 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva matemática Riesgos Laborales", 3)
e29_037 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva matemática BEPs", 3)
e30_040 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 3)
e31_045 = get_value(unidad2_AUTOMOVILES, "Constitución Reserva de Insuficiencia de Activos (resta)", 3)

# Calcular el total general
e32_999 = (e17_006 + e18_007 + e19_010 - e20_015 - e21_017 + e22_020 + e23_022 - e25_026 - e26_027 - e27_030 + e28_035 + e29_037 - e30_040 - e31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=5, value=e32_999)


<Cell 'Hoja1'.E32>

##  4   SOAT UNIDAD 2


In [48]:
# Filtrar y agrupar los datos
unidad2_SOAT = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_SOAT['TOTAL'] = unidad2_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 4, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 4, 18),
    ("Liberación Reserva Matemática", 4, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 4, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 4, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 4, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 4, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 4, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 4, 26),
    ("Constitución Reserva matemática (resta)", 4, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 4, 28),
    ("Constitución Reserva matemática BEPs", 4, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 4, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 4, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f17_006 = get_value(unidad2_SOAT, "Liberación Reserva Técnica de prima no devengada", 4)
f18_007 = get_value(unidad2_SOAT, "Liberación Reserva Técnica por insuficiencia de prima", 4)
f19_010 = get_value(unidad2_SOAT, "Liberación Reserva Matemática", 4)
f20_015 = get_value(unidad2_SOAT, "Liberación Reserva Matemática Riesgos Laborales (resta)", 4)
f21_017 = get_value(unidad2_SOAT, "Liberación Reserva Matemática BEPs (resta)", 3)
f22_020 = get_value(unidad2_SOAT, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 4)
f23_022 = get_value(unidad2_SOAT, "Liberación Reserva de Insuficiencia de Activos", 4)
f25_026 = get_value(unidad2_SOAT, "Constitución Reserva Técnica de prima no devengada (resta)", 4)
f26_027 = get_value(unidad2_SOAT, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 4)
f27_030 = get_value(unidad2_SOAT, "Constitución Reserva matemática (resta)", 4)
f28_035 = get_value(unidad2_SOAT, "Constitución Reserva matemática Riesgos Laborales", 4)
f29_037 = get_value(unidad2_SOAT, "Constitución Reserva matemática BEPs", 4)
f30_040 = get_value(unidad2_SOAT, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 4)
f31_045 = get_value(unidad2_SOAT, "Constitución Reserva de Insuficiencia de Activos (resta)", 4)

# Calcular el total general
f32_999 = (f17_006 + f18_007 + f19_010 - f20_015 - f21_017 + f22_020 + f23_022 - f25_026 - f26_027 - f27_030 + f28_035 + f29_037 - f30_040 - f31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=6, value=f32_999)


<Cell 'Hoja1'.F32>

##  5   CUMPLIMIENTO UNIDAD 2


In [49]:
# Filtrar y agrupar los datos
unidad2_CUMPLIMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_CUMPLIMIENTO['TOTAL'] = unidad2_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 5, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 5, 18),
    ("Liberación Reserva Matemática", 5, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 5, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 5, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 5, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 5, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 5, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 5, 26),
    ("Constitución Reserva matemática (resta)", 5, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 5, 28),
    ("Constitución Reserva matemática BEPs", 5, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 5, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 5, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g17_006 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Técnica de prima no devengada", 5)
g18_007 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Técnica por insuficiencia de prima", 5)
g19_010 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Matemática", 5)
g20_015 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 5)
g21_017 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Matemática BEPs (resta)", 5)
g22_020 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 5)
g23_022 = get_value(unidad2_CUMPLIMIENTO, "Liberación Reserva de Insuficiencia de Activos", 5)
g25_026 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva Técnica de prima no devengada (resta)", 5)
g26_027 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 5)
g27_030 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva matemática (resta)", 5)
g28_035 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva matemática Riesgos Laborales", 5)
g29_037 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva matemática BEPs", 5)
g30_040 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 5)
g31_045 = get_value(unidad2_CUMPLIMIENTO, "Constitución Reserva de Insuficiencia de Activos (resta)", 5)

# Calcular el total general
g32_999 = (g17_006 + g18_007 + g19_010 - g20_015 - g21_017 + g22_020 + g23_022 - g25_026 - g26_027 - g27_030 + g28_035 + g29_037 - g30_040 - g31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=7, value=g32_999)

<Cell 'Hoja1'.G32>

##  6   RESPONSABILIDAD CIVIL UNIDAD 2


In [50]:
# Filtrar y agrupar los datos
unidad2_RESPONSABILIDADCI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_RESPONSABILIDADCI['TOTAL'] = unidad2_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 6, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 6, 18),
    ("Liberación Reserva Matemática", 6, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 6, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 6, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 6, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 6, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 6, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 6, 26),
    ("Constitución Reserva matemática (resta)", 6, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 6, 28),
    ("Constitución Reserva matemática BEPs", 6, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 6, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 6, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h17_006 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Técnica de prima no devengada", 6)
h18_007 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Técnica por insuficiencia de prima", 6)
h19_010 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Matemática", 6)
h20_015 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Matemática Riesgos Laborales (resta)", 6)
h21_017 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Matemática BEPs (resta)", 6)
h22_020 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 6)
h23_022 = get_value(unidad2_RESPONSABILIDADCI, "Liberación Reserva de Insuficiencia de Activos", 6)
h25_026 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva Técnica de prima no devengada (resta)", 6)
h26_027 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 6)
h27_030 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva matemática (resta)", 6)
h28_035 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva matemática Riesgos Laborales", 6)
h29_037 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva matemática BEPs", 6)
h30_040 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 6)
h31_045 = get_value(unidad2_RESPONSABILIDADCI, "Constitución Reserva de Insuficiencia de Activos (resta)", 6)

# Calcular el total general
h32_999 = (h17_006 + h18_007 + h19_010 - h20_015 - h21_017 + h22_020 + h23_022 - h25_026 - h26_027 - h27_030 + h28_035 + h29_037 - h30_040 - h31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=8, value=h32_999)

<Cell 'Hoja1'.H32>

##  7   INCENDIO UNIDAD 2 


In [51]:
# Filtrar y agrupar los datos
unidad2_INCENDIO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_INCENDIO['TOTAL'] = unidad2_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 7, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 7, 18),
    ("Liberación Reserva Matemática", 7, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 7, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 7, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 7, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 7, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 7, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 7, 26),
    ("Constitución Reserva matemática (resta)", 7, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 7, 28),
    ("Constitución Reserva matemática BEPs", 7, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 7, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 7, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i17_006 = get_value(unidad2_INCENDIO, "Liberación Reserva Técnica de prima no devengada", 7)
i18_007 = get_value(unidad2_INCENDIO, "Liberación Reserva Técnica por insuficiencia de prima", 7)
i19_010 = get_value(unidad2_INCENDIO, "Liberación Reserva Matemática", 7)
i20_015 = get_value(unidad2_INCENDIO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 7)
i21_017 = get_value(unidad2_INCENDIO, "Liberación Reserva Matemática BEPs (resta)", 7)
i22_020 = get_value(unidad2_INCENDIO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 7)
i23_022 = get_value(unidad2_INCENDIO, "Liberación Reserva de Insuficiencia de Activos", 7)
i25_026 = get_value(unidad2_INCENDIO, "Constitución Reserva Técnica de prima no devengada (resta)", 7)
i26_027 = get_value(unidad2_INCENDIO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 7)
i27_030 = get_value(unidad2_INCENDIO, "Constitución Reserva matemática (resta)", 7)
i28_035 = get_value(unidad2_INCENDIO, "Constitución Reserva matemática Riesgos Laborales", 7)
i29_037 = get_value(unidad2_INCENDIO, "Constitución Reserva matemática BEPs", 7)
i30_040 = get_value(unidad2_INCENDIO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 7)
i31_045 = get_value(unidad2_INCENDIO, "Constitución Reserva de Insuficiencia de Activos (resta)", 7)

# Calcular el total general
i32_999 = (i17_006 + i18_007 + i19_010 - i20_015 - i21_017 + i22_020 + i23_022 - i25_026 - i26_027 - i27_030 + i28_035 + i29_037 - i30_040 - i31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=9, value=i32_999)

<Cell 'Hoja1'.I32>

##  8   TERREMOTO UNIDAD 2

In [52]:
# Filtrar y agrupar los datos
unidad2_TERREMOTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_TERREMOTO['TOTAL'] = unidad2_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 8, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 8, 18),
    ("Liberación Reserva Matemática", 8, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 8, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 8, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 8, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 8, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 8, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 8, 26),
    ("Constitución Reserva matemática (resta)", 8, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 8, 28),
    ("Constitución Reserva matemática BEPs", 8, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 8, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 8, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j17_006 = get_value(unidad2_TERREMOTO, "Liberación Reserva Técnica de prima no devengada", 8)
j18_007 = get_value(unidad2_TERREMOTO, "Liberación Reserva Técnica por insuficiencia de prima", 8)
j19_010 = get_value(unidad2_TERREMOTO, "Liberación Reserva Matemática", 8)
j20_015 = get_value(unidad2_TERREMOTO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 8)
j21_017 = get_value(unidad2_TERREMOTO, "Liberación Reserva Matemática BEPs (resta)", 8)
j22_020 = get_value(unidad2_TERREMOTO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 8)
j23_022 = get_value(unidad2_TERREMOTO, "Liberación Reserva de Insuficiencia de Activos", 8)
j25_026 = get_value(unidad2_TERREMOTO, "Constitución Reserva Técnica de prima no devengada (resta)", 8)
j26_027 = get_value(unidad2_TERREMOTO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 8)
j27_030 = get_value(unidad2_TERREMOTO, "Constitución Reserva matemática (resta)", 8)
j28_035 = get_value(unidad2_TERREMOTO, "Constitución Reserva matemática Riesgos Laborales", 8)
j29_037 = get_value(unidad2_TERREMOTO, "Constitución Reserva matemática BEPs", 8)
j30_040 = get_value(unidad2_TERREMOTO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 8)
j31_045 = get_value(unidad2_TERREMOTO, "Constitución Reserva de Insuficiencia de Activos (resta)", 8)

# Calcular el total general
j32_999 = (j17_006 + j18_007 + j19_010 - j20_015 - j21_017 + j22_020 + j23_022 - j25_026 - j26_027 - j27_030 + j28_035 + j29_037 - j30_040 - j31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=10, value=j32_999)

<Cell 'Hoja1'.J32>

##  9   SUSTRACCION UNIDAD 2

In [53]:
# Filtrar y agrupar los datos
unidad2_SUSTRACCION= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_SUSTRACCION['TOTAL'] = unidad2_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 9, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 9, 18),
    ("Liberación Reserva Matemática", 9, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 9, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 9, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 9, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 9, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 9, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 9, 26),
    ("Constitución Reserva matemática (resta)", 9, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 9, 28),
    ("Constitución Reserva matemática BEPs", 9, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 9, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 9, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k17_006 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Técnica de prima no devengada", 9)
k18_007 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Técnica por insuficiencia de prima", 9)
k19_010 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Matemática", 9)
k20_015 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Matemática Riesgos Laborales (resta)", 9)
k21_017 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Matemática BEPs (resta)", 9)
k22_020 = get_value(unidad2_SUSTRACCION, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 9)
k23_022 = get_value(unidad2_SUSTRACCION, "Liberación Reserva de Insuficiencia de Activos", 9)
k25_026 = get_value(unidad2_SUSTRACCION, "Constitución Reserva Técnica de prima no devengada (resta)", 9)
k26_027 = get_value(unidad2_SUSTRACCION, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 9)
k27_030 = get_value(unidad2_SUSTRACCION, "Constitución Reserva matemática (resta)", 9)
k28_035 = get_value(unidad2_SUSTRACCION, "Constitución Reserva matemática Riesgos Laborales", 9)
k29_037 = get_value(unidad2_SUSTRACCION, "Constitución Reserva matemática BEPs", 9)
k30_040 = get_value(unidad2_SUSTRACCION, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 9)
k31_045 = get_value(unidad2_SUSTRACCION, "Constitución Reserva de Insuficiencia de Activos (resta)", 9)

# Calcular el total general
k32_999 = (k17_006 + k18_007 + k19_010 - k20_015 - k21_017 + k22_020 + k23_022 - k25_026 - k26_027 - k27_030 + k28_035 + k29_037 - k30_040 - k31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=11, value=k32_999)

<Cell 'Hoja1'.K32>

##  10  TRANSPORTE UNIDAD 2


In [54]:
# Filtrar y agrupar los datos
unidad2_TRANSPORTE= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_TRANSPORTE['TOTAL'] = unidad2_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 10, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 10, 18),
    ("Liberación Reserva Matemática", 10, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 10, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 10, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 10, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 10, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 10, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 10, 26),
    ("Constitución Reserva matemática (resta)", 10, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 10, 28),
    ("Constitución Reserva matemática BEPs", 10, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 10, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 10, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l17_006 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Técnica de prima no devengada", 10)
l18_007 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Técnica por insuficiencia de prima", 10)
l19_010 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Matemática", 10)
l20_015 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Matemática Riesgos Laborales (resta)", 10)
l21_017 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Matemática BEPs (resta)", 10)
l22_020 = get_value(unidad2_TRANSPORTE, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 10)
l23_022 = get_value(unidad2_TRANSPORTE, "Liberación Reserva de Insuficiencia de Activos", 10)
l25_026 = get_value(unidad2_TRANSPORTE, "Constitución Reserva Técnica de prima no devengada (resta)", 10)
l26_027 = get_value(unidad2_TRANSPORTE, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 10)
l27_030 = get_value(unidad2_TRANSPORTE, "Constitución Reserva matemática (resta)", 10)
l28_035 = get_value(unidad2_TRANSPORTE, "Constitución Reserva matemática Riesgos Laborales", 10)
l29_037 = get_value(unidad2_TRANSPORTE, "Constitución Reserva matemática BEPs", 10)
l30_040 = get_value(unidad2_TRANSPORTE, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 10)
l31_045 = get_value(unidad2_TRANSPORTE, "Constitución Reserva de Insuficiencia de Activos (resta)", 10)

# Calcular el total general
l32_999 = (l17_006 +l18_007 + l19_010 - l20_015 - l21_017 + l22_020 + l23_022 - l25_026 - l26_027 - l27_030 + l28_035 + l29_037 - l30_040 - l31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=12, value=l32_999)

<Cell 'Hoja1'.L32>

##  11  CORRIENTE DÉBIL UNIDAD 2


In [55]:
# Filtrar y agrupar los datos
unidad2_CORRIENDEBIL= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_CORRIENDEBIL['TOTAL'] = unidad2_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 11, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 11, 18),
    ("Liberación Reserva Matemática", 11, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 11, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 11, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 11, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 11, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 11, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 11, 26),
    ("Constitución Reserva matemática (resta)", 11, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 11, 28),
    ("Constitución Reserva matemática BEPs", 11, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 11, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 11, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m17_006 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Técnica de prima no devengada", 11)
m18_007 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Técnica por insuficiencia de prima", 11)
m19_010 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Matemática", 11)
m20_015 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Matemática Riesgos Laborales (resta)", 11)
m21_017 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Matemática BEPs (resta)", 11)
m22_020 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 11)
m23_022 = get_value(unidad2_CORRIENDEBIL, "Liberación Reserva de Insuficiencia de Activos", 11)
m25_026 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva Técnica de prima no devengada (resta)", 11)
m26_027 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 11)
m27_030 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva matemática (resta)", 11)
m28_035 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva matemática Riesgos Laborales", 11)
m29_037 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva matemática BEPs", 11)
m30_040 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 11)
m31_045 = get_value(unidad2_CORRIENDEBIL, "Constitución Reserva de Insuficiencia de Activos (resta)", 11)

# Calcular el total general
m32_999 = (m17_006 +m18_007 + m19_010 - m20_015 - m21_017 + m22_020 + m23_022 - m25_026 - m26_027 - m27_030 + m28_035 + m29_037 - m30_040 - m31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=13, value=m32_999)

<Cell 'Hoja1'.M32>

##  12  TODO RIESGO CONTRATISTA UNIDAD 2


In [56]:
# Filtrar y agrupar los datos
unidad2_TODORIESGOCONTRATISTA= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_TODORIESGOCONTRATISTA['TOTAL'] = unidad2_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 12, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 12, 18),
    ("Liberación Reserva Matemática", 12, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 12, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 12, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 12, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 12, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 12, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 12, 26),
    ("Constitución Reserva matemática (resta)", 12, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 12, 28),
    ("Constitución Reserva matemática BEPs", 12, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 12, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 12, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n17_006 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Técnica de prima no devengada", 12)
n18_007 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Técnica por insuficiencia de prima", 12)
n19_010 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Matemática", 12)
n20_015 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Matemática Riesgos Laborales (resta)", 12)
n21_017 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Matemática BEPs (resta)", 12)
n22_020 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 12)
n23_022 = get_value(unidad2_TODORIESGOCONTRATISTA, "Liberación Reserva de Insuficiencia de Activos", 12)
n25_026 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva Técnica de prima no devengada (resta)", 12)
n26_027 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 12)
n27_030 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva matemática (resta)", 12)
n28_035 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva matemática Riesgos Laborales", 12)
n29_037 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva matemática BEPs", 12)
n30_040 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 12)
n31_045 = get_value(unidad2_TODORIESGOCONTRATISTA, "Constitución Reserva de Insuficiencia de Activos (resta)", 12)

# Calcular el total general
n32_999 = (n17_006 + n18_007 + n19_010 - n20_015 - n21_017 + n22_020 + n23_022 - n25_026 - n26_027 - n27_030 + n28_035 + n29_037 - n30_040 - n31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=14, value=n32_999)

<Cell 'Hoja1'.N32>

##  13  MANEJO UNIDAD 2

In [57]:
# Filtrar y agrupar los datos
unidad2_MANEJO= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_MANEJO['TOTAL'] = unidad2_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 13, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 13, 18),
    ("Liberación Reserva Matemática", 13, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 13, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 13, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 13, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 13, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 13, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 13, 26),
    ("Constitución Reserva matemática (resta)", 13, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 13, 28),
    ("Constitución Reserva matemática BEPs", 13, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 13, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 13, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o17_006 = get_value(unidad2_MANEJO, "Liberación Reserva Técnica de prima no devengada", 13)
o18_007 = get_value(unidad2_MANEJO, "Liberación Reserva Técnica por insuficiencia de prima", 13)
o19_010 = get_value(unidad2_MANEJO, "Liberación Reserva Matemática", 13)
o20_015 = get_value(unidad2_MANEJO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 13)
o21_017 = get_value(unidad2_MANEJO, "Liberación Reserva Matemática BEPs (resta)", 13)
o22_020 = get_value(unidad2_MANEJO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 13)
o23_022 = get_value(unidad2_MANEJO, "Liberación Reserva de Insuficiencia de Activos", 13)
o25_026 = get_value(unidad2_MANEJO, "Constitución Reserva Técnica de prima no devengada (resta)", 13)
o26_027 = get_value(unidad2_MANEJO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 13)
o27_030 = get_value(unidad2_MANEJO, "Constitución Reserva matemática (resta)", 13)
o28_035 = get_value(unidad2_MANEJO, "Constitución Reserva matemática Riesgos Laborales", 13)
o29_037 = get_value(unidad2_MANEJO, "Constitución Reserva matemática BEPs", 13)
o30_040 = get_value(unidad2_MANEJO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 13)
o31_045 = get_value(unidad2_MANEJO, "Constitución Reserva de Insuficiencia de Activos (resta)", 13)

# Calcular el total general
o32_999 = (o17_006 + o18_007 + o19_010 - o20_015 - o21_017 + o22_020 + o23_022 - o25_026 - o26_027 - o27_030 + o28_035 + o29_037 - o30_040 - o31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=15, value=o32_999)

<Cell 'Hoja1'.O32>

##  14  LUCRO CESANTE UNIDAD 2


In [58]:
# Filtrar y agrupar los datos
unidad2_LUCROCESANTE= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_LUCROCESANTE['TOTAL'] = unidad2_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 14, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 14, 18),
    ("Liberación Reserva Matemática", 14, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 14, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 14, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 14, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 14, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 14, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 14, 26),
    ("Constitución Reserva matemática (resta)", 14, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 14, 28),
    ("Constitución Reserva matemática BEPs", 14, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 14, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 14, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p17_006 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Técnica de prima no devengada", 14)
p18_007 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Técnica por insuficiencia de prima", 14)
p19_010 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Matemática", 14)
p20_015 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Matemática Riesgos Laborales (resta)", 14)
p21_017 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Matemática BEPs (resta)", 14)
p22_020 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 14)
p23_022 = get_value(unidad2_LUCROCESANTE, "Liberación Reserva de Insuficiencia de Activos", 14)
p25_026 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva Técnica de prima no devengada (resta)", 14)
p26_027 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 14)
p27_030 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva matemática (resta)", 14)
p28_035 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva matemática Riesgos Laborales", 14)
p29_037 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva matemática BEPs", 14)
p30_040 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 14)
p31_045 = get_value(unidad2_LUCROCESANTE, "Constitución Reserva de Insuficiencia de Activos (resta)", 14)

# Calcular el total general
p32_999 = (p17_006 + p18_007 + p19_010 - p20_015 - p21_017 + p22_020 + p23_022 - p25_026 - p26_027 - p27_030 + p28_035 + p29_037 - p30_040 - p31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=16, value=p32_999)

<Cell 'Hoja1'.P32>

##  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 2


In [59]:
# Filtrar y agrupar los datos
unidad2_MONTAJEYROTURA= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_MONTAJEYROTURA['TOTAL'] = unidad2_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 15, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 15, 18),
    ("Liberación Reserva Matemática", 15, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 15, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 15, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 15, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 15, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 15, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 15, 26),
    ("Constitución Reserva matemática (resta)", 15, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 15, 28),
    ("Constitución Reserva matemática BEPs", 15, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 15, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 15, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q17_006 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Técnica de prima no devengada", 15)
q18_007 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Técnica por insuficiencia de prima", 15)
q19_010 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Matemática", 15)
q20_015 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Matemática Riesgos Laborales (resta)", 15)
q21_017 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Matemática BEPs (resta)", 15)
q22_020 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 15)
q23_022 = get_value(unidad2_MONTAJEYROTURA, "Liberación Reserva de Insuficiencia de Activos", 15)
q25_026 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva Técnica de prima no devengada (resta)", 15)
q26_027 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 15)
q27_030 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva matemática (resta)", 15)
q28_035 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva matemática Riesgos Laborales", 15)
q29_037 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva matemática BEPs", 15)
q30_040 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 15)
q31_045 = get_value(unidad2_MONTAJEYROTURA, "Constitución Reserva de Insuficiencia de Activos (resta)", 15)

# Calcular el total general
q32_999 = (q17_006 + q18_007 + q19_010 - q20_015 - q21_017 + q22_020 + q23_022 - q25_026 - q26_027 - q27_030 + q28_035 + q29_037 - q30_040 - q31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=17, value=q32_999)

<Cell 'Hoja1'.Q32>

##  16  AVIACIÓN UNIDAD 2


In [60]:
# Filtrar y agrupar los datos
unidad2_AVIACION= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_AVIACION['TOTAL'] = unidad2_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 16, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 16, 18),
    ("Liberación Reserva Matemática", 16, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 16, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 16, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 16, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 16, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 16, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 16, 26),
    ("Constitución Reserva matemática (resta)", 16, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 16, 28),
    ("Constitución Reserva matemática BEPs", 16, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 16, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 16, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r17_006 = get_value(unidad2_AVIACION, "Liberación Reserva Técnica de prima no devengada", 16)
r18_007 = get_value(unidad2_AVIACION, "Liberación Reserva Técnica por insuficiencia de prima", 16)
r19_010 = get_value(unidad2_AVIACION, "Liberación Reserva Matemática", 16)
r20_015 = get_value(unidad2_AVIACION, "Liberación Reserva Matemática Riesgos Laborales (resta)", 16)
r21_017 = get_value(unidad2_AVIACION, "Liberación Reserva Matemática BEPs (resta)", 16)
r22_020 = get_value(unidad2_AVIACION, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 16)
r23_022 = get_value(unidad2_AVIACION, "Liberación Reserva de Insuficiencia de Activos", 16)
r25_026 = get_value(unidad2_AVIACION, "Constitución Reserva Técnica de prima no devengada (resta)", 16)
r26_027 = get_value(unidad2_AVIACION, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 16)
r27_030 = get_value(unidad2_AVIACION, "Constitución Reserva matemática (resta)", 16)
r28_035 = get_value(unidad2_AVIACION, "Constitución Reserva matemática Riesgos Laborales", 16)
r29_037 = get_value(unidad2_AVIACION, "Constitución Reserva matemática BEPs", 16)
r30_040 = get_value(unidad2_AVIACION, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 16)
r31_045 = get_value(unidad2_AVIACION, "Constitución Reserva de Insuficiencia de Activos (resta)", 16)

# Calcular el total general
r32_999 = (r17_006 + r18_007 + r19_010 - r20_015 - r21_017 + r22_020 + r23_022 - r25_026 - r26_027 - r27_030 + r28_035 + r29_037 - r30_040 - r31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=18, value=r32_999)

<Cell 'Hoja1'.R32>

##  17  NAVEGACIÓN Y CASCO UNIDAD 2


In [61]:
# Filtrar y agrupar los datos
unidad2_NAVEGACION= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_NAVEGACION['TOTAL'] = unidad2_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 17, 18),
    ("Liberación Reserva Matemática", 17, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 17, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 17, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 17, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 17, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 17, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 17, 26),
    ("Constitución Reserva matemática (resta)", 17, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 17, 28),
    ("Constitución Reserva matemática BEPs", 17, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 17, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 17, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s17_006 = get_value(unidad2_NAVEGACION, "Liberación Reserva Técnica de prima no devengada", 17)
s18_007 = get_value(unidad2_NAVEGACION, "Liberación Reserva Técnica por insuficiencia de prima", 17)
s19_010 = get_value(unidad2_NAVEGACION, "Liberación Reserva Matemática", 17)
s20_015 = get_value(unidad2_NAVEGACION, "Liberación Reserva Matemática Riesgos Laborales (resta)", 17)
s21_017 = get_value(unidad2_NAVEGACION, "Liberación Reserva Matemática BEPs (resta)", 17)
s22_020 = get_value(unidad2_NAVEGACION, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 17)
s23_022 = get_value(unidad2_NAVEGACION, "Liberación Reserva de Insuficiencia de Activos", 17)
s25_026 = get_value(unidad2_NAVEGACION, "Constitución Reserva Técnica de prima no devengada (resta)", 17)
s26_027 = get_value(unidad2_NAVEGACION, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 17)
s27_030 = get_value(unidad2_NAVEGACION, "Constitución Reserva matemática (resta)", 17)
s28_035 = get_value(unidad2_NAVEGACION, "Constitución Reserva matemática Riesgos Laborales", 17)
s29_037 = get_value(unidad2_NAVEGACION, "Constitución Reserva matemática BEPs", 17)
s30_040 = get_value(unidad2_NAVEGACION, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 17)
s31_045 = get_value(unidad2_NAVEGACION, "Constitución Reserva de Insuficiencia de Activos (resta)", 17)

# Calcular el total general
s32_999 = (s17_006 + s18_007 + s19_010 - s20_015 - s21_017 + s22_020 + s23_022 - s25_026 - s26_027 - s27_030 + s28_035 + s29_037 - s30_040 - s31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=19, value=s32_999)

<Cell 'Hoja1'.S32>

##  18  MINAS Y PETRÓLEOS UNIDAD 2


In [62]:
# Filtrar y agrupar los datos
unidad2_MINAS= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_MINAS['TOTAL'] = unidad2_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 18, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18, 18),
    ("Liberación Reserva Matemática", 18, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 18, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 18, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 18, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 18, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 18, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 18, 26),
    ("Constitución Reserva matemática (resta)", 18, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 18, 28),
    ("Constitución Reserva matemática BEPs", 18, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 18, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 18, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t17_006 = get_value(unidad2_MINAS, "Liberación Reserva Técnica de prima no devengada", 18)
t18_007 = get_value(unidad2_MINAS, "Liberación Reserva Técnica por insuficiencia de prima", 18)
t19_010 = get_value(unidad2_MINAS, "Liberación Reserva Matemática", 18)
t20_015 = get_value(unidad2_MINAS, "Liberación Reserva Matemática Riesgos Laborales (resta)", 18)
t21_017 = get_value(unidad2_MINAS, "Liberación Reserva Matemática BEPs (resta)", 18)
t22_020 = get_value(unidad2_MINAS, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 18)
t23_022 = get_value(unidad2_MINAS, "Liberación Reserva de Insuficiencia de Activos", 18)
t25_026 = get_value(unidad2_MINAS, "Constitución Reserva Técnica de prima no devengada (resta)", 18)
t26_027 = get_value(unidad2_MINAS, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 18)
t27_030 = get_value(unidad2_MINAS, "Constitución Reserva matemática (resta)", 18)
t28_035 = get_value(unidad2_MINAS, "Constitución Reserva matemática Riesgos Laborales", 18)
t29_037 = get_value(unidad2_MINAS, "Constitución Reserva matemática BEPs", 18)
t30_040 = get_value(unidad2_MINAS, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 18)
t31_045 = get_value(unidad2_MINAS, "Constitución Reserva de Insuficiencia de Activos (resta)", 18)

# Calcular el total general
t32_999 = (t17_006 + t18_007 + t19_010 - t20_015 - t21_017 + t22_020 + t23_022 - t25_026 - t26_027 - t27_030 + t28_035 + t29_037 - t30_040 - t31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=20, value=t32_999)

<Cell 'Hoja1'.T32>

##  19  VIDRIOS UNIDAD 2


In [63]:
# Filtrar y agrupar los datos
unidad2_VIDRIOS= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_VIDRIOS['TOTAL'] = unidad2_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 19, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 19, 18),
    ("Liberación Reserva Matemática", 19, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 19, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 19, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 19, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 19, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 19, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 19, 26),
    ("Constitución Reserva matemática (resta)", 19, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 19, 28),
    ("Constitución Reserva matemática BEPs", 19, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 19, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 19, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u17_006 = get_value(unidad2_VIDRIOS, "Liberación Reserva Técnica de prima no devengada", 19)
u18_007 = get_value(unidad2_VIDRIOS, "Liberación Reserva Técnica por insuficiencia de prima", 19)
u19_010 = get_value(unidad2_VIDRIOS, "Liberación Reserva Matemática", 19)
u20_015 = get_value(unidad2_VIDRIOS, "Liberación Reserva Matemática Riesgos Laborales (resta)", 19)
u21_017 = get_value(unidad2_VIDRIOS, "Liberación Reserva Matemática BEPs (resta)", 19)
u22_020 = get_value(unidad2_VIDRIOS, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 19)
u23_022 = get_value(unidad2_VIDRIOS, "Liberación Reserva de Insuficiencia de Activos", 19)
u25_026 = get_value(unidad2_VIDRIOS, "Constitución Reserva Técnica de prima no devengada (resta)", 19)
u26_027 = get_value(unidad2_VIDRIOS, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 19)
u27_030 = get_value(unidad2_VIDRIOS, "Constitución Reserva matemática (resta)", 19)
u28_035 = get_value(unidad2_VIDRIOS, "Constitución Reserva matemática Riesgos Laborales", 19)
u29_037 = get_value(unidad2_VIDRIOS, "Constitución Reserva matemática BEPs", 19)
u30_040 = get_value(unidad2_VIDRIOS, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 19)
u31_045 = get_value(unidad2_VIDRIOS, "Constitución Reserva de Insuficiencia de Activos (resta)", 19)

# Calcular el total general
u32_999 = (u17_006 + u18_007 + u19_010 - u20_015 - u21_017 + u22_020 + u23_022 - u25_026 - u26_027 - u27_030 + u28_035 + u29_037 - u30_040 - u31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=21, value=u32_999)

<Cell 'Hoja1'.U32>

##  20  CRÉDITO COMERCIAL UNIDAD 2


In [64]:
# Filtrar y agrupar los datos
unidad2_CREDITOCOMERCIAL= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_CREDITOCOMERCIAL['TOTAL'] = unidad2_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 20, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 20, 18),
    ("Liberación Reserva Matemática", 20, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 20, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 20, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 20, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 20, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 20, 26),
    ("Constitución Reserva matemática (resta)", 20, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 20, 28),
    ("Constitución Reserva matemática BEPs", 20, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 20, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 20, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v17_006 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Técnica de prima no devengada", 20)
v18_007 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Técnica por insuficiencia de prima", 20)
v19_010 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Matemática", 20)
v20_015 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Matemática Riesgos Laborales (resta)", 20)
v21_017 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Matemática BEPs (resta)", 20)
v22_020 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 20)
v23_022 = get_value(unidad2_CREDITOCOMERCIAL, "Liberación Reserva de Insuficiencia de Activos", 20)
v25_026 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva Técnica de prima no devengada (resta)", 20)
v26_027 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 20)
v27_030 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva matemática (resta)", 20)
v28_035 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva matemática Riesgos Laborales", 20)
v29_037 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva matemática BEPs", 20)
v30_040 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 20)
v31_045 = get_value(unidad2_CREDITOCOMERCIAL, "Constitución Reserva de Insuficiencia de Activos (resta)", 20)

# Calcular el total general
v32_999 = (v17_006 + v18_007 + v19_010 - v20_015 - v21_017 + v22_020 + v23_022 - v25_026 - v26_027 - v27_030 + v28_035 + v29_037 - v30_040 - v31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=22, value=v32_999)

<Cell 'Hoja1'.V32>

##  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 2


In [65]:
# Filtrar y agrupar los datos
unidad2_CREDITOEXPORTACION= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_CREDITOEXPORTACION['TOTAL'] = unidad2_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 21, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 21, 18),
    ("Liberación Reserva Matemática", 21, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 21, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 21, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 21, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 21, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 21, 26),
    ("Constitución Reserva matemática (resta)", 21, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 21, 28),
    ("Constitución Reserva matemática BEPs", 21, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 21, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 21, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w17_006 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Técnica de prima no devengada", 21)
w18_007 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Técnica por insuficiencia de prima", 21)
w19_010 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Matemática", 21)
w20_015 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Matemática Riesgos Laborales (resta)", 21)
w21_017 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Matemática BEPs (resta)", 21)
w22_020 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 21)
w23_022 = get_value(unidad2_CREDITOEXPORTACION, "Liberación Reserva de Insuficiencia de Activos", 21)
w25_026 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva Técnica de prima no devengada (resta)", 21)
w26_027 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 21)
w27_030 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva matemática (resta)", 21)
w28_035 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva matemática Riesgos Laborales", 21)
w29_037 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva matemática BEPs", 21)
w30_040 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 21)
w31_045 = get_value(unidad2_CREDITOEXPORTACION, "Constitución Reserva de Insuficiencia de Activos (resta)", 21)

# Calcular el total general
w32_999 = (w17_006 + w18_007 + w19_010 - w20_015 - w21_017 + w22_020 + w23_022 - w25_026 - w26_027 - w27_030 + w28_035 + w29_037 - w30_040 - w31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=23, value=w32_999)

<Cell 'Hoja1'.W32>

##  24  DESEMPLEO UNIDAD 2


In [66]:
# Filtrar y agrupar los datos
unidad2_DESEMPLEO= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_DESEMPLEO['TOTAL'] = unidad2_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 24, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 24, 18),
    ("Liberación Reserva Matemática", 24, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 24, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 24, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 24, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 24, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 24, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 24, 26),
    ("Constitución Reserva matemática (resta)", 24, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 24, 28),
    ("Constitución Reserva matemática BEPs", 24, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 24, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 24, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z17_006 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Técnica de prima no devengada", 24)
z18_007 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Técnica por insuficiencia de prima", 24)
z19_010 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Matemática", 24)
z20_015 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 24)
z21_017 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Matemática BEPs (resta)", 24)
z22_020 = get_value(unidad2_DESEMPLEO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 24)
z23_022 = get_value(unidad2_DESEMPLEO, "Liberación Reserva de Insuficiencia de Activos", 24)
z25_026 = get_value(unidad2_DESEMPLEO, "Constitución Reserva Técnica de prima no devengada (resta)", 24)
z26_027 = get_value(unidad2_DESEMPLEO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 24)
z27_030 = get_value(unidad2_DESEMPLEO, "Constitución Reserva matemática (resta)", 24)
z28_035 = get_value(unidad2_DESEMPLEO, "Constitución Reserva matemática Riesgos Laborales", 24)
z29_037 = get_value(unidad2_DESEMPLEO, "Constitución Reserva matemática BEPs", 24)
z30_040 = get_value(unidad2_DESEMPLEO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 24)
z31_045 = get_value(unidad2_DESEMPLEO, "Constitución Reserva de Insuficiencia de Activos (resta)", 24)

# Calcular el total general
z32_999 = (z17_006 + z18_007 + z19_010 - z20_015 - z21_017 + z22_020 + z23_022 - z25_026 - z26_027 - z27_030 + z28_035 + z29_037 - z30_040 - z31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=26, value=z32_999)

<Cell 'Hoja1'.Z32>

##  25  HOGAR UNIDAD 2


In [67]:
# Filtrar y agrupar los datos
unidad2_HOGAR= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_HOGAR['TOTAL'] = unidad2_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 25, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 25, 18),
    ("Liberación Reserva Matemática", 25, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 25, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 25, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 25, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 25, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 25, 26),
    ("Constitución Reserva matemática (resta)", 25, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 25, 28),
    ("Constitución Reserva matemática BEPs", 25, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 25, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 25, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa17_006 = get_value(unidad2_HOGAR, "Liberación Reserva Técnica de prima no devengada", 25)
aa18_007 = get_value(unidad2_HOGAR, "Liberación Reserva Técnica por insuficiencia de prima", 25)
aa19_010 = get_value(unidad2_HOGAR, "Liberación Reserva Matemática", 25)
aa20_015 = get_value(unidad2_HOGAR, "Liberación Reserva Matemática Riesgos Laborales (resta)", 25)
aa21_017 = get_value(unidad2_HOGAR, "Liberación Reserva Matemática BEPs (resta)", 25)
aa22_020 = get_value(unidad2_HOGAR, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 25)
aa23_022 = get_value(unidad2_HOGAR, "Liberación Reserva de Insuficiencia de Activos", 25)
aa25_026 = get_value(unidad2_HOGAR, "Constitución Reserva Técnica de prima no devengada (resta)", 25)
aa26_027 = get_value(unidad2_HOGAR, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 25)
aa27_030 = get_value(unidad2_HOGAR, "Constitución Reserva matemática (resta)", 25)
aa28_035 = get_value(unidad2_HOGAR, "Constitución Reserva matemática Riesgos Laborales", 25)
aa29_037 = get_value(unidad2_HOGAR, "Constitución Reserva matemática BEPs", 25)
aa30_040 = get_value(unidad2_HOGAR, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 25)
aa31_045 = get_value(unidad2_HOGAR, "Constitución Reserva de Insuficiencia de Activos (resta)", 25)

# Calcular el total general
aa32_999 = (aa17_006 + aa18_007 + aa19_010 - aa20_015 - aa21_017 + aa22_020 + aa23_022 - aa25_026 - aa26_027 - aa27_030 + aa28_035 + aa29_037 - aa30_040 - aa31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=27, value=aa32_999)

<Cell 'Hoja1'.AA32>

## 42 AGROPECUARIO UNIDAD 2


In [68]:
# Filtrar y agrupar los datos
unidad2_AGROPECUARIO= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_AGROPECUARIO['TOTAL'] = unidad2_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 42, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 42, 18),
    ("Liberación Reserva Matemática", 42, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 42, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 42, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 42, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 42, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 42, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 42, 26),
    ("Constitución Reserva matemática (resta)", 42, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 42, 28),
    ("Constitución Reserva matemática BEPs", 42, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 42, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 42, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar17_006 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Técnica de prima no devengada", 42)
ar18_007 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Técnica por insuficiencia de prima", 42)
ar19_010 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Matemática", 42)
ar20_015 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Matemática Riesgos Laborales (resta)", 42)
ar21_017 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Matemática BEPs (resta)", 42)
ar22_020 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 42)
ar23_022 = get_value(unidad2_AGROPECUARIO, "Liberación Reserva de Insuficiencia de Activos", 42)
ar25_026 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva Técnica de prima no devengada (resta)", 42)
ar26_027 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 42)
ar27_030 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva matemática (resta)", 42)
ar28_035 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva matemática Riesgos Laborales", 42)
ar29_037 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva matemática BEPs", 42)
ar30_040 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 42)
ar31_045 = get_value(unidad2_AGROPECUARIO, "Constitución Reserva de Insuficiencia de Activos (resta)", 42)

# Calcular el total general
ar32_999 = (ar17_006 + ar18_007 + ar19_010 - ar20_015 - ar21_017 + ar22_020 + ar23_022 - ar25_026 - ar26_027 - ar27_030 + ar28_035 + ar29_037 - ar30_040 - ar31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=44, value=ar32_999)

<Cell 'Hoja1'.AR32>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 2


In [69]:
# Filtrar y agrupar los datos
unidad2_OPERNOASIGNABLES= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_OPERNOASIGNABLES['TOTAL'] = unidad2_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 43, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 43, 18),
    ("Liberación Reserva Matemática", 43, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 43, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 43, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 43, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 43, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 43, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 43, 26),
    ("Constitución Reserva matemática (resta)", 43, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 43, 28),
    ("Constitución Reserva matemática BEPs", 43, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 43, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 43, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as17_006 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Técnica de prima no devengada", 43)
as18_007 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Técnica por insuficiencia de prima", 43)
as19_010 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática", 43)
as20_015 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática Riesgos Laborales (resta)", 43)
as21_017 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática BEPs (resta)", 43)
as22_020 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 43)
as23_022 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva de Insuficiencia de Activos", 43)
as25_026 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Técnica de prima no devengada (resta)", 43)
as26_027 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 43)
as27_030 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática (resta)", 43)
as28_035 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática Riesgos Laborales", 43)
as29_037 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática BEPs", 43)
as30_040 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 43)
as31_045 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva de Insuficiencia de Activos (resta)", 43)

# Calcular el total general
as32_999 = (as17_006 + as18_007 + as19_010 - as20_015 - as21_017 + as22_020 + as23_022 - as25_026 - as26_027 - as27_030 + as28_035 + as29_037 - as30_040 - as31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=45, value=as32_999)

<Cell 'Hoja1'.AS32>

## 44 DECENAL UNIDAD 2


In [70]:
# Filtrar y agrupar los datos
unidad2_DECENAL= bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_DECENAL['TOTAL'] = unidad2_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 44, 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 44, 18),
    ("Liberación Reserva Matemática", 44, 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 44, 20),
    ("Liberación Reserva Matemática BEPs (resta)", 44, 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 44, 22),
    ("Liberación Reserva de Insuficiencia de Activos", 44, 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 44, 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 44, 26),
    ("Constitución Reserva matemática (resta)", 44, 27),
    ("Constitución Reserva matemática Riesgos Laborales", 44, 28),
    ("Constitución Reserva matemática BEPs", 44, 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 44, 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 44, 31)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad2_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at17_006 = get_value(unidad2_DECENAL, "Liberación Reserva Técnica de prima no devengada", 44)
at18_007 = get_value(unidad2_DECENAL, "Liberación Reserva Técnica por insuficiencia de prima", 44)
at19_010 = get_value(unidad2_DECENAL, "Liberación Reserva Matemática", 44)
at20_015 = get_value(unidad2_DECENAL, "Liberación Reserva Matemática Riesgos Laborales (resta)", 44)
at21_017 = get_value(unidad2_DECENAL, "Liberación Reserva Matemática BEPs (resta)", 44)
at22_020 = get_value(unidad2_DECENAL, "Liberación Reserva Seguro de Vida Ahorro con Participa.", 44)
at23_022 = get_value(unidad2_DECENAL, "Liberación Reserva de Insuficiencia de Activos", 44)
at25_026 = get_value(unidad2_DECENAL, "Constitución Reserva Técnica de prima no devengada (resta)", 44)
at26_027 = get_value(unidad2_DECENAL, "Constitución Reserva Técnica por insuficiencia de prima (resta)", 44)
at27_030 = get_value(unidad2_DECENAL, "Constitución Reserva matemática (resta)", 44)
at28_035 = get_value(unidad2_DECENAL, "Constitución Reserva matemática Riesgos Laborales", 44)
at29_037 = get_value(unidad2_DECENAL, "Constitución Reserva matemática BEPs", 44)
at30_040 = get_value(unidad2_DECENAL, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 44)
at31_045 = get_value(unidad2_DECENAL, "Constitución Reserva de Insuficiencia de Activos (resta)", 44)

# Calcular el total general
at32_999 = (at17_006 + at18_007 + at19_010 - at20_015 - at21_017 + at22_020 + at23_022 - at25_026 - at26_027 - at27_030 + at28_035 + at29_037 - at30_040 - at31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=46, value=at32_999)

<Cell 'Hoja1'.AT32>

## 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 2


In [71]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

au17_006 = unidad2_CUMPLIMIENTOESTATALES_a[(unidad2_CUMPLIMIENTOESTATALES_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au17_006, 47, 17, 'TOTAL')

au18_007 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au18_007, 47, 18)

au19_010 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au19_010, 47, 19)

au20_015 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au20_015, 47, 20)

au21_017 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au21_017, 47, 21)

au22_020 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au22_020, 47, 22)

au23_022 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au23_022, 47, 23)

au25_026 = unidad2_CUMPLIMIENTOESTATALES_a[(unidad2_CUMPLIMIENTOESTATALES_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au25_026, 47, 25, 'TOTAL')

au26_027 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au26_027, 47, 26)

au27_030 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au27_030, 47, 27)

au28_035 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au28_035, 47, 28)

au29_037 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au29_037, 47, 29)

au30_040 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au30_040, 47, 30)

au31_045 = unidad2_CUMPLIMIENTOESTATALES[(unidad2_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au31_045, 47, 31)

# Calcular ak32_999
au32_999 = au17_006['TOTAL'].sum() + au18_007['TOTAL'].sum() + au19_010['TOTAL'].sum() + au20_015['TOTAL'].sum() + au21_017['TOTAL'].sum() + \
           au22_020['TOTAL'].sum() + au23_022['TOTAL'].sum() + au25_026['TOTAL'].sum() + au26_027['TOTAL'].sum() + au27_030['TOTAL'].sum() + \
           au28_035['TOTAL'].sum() + au29_037['TOTAL'].sum() + au30_040['TOTAL'].sum() + au31_045['TOTAL'].sum()

au32_999 = abs(au32_999)
# Crear DataFrame para escribir en Excel
df_au32_999 = pd.DataFrame({'total': [au32_999]})
write_to_excel(df_au32_999, 47, 32, 'total')


## 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 2


In [72]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

av17_006 = unidad2_CUMPLIMIENTOEMPRESASSERV_a[(unidad2_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av17_006, 48, 17, 'TOTAL')

av18_007 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av18_007, 48, 18)

av19_010 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av19_010, 48, 19)

av20_015 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av20_015, 48, 20)

av21_017 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av21_017, 48, 21)

av22_020 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av22_020, 48, 22)

av23_022 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av23_022, 48, 23)

av25_026 = unidad2_CUMPLIMIENTOEMPRESASSERV_a[(unidad2_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av25_026, 48, 25, 'TOTAL')

av26_027 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av26_027, 48, 26)

av27_030 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av27_030, 48, 27)

av28_035 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av28_035, 48, 28)

av29_037 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av29_037, 48, 29)

av30_040 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av30_040, 48, 30)

av31_045 = unidad2_CUMPLIMIENTOEMPRESASSERV[(unidad2_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av31_045, 48, 31)

# Calcular ak32_999
av32_999 = av17_006['TOTAL'].sum() + av18_007['TOTAL'].sum() + av19_010['TOTAL'].sum() + av20_015['TOTAL'].sum() + av21_017['TOTAL'].sum() + \
           av22_020['TOTAL'].sum() + av23_022['TOTAL'].sum() + av25_026['TOTAL'].sum() + av26_027['TOTAL'].sum() + av27_030['TOTAL'].sum() + \
           av28_035['TOTAL'].sum() + av29_037['TOTAL'].sum() + av30_040['TOTAL'].sum() + av31_045['TOTAL'].sum()

av32_999 = abs(av32_999)
# Crear DataFrame para escribir en Excel
df_av32_999 = pd.DataFrame({'total': [av32_999]})
write_to_excel(df_av32_999, 48, 32, 'total')

## 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 2


In [73]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOEMPRESASINDUSTRI= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

aw17_006 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a['COD_RAMO'] == 47)]
write_to_excel(aw17_006, 49, 17, 'TOTAL')

aw18_007 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw18_007, 49, 18)

aw19_010 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw19_010, 49, 19)

aw20_015 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw20_015, 49, 20)

aw21_017 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw21_017, 49, 21)

aw22_020 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw22_020, 49, 22)

aw23_022 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw23_022, 49, 23)

aw25_026 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI_a['COD_RAMO'] == 47)]
write_to_excel(aw25_026, 49, 25, 'TOTAL')

aw26_027 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw26_027, 49, 26)

aw27_030 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw27_030, 49, 27)

aw28_035 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw28_035, 49, 28)

aw29_037 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw29_037, 49, 29)

aw30_040 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw30_040, 49, 30)

aw31_045 = unidad2_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw31_045, 49, 31)

# Calcular ak32_999
aw32_999 = aw17_006['TOTAL'].sum() + aw18_007['TOTAL'].sum() + aw19_010['TOTAL'].sum() + aw20_015['TOTAL'].sum() + aw21_017['TOTAL'].sum() + \
           aw22_020['TOTAL'].sum() + aw23_022['TOTAL'].sum() + aw25_026['TOTAL'].sum() + aw26_027['TOTAL'].sum() + aw27_030['TOTAL'].sum() + \
           aw28_035['TOTAL'].sum() + aw29_037['TOTAL'].sum() + aw30_040['TOTAL'].sum() + aw31_045['TOTAL'].sum()

aw32_999 = abs(aw32_999)
# Crear DataFrame para escribir en Excel
df_aw32_999 = pd.DataFrame({'total': [aw32_999]})
write_to_excel(df_aw32_999, 49, 32, 'total')

## 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 2


In [74]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTODISPOLEG= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTODISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

ax17_006 = unidad2_CUMPLIMIENTODISPOLEG_a[(unidad2_CUMPLIMIENTODISPOLEG_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTODISPOLEG_a['COD_RAMO'] == 48)]
write_to_excel(ax17_006, 50, 17, 'TOTAL')

ax18_007 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax18_007, 50, 18)

ax19_010 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax19_010, 50, 19)

ax20_015 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax20_015, 50, 20)

ax21_017 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax21_017, 50, 21)

ax22_020 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax22_020, 50, 22)

ax23_022 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax23_022, 50, 23)

ax25_026 = unidad2_CUMPLIMIENTODISPOLEG_a[(unidad2_CUMPLIMIENTODISPOLEG_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTODISPOLEG_a['COD_RAMO'] == 48)]
write_to_excel(ax25_026, 50, 25, 'TOTAL')

ax26_027 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax26_027, 50, 26)

ax27_030 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax27_030, 50, 27)

ax28_035 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax28_035, 50, 28)

ax29_037 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax29_037, 50, 29)

ax30_040 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax30_040, 50, 30)

ax31_045 = unidad2_CUMPLIMIENTODISPOLEG[(unidad2_CUMPLIMIENTODISPOLEG['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTODISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax31_045, 50, 31)

# Calcular ak32_999
ax32_999 = ax17_006['TOTAL'].sum() + ax18_007['TOTAL'].sum() + ax19_010['TOTAL'].sum() + ax20_015['TOTAL'].sum() + ax21_017['TOTAL'].sum() + \
           ax22_020['TOTAL'].sum() + ax23_022['TOTAL'].sum() + ax25_026['TOTAL'].sum() + ax26_027['TOTAL'].sum() + ax27_030['TOTAL'].sum() + \
           ax28_035['TOTAL'].sum() + ax29_037['TOTAL'].sum() + ax30_040['TOTAL'].sum() + ax31_045['TOTAL'].sum()

ax32_999 = abs(ax32_999)
# Crear DataFrame para escribir en Excel
df_ax32_999 = pd.DataFrame({'total': [ax32_999]})
write_to_excel(df_ax32_999, 50, 32, 'total')

## 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 2


In [75]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

ay17_006 = unidad2_CUMPLIMIENTOCAUCIONES_a[(unidad2_CUMPLIMIENTOCAUCIONES_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOCAUCIONES_a['COD_RAMO'] == 49)]
write_to_excel(ay17_006, 51, 17, 'TOTAL')

ay18_007 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay18_007, 51, 18)

ay19_010 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay19_010, 51, 19)

ay20_015 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay20_015, 51, 20)

ay21_017 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay21_017, 51, 21)

ay22_020 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay22_020, 51, 22)

ay23_022 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay23_022, 51, 23)

ay25_026 = unidad2_CUMPLIMIENTOCAUCIONES_a[(unidad2_CUMPLIMIENTOCAUCIONES_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOCAUCIONES_a['COD_RAMO'] == 49)]
write_to_excel(ay25_026, 51, 25, 'TOTAL')

ay26_027 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay26_027, 51, 26)

ay27_030 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay27_030, 51, 27)

ay28_035 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay28_035, 51, 28)

ay29_037 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay29_037, 51, 29)

ay30_040 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay30_040, 51, 30)

ay31_045 = unidad2_CUMPLIMIENTOCAUCIONES[(unidad2_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay31_045, 51, 31)

# Calcular ay32_999
ay32_999 = ay17_006['TOTAL'].sum() + ay18_007['TOTAL'].sum() + ay19_010['TOTAL'].sum() + ay20_015['TOTAL'].sum() + ay21_017['TOTAL'].sum() + \
           ay22_020['TOTAL'].sum() + ay23_022['TOTAL'].sum() + ay25_026['TOTAL'].sum() + ay26_027['TOTAL'].sum() + ay27_030['TOTAL'].sum() + \
           ay28_035['TOTAL'].sum() + ay29_037['TOTAL'].sum() + ay30_040['TOTAL'].sum() + ay31_045['TOTAL'].sum()

ay32_999 = abs(ay32_999)
# Crear DataFrame para escribir en Excel
df_ay32_999 = pd.DataFrame({'total': [ay32_999]})
write_to_excel(df_ay32_999, 51, 32, 'total')


## 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 2


In [76]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

az17_006 = unidad2_CUMPLIMIENTOARRENDAMIENTO_a[(unidad2_CUMPLIMIENTOARRENDAMIENTO_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOARRENDAMIENTO_a['COD_RAMO'] == 50)]
write_to_excel(az17_006, 51, 17, 'TOTAL')

az18_007 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az18_007, 51, 18)

az19_010 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az19_010, 51, 19)

az20_015 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az20_015, 51, 20)

az21_017 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az21_017, 51, 21)

az22_020 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az22_020, 51, 22)

az23_022 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az23_022, 51, 23)

az25_026 = unidad2_CUMPLIMIENTOARRENDAMIENTO_a[(unidad2_CUMPLIMIENTOARRENDAMIENTO_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOARRENDAMIENTO_a['COD_RAMO'] == 50)]
write_to_excel(az25_026, 51, 25, 'TOTAL')

az26_027 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az26_027, 51, 26)

az27_030 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az27_030, 51, 27)

az28_035 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az28_035, 51, 28)

az29_037 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az29_037, 51, 29)

az30_040 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az30_040, 51, 30)

az31_045 = unidad2_CUMPLIMIENTOARRENDAMIENTO[(unidad2_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az31_045, 51, 31)

# Calcular az32_999
az32_999 = az17_006['TOTAL'].sum() + az18_007['TOTAL'].sum() + az19_010['TOTAL'].sum() + az20_015['TOTAL'].sum() + az21_017['TOTAL'].sum() + \
           az22_020['TOTAL'].sum() + az23_022['TOTAL'].sum() + az25_026['TOTAL'].sum() + az26_027['TOTAL'].sum() + az27_030['TOTAL'].sum() + \
           az28_035['TOTAL'].sum() + az29_037['TOTAL'].sum() + az30_040['TOTAL'].sum() + az31_045['TOTAL'].sum()

az32_999 = abs(az32_999)
# Crear DataFrame para escribir en Excel
df_az32_999 = pd.DataFrame({'total': [az32_999]})
write_to_excel(df_az32_999, 51, 32, 'total')


## 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 2

In [77]:
# Agrupar los datos según el concepto y el ramo
unidad2_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R

ba17_006 = unidad2_CUMPLIMIENTOPARTICULARES_a[(unidad2_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "5121001011") & (unidad2_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba17_006, 53, 17, 'TOTAL')

ba18_007 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba18_007, 53, 18)

ba19_010 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba19_010, 53, 19)

ba20_015 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba20_015, 53, 20)

ba21_017 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba21_017, 53, 21)

ba22_020 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba22_020, 53, 22)

ba23_022 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba23_022, 53, 23)

ba25_026 = unidad2_CUMPLIMIENTOPARTICULARES_a[(unidad2_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "5121001051") & (unidad2_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba25_026, 53, 25, 'TOTAL')

ba26_027 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba26_027, 53, 26)

ba27_030 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba27_030, 53, 27)

ba28_035 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba28_035, 53, 28)

ba29_037 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba29_037, 53, 29)

ba30_040 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba30_040, 53, 30)

ba31_045 = unidad2_CUMPLIMIENTOPARTICULARES[(unidad2_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba31_045, 53, 31)

# Calcular ba32_999
ba32_999 = ba17_006['TOTAL'].sum() + ba18_007['TOTAL'].sum() + ba19_010['TOTAL'].sum() + ba20_015['TOTAL'].sum() + ba21_017['TOTAL'].sum() + \
           ba22_020['TOTAL'].sum() + ba23_022['TOTAL'].sum() + ba25_026['TOTAL'].sum() + ba26_027['TOTAL'].sum() + ba27_030['TOTAL'].sum() + \
           ba28_035['TOTAL'].sum() + ba29_037['TOTAL'].sum() + ba30_040['TOTAL'].sum() + ba31_045['TOTAL'].sum()

ba32_999 = abs(ba32_999)
# Crear DataFrame para escribir en Excel
df_ba32_999 = pd.DataFrame({'total': [ba32_999]})
write_to_excel(df_ba32_999, 53, 32, 'total')


##  CONSULTA DE TOTALES RESERVAS TECNICAS Y MATEMATICAS UNIDAD 2

In [78]:
# Sumar los valores de E17 a AT17, considerando celdas vacías como 0

c17_006 = sum([ws.cell(row=17, column=col).value if ws.cell(row=17, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=17, column=3).value = c17_006  # Escribir en fila 17, columna 3

c18_007 = sum([ws.cell(row=18, column=col).value if ws.cell(row=18, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=18, column=3).value = c18_007  # Escribir en fila 18, columna 3

c19_010 = sum([ws.cell(row=19, column=col).value if ws.cell(row=19, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=19, column=3).value = c19_010  # Escribir en fila 19, columna 3

c20_015 = sum([ws.cell(row=20, column=col).value if ws.cell(row=20, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=20, column=3).value = c20_015  # Escribir en fila 20, columna 3

c21_017 = sum([ws.cell(row=21, column=col).value if ws.cell(row=21, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=21, column=3).value = c21_017  # Escribir en fila 21, columna 3

c22_020 = sum([ws.cell(row=22, column=col).value if ws.cell(row=22, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=22, column=3).value = c22_020  # Escribir en fila 22, columna 3

c23_022 = sum([ws.cell(row=23, column=col).value if ws.cell(row=23, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=23, column=3).value = c23_022  # Escribir en fila 23, columna 3

c25_026 = sum([ws.cell(row=25, column=col).value if ws.cell(row=25, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=25, column=3).value = c25_026  # Escribir en fila 25, columna 3

c26_027 = sum([ws.cell(row=26, column=col).value if ws.cell(row=26, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=26, column=3).value = c26_027  # Escribir en fila 26, columna 3

c27_030 = sum([ws.cell(row=27, column=col).value if ws.cell(row=27, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=27, column=3).value = c27_030  # Escribir en fila 27, columna 3

c28_035 = sum([ws.cell(row=28, column=col).value if ws.cell(row=28, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=28, column=3).value = c28_035  # Escribir en fila 28, columna 3

c29_037 = sum([ws.cell(row=29, column=col).value if ws.cell(row=29, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=29, column=3).value = c29_037  # Escribir en fila 29, columna 3

c30_040 = sum([ws.cell(row=30, column=col).value if ws.cell(row=30, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=30, column=3).value = c30_040  # Escribir en fila 30, columna 3

c31_045 = sum([ws.cell(row=31, column=col).value if ws.cell(row=31, column=col).value is not None else 0 for col in range(5, 46)])

# Calcular el total general según la fórmula dada
c32_999 = (c17_006 + c18_007 + c19_010 - c20_015 - c21_017 + c22_020 + c23_022 - c25_026 - c26_027 - c27_030 + c28_035 + c29_037 - c30_040 - c31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=3).value = c32_999

## CONSULTA SUBTOTAL - RAMOS RESERVAS TECNICAS Y MATEMATICAS UNIDAD 2  

In [79]:
# Sumar los valores de E17 a AT17, considerando celdas vacías como 0

d17_006 = sum([ws.cell(row=17, column=col).value if ws.cell(row=17, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=17, column=4).value = d17_006

d18_007 = sum([ws.cell(row=18, column=col).value if ws.cell(row=18, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=18, column=4).value = d18_007

d19_010 = sum([ws.cell(row=19, column=col).value if ws.cell(row=19, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=19, column=4).value = d19_010

d20_015 = sum([ws.cell(row=20, column=col).value if ws.cell(row=20, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=20, column=4).value = d20_015

d21_017 = sum([ws.cell(row=21, column=col).value if ws.cell(row=21, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=21, column=4).value = d21_017

d22_020 = sum([ws.cell(row=22, column=col).value if ws.cell(row=22, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=22, column=4).value = d22_020

d23_022 = sum([ws.cell(row=23, column=col).value if ws.cell(row=23, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=23, column=4).value = d23_022

d25_026 = sum([ws.cell(row=25, column=col).value if ws.cell(row=25, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=25, column=4).value = d25_026

d26_027 = sum([ws.cell(row=26, column=col).value if ws.cell(row=26, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=26, column=4).value = d26_027

d27_030 = sum([ws.cell(row=27, column=col).value if ws.cell(row=27, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=27, column=4).value = d27_030

d28_035 = sum([ws.cell(row=28, column=col).value if ws.cell(row=28, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=28, column=4).value = d28_035

d29_037 = sum([ws.cell(row=29, column=col).value if ws.cell(row=29, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=29, column=4).value = d29_037

d30_040 = sum([ws.cell(row=30, column=col).value if ws.cell(row=30, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=30, column=4).value = d30_040

d31_045 = sum([ws.cell(row=31, column=col).value if ws.cell(row=31, column=col).value is not None else 0 for col in range(5, 46)])


# Calcular el total general según la fórmula dada
d32_999 = (d17_006 + d18_007 + d19_010 - d20_015 - d21_017 + d22_020 + d23_022 - d25_026 - d26_027 - d27_030 + d28_035 + d29_037 - d30_040 - d31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=4).value = d32_999

#   UNIDAD DE CAPTURA 3  

## CONSULTA DE TOTALES PRIMAS DEVENGADAS  UNIDAD 3

In [80]:
# Sumar c14_999 y c32_999, y escribir en la celda correspondiente

c34_999 = c14_999 + c32_999
df_c34_999 = pd.DataFrame({'total': [c34_999]})
if df_c34_999.empty or c34_999 == 0:
    df_c34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=3).value = df_c34_999['total'].iloc[0]

# Repetir el mismo proceso para las otras variables
d34_999 = d32_999 + d32_999
df_d34_999 = pd.DataFrame({'total': [d34_999]})
if df_d34_999.empty or d34_999 == 0:
    df_d34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=4).value = df_d34_999['total'].iloc[0]

e34_999 = e14_999 + e32_999
df_e34_999 = pd.DataFrame({'total': [e34_999]})
if df_e34_999.empty or e34_999 == 0:
    df_e34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=5).value = df_e34_999['total'].iloc[0]

f34_999 = f14_999 + f32_999
df_f34_999 = pd.DataFrame({'total': [f34_999]})
if df_f34_999.empty or f34_999 == 0:
    df_f34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=6).value = df_f34_999['total'].iloc[0]

g34_999 = g14_999 + g32_999
df_g34_999 = pd.DataFrame({'total': [g34_999]})
if df_g34_999.empty or g34_999 == 0:
    df_g34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=7).value = df_g34_999['total'].iloc[0]

h34_999 = h14_999 + h32_999
df_h34_999 = pd.DataFrame({'total': [h34_999]})
if df_h34_999.empty or h34_999 == 0:
    df_h34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=8).value = df_h34_999['total'].iloc[0]

i34_999 = i14_999 + i32_999
df_i34_999 = pd.DataFrame({'total': [i34_999]})
if df_i34_999.empty or i34_999 == 0:
    df_i34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=9).value = df_i34_999['total'].iloc[0]

j34_999 = j14_999 + j32_999
df_j34_999 = pd.DataFrame({'total': [j34_999]})
if df_j34_999.empty or j34_999 == 0:
    df_j34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=10).value = df_j34_999['total'].iloc[0]

k34_999 = k14_999 + k32_999
df_k34_999 = pd.DataFrame({'total': [k34_999]})
if df_k34_999.empty or k34_999 == 0:
    df_k34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=11).value = df_k34_999['total'].iloc[0]

l34_999 = l14_999 + l32_999
df_l34_999 = pd.DataFrame({'total': [l34_999]})
if df_l34_999.empty or l34_999 == 0:
    df_l34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=12).value = df_l34_999['total'].iloc[0]

m34_999 = m14_999 + m32_999
df_m34_999 = pd.DataFrame({'total': [m34_999]})
if df_m34_999.empty or m34_999 == 0:
    df_m34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=13).value = df_m34_999['total'].iloc[0]

n34_999 = n14_999 + n32_999
df_n34_999 = pd.DataFrame({'total': [n34_999]})
if df_n34_999.empty or n34_999 == 0:
    df_n34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=14).value = df_n34_999['total'].iloc[0]

o34_999 = o14_999 + o32_999
df_o34_999 = pd.DataFrame({'total': [o34_999]})
if df_o34_999.empty or o34_999 == 0:
    df_o34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=15).value = df_o34_999['total'].iloc[0]

p34_999 = p14_999 + p32_999
df_p34_999 = pd.DataFrame({'total': [p34_999]})
if df_p34_999.empty or p34_999 == 0:
    df_p34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=16).value = df_p34_999['total'].iloc[0]

q34_999 = q14_999 + q32_999
df_q34_999 = pd.DataFrame({'total': [q34_999]})
if df_q34_999.empty or q34_999 == 0:
    df_q34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=17).value = df_q34_999['total'].iloc[0]

r34_999 = r14_999 + r32_999
df_r34_999 = pd.DataFrame({'total': [r34_999]})
if df_r34_999.empty or r34_999 == 0:
    df_r34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=18).value = df_r34_999['total'].iloc[0]

s34_999 = s14_999 + s32_999
df_s34_999 = pd.DataFrame({'total': [s34_999]})
if df_s34_999.empty or s34_999 == 0:
    df_s34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=19).value = df_s34_999['total'].iloc[0]

t34_999 = t14_999 + t32_999
df_t34_999 = pd.DataFrame({'total': [t34_999]})
if df_t34_999.empty or t34_999 == 0:
    df_t34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=20).value = df_t34_999['total'].iloc[0]

u34_999 = u14_999 + u32_999
df_u34_999 = pd.DataFrame({'total': [u34_999]})
if df_u34_999.empty or u34_999 == 0:
    df_u34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=21).value = df_u34_999['total'].iloc[0]

v34_999 = v14_999 + v32_999
df_v34_999 = pd.DataFrame({'total': [v34_999]})
if df_v34_999.empty or v34_999 == 0:
    df_v34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=22).value = df_v34_999['total'].iloc[0]

w34_999 = w14_999 + w32_999
df_w34_999 = pd.DataFrame({'total': [w34_999]})
if df_w34_999.empty or w34_999 == 0:
    df_w34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=23).value = df_w34_999['total'].iloc[0]

z34_999 = z14_999 + z32_999
df_z34_999 = pd.DataFrame({'total': [z34_999]})
if df_z34_999.empty or z34_999 == 0:
    df_z34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=26).value = df_z34_999['total'].iloc[0]

aa34_999 = aa14_999 + aa32_999
df_aa34_999 = pd.DataFrame({'total': [aa34_999]})
if df_aa34_999.empty or aa34_999 == 0:
    df_aa34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=27).value = df_aa34_999['total'].iloc[0]

ar34_999 = ar14_999 + ar32_999
df_ar34_999 = pd.DataFrame({'total': [ar34_999]})
if df_ar34_999.empty or ar34_999 == 0:
    df_ar34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=44).value = df_ar34_999['total'].iloc[0]

as34_999 = as14_999 + as32_999
df_as34_999 = pd.DataFrame({'total': [as34_999]})
if df_as34_999.empty or as34_999 == 0:
    df_as34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=45).value = df_as34_999['total'].iloc[0]

at34_999 = at14_999 + at32_999
df_at34_999 = pd.DataFrame({'total': [at34_999]})
if df_at34_999.empty or at34_999 == 0:
    df_at34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=46).value = df_at34_999['total'].iloc[0]

au34_999 = au14_999 + au32_999
df_au34_999 = pd.DataFrame({'total': [au34_999]})
if df_au34_999.empty or au34_999 == 0:
    df_au34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=47).value = df_au34_999['total'].iloc[0]

av34_999 = av14_999 + av32_999
df_av34_999 = pd.DataFrame({'total': [av34_999]})
if df_av34_999.empty or av34_999 == 0:
    df_av34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=48).value = df_av34_999['total'].iloc[0]

aw34_999 = aw14_999 + aw32_999
df_aw34_999 = pd.DataFrame({'total': [aw34_999]})
if df_aw34_999.empty or aw34_999 == 0:
    df_aw34_999= pd.DataFrame({'total': [0]})
ws.cell(row=34, column=49).value = df_aw34_999['total'].iloc[0]

ax34_999 = ax14_999 + ax32_999
df_ax34_999 = pd.DataFrame({'total': [ax34_999]})
if df_ax34_999.empty or ax34_999 == 0:
    df_ax34_999= pd.DataFrame({'total': [0]})
ws.cell(row=34, column=50).value = df_ax34_999['total'].iloc[0]

ay34_999 = ay14_999 + ay32_999
df_ay34_999 = pd.DataFrame({'total': [ay34_999]})
if df_ay34_999.empty or ay34_999 == 0:
    df_ay34_999= pd.DataFrame({'total': [0]})
ws.cell(row=34, column=51).value = df_ay34_999['total'].iloc[0]

az34_999 = az14_999 + az32_999
df_az34_999 = pd.DataFrame({'total': [az34_999]})
if df_az34_999.empty or az34_999 == 0:
    df_az34_999= pd.DataFrame({'total': [0]})
ws.cell(row=34, column=52).value = df_az34_999['total'].iloc[0]

ba34_999 = ba14_999 + ba32_999
df_ba34_999 = pd.DataFrame({'total': [ba34_999]})
if df_ba34_999.empty or ba34_999 == 0:
    df_ba34_999= pd.DataFrame({'total': [0]})
ws.cell(row=34, column=53).value = df_ba34_999 ['total'].iloc[0]


## CONSULTA DE SUBTOTAL - RAMOS PRIMAS DEVENGADAS UNIDAD 3

In [81]:
# Extrae el valor del diccionario d14_999 (si no existe 'total', usa 0)
d14_val = d14_999.get('total', 0)

# Suma con el valor numérico d32_999
d34_999 = d14_val + d32_999

# Crear DataFrame con el resultado
df_d34_999 = pd.DataFrame({'total': [d34_999]})

# Si el resultado es vacío o igual a 0, fuerza un DataFrame con 0
if df_d34_999.empty or d34_999 == 0:
    df_d34_999 = pd.DataFrame({'total': [0]})

# Escribir el valor en la celda correspondiente del Excel
ws.cell(row=34, column=4).value = df_d34_999['total'].iloc[0]

# UNIDAD DE CAPTURA 4 

##    3   AUTOMOVILES UNIDAD 4 

In [82]:
# Filtrar y agrupar los datos donde 'Unidad' es 4 y COD_RAMO es 3
unidad4_AUTOMOVILES = bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_AUTOMOVILES['TOTAL'] = unidad4_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 3, 36),
    ("Liberación Reserva Matemática BEPs", 3, 37),
    ("Liberación Reserva para Siniestros Avisados", 3, 38),
    ("Liberación Reserva para Siniestros No Avisados", 3, 39),
    ("Liberación Reservas desviación de siniestralidad", 3, 40),
    ("Liberación Reservas especiales", 3, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 3, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 3, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 3, 44),
    ("Constitución Reserva matemática BEPs (resta)", 3, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 3, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 3, 47),
    ("Constitución Reservas especiales (resta)", 3, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 3, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e36_005 = get_value(unidad4_AUTOMOVILES, "Liberación Reserva Matemática Riesgos Laborales", 3)
e37_007 = get_value(unidad4_AUTOMOVILES, "Liberación Reserva Matemática BEPs", 3)
e38_010 = get_value(unidad4_AUTOMOVILES, "Liberación Reserva para Siniestros Avisados", 3)
e39_015 = get_value(unidad4_AUTOMOVILES, "Liberación Reserva para Siniestros No Avisados", 3)
e40_020 = get_value(unidad4_AUTOMOVILES, "Liberación Reservas desviación de siniestralidad", 3)
e41_025 = get_value(unidad4_AUTOMOVILES, "Liberación Reservas especiales", 3)
e42_026 = get_value(unidad4_AUTOMOVILES, "Liberación Reserva de Riesgos Catastróficos", 3)
e43_030 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva matemática Riesgos Laborales (resta)", 3)
e44_035 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva Siniestros Avisados  (resta)", 3)
e45_037 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva matemática BEPs (resta)", 3)
e46_040 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva Siniestros No Avisados  (resta)", 3)
e47_045 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva desviación siniestralidad (resta)", 3)
e48_050 = get_value(unidad4_AUTOMOVILES, "Constitución Reservas especiales (resta)", 3)
e49_051 = get_value(unidad4_AUTOMOVILES, "Constitución Reserva de Riesgos Catastróficos (resta)",3)


e50_999 = (e36_005 + e37_007 + e38_010 + e39_015 + e40_020 + e41_025 + e42_026 -
            e43_030 - e44_035 - e45_037 - e46_040 - e47_045 - e48_050 - e49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=5, value=e50_999)


<Cell 'Hoja1'.E50>

##     4   SOAT UNIDAD 4 

In [83]:
# Filtrar y agrupar los datos donde 'Unidad' es 4 y COD_RAMO es 4
unidad4_SOAT = bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_SOAT['TOTAL'] = unidad4_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 4, 36),
    ("Liberación Reserva Matemática BEPs", 4, 37),
    ("Liberación Reserva para Siniestros Avisados", 4, 38),
    ("Liberación Reserva para Siniestros No Avisados", 4, 39),
    ("Liberación Reservas desviación de siniestralidad", 4, 40),
    ("Liberación Reservas especiales", 4, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 4, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 4, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 4, 44),
    ("Constitución Reserva matemática BEPs (resta)", 4, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 4, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 4, 47),
    ("Constitución Reservas especiales (resta)", 4, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 4, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f36_005 = get_value(unidad4_SOAT, "Liberación Reserva Matemática Riesgos Laborales", 4)
f37_007 = get_value(unidad4_SOAT, "Liberación Reserva Matemática BEPs", 4)
f38_010 = get_value(unidad4_SOAT, "Liberación Reserva para Siniestros Avisados", 4)
f39_015 = get_value(unidad4_SOAT, "Liberación Reserva para Siniestros No Avisados", 4)
f40_020 = get_value(unidad4_SOAT, "Liberación Reservas desviación de siniestralidad", 4)
f41_025 = get_value(unidad4_SOAT, "Liberación Reservas especiales", 4)
f42_026 = get_value(unidad4_SOAT, "Liberación Reserva de Riesgos Catastróficos", 4)
f43_030 = get_value(unidad4_SOAT, "Constitución Reserva matemática Riesgos Laborales (resta)", 4)
f44_035 = get_value(unidad4_SOAT, "Constitución Reserva Siniestros Avisados  (resta)", 4)
f45_037 = get_value(unidad4_SOAT, "Constitución Reserva matemática BEPs (resta)", 4)
f46_040 = get_value(unidad4_SOAT, "Constitución Reserva Siniestros No Avisados  (resta)", 4)
f47_045 = get_value(unidad4_SOAT, "Constitución Reserva desviación siniestralidad (resta)", 4)
f48_050 = get_value(unidad4_SOAT, "Constitución Reservas especiales (resta)", 4)
f49_051 = get_value(unidad4_SOAT, "Constitución Reserva de Riesgos Catastróficos (resta)",4)


f50_999 = (f36_005 + f37_007 + f38_010 + f39_015 + f40_020 + f41_025 + f42_026 -
            f43_030 - f44_035 - f45_037 - f46_040 - f47_045 - f48_050 - f49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=6, value=f50_999)

<Cell 'Hoja1'.F50>

##  5   CUMPLIMIENTO UNIDAD 4

In [84]:
unidad4_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_CUMPLIMIENTO['TOTAL'] = unidad4_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 5, 36),
    ("Liberación Reserva Matemática BEPs", 5, 37),
    ("Liberación Reserva para Siniestros Avisados", 5, 38),
    ("Liberación Reserva para Siniestros No Avisados", 5, 39),
    ("Liberación Reservas desviación de siniestralidad", 5, 40),
    ("Liberación Reservas especiales", 5, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 5, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 5, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 5, 44),
    ("Constitución Reserva matemática BEPs (resta)", 5, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 5, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 5, 47),
    ("Constitución Reservas especiales (resta)", 5, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 5, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g36_005 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reserva Matemática Riesgos Laborales", 5)
g37_007 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reserva Matemática BEPs", 5)
g38_010 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reserva para Siniestros Avisados", 5)
g39_015 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reserva para Siniestros No Avisados", 5)
g40_020 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reservas desviación de siniestralidad", 5)
g41_025 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reservas especiales", 5)
g42_026 = get_value(unidad4_CUMPLIMIENTO, "Liberación Reserva de Riesgos Catastróficos", 5)
g43_030 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva matemática Riesgos Laborales (resta)", 5)
g44_035 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva Siniestros Avisados  (resta)", 5)
g45_037 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva matemática BEPs (resta)", 5)
g46_040 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva Siniestros No Avisados  (resta)", 5)
g47_045 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva desviación siniestralidad (resta)", 5)
g48_050 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reservas especiales (resta)", 5)
g49_051 = get_value(unidad4_CUMPLIMIENTO, "Constitución Reserva de Riesgos Catastróficos (resta)",5)


g50_999 = (g36_005 + g37_007 + g38_010 + g39_015 + g40_020 + g41_025 + g42_026 -
            g43_030 - g44_035 - g45_037 - g46_040 - g47_045 - g48_050 - g49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=7, value=g50_999)

<Cell 'Hoja1'.G50>

##  6   RESPONSABILIDAD CIVIL UNIDAD 4

In [85]:
unidad4_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_RESPONSABILIDADCI['TOTAL'] = unidad4_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 6, 36),
    ("Liberación Reserva Matemática BEPs", 6, 37),
    ("Liberación Reserva para Siniestros Avisados", 6, 38),
    ("Liberación Reserva para Siniestros No Avisados", 6, 39),
    ("Liberación Reservas desviación de siniestralidad", 6, 40),
    ("Liberación Reservas especiales", 6, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 6, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 6, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 6, 44),
    ("Constitución Reserva matemática BEPs (resta)", 6, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 6, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 6, 47),
    ("Constitución Reservas especiales (resta)", 6, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 6, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h36_005 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reserva Matemática Riesgos Laborales", 6)
h37_007 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reserva Matemática BEPs", 6)
h38_010 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reserva para Siniestros Avisados", 6)
h39_015 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reserva para Siniestros No Avisados", 6)
h40_020 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reservas desviación de siniestralidad", 6)
h41_025 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reservas especiales", 6)
h42_026 = get_value(unidad4_RESPONSABILIDADCI, "Liberación Reserva de Riesgos Catastróficos", 6)
h43_030 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva matemática Riesgos Laborales (resta)", 6)
h44_035 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva Siniestros Avisados  (resta)", 6)
h45_037 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva matemática BEPs (resta)", 6)
h46_040 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva Siniestros No Avisados  (resta)", 6)
h47_045 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva desviación siniestralidad (resta)", 6)
h48_050 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reservas especiales (resta)", 6)
h49_051 = get_value(unidad4_RESPONSABILIDADCI, "Constitución Reserva de Riesgos Catastróficos (resta)",6)


h50_999 = (h36_005 + h37_007 + h38_010 + h39_015 + h40_020 + h41_025 + h42_026 -
            h43_030 - h44_035 - h45_037 - h46_040 - h47_045 - h48_050 - h49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=8, value=h50_999)

<Cell 'Hoja1'.H50>

##  7   INCENDIO UNIDAD 4

In [86]:
unidad4_INCENDIO = bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_INCENDIO['TOTAL'] = unidad4_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 7, 36),
    ("Liberación Reserva Matemática BEPs", 7, 37),
    ("Liberación Reserva para Siniestros Avisados", 7, 38),
    ("Liberación Reserva para Siniestros No Avisados", 7, 39),
    ("Liberación Reservas desviación de siniestralidad", 7, 40),
    ("Liberación Reservas especiales", 7, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 7, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 7, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 7, 44),
    ("Constitución Reserva matemática BEPs (resta)", 7, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 7, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 7, 47),
    ("Constitución Reservas especiales (resta)", 7, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 7, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i36_005 = get_value(unidad4_INCENDIO, "Liberación Reserva Matemática Riesgos Laborales", 7)
i37_007 = get_value(unidad4_INCENDIO, "Liberación Reserva Matemática BEPs", 7)
i38_010 = get_value(unidad4_INCENDIO, "Liberación Reserva para Siniestros Avisados", 7)
i39_015 = get_value(unidad4_INCENDIO, "Liberación Reserva para Siniestros No Avisados", 7)
i40_020 = get_value(unidad4_INCENDIO, "Liberación Reservas desviación de siniestralidad", 7)
i41_025 = get_value(unidad4_INCENDIO, "Liberación Reservas especiales", 7)
i42_026 = get_value(unidad4_INCENDIO, "Liberación Reserva de Riesgos Catastróficos", 7)
i43_030 = get_value(unidad4_INCENDIO, "Constitución Reserva matemática Riesgos Laborales (resta)", 7)
i44_035 = get_value(unidad4_INCENDIO, "Constitución Reserva Siniestros Avisados  (resta)", 7)
i45_037 = get_value(unidad4_INCENDIO, "Constitución Reserva matemática BEPs (resta)", 7)
i46_040 = get_value(unidad4_INCENDIO, "Constitución Reserva Siniestros No Avisados  (resta)", 7)
i47_045 = get_value(unidad4_INCENDIO, "Constitución Reserva desviación siniestralidad (resta)", 7)
i48_050 = get_value(unidad4_INCENDIO, "Constitución Reservas especiales (resta)", 7)
i49_051 = get_value(unidad4_INCENDIO, "Constitución Reserva de Riesgos Catastróficos (resta)",7)


i50_999 = (i36_005 + i37_007 + i38_010 + i39_015 + i40_020 + i41_025 + i42_026 -
            i43_030 - i44_035 - i45_037 - i46_040 - i47_045 - i48_050 - i49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=9, value=i50_999)

<Cell 'Hoja1'.I50>

## 8   TERREMOTO UNIDAD 4

In [87]:
unidad4_TERREMOTO= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_TERREMOTO['TOTAL'] = unidad4_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 8, 36),
    ("Liberación Reserva Matemática BEPs", 8, 37),
    ("Liberación Reserva para Siniestros Avisados", 8, 38),
    ("Liberación Reserva para Siniestros No Avisados", 8, 39),
    ("Liberación Reservas desviación de siniestralidad", 8, 40),
    ("Liberación Reservas especiales", 8, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 8, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 8, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 8, 44),
    ("Constitución Reserva matemática BEPs (resta)", 8, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 8, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 8, 47),
    ("Constitución Reservas especiales (resta)", 8, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 8, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j36_005 = get_value(unidad4_TERREMOTO, "Liberación Reserva Matemática Riesgos Laborales", 8)
j37_007 = get_value(unidad4_TERREMOTO, "Liberación Reserva Matemática BEPs", 8)
j38_010 = get_value(unidad4_TERREMOTO, "Liberación Reserva para Siniestros Avisados", 8)
j39_015 = get_value(unidad4_TERREMOTO, "Liberación Reserva para Siniestros No Avisados", 8)
j40_020 = get_value(unidad4_TERREMOTO, "Liberación Reservas desviación de siniestralidad", 8)
j41_025 = get_value(unidad4_TERREMOTO, "Liberación Reservas especiales", 8)
j42_026 = get_value(unidad4_TERREMOTO, "Liberación Reserva de Riesgos Catastróficos", 8)
j43_030 = get_value(unidad4_TERREMOTO, "Constitución Reserva matemática Riesgos Laborales (resta)", 8)
j44_035 = get_value(unidad4_TERREMOTO, "Constitución Reserva Siniestros Avisados  (resta)", 8)
j45_037 = get_value(unidad4_TERREMOTO, "Constitución Reserva matemática BEPs (resta)", 8)
j46_040 = get_value(unidad4_TERREMOTO, "Constitución Reserva Siniestros No Avisados  (resta)", 8)
j47_045 = get_value(unidad4_TERREMOTO, "Constitución Reserva desviación siniestralidad (resta)", 8)
j48_050 = get_value(unidad4_TERREMOTO, "Constitución Reservas especiales (resta)", 8)
j49_051 = get_value(unidad4_TERREMOTO, "Constitución Reserva de Riesgos Catastróficos (resta)",8)


j50_999 = (j36_005 + j37_007 + j38_010 + j39_015 + j40_020 + j41_025 + j42_026 -
            j43_030 - j44_035 - j45_037 - j46_040 - j47_045 - j48_050 - j49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=10, value=j50_999)

<Cell 'Hoja1'.J50>

##  9   SUSTRACCION UNIDAD 4

In [88]:
unidad4_SUSTRACCION= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_SUSTRACCION['TOTAL'] = unidad4_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 9, 36),
    ("Liberación Reserva Matemática BEPs", 9, 37),
    ("Liberación Reserva para Siniestros Avisados", 9, 38),
    ("Liberación Reserva para Siniestros No Avisados", 9, 39),
    ("Liberación Reservas desviación de siniestralidad", 9, 40),
    ("Liberación Reservas especiales", 9, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 9, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 9, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 9, 44),
    ("Constitución Reserva matemática BEPs (resta)", 9, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 9, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 9, 47),
    ("Constitución Reservas especiales (resta)", 9, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 9, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k36_005 = get_value(unidad4_SUSTRACCION, "Liberación Reserva Matemática Riesgos Laborales", 9)
k37_007 = get_value(unidad4_SUSTRACCION, "Liberación Reserva Matemática BEPs", 9)
k38_010 = get_value(unidad4_SUSTRACCION, "Liberación Reserva para Siniestros Avisados", 9)
k39_015 = get_value(unidad4_SUSTRACCION, "Liberación Reserva para Siniestros No Avisados", 9)
k40_020 = get_value(unidad4_SUSTRACCION, "Liberación Reservas desviación de siniestralidad", 9)
k41_025 = get_value(unidad4_SUSTRACCION, "Liberación Reservas especiales", 9)
k42_026 = get_value(unidad4_SUSTRACCION, "Liberación Reserva de Riesgos Catastróficos", 9)
k43_030 = get_value(unidad4_SUSTRACCION, "Constitución Reserva matemática Riesgos Laborales (resta)", 9)
k44_035 = get_value(unidad4_SUSTRACCION, "Constitución Reserva Siniestros Avisados  (resta)", 9)
k45_037 = get_value(unidad4_SUSTRACCION, "Constitución Reserva matemática BEPs (resta)", 9)
k46_040 = get_value(unidad4_SUSTRACCION, "Constitución Reserva Siniestros No Avisados  (resta)", 9)
k47_045 = get_value(unidad4_SUSTRACCION, "Constitución Reserva desviación siniestralidad (resta)", 9)
k48_050 = get_value(unidad4_SUSTRACCION, "Constitución Reservas especiales (resta)", 9)
k49_051 = get_value(unidad4_SUSTRACCION, "Constitución Reserva de Riesgos Catastróficos (resta)",9)


k50_999 = (k36_005 + k37_007 + k38_010 + k39_015 + k40_020 + k41_025 + k42_026 -
            k43_030 - k44_035 - k45_037 - k46_040 - k47_045 - k48_050 - k49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=11, value=k50_999)

<Cell 'Hoja1'.K50>

##  10  TRANSPORTE UNIDAD 4

In [89]:
unidad4_TRANSPORTE= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_TRANSPORTE['TOTAL'] = unidad4_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 10, 36),
    ("Liberación Reserva Matemática BEPs", 10, 37),
    ("Liberación Reserva para Siniestros Avisados", 10, 38),
    ("Liberación Reserva para Siniestros No Avisados", 10, 39),
    ("Liberación Reservas desviación de siniestralidad", 10, 40),
    ("Liberación Reservas especiales", 10, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 10, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 10, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 10, 44),
    ("Constitución Reserva matemática BEPs (resta)", 10, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 10, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 10, 47),
    ("Constitución Reservas especiales (resta)", 10, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 10, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l36_005 = get_value(unidad4_TRANSPORTE, "Liberación Reserva Matemática Riesgos Laborales", 10)
l37_007 = get_value(unidad4_TRANSPORTE, "Liberación Reserva Matemática BEPs", 10)
l38_010 = get_value(unidad4_TRANSPORTE, "Liberación Reserva para Siniestros Avisados", 10)
l39_015 = get_value(unidad4_TRANSPORTE, "Liberación Reserva para Siniestros No Avisados", 10)
l40_020 = get_value(unidad4_TRANSPORTE, "Liberación Reservas desviación de siniestralidad", 10)
l41_025 = get_value(unidad4_TRANSPORTE, "Liberación Reservas especiales", 10)
l42_026 = get_value(unidad4_TRANSPORTE, "Liberación Reserva de Riesgos Catastróficos", 10)
l43_030 = get_value(unidad4_TRANSPORTE, "Constitución Reserva matemática Riesgos Laborales (resta)", 10)
l44_035 = get_value(unidad4_TRANSPORTE, "Constitución Reserva Siniestros Avisados  (resta)", 10)
l45_037 = get_value(unidad4_TRANSPORTE, "Constitución Reserva matemática BEPs (resta)", 10)
l46_040 = get_value(unidad4_TRANSPORTE, "Constitución Reserva Siniestros No Avisados  (resta)", 10)
l47_045 = get_value(unidad4_TRANSPORTE, "Constitución Reserva desviación siniestralidad (resta)", 10)
l48_050 = get_value(unidad4_TRANSPORTE, "Constitución Reservas especiales (resta)", 10)
l49_051 = get_value(unidad4_TRANSPORTE, "Constitución Reserva de Riesgos Catastróficos (resta)",10)


l50_999 = (l36_005 + l37_007 + l38_010 + l39_015 + l40_020 + l41_025 + l42_026 -
            l43_030 - l44_035 - l45_037 - l46_040 - l47_045 - l48_050 - l49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=12, value=l50_999)

<Cell 'Hoja1'.L50>

##  11  CORRIENTE DÉBIL UNIDAD 4

In [90]:
unidad4_CORRIENDEBIL= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_CORRIENDEBIL['TOTAL'] = unidad4_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 11, 36),
    ("Liberación Reserva Matemática BEPs", 11, 37),
    ("Liberación Reserva para Siniestros Avisados", 11, 38),
    ("Liberación Reserva para Siniestros No Avisados", 11, 39),
    ("Liberación Reservas desviación de siniestralidad", 11, 40),
    ("Liberación Reservas especiales", 11, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 11, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 11, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 11, 44),
    ("Constitución Reserva matemática BEPs (resta)", 11, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 11, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 11, 47),
    ("Constitución Reservas especiales (resta)", 11, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 11, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m36_005 = get_value(unidad4_CORRIENDEBIL, "Liberación Reserva Matemática Riesgos Laborales", 11)
m37_007 = get_value(unidad4_CORRIENDEBIL, "Liberación Reserva Matemática BEPs", 11)
m38_010 = get_value(unidad4_CORRIENDEBIL, "Liberación Reserva para Siniestros Avisados", 11)
m39_015 = get_value(unidad4_CORRIENDEBIL, "Liberación Reserva para Siniestros No Avisados", 11)
m40_020 = get_value(unidad4_CORRIENDEBIL, "Liberación Reservas desviación de siniestralidad", 11)
m41_025 = get_value(unidad4_CORRIENDEBIL, "Liberación Reservas especiales", 11)
m42_026 = get_value(unidad4_CORRIENDEBIL, "Liberación Reserva de Riesgos Catastróficos", 11)
m43_030 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva matemática Riesgos Laborales (resta)", 11)
m44_035 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva Siniestros Avisados  (resta)", 11)
m45_037 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva matemática BEPs (resta)", 11)
m46_040 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva Siniestros No Avisados  (resta)", 11)
m47_045 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva desviación siniestralidad (resta)", 11)
m48_050 = get_value(unidad4_CORRIENDEBIL, "Constitución Reservas especiales (resta)", 11)
m49_051 = get_value(unidad4_CORRIENDEBIL, "Constitución Reserva de Riesgos Catastróficos (resta)",11)


m50_999 = (m36_005 + m37_007 + m38_010 + m39_015 + m40_020 + m41_025 + m42_026 -
            m43_030 - m44_035 - m45_037 - m46_040 - m47_045 - m48_050 - m49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=13, value=m50_999)

<Cell 'Hoja1'.M50>

##  12  TODO RIESGO CONTRATISTA UNIDAD 4

In [91]:
unidad4_TODORIESGOCONTRATISTA= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_TODORIESGOCONTRATISTA['TOTAL'] = unidad4_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 12, 36),
    ("Liberación Reserva Matemática BEPs", 12, 37),
    ("Liberación Reserva para Siniestros Avisados", 12, 38),
    ("Liberación Reserva para Siniestros No Avisados", 12, 39),
    ("Liberación Reservas desviación de siniestralidad", 12, 40),
    ("Liberación Reservas especiales", 12, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 12, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 12, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 12, 44),
    ("Constitución Reserva matemática BEPs (resta)", 12, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 12, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 12, 47),
    ("Constitución Reservas especiales (resta)", 12, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 12, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n36_005 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reserva Matemática Riesgos Laborales", 12)
n37_007 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reserva Matemática BEPs", 12)
n38_010 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reserva para Siniestros Avisados", 12)
n39_015 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reserva para Siniestros No Avisados", 12)
n40_020 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reservas desviación de siniestralidad", 12)
n41_025 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reservas especiales", 12)
n42_026 = get_value(unidad4_TODORIESGOCONTRATISTA, "Liberación Reserva de Riesgos Catastróficos", 12)
n43_030 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva matemática Riesgos Laborales (resta)", 12)
n44_035 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva Siniestros Avisados  (resta)", 12)
n45_037 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva matemática BEPs (resta)", 12)
n46_040 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva Siniestros No Avisados  (resta)", 12)
n47_045 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva desviación siniestralidad (resta)", 12)
n48_050 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reservas especiales (resta)", 12)
n49_051 = get_value(unidad4_TODORIESGOCONTRATISTA, "Constitución Reserva de Riesgos Catastróficos (resta)",12)


n50_999 = (n36_005 + n37_007 + n38_010 + n39_015 + n40_020 + n41_025 + n42_026 -
            n43_030 - n44_035 - n45_037 - n46_040 - n47_045 - n48_050 - n49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=14, value=n50_999)

<Cell 'Hoja1'.N50>

##  13  MANEJO UNIDAD 4


In [92]:
unidad4_MANEJO= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_MANEJO['TOTAL'] = unidad4_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 13, 36),
    ("Liberación Reserva Matemática BEPs", 13, 37),
    ("Liberación Reserva para Siniestros Avisados", 13, 38),
    ("Liberación Reserva para Siniestros No Avisados", 13, 39),
    ("Liberación Reservas desviación de siniestralidad", 13, 40),
    ("Liberación Reservas especiales", 13, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 13, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 13, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 13, 44),
    ("Constitución Reserva matemática BEPs (resta)", 13, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 13, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 13, 47),
    ("Constitución Reservas especiales (resta)", 13, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 13, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o36_005 = get_value(unidad4_MANEJO, "Liberación Reserva Matemática Riesgos Laborales", 13)
o37_007 = get_value(unidad4_MANEJO, "Liberación Reserva Matemática BEPs", 13)
o38_010 = get_value(unidad4_MANEJO, "Liberación Reserva para Siniestros Avisados", 13)
o39_015 = get_value(unidad4_MANEJO, "Liberación Reserva para Siniestros No Avisados", 13)
o40_020 = get_value(unidad4_MANEJO, "Liberación Reservas desviación de siniestralidad", 13)
o41_025 = get_value(unidad4_MANEJO, "Liberación Reservas especiales", 13)
o42_026 = get_value(unidad4_MANEJO, "Liberación Reserva de Riesgos Catastróficos", 13)
o43_030 = get_value(unidad4_MANEJO, "Constitución Reserva matemática Riesgos Laborales (resta)", 13)
o44_035 = get_value(unidad4_MANEJO, "Constitución Reserva Siniestros Avisados  (resta)", 13)
o45_037 = get_value(unidad4_MANEJO, "Constitución Reserva matemática BEPs (resta)", 13)
o46_040 = get_value(unidad4_MANEJO, "Constitución Reserva Siniestros No Avisados  (resta)", 13)
o47_045 = get_value(unidad4_MANEJO, "Constitución Reserva desviación siniestralidad (resta)", 13)
o48_050 = get_value(unidad4_MANEJO, "Constitución Reservas especiales (resta)", 13)
o49_051 = get_value(unidad4_MANEJO, "Constitución Reserva de Riesgos Catastróficos (resta)",13)


o50_999 = (o36_005 + o37_007 + o38_010 + o39_015 + o40_020 + o41_025 + o42_026 -
            o43_030 - o44_035 - o45_037 - o46_040 - o47_045 - o48_050 - o49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=15, value=o50_999)

<Cell 'Hoja1'.O50>

##  14  LUCRO CESANTE UNIDAD 4


In [93]:
unidad4_LUCROCESANTE= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_LUCROCESANTE['TOTAL'] = unidad4_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 14, 36),
    ("Liberación Reserva Matemática BEPs", 14, 37),
    ("Liberación Reserva para Siniestros Avisados", 14, 38),
    ("Liberación Reserva para Siniestros No Avisados", 14, 39),
    ("Liberación Reservas desviación de siniestralidad", 14, 40),
    ("Liberación Reservas especiales", 14, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 14, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 14, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 14, 44),
    ("Constitución Reserva matemática BEPs (resta)", 14, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 14, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 14, 47),
    ("Constitución Reservas especiales (resta)", 14, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 14, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p36_005 = get_value(unidad4_LUCROCESANTE, "Liberación Reserva Matemática Riesgos Laborales", 14)
p37_007 = get_value(unidad4_LUCROCESANTE, "Liberación Reserva Matemática BEPs", 14)
p38_010 = get_value(unidad4_LUCROCESANTE, "Liberación Reserva para Siniestros Avisados", 14)
p39_015 = get_value(unidad4_LUCROCESANTE, "Liberación Reserva para Siniestros No Avisados", 14)
p40_020 = get_value(unidad4_LUCROCESANTE, "Liberación Reservas desviación de siniestralidad", 14)
p41_025 = get_value(unidad4_LUCROCESANTE, "Liberación Reservas especiales", 14)
p42_026 = get_value(unidad4_LUCROCESANTE, "Liberación Reserva de Riesgos Catastróficos", 14)
p43_030 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva matemática Riesgos Laborales (resta)", 14)
p44_035 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva Siniestros Avisados  (resta)", 14)
p45_037 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva matemática BEPs (resta)", 14)
p46_040 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva Siniestros No Avisados  (resta)", 14)
p47_045 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva desviación siniestralidad (resta)", 14)
p48_050 = get_value(unidad4_LUCROCESANTE, "Constitución Reservas especiales (resta)", 14)
p49_051 = get_value(unidad4_LUCROCESANTE, "Constitución Reserva de Riesgos Catastróficos (resta)",14)


p50_999 = (p36_005 + p37_007 + p38_010 + p39_015 + p40_020 + p41_025 + p42_026 -
            p43_030 - p44_035 - p45_037 - p46_040 - p47_045 - p48_050 - p49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=16, value=p50_999)

<Cell 'Hoja1'.P50>

##  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 4


In [94]:
unidad4_MONTAJEYROTURA= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_MONTAJEYROTURA['TOTAL'] = unidad4_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 15, 36),
    ("Liberación Reserva Matemática BEPs", 15, 37),
    ("Liberación Reserva para Siniestros Avisados", 15, 38),
    ("Liberación Reserva para Siniestros No Avisados", 15, 39),
    ("Liberación Reservas desviación de siniestralidad", 15, 40),
    ("Liberación Reservas especiales", 15, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 15, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 15, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 15, 44),
    ("Constitución Reserva matemática BEPs (resta)", 15, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 15, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 15, 47),
    ("Constitución Reservas especiales (resta)", 15, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 15, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q36_005 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reserva Matemática Riesgos Laborales", 15)
q37_007 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reserva Matemática BEPs", 15)
q38_010 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reserva para Siniestros Avisados", 15)
q39_015 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reserva para Siniestros No Avisados", 15)
q40_020 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reservas desviación de siniestralidad", 15)
q41_025 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reservas especiales", 15)
q42_026 = get_value(unidad4_MONTAJEYROTURA, "Liberación Reserva de Riesgos Catastróficos", 15)
q43_030 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva matemática Riesgos Laborales (resta)", 15)
q44_035 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva Siniestros Avisados  (resta)", 15)
q45_037 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva matemática BEPs (resta)", 15)
q46_040 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva Siniestros No Avisados  (resta)", 15)
q47_045 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva desviación siniestralidad (resta)", 15)
q48_050 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reservas especiales (resta)", 15)
q49_051 = get_value(unidad4_MONTAJEYROTURA, "Constitución Reserva de Riesgos Catastróficos (resta)",15)


q50_999 = (q36_005 + q37_007 + q38_010 + q39_015 + q40_020 + q41_025 + q42_026 -
            q43_030 - q44_035 - q45_037 - q46_040 - q47_045 - q48_050 - q49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=17, value=q50_999)

<Cell 'Hoja1'.Q50>

##  16  AVIACIÓN UNIDAD 4


In [95]:
unidad4_AVIACION= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_AVIACION['TOTAL'] = unidad4_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 16, 36),
    ("Liberación Reserva Matemática BEPs", 16, 37),
    ("Liberación Reserva para Siniestros Avisados", 16, 38),
    ("Liberación Reserva para Siniestros No Avisados", 16, 39),
    ("Liberación Reservas desviación de siniestralidad", 16, 40),
    ("Liberación Reservas especiales", 16, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 16, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 16, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 16, 44),
    ("Constitución Reserva matemática BEPs (resta)", 16, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 16, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 16, 47),
    ("Constitución Reservas especiales (resta)", 16, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 16, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r36_005 = get_value(unidad4_AVIACION, "Liberación Reserva Matemática Riesgos Laborales", 16)
r37_007 = get_value(unidad4_AVIACION, "Liberación Reserva Matemática BEPs", 16)
r38_010 = get_value(unidad4_AVIACION, "Liberación Reserva para Siniestros Avisados", 16)
r39_015 = get_value(unidad4_AVIACION, "Liberación Reserva para Siniestros No Avisados", 16)
r40_020 = get_value(unidad4_AVIACION, "Liberación Reservas desviación de siniestralidad", 16)
r41_025 = get_value(unidad4_AVIACION, "Liberación Reservas especiales", 16)
r42_026 = get_value(unidad4_AVIACION, "Liberación Reserva de Riesgos Catastróficos", 16)
r43_030 = get_value(unidad4_AVIACION, "Constitución Reserva matemática Riesgos Laborales (resta)", 16)
r44_035 = get_value(unidad4_AVIACION, "Constitución Reserva Siniestros Avisados  (resta)", 16)
r45_037 = get_value(unidad4_AVIACION, "Constitución Reserva matemática BEPs (resta)", 16)
r46_040 = get_value(unidad4_AVIACION, "Constitución Reserva Siniestros No Avisados  (resta)", 16)
r47_045 = get_value(unidad4_AVIACION, "Constitución Reserva desviación siniestralidad (resta)", 16)
r48_050 = get_value(unidad4_AVIACION, "Constitución Reservas especiales (resta)", 16)
r49_051 = get_value(unidad4_AVIACION, "Constitución Reserva de Riesgos Catastróficos (resta)",16)


r50_999 = (r36_005 + r37_007 + r38_010 + r39_015 + r40_020 + r41_025 + r42_026 -
            r43_030 - r44_035 - r45_037 - r46_040 - r47_045 - r48_050 - r49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=18, value=r50_999)

<Cell 'Hoja1'.R50>

##  17  NAVEGACIÓN Y CASCO UNIDAD 4


In [96]:
unidad4_NAVEGACION= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_NAVEGACION['TOTAL'] = unidad4_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 17, 36),
    ("Liberación Reserva Matemática BEPs", 17, 37),
    ("Liberación Reserva para Siniestros Avisados", 17, 38),
    ("Liberación Reserva para Siniestros No Avisados", 17, 39),
    ("Liberación Reservas desviación de siniestralidad", 17, 40),
    ("Liberación Reservas especiales", 17, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 17, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 17, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 17, 44),
    ("Constitución Reserva matemática BEPs (resta)", 17, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 17, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 17, 47),
    ("Constitución Reservas especiales (resta)", 17, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 17, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s36_005 = get_value(unidad4_NAVEGACION, "Liberación Reserva Matemática Riesgos Laborales", 17)
s37_007 = get_value(unidad4_NAVEGACION, "Liberación Reserva Matemática BEPs", 17)
s38_010 = get_value(unidad4_NAVEGACION, "Liberación Reserva para Siniestros Avisados", 17)
s39_015 = get_value(unidad4_NAVEGACION, "Liberación Reserva para Siniestros No Avisados", 17)
s40_020 = get_value(unidad4_NAVEGACION, "Liberación Reservas desviación de siniestralidad", 17)
s41_025 = get_value(unidad4_NAVEGACION, "Liberación Reservas especiales", 17)
s42_026 = get_value(unidad4_NAVEGACION, "Liberación Reserva de Riesgos Catastróficos", 17)
s43_030 = get_value(unidad4_NAVEGACION, "Constitución Reserva matemática Riesgos Laborales (resta)", 17)
s44_035 = get_value(unidad4_NAVEGACION, "Constitución Reserva Siniestros Avisados  (resta)", 17)
s45_037 = get_value(unidad4_NAVEGACION, "Constitución Reserva matemática BEPs (resta)", 17)
s46_040 = get_value(unidad4_NAVEGACION, "Constitución Reserva Siniestros No Avisados  (resta)", 17)
s47_045 = get_value(unidad4_NAVEGACION, "Constitución Reserva desviación siniestralidad (resta)", 17)
s48_050 = get_value(unidad4_NAVEGACION, "Constitución Reservas especiales (resta)", 17)
s49_051 = get_value(unidad4_NAVEGACION, "Constitución Reserva de Riesgos Catastróficos (resta)",17)


s50_999 = (s36_005 + s37_007 + s38_010 + s39_015 + s40_020 + s41_025 + s42_026 -
            s43_030 - s44_035 - s45_037 - s46_040 - s47_045 - s48_050 - s49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=19, value=s50_999)

<Cell 'Hoja1'.S50>

##  18  MINAS Y PETRÓLEOS UNIDAD 4


In [97]:
unidad4_MINAS= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_MINAS['TOTAL'] = unidad4_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 18, 36),
    ("Liberación Reserva Matemática BEPs", 18, 37),
    ("Liberación Reserva para Siniestros Avisados", 18, 38),
    ("Liberación Reserva para Siniestros No Avisados", 18, 39),
    ("Liberación Reservas desviación de siniestralidad", 18, 40),
    ("Liberación Reservas especiales", 18, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 18, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 18, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 18, 44),
    ("Constitución Reserva matemática BEPs (resta)", 18, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 18, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 18, 47),
    ("Constitución Reservas especiales (resta)", 18, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 18, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t36_005 = get_value(unidad4_MINAS, "Liberación Reserva Matemática Riesgos Laborales", 18)
t37_007 = get_value(unidad4_MINAS, "Liberación Reserva Matemática BEPs", 18)
t38_010 = get_value(unidad4_MINAS, "Liberación Reserva para Siniestros Avisados", 18)
t39_015 = get_value(unidad4_MINAS, "Liberación Reserva para Siniestros No Avisados", 18)
t40_020 = get_value(unidad4_MINAS, "Liberación Reservas desviación de siniestralidad", 18)
t41_025 = get_value(unidad4_MINAS, "Liberación Reservas especiales", 18)
t42_026 = get_value(unidad4_MINAS, "Liberación Reserva de Riesgos Catastróficos", 18)
t43_030 = get_value(unidad4_MINAS, "Constitución Reserva matemática Riesgos Laborales (resta)", 18)
t44_035 = get_value(unidad4_MINAS, "Constitución Reserva Siniestros Avisados  (resta)", 18)
t45_037 = get_value(unidad4_MINAS, "Constitución Reserva matemática BEPs (resta)", 18)
t46_040 = get_value(unidad4_MINAS, "Constitución Reserva Siniestros No Avisados  (resta)", 18)
t47_045 = get_value(unidad4_MINAS, "Constitución Reserva desviación siniestralidad (resta)", 18)
t48_050 = get_value(unidad4_MINAS, "Constitución Reservas especiales (resta)", 18)
t49_051 = get_value(unidad4_MINAS, "Constitución Reserva de Riesgos Catastróficos (resta)",18)


t50_999 = (t36_005 + t37_007 + t38_010 + t39_015 + t40_020 + t41_025 + t42_026 -
            t43_030 - t44_035 - t45_037 - t46_040 - t47_045 - t48_050 - t49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=20, value=t50_999)

<Cell 'Hoja1'.T50>

##  19  VIDRIOS UNIDAD 4


In [98]:
unidad4_VIDRIOS= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_VIDRIOS['TOTAL'] = unidad4_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 19, 36),
    ("Liberación Reserva Matemática BEPs", 19, 37),
    ("Liberación Reserva para Siniestros Avisados", 19, 38),
    ("Liberación Reserva para Siniestros No Avisados", 19, 39),
    ("Liberación Reservas desviación de siniestralidad", 19, 40),
    ("Liberación Reservas especiales", 19, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 19, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 19, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 19, 44),
    ("Constitución Reserva matemática BEPs (resta)", 19, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 19, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 19, 47),
    ("Constitución Reservas especiales (resta)", 19, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 19, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u36_005 = get_value(unidad4_VIDRIOS, "Liberación Reserva Matemática Riesgos Laborales", 19)
u37_007 = get_value(unidad4_VIDRIOS, "Liberación Reserva Matemática BEPs", 19)
u38_010 = get_value(unidad4_VIDRIOS, "Liberación Reserva para Siniestros Avisados", 19)
u39_015 = get_value(unidad4_VIDRIOS, "Liberación Reserva para Siniestros No Avisados", 19)
u40_020 = get_value(unidad4_VIDRIOS, "Liberación Reservas desviación de siniestralidad", 19)
u41_025 = get_value(unidad4_VIDRIOS, "Liberación Reservas especiales", 19)
u42_026 = get_value(unidad4_VIDRIOS, "Liberación Reserva de Riesgos Catastróficos", 19)
u43_030 = get_value(unidad4_VIDRIOS, "Constitución Reserva matemática Riesgos Laborales (resta)", 19)
u44_035 = get_value(unidad4_VIDRIOS, "Constitución Reserva Siniestros Avisados  (resta)", 19)
u45_037 = get_value(unidad4_VIDRIOS, "Constitución Reserva matemática BEPs (resta)", 19)
u46_040 = get_value(unidad4_VIDRIOS, "Constitución Reserva Siniestros No Avisados  (resta)", 19)
u47_045 = get_value(unidad4_VIDRIOS, "Constitución Reserva desviación siniestralidad (resta)", 19)
u48_050 = get_value(unidad4_VIDRIOS, "Constitución Reservas especiales (resta)", 19)
u49_051 = get_value(unidad4_VIDRIOS, "Constitución Reserva de Riesgos Catastróficos (resta)",19)


u50_999 = (u36_005 + u37_007 + u38_010 + u39_015 + u40_020 + u41_025 + u42_026 -
            u43_030 - u44_035 - u45_037 - u46_040 - u47_045 - u48_050 - u49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=21, value=u50_999)

<Cell 'Hoja1'.U50>

##  20  CRÉDITO COMERCIAL UNIDAD 4


In [99]:
unidad4_CREDITOCOMERCIAL= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_CREDITOCOMERCIAL['TOTAL'] = unidad4_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 20, 36),
    ("Liberación Reserva Matemática BEPs", 20, 37),
    ("Liberación Reserva para Siniestros Avisados", 20, 38),
    ("Liberación Reserva para Siniestros No Avisados", 20, 39),
    ("Liberación Reservas desviación de siniestralidad", 20, 40),
    ("Liberación Reservas especiales", 20, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 20, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 20, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 20, 44),
    ("Constitución Reserva matemática BEPs (resta)", 20, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 20, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 20, 47),
    ("Constitución Reservas especiales (resta)", 20, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 20, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v36_005 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reserva Matemática Riesgos Laborales", 20)
v37_007 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reserva Matemática BEPs", 20)
v38_010 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reserva para Siniestros Avisados", 20)
v39_015 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reserva para Siniestros No Avisados", 20)
v40_020 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reservas desviación de siniestralidad", 20)
v41_025 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reservas especiales", 20)
v42_026 = get_value(unidad4_CREDITOCOMERCIAL, "Liberación Reserva de Riesgos Catastróficos", 20)
v43_030 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva matemática Riesgos Laborales (resta)", 20)
v44_035 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva Siniestros Avisados  (resta)", 20)
v45_037 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva matemática BEPs (resta)", 20)
v46_040 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva Siniestros No Avisados  (resta)", 20)
v47_045 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva desviación siniestralidad (resta)", 20)
v48_050 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reservas especiales (resta)", 20)
v49_051 = get_value(unidad4_CREDITOCOMERCIAL, "Constitución Reserva de Riesgos Catastróficos (resta)",20)


v50_999 = (v36_005 + v37_007 + v38_010 + v39_015 + v40_020 + v41_025 + v42_026 -
            v43_030 - v44_035 - v45_037 - v46_040 - v47_045 - v48_050 - v49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=22, value=v50_999)

<Cell 'Hoja1'.V50>

##  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 4


In [100]:
unidad4_CREDITOEXPORTACION= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_CREDITOEXPORTACION['TOTAL'] = unidad4_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 21, 36),
    ("Liberación Reserva Matemática BEPs", 21, 37),
    ("Liberación Reserva para Siniestros Avisados", 21, 38),
    ("Liberación Reserva para Siniestros No Avisados", 21, 39),
    ("Liberación Reservas desviación de siniestralidad", 21, 40),
    ("Liberación Reservas especiales", 21, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 21, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 21, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 21, 44),
    ("Constitución Reserva matemática BEPs (resta)", 21, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 21, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 21, 47),
    ("Constitución Reservas especiales (resta)", 21, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 21, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w36_005 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reserva Matemática Riesgos Laborales", 21)
w37_007 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reserva Matemática BEPs", 21)
w38_010 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reserva para Siniestros Avisados", 21)
w39_015 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reserva para Siniestros No Avisados", 21)
w40_020 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reservas desviación de siniestralidad", 21)
w41_025 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reservas especiales", 21)
w42_026 = get_value(unidad4_CREDITOEXPORTACION, "Liberación Reserva de Riesgos Catastróficos", 21)
w43_030 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva matemática Riesgos Laborales (resta)", 21)
w44_035 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva Siniestros Avisados  (resta)", 21)
w45_037 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva matemática BEPs (resta)", 21)
w46_040 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva Siniestros No Avisados  (resta)", 21)
w47_045 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva desviación siniestralidad (resta)", 21)
w48_050 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reservas especiales (resta)", 21)
w49_051 = get_value(unidad4_CREDITOEXPORTACION, "Constitución Reserva de Riesgos Catastróficos (resta)",21)


w50_999 = (w36_005 + w37_007 + w38_010 + w39_015 + w40_020 + w41_025 + w42_026 -
            w43_030 - w44_035 - w45_037 - w46_040 - w47_045 - w48_050 - w49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=23, value=w50_999)

<Cell 'Hoja1'.W50>

## 24  DESEMPLEO UNIDAD 4


In [101]:
unidad4_DESEMPLEO= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_DESEMPLEO['TOTAL'] = unidad4_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 24, 36),
    ("Liberación Reserva Matemática BEPs", 24, 37),
    ("Liberación Reserva para Siniestros Avisados", 24, 38),
    ("Liberación Reserva para Siniestros No Avisados", 24, 39),
    ("Liberación Reservas desviación de siniestralidad", 24, 40),
    ("Liberación Reservas especiales", 24, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 24, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 24, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 24, 44),
    ("Constitución Reserva matemática BEPs (resta)", 24, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 24, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 24, 47),
    ("Constitución Reservas especiales (resta)", 24, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 24, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z36_005 = get_value(unidad4_DESEMPLEO, "Liberación Reserva Matemática Riesgos Laborales", 24)
z37_007 = get_value(unidad4_DESEMPLEO, "Liberación Reserva Matemática BEPs", 24)
z38_010 = get_value(unidad4_DESEMPLEO, "Liberación Reserva para Siniestros Avisados", 24)
z39_015 = get_value(unidad4_DESEMPLEO, "Liberación Reserva para Siniestros No Avisados", 24)
z40_020 = get_value(unidad4_DESEMPLEO, "Liberación Reservas desviación de siniestralidad", 24)
z41_025 = get_value(unidad4_DESEMPLEO, "Liberación Reservas especiales", 24)
z42_026 = get_value(unidad4_DESEMPLEO, "Liberación Reserva de Riesgos Catastróficos", 24)
z43_030 = get_value(unidad4_DESEMPLEO, "Constitución Reserva matemática Riesgos Laborales (resta)", 24)
z44_035 = get_value(unidad4_DESEMPLEO, "Constitución Reserva Siniestros Avisados  (resta)", 24)
z45_037 = get_value(unidad4_DESEMPLEO, "Constitución Reserva matemática BEPs (resta)", 24)
z46_040 = get_value(unidad4_DESEMPLEO, "Constitución Reserva Siniestros No Avisados  (resta)", 24)
z47_045 = get_value(unidad4_DESEMPLEO, "Constitución Reserva desviación siniestralidad (resta)", 24)
z48_050 = get_value(unidad4_DESEMPLEO, "Constitución Reservas especiales (resta)", 24)
z49_051 = get_value(unidad4_DESEMPLEO, "Constitución Reserva de Riesgos Catastróficos (resta)",24)


z50_999 = (z36_005 + z37_007 + z38_010 + z39_015 + z40_020 + z41_025 + z42_026 -
            z43_030 - z44_035 - z45_037 - z46_040 - z47_045 - z48_050 - z49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=26, value=z50_999)

<Cell 'Hoja1'.Z50>

##  25  HOGAR UNIDAD 4


In [102]:
unidad4_HOGAR= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_HOGAR['TOTAL'] = unidad4_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 25, 36),
    ("Liberación Reserva Matemática BEPs", 25, 37),
    ("Liberación Reserva para Siniestros Avisados", 25, 38),
    ("Liberación Reserva para Siniestros No Avisados", 25, 39),
    ("Liberación Reservas desviación de siniestralidad", 25, 40),
    ("Liberación Reservas especiales", 25, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 25, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 25, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 25, 44),
    ("Constitución Reserva matemática BEPs (resta)", 25, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 25, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 25, 47),
    ("Constitución Reservas especiales (resta)", 25, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 25, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa36_005 = get_value(unidad4_HOGAR, "Liberación Reserva Matemática Riesgos Laborales", 25)
aa37_007 = get_value(unidad4_HOGAR, "Liberación Reserva Matemática BEPs", 25)
aa38_010 = get_value(unidad4_HOGAR, "Liberación Reserva para Siniestros Avisados", 25)
aa39_015 = get_value(unidad4_HOGAR, "Liberación Reserva para Siniestros No Avisados", 25)
aa40_020 = get_value(unidad4_HOGAR, "Liberación Reservas desviación de siniestralidad", 25)
aa41_025 = get_value(unidad4_HOGAR, "Liberación Reservas especiales", 25)
aa42_026 = get_value(unidad4_HOGAR, "Liberación Reserva de Riesgos Catastróficos", 25)
aa43_030 = get_value(unidad4_HOGAR, "Constitución Reserva matemática Riesgos Laborales (resta)", 25)
aa44_035 = get_value(unidad4_HOGAR, "Constitución Reserva Siniestros Avisados  (resta)", 25)
aa45_037 = get_value(unidad4_HOGAR, "Constitución Reserva matemática BEPs (resta)", 25)
aa46_040 = get_value(unidad4_HOGAR, "Constitución Reserva Siniestros No Avisados  (resta)", 25)
aa47_045 = get_value(unidad4_HOGAR, "Constitución Reserva desviación siniestralidad (resta)", 25)
aa48_050 = get_value(unidad4_HOGAR, "Constitución Reservas especiales (resta)", 25)
aa49_051 = get_value(unidad4_HOGAR, "Constitución Reserva de Riesgos Catastróficos (resta)",25)


aa50_999 = (aa36_005 + aa37_007 + aa38_010 + aa39_015 + aa40_020 + aa41_025 + aa42_026 -
            aa43_030 - aa44_035 - aa45_037 - aa46_040 - aa47_045 - aa48_050 - aa49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=27, value=aa50_999)

<Cell 'Hoja1'.AA50>

## 42 AGROPECUARIO UNIDAD 4


In [103]:
unidad4_AGROPECUARIO= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_AGROPECUARIO['TOTAL'] = unidad4_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 42, 36),
    ("Liberación Reserva Matemática BEPs", 42, 37),
    ("Liberación Reserva para Siniestros Avisados", 42, 38),
    ("Liberación Reserva para Siniestros No Avisados", 42, 39),
    ("Liberación Reservas desviación de siniestralidad", 42, 40),
    ("Liberación Reservas especiales", 42, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 42, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 42, 44),
    ("Constitución Reserva matemática BEPs (resta)", 42, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 42, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 42, 47),
    ("Constitución Reservas especiales (resta)", 42, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 42, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar36_005 = get_value(unidad4_AGROPECUARIO, "Liberación Reserva Matemática Riesgos Laborales", 42)
ar37_007 = get_value(unidad4_AGROPECUARIO, "Liberación Reserva Matemática BEPs", 42)
ar38_010 = get_value(unidad4_AGROPECUARIO, "Liberación Reserva para Siniestros Avisados", 42)
ar39_015 = get_value(unidad4_AGROPECUARIO, "Liberación Reserva para Siniestros No Avisados", 42)
ar40_020 = get_value(unidad4_AGROPECUARIO, "Liberación Reservas desviación de siniestralidad", 42)
ar41_025 = get_value(unidad4_AGROPECUARIO, "Liberación Reservas especiales", 42)
ar42_026 = get_value(unidad4_AGROPECUARIO, "Liberación Reserva de Riesgos Catastróficos", 42)
ar43_030 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva matemática Riesgos Laborales (resta)", 42)
ar44_035 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva Siniestros Avisados  (resta)", 42)
ar45_037 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva matemática BEPs (resta)", 42)
ar46_040 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva Siniestros No Avisados  (resta)", 42)
ar47_045 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva desviación siniestralidad (resta)", 42)
ar48_050 = get_value(unidad4_AGROPECUARIO, "Constitución Reservas especiales (resta)", 42)
ar49_051 = get_value(unidad4_AGROPECUARIO, "Constitución Reserva de Riesgos Catastróficos (resta)",42)


ar50_999 = (ar36_005 + ar37_007 + ar38_010 + ar39_015 + ar40_020 + ar41_025 + ar42_026 -
            ar43_030 - ar44_035 - ar45_037 - ar46_040 - ar47_045 - ar48_050 - ar49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=44, value=ar50_999)

<Cell 'Hoja1'.AR50>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 4


In [104]:
unidad4_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_OPERNOASIGNABLES['TOTAL'] = unidad4_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 43, 36),
    ("Liberación Reserva Matemática BEPs", 43, 37),
    ("Liberación Reserva para Siniestros Avisados", 43, 38),
    ("Liberación Reserva para Siniestros No Avisados", 43, 39),
    ("Liberación Reservas desviación de siniestralidad", 43, 40),
    ("Liberación Reservas especiales", 43, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 43, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 43, 44),
    ("Constitución Reserva matemática BEPs (resta)", 43, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 43, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 43, 47),
    ("Constitución Reservas especiales (resta)", 43, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 43, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as36_005 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva Matemática Riesgos Laborales", 43)
as37_007 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva Matemática BEPs", 43)
as38_010 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva para Siniestros Avisados", 43)
as39_015 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva para Siniestros No Avisados", 43)
as40_020 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reservas desviación de siniestralidad", 43)
as41_025 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reservas especiales", 43)
as42_026 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva de Riesgos Catastróficos", 43)
as43_030 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva matemática Riesgos Laborales (resta)", 43)
as44_035 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva Siniestros Avisados  (resta)", 43)
as45_037 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva matemática BEPs (resta)", 43)
as46_040 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva Siniestros No Avisados  (resta)", 43)
as47_045 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva desviación siniestralidad (resta)", 43)
as48_050 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reservas especiales (resta)", 43)
as49_051 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva de Riesgos Catastróficos (resta)",43)


as50_999 = (as36_005 + as37_007 + as38_010 + as39_015 + as40_020 + as41_025 + as42_026 -
            as43_030 - as44_035 - as45_037 - as46_040 - as47_045 - as48_050 - as49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=45, value=as50_999)

<Cell 'Hoja1'.AS50>

## 44 DECENAL UNIDAD 4


In [105]:
unidad4_DECENAL= bd[(bd['UNIDAD'] == 4) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_DECENAL['TOTAL'] = unidad4_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 44, 36),
    ("Liberación Reserva Matemática BEPs", 44, 37),
    ("Liberación Reserva para Siniestros Avisados", 44, 38),
    ("Liberación Reserva para Siniestros No Avisados", 44, 39),
    ("Liberación Reservas desviación de siniestralidad", 44, 40),
    ("Liberación Reservas especiales", 44, 41),
    ("Liberación Reserva de Riesgos Catastróficos", 44, 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 44, 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44, 44),
    ("Constitución Reserva matemática BEPs (resta)", 44, 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 44, 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 44, 47),
    ("Constitución Reservas especiales (resta)", 44, 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 44, 49)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad4_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at36_005 = get_value(unidad4_DECENAL, "Liberación Reserva Matemática Riesgos Laborales", 44)
at37_007 = get_value(unidad4_DECENAL, "Liberación Reserva Matemática BEPs", 44)
at38_010 = get_value(unidad4_DECENAL, "Liberación Reserva para Siniestros Avisados", 44)
at39_015 = get_value(unidad4_DECENAL, "Liberación Reserva para Siniestros No Avisados", 44)
at40_020 = get_value(unidad4_DECENAL, "Liberación Reservas desviación de siniestralidad", 44)
at41_025 = get_value(unidad4_DECENAL, "Liberación Reservas especiales", 44)
at42_026 = get_value(unidad4_DECENAL, "Liberación Reserva de Riesgos Catastróficos", 44)
at43_030 = get_value(unidad4_DECENAL, "Constitución Reserva matemática Riesgos Laborales (resta)", 44)
at44_035 = get_value(unidad4_DECENAL, "Constitución Reserva Siniestros Avisados  (resta)", 44)
at45_037 = get_value(unidad4_DECENAL, "Constitución Reserva matemática BEPs (resta)", 44)
at46_040 = get_value(unidad4_DECENAL, "Constitución Reserva Siniestros No Avisados  (resta)", 44)
at47_045 = get_value(unidad4_DECENAL, "Constitución Reserva desviación siniestralidad (resta)", 44)
at48_050 = get_value(unidad4_DECENAL, "Constitución Reservas especiales (resta)", 44)
at49_051 = get_value(unidad4_DECENAL, "Constitución Reserva de Riesgos Catastróficos (resta)",44)


at50_999 = (at36_005 + at37_007 + at38_010 + at39_015 + at40_020 + at41_025 + at42_026 -
            at43_030 - at44_035 - at45_037 - at46_040 - at47_045 - at48_050 - at49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=46, value=at50_999)

<Cell 'Hoja1'.AT50>

## 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 4


In [106]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au36_005 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au36_005, 47, 36)

au37_007 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au37_007, 47, 37)

au38_010 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au38_010, 47, 38)

au39_015 = unidad4_CUMPLIMIENTOESTATALES_a[(unidad4_CUMPLIMIENTOESTATALES_a['CUENTA'] == "6112102011") & (unidad4_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au39_015, 47, 39, 'TOTAL')

au40_020 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au40_020, 47, 40)

au41_025 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au41_025, 47, 41)

au42_026 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au42_026, 47, 42)

au43_030 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au43_030, 47, 43)

au44_035 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au44_035, 47, 44)

au45_037 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au45_037, 47, 45)

au46_040 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au46_040, 47, 46)

au47_045 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au47_045, 47, 47)

au48_050 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au48_050, 47, 48)

au49_051 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au49_051, 47, 49)


# Calcular ak32_999
au50_999 = au36_005['TOTAL'].sum() + au37_007['TOTAL'].sum() + au38_010['TOTAL'].sum() + au39_015['TOTAL'].sum() + au40_020['TOTAL'].sum() + \
           au41_025['TOTAL'].sum() + au42_026['TOTAL'].sum() + au43_030['TOTAL'].sum() + au44_035['TOTAL'].sum() + au45_037['TOTAL'].sum() + \
           au46_040['TOTAL'].sum() + au47_045['TOTAL'].sum() + au48_050['TOTAL'].sum() + au49_051['TOTAL'].sum()

au50_999 = abs(au50_999)
# Crear DataFrame para escribir en Excel
df_au50_999 = pd.DataFrame({'total': [au50_999]})
write_to_excel(df_au50_999, 47, 50, 'total')

## 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 4


In [107]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

av36_005 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av36_005, 48, 36)

av37_007 = unidad4_CUMPLIMIENTOESTATALES[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av37_007, 48, 37)

av38_010 = unidad4_CUMPLIMIENTOEMPRESASSERV_a[(unidad4_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "6112201011") & (unidad4_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av38_010, 48, 38, 'TOTAL')

av39_015 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva para Siniestros No Avisados") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av39_015, 48, 39)

av40_020 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av40_020, 48, 40)

av41_025 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av41_025, 48, 41)

av42_026 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av42_026, 48, 42)

av43_030 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av43_030, 48, 43)

av44_035 = unidad4_CUMPLIMIENTOEMPRESASSERV_a[(unidad4_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "6112101051") & (unidad4_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av44_035, 48, 44, 'TOTAL')

av45_037 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av45_037, 48, 45)

av46_040 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av46_040, 48, 46)

av47_045 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av46_040, 48, 47)

av48_050 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av48_050, 48, 48)

av49_051 = unidad4_CUMPLIMIENTOEMPRESASSERV[(unidad4_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av49_051, 48, 49)

# Calcular ak32_999
av50_999 = av36_005['TOTAL'].sum() + av37_007['TOTAL'].sum() + av38_010['TOTAL'].sum() + av39_015['TOTAL'].sum() + av40_020['TOTAL'].sum() + \
           av41_025['TOTAL'].sum() + av42_026['TOTAL'].sum() + av43_030['TOTAL'].sum() + av44_035['TOTAL'].sum() + av45_037['TOTAL'].sum() + \
           av46_040['TOTAL'].sum() + av47_045['TOTAL'].sum() + av48_050['TOTAL'].sum() + av49_051['TOTAL'].sum()

av50_999 = abs(av50_999)
# Crear DataFrame para escribir en Excel
df_av50_999 = pd.DataFrame({'total': [av50_999]})
write_to_excel(df_av50_999, 48, 50, 'total')


## 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 4


In [108]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Consultas ajustadas para COD_RAMO 47 y escritura en columna AW (columna 49)
aw36_005 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw36_005, 49, 36)

aw37_007 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw37_007, 49, 37)

aw38_010 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw38_010, 49, 38)

aw39_015 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva para Siniestros No Avisados") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw39_015, 49, 39)

aw40_020 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw40_020, 49, 40)

aw41_025 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw41_025, 49, 41)

aw42_026 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw42_026, 49, 42)

aw43_030 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw43_030, 49, 43)

aw44_035 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw44_035, 49, 44)

aw45_037 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw45_037, 49, 45)

aw46_040 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw46_040, 49, 46)

aw47_045 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw47_045, 49, 47)

aw48_050 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw48_050, 49, 48)

aw49_051 = unidad4_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw49_051, 49, 49)

# Calcular aw50_999
aw50_999 = aw36_005['TOTAL'].sum() + aw37_007['TOTAL'].sum() + aw38_010['TOTAL'].sum() + aw39_015['TOTAL'].sum() + aw40_020['TOTAL'].sum() + \
           aw41_025['TOTAL'].sum() + aw42_026['TOTAL'].sum() + aw43_030['TOTAL'].sum() + aw44_035['TOTAL'].sum() + aw45_037['TOTAL'].sum() + \
           aw46_040['TOTAL'].sum() + aw47_045['TOTAL'].sum() + aw48_050['TOTAL'].sum() + aw49_051['TOTAL'].sum()

aw50_999 = abs(aw50_999)

# Crear DataFrame para escribir en Excel
df_aw50_999 = pd.DataFrame({'total': [aw50_999]})
write_to_excel(df_aw50_999, 49, 50, 'total')


## 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 4


In [109]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax36_005 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax36_005, 50, 36)

ax37_007 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax37_007, 50, 37)

ax38_010 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax38_010, 50, 38)

ax39_015 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG_a[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG_a['CUENTA'] == "6112102011") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG_a['COD_RAMO'] == 48)]
write_to_excel(ax39_015, 50, 39, 'TOTAL')

ax40_020 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax40_020, 50, 40)

ax41_025 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax41_025, 50, 41)

ax42_026 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax42_026, 50, 42)

ax43_030 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax43_030, 50, 43)

ax44_035 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax44_035, 50, 44)

ax45_037 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax45_037, 50, 45)

ax46_040 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax46_040, 50, 46)

ax47_045 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax47_045, 50, 47)

ax48_050 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax48_050, 50, 48)

ax49_051 = unidad4_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax49_051, 50, 49)


# Calcular ax50_999
ax50_999 = ax36_005['TOTAL'].sum() + ax37_007['TOTAL'].sum() + ax38_010['TOTAL'].sum() + ax39_015['TOTAL'].sum() + ax40_020['TOTAL'].sum() + \
           ax41_025['TOTAL'].sum() + ax42_026['TOTAL'].sum() + ax43_030['TOTAL'].sum() + ax44_035['TOTAL'].sum() + ax45_037['TOTAL'].sum() + \
           ax46_040['TOTAL'].sum() + ax47_045['TOTAL'].sum() + ax48_050['TOTAL'].sum() + ax49_051['TOTAL'].sum()

ax50_999 = abs(ax50_999)
# Crear DataFrame para escribir en Excel
df_ax50_999 = pd.DataFrame({'total': [ax50_999]})
write_to_excel(df_ax50_999, 50, 50, 'total')  


## 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 4


In [110]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Consultas ajustadas para COD_RAMO 49 y escritura en columna AY (columna 50)
ay36_005 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay36_005, 51, 36)

ay37_007 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay37_007, 51, 37)

ay38_010 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay38_010, 51, 38)

ay39_015 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva para Siniestros No Avisados") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay39_015, 51, 39)

ay40_020 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay40_020, 51, 40)

ay41_025 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay41_025, 51, 41)

ay42_026 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay42_026, 51, 42)

ay43_030 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay43_030, 51, 43)

ay44_035 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay44_035, 51, 44)

ay45_037 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay45_037, 51, 45)

ay46_040 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay46_040, 51, 46)

ay47_045 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay47_045, 51, 47)

ay48_050 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay48_050, 51, 48)

ay49_051 = unidad4_CUMPLIMIENTOCAUCIONES[(unidad4_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay49_051, 51, 49)

# Calcular ay50_999
ay50_999 = ay36_005['TOTAL'].sum() + ay37_007['TOTAL'].sum() + ay38_010['TOTAL'].sum() + ay39_015['TOTAL'].sum() + ay40_020['TOTAL'].sum() + \
           ay41_025['TOTAL'].sum() + ay42_026['TOTAL'].sum() + ay43_030['TOTAL'].sum() + ay44_035['TOTAL'].sum() + ay45_037['TOTAL'].sum() + \
           ay46_040['TOTAL'].sum() + ay47_045['TOTAL'].sum() + ay48_050['TOTAL'].sum() + ay49_051['TOTAL'].sum()

ay50_999 = abs(ay50_999)

# Crear DataFrame para escribir en Excel
df_ay50_999 = pd.DataFrame({'total': [ay50_999]})
write_to_excel(df_ay50_999, 51, 50, 'total')


## 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 4


In [111]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Consultas ajustadas para COD_RAMO 50 y escritura en columna AZ (columna 52)
az36_005 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az36_005, 52, 36)

az37_007 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az37_007, 52, 37)

az38_010 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az38_010, 52, 38)

az39_015 = unidad4_CUMPLIMIENTOARRENDAMIENTO_a[(unidad4_CUMPLIMIENTOARRENDAMIENTO_a['CUENTA'] == "6112102011") & (unidad4_CUMPLIMIENTOARRENDAMIENTO_a['COD_RAMO'] == 50)]
write_to_excel(az39_015, 52, 39, 'TOTAL')

az40_020 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az40_020, 52, 40)

az41_025 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az41_025, 52, 41)

az42_026 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az42_026, 52, 42)

az43_030 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az43_030, 52, 43)

az44_035 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az44_035, 52, 44)

az45_037 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az45_037, 52, 45)

az46_040 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az46_040, 52, 46)

az47_045 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az47_045, 52, 47)

az48_050 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az48_050, 52, 48)

az49_051 = unidad4_CUMPLIMIENTOARRENDAMIENTO[(unidad4_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az49_051, 52, 49)

# Calcular az50_999
az50_999 = az36_005['TOTAL'].sum() + az37_007['TOTAL'].sum() + az38_010['TOTAL'].sum() + az39_015['TOTAL'].sum() + az40_020['TOTAL'].sum() + \
           az41_025['TOTAL'].sum() + az42_026['TOTAL'].sum() + az43_030['TOTAL'].sum() + az44_035['TOTAL'].sum() + az45_037['TOTAL'].sum() + \
           az46_040['TOTAL'].sum() + az47_045['TOTAL'].sum() + az48_050['TOTAL'].sum() + az49_051['TOTAL'].sum()

az50_999 = abs(az50_999)

# Crear DataFrame para escribir en Excel
df_az50_999 = pd.DataFrame({'total': [az50_999]})
write_to_excel(df_az50_999, 52, 50, 'total')


## 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 4


In [112]:
# Agrupar los datos según el concepto y el ramo
unidad4_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad4_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba36_005 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba36_005, 53, 36)

ba37_007 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva Matemática BEPs") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba37_007, 53, 37)

ba38_010 = unidad4_CUMPLIMIENTOPARTICULARES_a[(unidad4_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6112201011") & (unidad4_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba38_010, 53, 38, 'TOTAL')

ba39_015 = unidad4_CUMPLIMIENTOPARTICULARES_a[(unidad4_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6112102011") & (unidad4_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba39_015, 53, 39, 'TOTAL')

ba40_020 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba40_020, 53, 40)

ba41_025 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reservas especiales") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba41_025, 53, 41)

ba42_026 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba42_026, 53, 42)

ba43_030 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva matemática Riesgos Laborales (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba43_030, 53, 43)

ba44_035 = unidad4_CUMPLIMIENTOPARTICULARES_a[(unidad4_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6112101051") & (unidad4_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba44_035, 53, 44, 'TOTAL')

ba45_037 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba45_037, 53, 45)

ba46_040 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba46_040, 53, 46)

ba47_045 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba47_045, 53, 47)

ba48_050 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reservas especiales (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba48_050, 53, 48)

ba49_051 = unidad4_CUMPLIMIENTOPARTICULARES[(unidad4_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") & (unidad4_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba49_051, 53, 49)


# Sumar los valores de ba50_999
ba50_999 = ba36_005['TOTAL'].sum() + ba37_007['TOTAL'].sum() + ba38_010['TOTAL'].sum() + ba39_015['TOTAL'].sum() + ba40_020['TOTAL'].sum() + \
           ba41_025['TOTAL'].sum() + ba42_026['TOTAL'].sum() + ba43_030['TOTAL'].sum() + ba44_035['TOTAL'].sum() + ba45_037['TOTAL'].sum() + \
           ba46_040['TOTAL'].sum() + ba47_045['TOTAL'].sum() + ba48_050['TOTAL'].sum() + ba49_051['TOTAL'].sum()

ba50_999 = abs(ba50_999)

# Crear DataFrame para escribir en Excel
df_ba50_999 = pd.DataFrame({'total': [ba50_999]})
write_to_excel(df_ba50_999, 53, 50, 'total')

## CONSULTA DE TOTALES LIBERACIÓN Y CONSTITUCION DE RESERVAS SINIESTROS UNIDAD 4


In [113]:
# Sumar los valores de E36 a AT36, considerando celdas vacías como 0

c36_005 = sum([ws.cell(row=36, column=col).value if ws.cell(row=36, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=36, column=3).value = c36_005  # Escribir en fila 36, columna 3

c37_007 = sum([ws.cell(row=37, column=col).value if ws.cell(row=37, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=37, column=3).value = c37_007  # Escribir en fila 37, columna 3

c38_010 = sum([ws.cell(row=38, column=col).value if ws.cell(row=38, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=38, column=3).value = c38_010  # Escribir en fila 38, columna 3

c39_015 = sum([ws.cell(row=39, column=col).value if ws.cell(row=39, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=39, column=3).value = c39_015  # Escribir en fila 39, columna 3

c40_020 = sum([ws.cell(row=40, column=col).value if ws.cell(row=40, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=40, column=3).value = c40_020  # Escribir en fila 40, columna 3

c41_025 = sum([ws.cell(row=41, column=col).value if ws.cell(row=41, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=41, column=3).value = c41_025  # Escribir en fila 41, columna 3

c42_026 = sum([ws.cell(row=42, column=col).value if ws.cell(row=42, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=42, column=3).value = c42_026  # Escribir en fila 42, columna 3

c43_030 = sum([ws.cell(row=43, column=col).value if ws.cell(row=43, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=43, column=3).value = c43_030  # Escribir en fila 43, columna 3

c44_035 = sum([ws.cell(row=44, column=col).value if ws.cell(row=44, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=44, column=3).value = c44_035  # Escribir en fila 44, columna 3

c45_037 = sum([ws.cell(row=45, column=col).value if ws.cell(row=45, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=45, column=3).value = c45_037  # Escribir en fila 45, columna 3

c46_040 = sum([ws.cell(row=46, column=col).value if ws.cell(row=46, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=46, column=3).value = c46_040  # Escribir en fila 46, columna 3

c47_045 = sum([ws.cell(row=47, column=col).value if ws.cell(row=47, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=47, column=3).value = c47_045  # Escribir en fila 47, columna 3

c48_050 = sum([ws.cell(row=48, column=col).value if ws.cell(row=48, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=48, column=3).value = c48_050  # Escribir en fila 48, columna 3

c49_051 = sum([ws.cell(row=49, column=col).value if ws.cell(row=49, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=49, column=3).value = c49_051  # Escribir en fila 49, columna 3

# Calcular el total general según la fórmula dada
c50_999 = (c36_005 + c37_007 + c38_010 + c39_015 + c40_020 + c41_025 + c42_026 - c43_030 - c44_035 - c45_037 - c46_040 - c47_045 - c48_050 - c49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=3).value = c50_999

## CONSULTA DE SUBTOTAL - RAMOS LIBERACIÓN Y CONSTITUCION DE RESERVAS SINIESTROS UNIDAD 4

In [114]:
# Sumar los valores de E36 a AT36, considerando celdas vacías como 0 para la columna D

d36_005 = sum([ws.cell(row=36, column=col).value if ws.cell(row=36, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=36, column=4).value = d36_005  # Escribir en fila 36, columna 4

d37_007 = sum([ws.cell(row=37, column=col).value if ws.cell(row=37, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=37, column=4).value = d37_007  # Escribir en fila 37, columna 4

d38_010 = sum([ws.cell(row=38, column=col).value if ws.cell(row=38, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=38, column=4).value = d38_010  # Escribir en fila 38, columna 4

d39_015 = sum([ws.cell(row=39, column=col).value if ws.cell(row=39, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=39, column=4).value = d39_015  # Escribir en fila 39, columna 4

d40_020 = sum([ws.cell(row=40, column=col).value if ws.cell(row=40, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=40, column=4).value = d40_020  # Escribir en fila 40, columna 4

d41_025 = sum([ws.cell(row=41, column=col).value if ws.cell(row=41, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=41, column=4).value = d41_025  # Escribir en fila 41, columna 4

d42_026 = sum([ws.cell(row=42, column=col).value if ws.cell(row=42, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=42, column=4).value = d42_026  # Escribir en fila 42, columna 4

d43_030 = sum([ws.cell(row=43, column=col).value if ws.cell(row=43, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=43, column=4).value = d43_030  # Escribir en fila 43, columna 4

d44_035 = sum([ws.cell(row=44, column=col).value if ws.cell(row=44, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=44, column=4).value = d44_035  # Escribir en fila 44, columna 4

d45_037 = sum([ws.cell(row=45, column=col).value if ws.cell(row=45, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=45, column=4).value = d45_037  # Escribir en fila 45, columna 4

d46_040 = sum([ws.cell(row=46, column=col).value if ws.cell(row=46, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=46, column=4).value = d46_040  # Escribir en fila 46, columna 4

d47_045 = sum([ws.cell(row=47, column=col).value if ws.cell(row=47, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=47, column=4).value = d47_045  # Escribir en fila 47, columna 4

d48_050 = sum([ws.cell(row=48, column=col).value if ws.cell(row=48, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=48, column=4).value = d48_050  # Escribir en fila 48, columna 4

d49_051 = sum([ws.cell(row=49, column=col).value if ws.cell(row=49, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=49, column=4).value = d49_051  # Escribir en fila 49, columna 4

# Calcular el total general según la fórmula dada
d50_999 = (d36_005 + d37_007 + d38_010 + d39_015 + d40_020 + d41_025 + d42_026 - d43_030 - d44_035 - d45_037 - d46_040 - d47_045 - d48_050 - d49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=4).value = d50_999  # Escribir en fila 50, columna 4

## UNIDAD DE CAPTURA 5

#  3   AUTOMOVILES UNIDAD 5


In [115]:
unidad5_AUTOMOVILES = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_AUTOMOVILES['TOTAL'] = unidad5_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 3, 52),
    ("Siniestros de aceptaciones reaseguro interior", 3, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 3, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 3, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 3, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e52_005 = get_value(unidad5_AUTOMOVILES, "Siniestros liquidados", 3)
e53_010 = get_value(unidad5_AUTOMOVILES, "Siniestros de aceptaciones reaseguro interior", 3)
e54_015 = get_value(unidad5_AUTOMOVILES, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 3)
e55_020 = get_value(unidad5_AUTOMOVILES, "Siniestros de aceptaciones reaseguro exterior", 3)
e56_025 = get_value(unidad5_AUTOMOVILES, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 3)


e57_999 = (e52_005 + e53_010 + e54_015 + e55_020 + e56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=5, value=e57_999)

<Cell 'Hoja1'.E57>

#  4   SOAT UNIDAD 5


In [116]:
unidad5_SOAT = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_SOAT['TOTAL'] = unidad5_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 4, 52),
    ("Siniestros de aceptaciones reaseguro interior", 4, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 4, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 4, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 4, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f52_005 = get_value(unidad5_SOAT, "Siniestros liquidados", 4)
f53_010 = get_value(unidad5_SOAT, "Siniestros de aceptaciones reaseguro interior", 4)
f54_015 = get_value(unidad5_SOAT, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 4)
f55_020 = get_value(unidad5_SOAT, "Siniestros de aceptaciones reaseguro exterior", 4)
f56_025 = get_value(unidad5_SOAT, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 4)


f57_999 = (f52_005 + f53_010 + f54_015 + f55_020 + f56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=6, value=f57_999)

<Cell 'Hoja1'.F57>

#  5   CUMPLIMIENTO UNIDAD 5


In [117]:
unidad5_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_CUMPLIMIENTO['TOTAL'] = unidad5_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 5, 52),
    ("Siniestros de aceptaciones reaseguro interior", 5, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 5, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 5, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 5, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g52_005 = get_value(unidad5_CUMPLIMIENTO, "Siniestros liquidados", 5)
g53_010 = get_value(unidad5_CUMPLIMIENTO, "Siniestros de aceptaciones reaseguro interior", 5)
g54_015 = get_value(unidad5_CUMPLIMIENTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 5)
g55_020 = get_value(unidad5_CUMPLIMIENTO, "Siniestros de aceptaciones reaseguro exterior", 5)
g56_025 = get_value(unidad5_CUMPLIMIENTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 5)


g57_999 = (g52_005 + g53_010 + g54_015 + g55_020 + g56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=7, value=g57_999)

<Cell 'Hoja1'.G57>

#  6   RESPONSABILIDAD CIVIL UNIDAD 5

In [118]:
unidad5_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_RESPONSABILIDADCI['TOTAL'] = unidad5_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 6, 52),
    ("Siniestros de aceptaciones reaseguro interior", 6, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 6, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 6, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 6, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h52_005 = get_value(unidad5_RESPONSABILIDADCI, "Siniestros liquidados", 6)
h53_010 = get_value(unidad5_RESPONSABILIDADCI, "Siniestros de aceptaciones reaseguro interior", 6)
h54_015 = get_value(unidad5_RESPONSABILIDADCI, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 6)
h55_020 = get_value(unidad5_RESPONSABILIDADCI, "Siniestros de aceptaciones reaseguro exterior", 6)
h56_025 = get_value(unidad5_CUMPLIMIENTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 6)


h57_999 = (h52_005 + h53_010 + h54_015 + h55_020 + h56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=8, value=h57_999)

<Cell 'Hoja1'.H57>

#  7   INCENDIO UNIDAD 5

In [119]:
unidad5_INCENDIO = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_INCENDIO['TOTAL'] = unidad5_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 7, 52),
    ("Siniestros de aceptaciones reaseguro interior", 7, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 7, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 7, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 7, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i52_005 = get_value(unidad5_INCENDIO, "Siniestros liquidados", 7)
i53_010 = get_value(unidad5_INCENDIO, "Siniestros de aceptaciones reaseguro interior", 7)
i54_015 = get_value(unidad5_INCENDIO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 7)
i55_020 = get_value(unidad5_INCENDIO, "Siniestros de aceptaciones reaseguro exterior", 7)
i56_025 = get_value(unidad5_INCENDIO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 7)


i57_999 = (i52_005 + i53_010 + i54_015 + i55_020 + i56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=9, value=i57_999)

<Cell 'Hoja1'.I57>

#  8   TERREMOTO UNIDAD 5

In [120]:
unidad5_TERREMOTO = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_TERREMOTO['TOTAL'] = unidad5_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 8, 52),
    ("Siniestros de aceptaciones reaseguro interior", 8, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 8, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 8, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 8, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j52_005 = get_value(unidad5_TERREMOTO, "Siniestros liquidados", 8)
j53_010 = get_value(unidad5_TERREMOTO, "Siniestros de aceptaciones reaseguro interior", 8)
j54_015 = get_value(unidad5_TERREMOTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 8)
j55_020 = get_value(unidad5_TERREMOTO, "Siniestros de aceptaciones reaseguro exterior", 8)
j56_025 = get_value(unidad5_TERREMOTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 8)


j57_999 = (j52_005 + j53_010 + j54_015 + j55_020 + j56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=10, value=j57_999)

<Cell 'Hoja1'.J57>

#  9   SUSTRACCION UNIDAD 5

In [121]:
unidad5_SUSTRACCION = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_SUSTRACCION['TOTAL'] = unidad5_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 9, 52),
    ("Siniestros de aceptaciones reaseguro interior", 9, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 9, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 9, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 9, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k52_005 = get_value(unidad5_SUSTRACCION, "Siniestros liquidados", 9)
k53_010 = get_value(unidad5_SUSTRACCION, "Siniestros de aceptaciones reaseguro interior", 9)
k54_015 = get_value(unidad5_SUSTRACCION, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 9)
k55_020 = get_value(unidad5_SUSTRACCION, "Siniestros de aceptaciones reaseguro exterior", 9)
k56_025 = get_value(unidad5_SUSTRACCION, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 9)


k57_999 = (k52_005 + k53_010 + k54_015 + k55_020 + k56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=11, value=k57_999)

<Cell 'Hoja1'.K57>

#  10  TRANSPORTE UNIDAD 5

In [122]:
unidad5_TRANSPORTE = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_TRANSPORTE['TOTAL'] = unidad5_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 10, 52),
    ("Siniestros de aceptaciones reaseguro interior", 10, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 10, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 10, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 10, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l52_005 = get_value(unidad5_TRANSPORTE, "Siniestros liquidados", 10)
l53_010 = get_value(unidad5_TRANSPORTE, "Siniestros de aceptaciones reaseguro interior", 10)
l54_015 = get_value(unidad5_TRANSPORTE, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 10)
l55_020 = get_value(unidad5_TRANSPORTE, "Siniestros de aceptaciones reaseguro exterior", 10)
l56_025 = get_value(unidad5_TRANSPORTE, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 10)


l57_999 = (l52_005 + l53_010 + l54_015 + l55_020 + l56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=12, value=l57_999)

<Cell 'Hoja1'.L57>

#  11  CORRIENTE DÉBIL UNIDAD 5

In [123]:
unidad5_CORRIENDEBIL = bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_CORRIENDEBIL['TOTAL'] = unidad5_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 11, 52),
    ("Siniestros de aceptaciones reaseguro interior", 11, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 11, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 11, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 11, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m52_005 = get_value(unidad5_CORRIENDEBIL, "Siniestros liquidados", 11)
m53_010 = get_value(unidad5_CORRIENDEBIL, "Siniestros de aceptaciones reaseguro interior", 11)
m54_015 = get_value(unidad5_CORRIENDEBIL, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 11)
m55_020 = get_value(unidad5_CORRIENDEBIL, "Siniestros de aceptaciones reaseguro exterior", 11)
m56_025 = get_value(unidad5_CORRIENDEBIL, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 11)


m57_999 = (m52_005 + m53_010 + m54_015 + m55_020 + m56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=13, value=m57_999)

<Cell 'Hoja1'.M57>

#  12  TODO RIESGO CONTRATISTA UNIDAD 5

In [124]:
unidad5_TODORIESGOCONTRATISTA= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_TODORIESGOCONTRATISTA['TOTAL'] = unidad5_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 12, 52),
    ("Siniestros de aceptaciones reaseguro interior", 12, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 12, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 12, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 12, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n52_005 = get_value(unidad5_TODORIESGOCONTRATISTA, "Siniestros liquidados", 12)
n53_010 = get_value(unidad5_TODORIESGOCONTRATISTA, "Siniestros de aceptaciones reaseguro interior", 12)
n54_015 = get_value(unidad5_TODORIESGOCONTRATISTA, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 12)
n55_020 = get_value(unidad5_TODORIESGOCONTRATISTA, "Siniestros de aceptaciones reaseguro exterior", 12)
n56_025 = get_value(unidad5_TODORIESGOCONTRATISTA, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 12)


n57_999 = (n52_005 + n53_010 + n54_015 + n55_020 + n56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=14, value= n57_999)

<Cell 'Hoja1'.N57>

#  13  MANEJO UNIDAD 5

In [125]:
unidad5_MANEJO= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_MANEJO['TOTAL'] = unidad5_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 13, 52),
    ("Siniestros de aceptaciones reaseguro interior", 13, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 13, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 13, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 13, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o52_005 = get_value(unidad5_MANEJO, "Siniestros liquidados", 13)
o53_010 = get_value(unidad5_MANEJO, "Siniestros de aceptaciones reaseguro interior", 13)
o54_015 = get_value(unidad5_MANEJO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 13)
o55_020 = get_value(unidad5_MANEJO, "Siniestros de aceptaciones reaseguro exterior", 13)
o56_025 = get_value(unidad5_MANEJO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 13)


o57_999 = (o52_005 + o53_010 + o54_015 + o55_020 + o56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=15, value= o57_999)

<Cell 'Hoja1'.O57>

#  14  LUCRO CESANTE UNIDAD 5


In [126]:
unidad5_LUCROCESANTE= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_LUCROCESANTE['TOTAL'] = unidad5_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 14, 52),
    ("Siniestros de aceptaciones reaseguro interior", 14, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 14, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 14, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 14, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p52_005 = get_value(unidad5_LUCROCESANTE, "Siniestros liquidados", 14)
p53_010 = get_value(unidad5_LUCROCESANTE, "Siniestros de aceptaciones reaseguro interior", 14)
p54_015 = get_value(unidad5_LUCROCESANTE, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 14)
p55_020 = get_value(unidad5_LUCROCESANTE, "Siniestros de aceptaciones reaseguro exterior", 14)
p56_025 = get_value(unidad5_LUCROCESANTE, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 14)


p57_999 = (p52_005 + p53_010 + p54_015 + p55_020 + p56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=16, value= p57_999)

<Cell 'Hoja1'.P57>

# 15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 5

In [127]:
unidad5_MONTAJEYROTURA= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_MONTAJEYROTURA['TOTAL'] = unidad5_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 15, 52),
    ("Siniestros de aceptaciones reaseguro interior", 15, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 15, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 15, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 15, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q52_005 = get_value(unidad5_MONTAJEYROTURA, "Siniestros liquidados", 15)
q53_010 = get_value(unidad5_MONTAJEYROTURA, "Siniestros de aceptaciones reaseguro interior", 15)
q54_015 = get_value(unidad5_MONTAJEYROTURA, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 15)
q55_020 = get_value(unidad5_MONTAJEYROTURA, "Siniestros de aceptaciones reaseguro exterior", 15)
q56_025 = get_value(unidad5_MONTAJEYROTURA, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 15)


q57_999 = (q52_005 + q53_010 + q54_015 + q55_020 + q56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=17, value= q57_999)

<Cell 'Hoja1'.Q57>

#  16  AVIACIÓN UNIDAD 5


In [128]:
unidad5_AVIACION= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_AVIACION['TOTAL'] = unidad5_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 16, 52),
    ("Siniestros de aceptaciones reaseguro interior", 16, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 16, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 16, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 16, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r52_005 = get_value(unidad5_AVIACION, "Siniestros liquidados", 16)
r53_010 = get_value(unidad5_AVIACION, "Siniestros de aceptaciones reaseguro interior", 16)
r54_015 = get_value(unidad5_AVIACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 16)
r55_020 = get_value(unidad5_AVIACION, "Siniestros de aceptaciones reaseguro exterior", 16)
r56_025 = get_value(unidad5_AVIACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 16)


r57_999 = (r52_005 + r53_010 + r54_015 + r55_020 + r56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=18, value= r57_999)

<Cell 'Hoja1'.R57>

#  17  NAVEGACIÓN Y CASCO UNIDAD 5

In [129]:
unidad5_NAVEGACION= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_NAVEGACION['TOTAL'] = unidad5_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 17, 52),
    ("Siniestros de aceptaciones reaseguro interior", 17, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 17, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 17, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 17, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s52_005 = get_value(unidad5_NAVEGACION, "Siniestros liquidados", 17)
s53_010 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones reaseguro interior", 17)
s54_015 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 17)
s55_020 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones reaseguro exterior", 17)
s56_025 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 17)


s57_999 = (s52_005 + s53_010 + s54_015 + s55_020 + s56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=19, value= s57_999)

<Cell 'Hoja1'.S57>

#  18  MINAS Y PETRÓLEOS UNIDAD 5

In [130]:
unidad5_MINAS= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_MINAS['TOTAL'] = unidad5_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 18, 52),
    ("Siniestros de aceptaciones reaseguro interior", 18, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 18, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 18, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 18, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t52_005 = get_value(unidad5_NAVEGACION, "Siniestros liquidados", 18)
t53_010 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones reaseguro interior", 18)
t54_015 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 18)
t55_020 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones reaseguro exterior", 18)
t56_025 = get_value(unidad5_NAVEGACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 18)


t57_999 = (t52_005 + t53_010 + t54_015 + t55_020 + t56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=20, value= t57_999)

<Cell 'Hoja1'.T57>

#  19  VIDRIOS UNIDAD 5

In [131]:
unidad5_VIDRIOS= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_VIDRIOS['TOTAL'] = unidad5_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 19, 52),
    ("Siniestros de aceptaciones reaseguro interior", 19, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 19, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 19, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 19, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u52_005 = get_value(unidad5_VIDRIOS, "Siniestros liquidados", 19)
u53_010 = get_value(unidad5_VIDRIOS, "Siniestros de aceptaciones reaseguro interior", 19)
u54_015 = get_value(unidad5_VIDRIOS, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 19)
u55_020 = get_value(unidad5_VIDRIOS, "Siniestros de aceptaciones reaseguro exterior", 19)
u56_025 = get_value(unidad5_VIDRIOS, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 19)


u57_999 = (u52_005 + u53_010 + u54_015 + u55_020 + u56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=21, value= u57_999)

<Cell 'Hoja1'.U57>

#  20  CRÉDITO COMERCIAL UNIDAD 5


In [132]:
unidad5_CREDITOCOMERCIAL= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_CREDITOCOMERCIAL['TOTAL'] = unidad5_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 20, 52),
    ("Siniestros de aceptaciones reaseguro interior", 20, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 20, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 20, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 20, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v52_005 = get_value(unidad5_CREDITOCOMERCIAL, "Siniestros liquidados", 20)
v53_010 = get_value(unidad5_CREDITOCOMERCIAL, "Siniestros de aceptaciones reaseguro interior", 20)
v54_015 = get_value(unidad5_CREDITOCOMERCIAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 20)
v55_020 = get_value(unidad5_CREDITOCOMERCIAL, "Siniestros de aceptaciones reaseguro exterior", 20)
v56_025 = get_value(unidad5_CREDITOCOMERCIAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 20)


v57_999 = (v52_005 + v53_010 + v54_015 + v55_020 + v56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=22, value= v57_999)

<Cell 'Hoja1'.V57>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 5

In [133]:
unidad5_CREDITOEXPORTACION= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_CREDITOEXPORTACION['TOTAL'] = unidad5_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 21, 52),
    ("Siniestros de aceptaciones reaseguro interior", 21, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 21, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 21, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 21, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w52_005 = get_value(unidad5_CREDITOEXPORTACION, "Siniestros liquidados", 21)
w53_010 = get_value(unidad5_CREDITOEXPORTACION, "Siniestros de aceptaciones reaseguro interior", 21)
w54_015 = get_value(unidad5_CREDITOEXPORTACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 21)
w55_020 = get_value(unidad5_CREDITOEXPORTACION, "Siniestros de aceptaciones reaseguro exterior", 21)
w56_025 = get_value(unidad5_CREDITOEXPORTACION, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 21)


w57_999 = (w52_005 + w53_010 + w54_015 + w55_020 + w56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=23, value= w57_999)

<Cell 'Hoja1'.W57>

#  24  DESEMPLEO UNIDAD 5

In [134]:
unidad5_DESEMPLEO= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_DESEMPLEO['TOTAL'] = unidad5_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 24, 52),
    ("Siniestros de aceptaciones reaseguro interior", 24, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 24, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 24, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 24, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z52_005 = get_value(unidad5_DESEMPLEO, "Siniestros liquidados", 24)
z53_010 = get_value(unidad5_DESEMPLEO, "Siniestros de aceptaciones reaseguro interior", 24)
z54_015 = get_value(unidad5_DESEMPLEO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 24)
z55_020 = get_value(unidad5_DESEMPLEO, "Siniestros de aceptaciones reaseguro exterior", 24)
z56_025 = get_value(unidad5_DESEMPLEO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 24)


z57_999 = (z52_005 + z53_010 + z54_015 + z55_020 + z56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=26, value= z57_999)

<Cell 'Hoja1'.Z57>

#  25  HOGAR UNIDAD 5

In [135]:
unidad5_HOGAR= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_HOGAR['TOTAL'] = unidad5_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 25, 52),
    ("Siniestros de aceptaciones reaseguro interior", 25, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 25, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 25, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 25, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa52_005 = get_value(unidad5_HOGAR, "Siniestros liquidados", 25)
aa53_010 = get_value(unidad5_HOGAR, "Siniestros de aceptaciones reaseguro interior", 25)
aa54_015 = get_value(unidad5_HOGAR, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 25)
aa55_020 = get_value(unidad5_HOGAR, "Siniestros de aceptaciones reaseguro exterior", 25)
aa56_025 = get_value(unidad5_HOGAR, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 25)


aa57_999 = (aa52_005 + aa53_010 + aa54_015 + aa55_020 + aa56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=27, value= aa57_999)

<Cell 'Hoja1'.AA57>

# 42 AGROPECUARIO UNIDAD 5

In [136]:
unidad5_AGROPECUARIO= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_AGROPECUARIO['TOTAL'] = unidad5_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 42, 52),
    ("Siniestros de aceptaciones reaseguro interior", 42, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 42, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 42, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 42, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar52_005 = get_value(unidad5_AGROPECUARIO, "Siniestros liquidados", 42)
ar53_010 = get_value(unidad5_AGROPECUARIO, "Siniestros de aceptaciones reaseguro interior", 42)
ar54_015 = get_value(unidad5_AGROPECUARIO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 42)
ar55_020 = get_value(unidad5_AGROPECUARIO, "Siniestros de aceptaciones reaseguro exterior", 42)
ar56_025 = get_value(unidad5_AGROPECUARIO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 42)


ar57_999 = (ar52_005 + ar53_010 + ar54_015 + ar55_020 + ar56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=44, value= ar57_999)

<Cell 'Hoja1'.AR57>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 5

In [137]:
unidad5_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_OPERNOASIGNABLES['TOTAL'] = unidad5_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 43, 52),
    ("Siniestros de aceptaciones reaseguro interior", 43, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 43, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 43, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 43, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as52_005 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros liquidados", 43)
as53_010 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones reaseguro interior", 43)
as54_015 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 43)
as55_020 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones reaseguro exterior", 43)
as56_025 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 43)


as57_999 = (as52_005 + as53_010 + as54_015 + as55_020 + as56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=45, value= as57_999)

<Cell 'Hoja1'.AS57>

# 44 DECENAL UNIDAD 5

In [138]:
unidad5_DECENAL=bd[(bd['UNIDAD'] == 5) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_DECENAL['TOTAL'] = unidad5_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 44, 52),
    ("Siniestros de aceptaciones reaseguro interior", 44, 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 44, 54),
    ("Siniestros de aceptaciones reaseguro exterior", 44, 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 44, 56),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad5_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at52_005 = get_value(unidad5_DECENAL, "Siniestros liquidados", 44)
at53_010 = get_value(unidad5_DECENAL, "Siniestros de aceptaciones reaseguro interior", 44)
at54_015 = get_value(unidad5_DECENAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 44)
at55_020 = get_value(unidad5_DECENAL, "Siniestros de aceptaciones reaseguro exterior", 44)
at56_025 = get_value(unidad5_DECENAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 44)


at57_999 = (at52_005 + at53_010 + at54_015 + at55_020 + at56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=46, value= at57_999)

<Cell 'Hoja1'.AT57>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 5

In [139]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos
au52_005 = unidad5_CUMPLIMIENTOESTATALES_a[(unidad5_CUMPLIMIENTOESTATALES_a['CUENTA'] == "6111201091") & (unidad5_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au52_005, 47, 52, 'TOTAL')

au53_010 = unidad5_CUMPLIMIENTOESTATALES[(unidad5_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au53_010, 47, 53)

au54_015 = unidad5_CUMPLIMIENTOESTATALES[(unidad5_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au54_015, 47, 54)

au55_020 = unidad5_CUMPLIMIENTOESTATALES[(unidad5_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au55_020, 47, 55)

au56_025 = unidad5_CUMPLIMIENTOESTATALES[(unidad5_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au56_025, 47, 56)

# Calcular el total general solo con las variables seleccionadas
au57_999 = au52_005['TOTAL'].sum() + au53_010['TOTAL'].sum() + au54_015['TOTAL'].sum() + au55_020['TOTAL'].sum() + au56_025['TOTAL'].sum()

au57_999 = abs(au57_999)
# Crear DataFrame para escribir en Excel
df_au57_999 = pd.DataFrame({'total': [au57_999]})
write_to_excel(df_au57_999, 47, 57, 'total')


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 5

In [140]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos
av52_005 = unidad5_CUMPLIMIENTOESTATALES_a[(unidad5_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "6111201091") & (unidad5_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av52_005, 48, 52, 'TOTAL')

av53_010 = unidad5_CUMPLIMIENTOEMPRESASSERV[(unidad5_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av53_010, 48, 53)

av54_015 = unidad5_CUMPLIMIENTOEMPRESASSERV[(unidad5_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av54_015, 48, 54)

av55_020 = unidad5_CUMPLIMIENTOEMPRESASSERV[(unidad5_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av55_020, 48, 55)

av56_025 = unidad5_CUMPLIMIENTOEMPRESASSERV[(unidad5_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av56_025, 48, 56)

# Calcular el total general solo con las variables seleccionadas
av57_999 = av52_005['TOTAL'].sum() + av53_010['TOTAL'].sum() + av54_015['TOTAL'].sum() + av55_020['TOTAL'].sum() + av56_025['TOTAL'].sum()

av57_999 = abs(av57_999)
# Crear DataFrame para escribir en Excel
df_av57_999 = pd.DataFrame({'total': [av57_999]})
write_to_excel(df_av57_999, 48, 57, 'total')


# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 5

In [141]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos

aw52_005 = unidad5_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Siniestros liquidados") & (unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw52_005, 49, 52)

aw53_010 = unidad5_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw53_010, 49, 53)

aw54_015 = unidad5_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw54_015, 49, 54)

aw55_020 = unidad5_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw55_020, 49, 55)

aw56_025 = unidad5_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw56_025, 49, 56)

# Calcular el total general solo con las variables seleccionadas
aw57_999 = aw52_005['TOTAL'].sum() + aw53_010['TOTAL'].sum() + aw54_015['TOTAL'].sum() + aw55_020['TOTAL'].sum() + aw56_025['TOTAL'].sum()

aw57_999 = abs(aw57_999)
# Crear DataFrame para escribir en Excel
df_aw57_999 = pd.DataFrame({'total': [aw57_999]})
write_to_excel(df_aw57_999, 49, 57, 'total')


# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 5

In [142]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos

ax52_005 = unidad5_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Siniestros liquidados") & (unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax52_005, 50, 52)

ax53_010 = unidad5_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax53_010, 50, 53)

ax54_015 = unidad5_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax54_015, 50, 54)

ax55_020 = unidad5_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax55_020, 50, 55)

ax56_025 = unidad5_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax56_025, 50, 56)

# Calcular el total general solo con las variables seleccionadas
ax57_999 = ax52_005['TOTAL'].sum() + ax53_010['TOTAL'].sum() + ax54_015['TOTAL'].sum() + ax55_020['TOTAL'].sum() + ax56_025['TOTAL'].sum()

ax57_999 = abs(ax57_999)
# Crear DataFrame para escribir en Excel
df_ax57_999 = pd.DataFrame({'total': [ax57_999]})
write_to_excel(df_ax57_999, 50, 57, 'total')


# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 5

In [143]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos

ay52_005 = unidad5_CUMPLIMIENTOCAUCIONES[(unidad5_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Siniestros liquidados") & (unidad5_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay52_005, 51, 52)

ay53_010 = unidad5_CUMPLIMIENTOCAUCIONES[(unidad5_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay53_010, 51, 53)

ay54_015 = unidad5_CUMPLIMIENTOCAUCIONES[(unidad5_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay54_015, 51, 54)

ay55_020 = unidad5_CUMPLIMIENTOCAUCIONES[(unidad5_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay55_020, 51, 55)

ay56_025 = unidad5_CUMPLIMIENTOCAUCIONES[(unidad5_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay56_025, 51, 56)

# Calcular el total general solo con las variables seleccionadas
ay57_999 = ay52_005['TOTAL'].sum() + ay53_010['TOTAL'].sum() + ay54_015['TOTAL'].sum() + ay55_020['TOTAL'].sum() + ay56_025['TOTAL'].sum()

ay57_999 = abs(ay57_999)
# Crear DataFrame para escribir en Excel
df_ay57_999 = pd.DataFrame({'total': [ay57_999]})
write_to_excel(df_ay57_999, 51, 57, 'total')


# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 5

In [144]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos

az52_005 = unidad5_CUMPLIMIENTOARRENDAMIENTO[(unidad5_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Siniestros liquidados") & (unidad5_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az52_005, 52, 52)

az53_010 = unidad5_CUMPLIMIENTOARRENDAMIENTO[(unidad5_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az53_010, 52, 53)

az54_015 = unidad5_CUMPLIMIENTOARRENDAMIENTO[(unidad5_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az54_015, 52, 54)

az55_020 = unidad5_CUMPLIMIENTOARRENDAMIENTO[(unidad5_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az55_020, 52, 55)

az56_025 = unidad5_CUMPLIMIENTOARRENDAMIENTO[(unidad5_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az56_025, 52, 56)

# Calcular el total general solo con las variables seleccionadas
az57_999 = az52_005['TOTAL'].sum() + az53_010['TOTAL'].sum() + az54_015['TOTAL'].sum() + az55_020['TOTAL'].sum() + az56_025['TOTAL'].sum()

az57_999 = abs(az57_999)
# Crear DataFrame para escribir en Excel
df_az57_999 = pd.DataFrame({'total': [az57_999]})
write_to_excel(df_az57_999, 52, 57, 'total')


# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 5

In [145]:
# Agrupar los datos según el concepto y el ramo
unidad5_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad5_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Obtener datos específicos

ba52_005 = unidad5_CUMPLIMIENTOPARTICULARES_a[(unidad5_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6111201091") & (unidad5_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba52_005, 53, 52, 'TOTAL')

ba53_010 = unidad5_CUMPLIMIENTOPARTICULARES[(unidad5_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro interior") & (unidad5_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba53_010, 53, 53)

ba54_015 = unidad5_CUMPLIMIENTOPARTICULARES[(unidad5_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros interior") & (unidad5_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba54_015, 53, 54)

ba55_020 = unidad5_CUMPLIMIENTOPARTICULARES[(unidad5_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Siniestros de aceptaciones reaseguro exterior") & (unidad5_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba55_020, 53, 55)

ba56_025 = unidad5_CUMPLIMIENTOPARTICULARES[(unidad5_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior") & (unidad5_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba56_025, 53, 56)

# Calcular el total general solo con las variables seleccionadas
ba57_999 = ba52_005['TOTAL'].sum() + ba53_010['TOTAL'].sum() + ba54_015['TOTAL'].sum() + ba55_020['TOTAL'].sum() + ba56_025['TOTAL'].sum()

ba57_999 = abs(ba57_999)
# Crear DataFrame para escribir en Excel
df_ba57_999 = pd.DataFrame({'total': [ba57_999]})
write_to_excel(df_ba57_999, 53, 57, 'total')

#  1   TOTAL UNIDAD 5

In [146]:
c52_005 = sum([ws.cell(row=52, column=col).value if ws.cell(row=52, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=52, column=3).value = c52_005  # Escribir en fila 36, columna 3

c53_010 = sum([ws.cell(row=53, column=col).value if ws.cell(row=53, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=53, column=3).value = c53_010  # Escribir en fila 37, columna 3

c54_015 = sum([ws.cell(row=54, column=col).value if ws.cell(row=54, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=54, column=3).value = c54_015  # Escribir en fila 38, columna 3

c55_020 = sum([ws.cell(row=55, column=col).value if ws.cell(row=55, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=55, column=3).value = c55_020  # Escribir en fila 39, columna 3

c56_025 = sum([ws.cell(row=56, column=col).value if ws.cell(row=56, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=56, column=3).value = c56_025  # Escribir en fila 40, columna 3

# Calcular el total general según la fórmula dada
c57_999 = (c52_005 + c53_010 + c54_015 + c55_020 + c56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=3).value = c57_999

#  2   SUBTOTAL - RAMOS UNIDAD 5

In [147]:
d52_005 = sum([ws.cell(row=52, column=col).value if ws.cell(row=52, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=52, column=4).value = d52_005  # Escribir en fila 36, columna 4

d53_010 = sum([ws.cell(row=53, column=col).value if ws.cell(row=53, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=53, column=4).value = d53_010  # Escribir en fila 37, columna 4

d54_015 = sum([ws.cell(row=54, column=col).value if ws.cell(row=54, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=54, column=4).value = d54_015  # Escribir en fila 38, columna 4

d55_020 = sum([ws.cell(row=55, column=col).value if ws.cell(row=55, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=55, column=4).value = d55_020  # Escribir en fila 39, columna 4

d56_025 = sum([ws.cell(row=56, column=col).value if ws.cell(row=56, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=56, column=4).value = d56_025  # Escribir en fila 40, columna 4

# Calcular el total general según la fórmula dada
d57_999 = (d52_005 + d53_010 + d54_015 + d55_020 + d56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=4).value = d57_999

## UNIDAD DE CAPTURA 6

#  3   AUTOMOVILES UNIDAD 6

In [148]:
unidad6_AUTOMOVILES = bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_AUTOMOVILES['TOTAL'] = unidad6_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 3, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 3, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 3, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 3, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 3, 63),
    ("Reembolsos por enfermedad profesional", 3, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e59_005 = get_value(unidad6_AUTOMOVILES, "Reembolso de siniestros sobre cesiones reaseguro interior", 3)
e60_010 = get_value(unidad6_AUTOMOVILES, "Reembolso siniestros contratos no proporcionales reaseguro interior", 3)
e61_015 = get_value(unidad6_AUTOMOVILES, "Reembolso de siniestros sobre cesiones reaseguro exterior", 3)
e62_020 = get_value(unidad6_AUTOMOVILES, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 3)
e63_025 = get_value(unidad6_AUTOMOVILES, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 3)
e64_030 = get_value(unidad6_AUTOMOVILES, "Reembolsos por enfermedad profesional", 3)

e65_999 = (e59_005 + e60_010 + e61_015 + e62_020 + e63_025 + e64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=5, value=e65_999)

<Cell 'Hoja1'.E65>

#  4   SOAT UNIDAD 6


In [149]:
unidad6_SOAT = bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_SOAT['TOTAL'] = unidad6_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 4, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 4, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 4, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 4, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 4, 63),
    ("Reembolsos por enfermedad profesional", 4, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f59_005 = get_value(unidad6_SOAT, "Reembolso de siniestros sobre cesiones reaseguro interior", 4)
f60_010 = get_value(unidad6_SOAT, "Reembolso siniestros contratos no proporcionales reaseguro interior", 4)
f61_015 = get_value(unidad6_SOAT, "Reembolso de siniestros sobre cesiones reaseguro exterior", 4)
f62_020 = get_value(unidad6_SOAT, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 4)
f63_025 = get_value(unidad6_SOAT, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 4)
f64_030 = get_value(unidad6_SOAT, "Reembolsos por enfermedad profesional", 4)

f65_999 = (f59_005 + f60_010 + f61_015 + f62_020 + f63_025 + f64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=6, value=f65_999)

<Cell 'Hoja1'.F65>

#  5   CUMPLIMIENTO UNIDAD 6

In [150]:
unidad6_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_CUMPLIMIENTO['TOTAL'] = unidad6_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 5, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 5, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 5, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 5, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 5, 63),
    ("Reembolsos por enfermedad profesional", 5, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g59_005 = get_value(unidad6_CUMPLIMIENTO, "Reembolso de siniestros sobre cesiones reaseguro interior", 5)
g60_010 = get_value(unidad6_CUMPLIMIENTO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 5)
g61_015 = get_value(unidad6_CUMPLIMIENTO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 5)
g62_020 = get_value(unidad6_CUMPLIMIENTO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 5)
g63_025 = get_value(unidad6_CUMPLIMIENTO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 5)
g64_030 = get_value(unidad6_CUMPLIMIENTO, "Reembolsos por enfermedad profesional", 5)

g65_999 = (g59_005 + g60_010 + g61_015 + g62_020 + g63_025 + g64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=7, value=g65_999)

<Cell 'Hoja1'.G65>

#  6   RESPONSABILIDAD CIVIL UNIDAD 6

In [151]:
unidad6_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_RESPONSABILIDADCI['TOTAL'] = unidad6_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 6, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 6, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 6, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 6, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 6, 63),
    ("Reembolsos por enfermedad profesional", 6, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h59_005 = get_value(unidad6_RESPONSABILIDADCI, "Reembolso de siniestros sobre cesiones reaseguro interior", 6)
h60_010 = get_value(unidad6_RESPONSABILIDADCI, "Reembolso siniestros contratos no proporcionales reaseguro interior", 6)
h61_015 = get_value(unidad6_RESPONSABILIDADCI, "Reembolso de siniestros sobre cesiones reaseguro exterior", 6)
h62_020 = get_value(unidad6_RESPONSABILIDADCI, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 6)
h63_025 = get_value(unidad6_RESPONSABILIDADCI, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 6)
h64_030 = get_value(unidad6_RESPONSABILIDADCI, "Reembolsos por enfermedad profesional", 6)

h65_999 = (h59_005 + h60_010 + h61_015 + h62_020 + h63_025 + h64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=8, value=h65_999)

<Cell 'Hoja1'.H65>

#  7   INCENDIO UNIDAD 6

In [152]:
unidad6_INCENDIO = bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_INCENDIO['TOTAL'] = unidad6_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 7, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 7, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 7, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 7, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 7, 63),
    ("Reembolsos por enfermedad profesional", 7, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i59_005 = get_value(unidad6_INCENDIO, "Reembolso de siniestros sobre cesiones reaseguro interior", 7)
i60_010 = get_value(unidad6_INCENDIO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 7)
i61_015 = get_value(unidad6_INCENDIO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 7)
i62_020 = get_value(unidad6_INCENDIO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 7)
i63_025 = get_value(unidad6_INCENDIO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 7)
i64_030 = get_value(unidad6_INCENDIO, "Reembolsos por enfermedad profesional", 7)

i65_999 = (i59_005 + i60_010 + i61_015 + i62_020 + i63_025 + i64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=9, value=i65_999)

<Cell 'Hoja1'.I65>

#  8   TERREMOTO UNIDAD 6

In [153]:
unidad6_TERREMOTO= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_TERREMOTO['TOTAL'] = unidad6_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 8, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 8, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 8, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 8, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 8, 63),
    ("Reembolsos por enfermedad profesional", 8, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j59_005 = get_value(unidad6_TERREMOTO, "Reembolso de siniestros sobre cesiones reaseguro interior", 8)
j60_010 = get_value(unidad6_TERREMOTO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 8)
j61_015 = get_value(unidad6_TERREMOTO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 8)
j62_020 = get_value(unidad6_TERREMOTO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 8)
j63_025 = get_value(unidad6_TERREMOTO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 8)
j64_030 = get_value(unidad6_TERREMOTO, "Reembolsos por enfermedad profesional", 8)

j65_999 = (j59_005 + j60_010 + j61_015 + j62_020 + j63_025 + j64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=10, value=j65_999)

<Cell 'Hoja1'.J65>

#  9   SUSTRACCION UNIDAD 6

In [154]:
unidad6_SUSTRACCION= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_SUSTRACCION['TOTAL'] = unidad6_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 9, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 9, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 9, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 9, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 9, 63),
    ("Reembolsos por enfermedad profesional", 9, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k59_005 = get_value(unidad6_SUSTRACCION, "Reembolso de siniestros sobre cesiones reaseguro interior", 9)
k60_010 = get_value(unidad6_SUSTRACCION, "Reembolso siniestros contratos no proporcionales reaseguro interior", 9)
k61_015 = get_value(unidad6_SUSTRACCION, "Reembolso de siniestros sobre cesiones reaseguro exterior", 9)
k62_020 = get_value(unidad6_SUSTRACCION, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 9)
k63_025 = get_value(unidad6_SUSTRACCION, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 9)
k64_030 = get_value(unidad6_SUSTRACCION, "Reembolsos por enfermedad profesional", 9)

k65_999 = (k59_005 + k60_010 + k61_015 + k62_020 + k63_025 + k64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=11, value=k65_999)


<Cell 'Hoja1'.K65>

#  10  TRANSPORTE UNIDAD 6

In [155]:
unidad6_TRANSPORTE= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_TRANSPORTE['TOTAL'] = unidad6_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 10, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 10, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 10, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 10, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 10, 63),
    ("Reembolsos por enfermedad profesional", 10, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l59_005 = get_value(unidad6_TRANSPORTE, "Reembolso de siniestros sobre cesiones reaseguro interior", 10)
l60_010 = get_value(unidad6_TRANSPORTE, "Reembolso siniestros contratos no proporcionales reaseguro interior", 10)
l61_015 = get_value(unidad6_TRANSPORTE, "Reembolso de siniestros sobre cesiones reaseguro exterior", 10)
l62_020 = get_value(unidad6_TRANSPORTE, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 10)
l63_025 = get_value(unidad6_TRANSPORTE, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 10)
l64_030 = get_value(unidad6_TRANSPORTE, "Reembolsos por enfermedad profesional", 10)

l65_999 = (l59_005 + l60_010 + l61_015 + l62_020 + l63_025 + l64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=12, value=l65_999)


<Cell 'Hoja1'.L65>

# 11  CORRIENTE DÉBIL UNIDAD 6

In [156]:
unidad6_CORRIENDEBIL= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_CORRIENDEBIL['TOTAL'] = unidad6_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 11, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 11, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 11, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 11, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 11, 63),
    ("Reembolsos por enfermedad profesional", 11, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m59_005 = get_value(unidad6_CORRIENDEBIL, "Reembolso de siniestros sobre cesiones reaseguro interior", 11)
m60_010 = get_value(unidad6_CORRIENDEBIL, "Reembolso siniestros contratos no proporcionales reaseguro interior", 11)
m61_015 = get_value(unidad6_CORRIENDEBIL, "Reembolso de siniestros sobre cesiones reaseguro exterior", 11)
m62_020 = get_value(unidad6_CORRIENDEBIL, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 11)
m63_025 = get_value(unidad6_CORRIENDEBIL, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 11)
m64_030 = get_value(unidad6_CORRIENDEBIL, "Reembolsos por enfermedad profesional", 11)

m65_999 = (m59_005 + m60_010 + m61_015 + m62_020 + m63_025 + m64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=13, value=m65_999)


<Cell 'Hoja1'.M65>

#  12  TODO RIESGO CONTRATISTA UNIDAD 6

In [157]:
unidad6_TODORIESGOCONTRATISTA= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_TODORIESGOCONTRATISTA['TOTAL'] = unidad6_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 12, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 12, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 12, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 12, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 12, 63),
    ("Reembolsos por enfermedad profesional", 12, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n59_005 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolso de siniestros sobre cesiones reaseguro interior", 12)
n60_010 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolso siniestros contratos no proporcionales reaseguro interior", 12)
n61_015 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolso de siniestros sobre cesiones reaseguro exterior", 12)
n62_020 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 12)
n63_025 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 12)
n64_030 = get_value(unidad6_TODORIESGOCONTRATISTA, "Reembolsos por enfermedad profesional", 12)

n65_999 = (n59_005 + n60_010 + n61_015 + n62_020 + n63_025 + n64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=14, value=n65_999)

<Cell 'Hoja1'.N65>

#  13  MANEJO UNIDAD 6

In [158]:
unidad6_MANEJO= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_MANEJO['TOTAL'] = unidad6_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 13, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 13, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 13, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 13, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 13, 63),
    ("Reembolsos por enfermedad profesional", 13, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o59_005 = get_value(unidad6_MANEJO, "Reembolso de siniestros sobre cesiones reaseguro interior", 13)
o60_010 = get_value(unidad6_MANEJO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 13)
o61_015 = get_value(unidad6_MANEJO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 13)
o62_020 = get_value(unidad6_MANEJO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 13)
o63_025 = get_value(unidad6_MANEJO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 13)
o64_030 = get_value(unidad6_MANEJO, "Reembolsos por enfermedad profesional", 13)

o65_999 = (o59_005 + o60_010 + o61_015 + o62_020 + o63_025 + o64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=15, value=o65_999)

<Cell 'Hoja1'.O65>

# 14  LUCRO CESANTE UNIDAD 6

In [159]:
unidad6_LUCROCESANTE= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_LUCROCESANTE['TOTAL'] = unidad6_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 14, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 14, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 14, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 14, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 14, 63),
    ("Reembolsos por enfermedad profesional", 14, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p59_005 = get_value(unidad6_LUCROCESANTE, "Reembolso de siniestros sobre cesiones reaseguro interior", 14)
p60_010 = get_value(unidad6_LUCROCESANTE, "Reembolso siniestros contratos no proporcionales reaseguro interior", 14)
p61_015 = get_value(unidad6_LUCROCESANTE, "Reembolso de siniestros sobre cesiones reaseguro exterior", 14)
p62_020 = get_value(unidad6_LUCROCESANTE, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 14)
p63_025 = get_value(unidad6_LUCROCESANTE, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 14)
p64_030 = get_value(unidad6_LUCROCESANTE, "Reembolsos por enfermedad profesional", 14)

p65_999 = (p59_005 + p60_010 + p61_015 + p62_020 + p63_025 + p64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=16, value=p65_999)

<Cell 'Hoja1'.P65>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 6

In [160]:
unidad6_MONTAJEYROTURA= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_MONTAJEYROTURA['TOTAL'] = unidad6_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 15, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 15, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 15, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 15, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 15, 63),
    ("Reembolsos por enfermedad profesional", 15, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q59_005 = get_value(unidad6_MONTAJEYROTURA, "Reembolso de siniestros sobre cesiones reaseguro interior", 15)
q60_010 = get_value(unidad6_MONTAJEYROTURA, "Reembolso siniestros contratos no proporcionales reaseguro interior", 15)
q61_015 = get_value(unidad6_MONTAJEYROTURA, "Reembolso de siniestros sobre cesiones reaseguro exterior", 15)
q62_020 = get_value(unidad6_MONTAJEYROTURA, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 15)
q63_025 = get_value(unidad6_MONTAJEYROTURA, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 15)
q64_030 = get_value(unidad6_MONTAJEYROTURA, "Reembolsos por enfermedad profesional", 15)

q65_999 = (q59_005 + q60_010 + q61_015 + q62_020 + q63_025 + q64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=17, value=q65_999)

<Cell 'Hoja1'.Q65>

#  16  AVIACIÓN UNIDAD 6

In [161]:
unidad6_AVIACION= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_AVIACION['TOTAL'] = unidad6_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 16, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 16, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 16, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 16, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 16, 63),
    ("Reembolsos por enfermedad profesional", 16, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r59_005 = get_value(unidad6_AVIACION, "Reembolso de siniestros sobre cesiones reaseguro interior", 16)
r60_010 = get_value(unidad6_AVIACION, "Reembolso siniestros contratos no proporcionales reaseguro interior", 16)
r61_015 = get_value(unidad6_AVIACION, "Reembolso de siniestros sobre cesiones reaseguro exterior", 16)
r62_020 = get_value(unidad6_AVIACION, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 16)
r63_025 = get_value(unidad6_AVIACION, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 16)
r64_030 = get_value(unidad6_AVIACION, "Reembolsos por enfermedad profesional", 16)

r65_999 = (r59_005 + r60_010 + r61_015 + r62_020 + r63_025 + r64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=18, value=r65_999)

<Cell 'Hoja1'.R65>

#  17  NAVEGACIÓN Y CASCO UNIDAD 6

In [162]:
unidad6_NAVEGACION= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_NAVEGACION['TOTAL'] = unidad6_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 17, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 17, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 17, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 17, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 17, 63),
    ("Reembolsos por enfermedad profesional", 17, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s59_005 = get_value(unidad6_NAVEGACION, "Reembolso de siniestros sobre cesiones reaseguro interior", 17)
s60_010 = get_value(unidad6_NAVEGACION, "Reembolso siniestros contratos no proporcionales reaseguro interior", 17)
s61_015 = get_value(unidad6_NAVEGACION, "Reembolso de siniestros sobre cesiones reaseguro exterior", 17)
s62_020 = get_value(unidad6_NAVEGACION, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 17)
s63_025 = get_value(unidad6_NAVEGACION, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 17)
s64_030 = get_value(unidad6_NAVEGACION, "Reembolsos por enfermedad profesional", 17)

s65_999 = (s59_005 + s60_010 + s61_015 + s62_020 + s63_025 + s64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=19, value=s65_999)

<Cell 'Hoja1'.S65>

#  18  MINAS Y PETRÓLEOS UNIDAD 6

In [163]:
unidad6_MINAS= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_MINAS['TOTAL'] = unidad6_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 18, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 18, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 18, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 18, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 18, 63),
    ("Reembolsos por enfermedad profesional", 18, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t59_005 = get_value(unidad6_MINAS, "Reembolso de siniestros sobre cesiones reaseguro interior", 18)
t60_010 = get_value(unidad6_MINAS, "Reembolso siniestros contratos no proporcionales reaseguro interior", 18)
t61_015 = get_value(unidad6_MINAS, "Reembolso de siniestros sobre cesiones reaseguro exterior", 18)
t62_020 = get_value(unidad6_MINAS, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 18)
t63_025 = get_value(unidad6_MINAS, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 18)
t64_030 = get_value(unidad6_MINAS, "Reembolsos por enfermedad profesional", 18)

t65_999 = (t59_005 + t60_010 + t61_015 + t62_020 + t63_025 + t64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=20, value=t65_999)

<Cell 'Hoja1'.T65>

#  19  VIDRIOS UNIDAD 6

In [164]:
unidad6_VIDRIOS= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_VIDRIOS['TOTAL'] = unidad6_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 19, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 19, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 19, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 19, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 19, 63),
    ("Reembolsos por enfermedad profesional", 19, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u59_005 = get_value(unidad6_VIDRIOS, "Reembolso de siniestros sobre cesiones reaseguro interior", 19)
u60_010 = get_value(unidad6_VIDRIOS, "Reembolso siniestros contratos no proporcionales reaseguro interior", 19)
u61_015 = get_value(unidad6_VIDRIOS, "Reembolso de siniestros sobre cesiones reaseguro exterior", 19)
u62_020 = get_value(unidad6_VIDRIOS, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 19)
u63_025 = get_value(unidad6_VIDRIOS, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 19)
u64_030 = get_value(unidad6_VIDRIOS, "Reembolsos por enfermedad profesional", 19)

u65_999 = (u59_005 + u60_010 + u61_015 + u62_020 + u63_025 + u64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=21, value=u65_999)

<Cell 'Hoja1'.U65>

#  20  CRÉDITO COMERCIAL UNIDAD 6

In [165]:
unidad6_CREDITOCOMERCIAL= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_CREDITOCOMERCIAL['TOTAL'] = unidad6_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 20, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 20, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 20, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 20, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 20, 63),
    ("Reembolsos por enfermedad profesional", 20, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v59_005 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolso de siniestros sobre cesiones reaseguro interior", 20)
v60_010 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolso siniestros contratos no proporcionales reaseguro interior", 20)
v61_015 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolso de siniestros sobre cesiones reaseguro exterior", 20)
v62_020 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 20)
v63_025 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 20)
v64_030 = get_value(unidad6_CREDITOCOMERCIAL, "Reembolsos por enfermedad profesional", 20)

v65_999 = (v59_005 + v60_010 + v61_015 + v62_020 + v63_025 + v64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=22, value=v65_999)

<Cell 'Hoja1'.V65>

# 21  CRÉDITO A LA EXPORTACIÓN UNIDAD 6

In [166]:
unidad6_CREDITOEXPORTACION= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_CREDITOEXPORTACION['TOTAL'] = unidad6_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 21, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 21, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 21, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 21, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 21, 63),
    ("Reembolsos por enfermedad profesional", 21, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w59_005 = get_value(unidad6_CREDITOEXPORTACION, "Reembolso de siniestros sobre cesiones reaseguro interior", 21)
w60_010 = get_value(unidad6_CREDITOEXPORTACION, "Reembolso siniestros contratos no proporcionales reaseguro interior", 21)
w61_015 = get_value(unidad6_CREDITOEXPORTACION, "Reembolso de siniestros sobre cesiones reaseguro exterior", 21)
w62_020 = get_value(unidad6_CREDITOEXPORTACION, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 21)
w63_025 = get_value(unidad6_CREDITOEXPORTACION, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 21)
w64_030 = get_value(unidad6_CREDITOEXPORTACION, "Reembolsos por enfermedad profesional", 21)

w65_999 = (w59_005 + w60_010 + w61_015 + w62_020 + w63_025 + w64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=23, value=w65_999)

<Cell 'Hoja1'.W65>

#  24  DESEMPLEO UNIDAD 6

In [167]:
unidad6_DESEMPLEO= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_DESEMPLEO['TOTAL'] = unidad6_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 24, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 24, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 24, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 24, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 24, 63),
    ("Reembolsos por enfermedad profesional", 24, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z59_005 = get_value(unidad6_DESEMPLEO, "Reembolso de siniestros sobre cesiones reaseguro interior", 24)
z60_010 = get_value(unidad6_DESEMPLEO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 24)
z61_015 = get_value(unidad6_DESEMPLEO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 24)
z62_020 = get_value(unidad6_DESEMPLEO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 24)
z63_025 = get_value(unidad6_DESEMPLEO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 24)
z64_030 = get_value(unidad6_DESEMPLEO, "Reembolsos por enfermedad profesional", 24)

z65_999 = (z59_005 + z60_010 + z61_015 + z62_020 + z63_025 + z64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=26, value=z65_999)

<Cell 'Hoja1'.Z65>

#  25  HOGAR UNIDAD 6

In [168]:
unidad6_HOGAR= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_HOGAR['TOTAL'] = unidad6_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 25, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 25, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 25, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 25, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 25, 63),
    ("Reembolsos por enfermedad profesional", 25, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa59_005 = get_value(unidad6_HOGAR, "Reembolso de siniestros sobre cesiones reaseguro interior", 25)
aa60_010 = get_value(unidad6_HOGAR, "Reembolso siniestros contratos no proporcionales reaseguro interior", 25)
aa61_015 = get_value(unidad6_HOGAR, "Reembolso de siniestros sobre cesiones reaseguro exterior", 25)
aa62_020 = get_value(unidad6_HOGAR, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 25)
aa63_025 = get_value(unidad6_HOGAR, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 25)
aa64_030 = get_value(unidad6_HOGAR, "Reembolsos por enfermedad profesional", 25)

aa65_999 = (aa59_005 + aa60_010 + aa61_015 + aa62_020 + aa63_025 + aa64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=27, value=aa65_999)

<Cell 'Hoja1'.AA65>

# 42 AGROPECUARIO UNIDAD 6

In [169]:
unidad6_AGROPECUARIO= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_AGROPECUARIO['TOTAL'] = unidad6_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 42, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 42, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 42, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 42, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 42, 63),
    ("Reembolsos por enfermedad profesional", 42, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar59_005 = get_value(unidad6_AGROPECUARIO, "Reembolso de siniestros sobre cesiones reaseguro interior", 42)
ar60_010 = get_value(unidad6_AGROPECUARIO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 42)
ar61_015 = get_value(unidad6_AGROPECUARIO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 42)
ar62_020 = get_value(unidad6_AGROPECUARIO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 42)
ar63_025 = get_value(unidad6_AGROPECUARIO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 42)
ar64_030 = get_value(unidad6_AGROPECUARIO, "Reembolsos por enfermedad profesional", 42)

ar65_999 = (ar59_005 + ar60_010 + ar61_015 + ar62_020 + ar63_025 + ar64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=44, value=ar65_999)

<Cell 'Hoja1'.AR65>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 6

In [170]:
unidad6_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_OPERNOASIGNABLES['TOTAL'] = unidad6_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 43, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 43, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 43, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 43, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 43, 63),
    ("Reembolsos por enfermedad profesional", 43, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as59_005 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros sobre cesiones reaseguro interior", 43)
as60_010 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso siniestros contratos no proporcionales reaseguro interior", 43)
as61_015 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros sobre cesiones reaseguro exterior", 43)
as62_020 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 43)
as63_025 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 43)
as64_030 = get_value(unidad6_OPERNOASIGNABLES, "Reembolsos por enfermedad profesional", 43)

as65_999 = (as59_005 + as60_010 + as61_015 + as62_020 + as63_025 + as64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=45, value=as65_999)

<Cell 'Hoja1'.AS65>

# 44 DECENAL UNIDAD 6

In [171]:
unidad6_DECENAL= bd[(bd['UNIDAD'] == 6) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_DECENAL['TOTAL'] = unidad6_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 44, 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 44, 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 44, 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 44, 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 44, 63),
    ("Reembolsos por enfermedad profesional", 44, 64),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad6_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at59_005 = get_value(unidad6_DECENAL, "Reembolso de siniestros sobre cesiones reaseguro interior", 44)
at60_010 = get_value(unidad6_DECENAL, "Reembolso siniestros contratos no proporcionales reaseguro interior", 44)
at61_015 = get_value(unidad6_DECENAL, "Reembolso de siniestros sobre cesiones reaseguro exterior", 44)
at62_020 = get_value(unidad6_DECENAL, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 44)
at63_025 = get_value(unidad6_DECENAL, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 44)
at64_030 = get_value(unidad6_DECENAL, "Reembolsos por enfermedad profesional", 44)

at65_999 = (at59_005 + at60_010 + at61_015 + at62_020 + at63_025 + at64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=46, value=at65_999)

<Cell 'Hoja1'.AT65>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 6

In [172]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au59_005 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au59_005, 47, 59)

au60_010 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au60_010, 47, 60)

au61_015 = unidad6_CUMPLIMIENTOESTATALES_a[(unidad6_CUMPLIMIENTOESTATALES_a['CUENTA'] == "6121101071") & (unidad6_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au61_015, 47, 61, 'TOTAL')

au62_020 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au62_020, 47, 62)

au63_025 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au63_025, 47, 63)

au64_030 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au64_030, 47, 64)


# Calcular ak32_999
au65_999 = au59_005['TOTAL'].sum() + au60_010['TOTAL'].sum() + au61_015['TOTAL'].sum() + au62_020['TOTAL'].sum() + au63_025['TOTAL'].sum() + \
           au64_030['TOTAL'].sum() 

au65_999 = abs(au65_999)
# Crear DataFrame para escribir en Excel
df_au65_999 = pd.DataFrame({'total': [au65_999]})
write_to_excel(df_au65_999, 47, 65, 'total')


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 6

In [173]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av59_005 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av59_005, 48, 59)

av60_010 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av60_010, 48, 60)

av61_015 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av61_015, 48, 61)

av62_020 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av62_020, 48, 62)

av63_025 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(au63_025, 48, 63)

av64_030 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av64_030, 48, 64)


# Calcular ak32_999
av65_999 = av59_005['TOTAL'].sum() + av60_010['TOTAL'].sum() + av61_015['TOTAL'].sum() + av62_020['TOTAL'].sum() + av63_025['TOTAL'].sum() + \
           av64_030['TOTAL'].sum() 

av65_999 = abs(av65_999)
# Crear DataFrame para escribir en Excel
df_av65_999 = pd.DataFrame({'total': [av65_999]})
write_to_excel(df_av65_999, 48, 65, 'total')

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 6

In [174]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


aw59_005 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw59_005, 49, 59)

aw60_010 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw60_010, 49, 60)

aw61_015 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw61_015, 49, 61)

aw62_020 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw62_020, 49, 62)

aw63_025 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw63_025, 49, 63)

aw64_030 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw64_030, 49, 64)


# Calcular ak32_999
aw65_999 = aw59_005['TOTAL'].sum() + aw60_010['TOTAL'].sum() + aw61_015['TOTAL'].sum() + aw62_020['TOTAL'].sum() + aw63_025['TOTAL'].sum() + \
           aw64_030['TOTAL'].sum() 

aw65_999 = abs(aw65_999)
# Crear DataFrame para escribir en Excel
df_aw65_999 = pd.DataFrame({'total': [aw65_999]})
write_to_excel(df_aw65_999, 49, 65, 'total')

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 6

In [175]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax59_005 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax59_005, 50, 59)

ax60_010 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax60_010, 50, 60)

ax61_015 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax61_015, 50, 61)

ax62_020 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax62_020, 50, 62)

ax63_025 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax63_025, 50, 63)

ax64_030 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax64_030, 50, 64)


# Calcular ak32_999
ax65_999 = ax59_005['TOTAL'].sum() + ax60_010['TOTAL'].sum() + ax61_015['TOTAL'].sum() + ax62_020['TOTAL'].sum() + ax63_025['TOTAL'].sum() + \
           ax64_030['TOTAL'].sum() 

ax65_999 = abs(ax65_999)
# Crear DataFrame para escribir en Excel
df_ax65_999 = pd.DataFrame({'total': [ax65_999]})
write_to_excel(df_ax65_999, 50, 65, 'total')

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 6

In [176]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ay59_005 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay59_005, 51, 59)

ay60_010 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay60_010, 51, 60)

ay61_015 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro exterior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay61_015, 51, 61)

ay62_020 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay62_020, 51, 62)

ay63_025 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay63_025, 51, 63)

ay64_030 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay64_030, 51, 64)


# Calcular ak32_999
ay65_999 = ay59_005['TOTAL'].sum() + ay60_010['TOTAL'].sum() + ay61_015['TOTAL'].sum() + ay62_020['TOTAL'].sum() + ay63_025['TOTAL'].sum() + \
           ay64_030['TOTAL'].sum() 

ay65_999 = abs(ay65_999)
# Crear DataFrame para escribir en Excel
df_ay65_999 = pd.DataFrame({'total': [ay65_999]})
write_to_excel(df_ay65_999, 51, 65, 'total')

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 6

In [177]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


az59_005 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az59_005, 52, 59)

az60_010 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az60_010, 52, 60)

az61_015 = unidad6_CUMPLIMIENTOARRENDAMIENTO_a[(unidad6_CUMPLIMIENTOARRENDAMIENTO_a['CUENTA'] == "6121101071") & (unidad6_CUMPLIMIENTOARRENDAMIENTO_a['COD_RAMO'] == 50)]
write_to_excel(az61_015, 52, 61, 'TOTAL')

az62_020 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az62_020, 52, 62)

az63_025 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az63_025, 52, 63)

az64_030 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az64_030, 52, 64)


# Calcular ak32_999
az65_999 = az59_005['TOTAL'].sum() + az60_010['TOTAL'].sum() + az61_015['TOTAL'].sum() + az62_020['TOTAL'].sum() + az63_025['TOTAL'].sum() + \
           az64_030['TOTAL'].sum() 

az65_999 = abs(az65_999)
# Crear DataFrame para escribir en Excel
df_az65_999 = pd.DataFrame({'total': [az65_999]})
write_to_excel(df_az65_999, 52, 65, 'total')

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 6

In [178]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba59_005 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Reembolso de siniestros sobre cesiones reaseguro interior") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba59_005, 53, 59)

ba60_010 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Reembolso siniestros contratos no proporcionales reaseguro interior") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba60_010, 53, 60)

ba61_015 = unidad6_CUMPLIMIENTOPARTICULARES_a[(unidad6_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6121101071") & (unidad6_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba61_015, 53, 61, 'TOTAL')

ba62_020 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Reembolso de siniestros contratos no proporcionales reaseguro exterior") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba62_020, 53, 62)

ba63_025 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba63_025, 53, 63)

ba64_030 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Reembolsos por enfermedad profesional") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba64_030, 53, 64)


# Calcular ak32_999
ba65_999 = ba59_005['TOTAL'].sum() + ba60_010['TOTAL'].sum() + ba61_015['TOTAL'].sum() + ba62_020['TOTAL'].sum() + ba63_025['TOTAL'].sum() + \
           ba64_030['TOTAL'].sum() 

ba65_999 = abs(ba65_999)
# Crear DataFrame para escribir en Excel
df_ba65_999 = pd.DataFrame({'total': [ba65_999]})
write_to_excel(df_ba65_999, 53, 65, 'total')

#  1   TOTAL UNIDAD 6

In [179]:
c59_005 = sum([ws.cell(row=59, column=col).value if ws.cell(row=59, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=59, column=3).value = c59_005  

c60_010 = sum([ws.cell(row=60, column=col).value if ws.cell(row=60, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=60, column=3).value = c60_010 

c61_015 = sum([ws.cell(row=61, column=col).value if ws.cell(row=61, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=61, column=3).value = c61_015 

c62_020 = sum([ws.cell(row=62, column=col).value if ws.cell(row=62, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=62, column=3).value = c62_020  

c63_025 = sum([ws.cell(row=63, column=col).value if ws.cell(row=63, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=63, column=3).value = c63_025  

c64_030 = sum([ws.cell(row=64, column=col).value if ws.cell(row=64, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=64, column=3).value = c64_030  

# Calcular el total general según la fórmula dada
c65_999 = (c59_005 + c60_010 + c61_015 + c62_020 + c63_025 + c64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=3).value = c65_999

#  2   SUBTOTAL - RAMOS UNIDAD 6

In [180]:
d59_005 = sum([ws.cell(row=59, column=col).value if ws.cell(row=59, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=59, column=4).value = d59_005  

d60_010 = sum([ws.cell(row=60, column=col).value if ws.cell(row=60, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=60, column=4).value = d60_010 

d61_015 = sum([ws.cell(row=61, column=col).value if ws.cell(row=61, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=61, column=4).value = d61_015 

d62_020 = sum([ws.cell(row=62, column=col).value if ws.cell(row=62, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=62, column=4).value = d62_020  

d63_025 = sum([ws.cell(row=63, column=col).value if ws.cell(row=63, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=63, column=4).value = d63_025  

d64_030 = sum([ws.cell(row=64, column=col).value if ws.cell(row=64, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=64, column=4).value = d64_030  

# Calcular el total general según la fórmula dada
d65_999 = (d59_005 + d60_010 + d61_015 + d62_020 + d63_025 + d64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=4).value = d65_999

## UNIDAD DE CAPTURA 7

#  3   AUTOMOVILES UNIDAD 7

In [181]:
unidad7_AUTOMOVILES = bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_AUTOMOVILES['TOTAL'] = unidad7_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 3, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 3, 68),
    ("Salvamentos liquidados y realizados", 3, 69),
    ("Recobros", 3, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 3, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 3, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e67_005 = get_value(unidad7_AUTOMOVILES, "Salvamentos de aceptaciones reaseguro interior", 3)
e68_010 = get_value(unidad7_AUTOMOVILES, "Salvamentos de aceptaciones reaseguro exterior", 3)
e69_015 = get_value(unidad7_AUTOMOVILES, "Salvamentos liquidados y realizados", 3)
e70_020 = get_value(unidad7_AUTOMOVILES, "Recobros", 3)
e71_025 = get_value(unidad7_AUTOMOVILES, "Gastos Salvamentos parte reasegurada interior (resta)", 3)
e72_030 = get_value(unidad7_AUTOMOVILES, "Gastos Salvamentos parte reasegurada exterior (resta)", 3)

e73_999 = (e67_005 + e68_010 + e69_015 + e70_020 + e71_025 + e72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=5, value=e73_999)

<Cell 'Hoja1'.E73>

# 4   SOAT UNIDAD 7

In [182]:
unidad7_SOAT= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_SOAT['TOTAL'] = unidad7_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 4, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 4, 68),
    ("Salvamentos liquidados y realizados", 4, 69),
    ("Recobros", 4, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 4, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 4, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f67_005 = get_value(unidad7_SOAT, "Salvamentos de aceptaciones reaseguro interior", 4)
f68_010 = get_value(unidad7_SOAT, "Salvamentos de aceptaciones reaseguro exterior", 4)
f69_015 = get_value(unidad7_SOAT, "Salvamentos liquidados y realizados", 4)
f70_020 = get_value(unidad7_SOAT, "Recobros", 4)
f71_025 = get_value(unidad7_SOAT, "Gastos Salvamentos parte reasegurada interior (resta)", 4)
f72_030 = get_value(unidad7_SOAT, "Gastos Salvamentos parte reasegurada exterior (resta)", 4)

f73_999 = (f67_005 + f68_010 + f69_015 + f70_020 + f71_025 + f72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=6, value=f73_999)

<Cell 'Hoja1'.F73>

#  5   CUMPLIMIENTO UNIDAD 7

In [183]:
unidad7_CUMPLIMIENTO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_CUMPLIMIENTO['TOTAL'] = unidad7_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 5, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 5, 68),
    ("Salvamentos liquidados y realizados", 5, 69),
    ("Recobros", 5, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 5, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 5, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g67_005 = get_value(unidad7_CUMPLIMIENTO, "Salvamentos de aceptaciones reaseguro interior", 5)
g68_010 = get_value(unidad7_CUMPLIMIENTO, "Salvamentos de aceptaciones reaseguro exterior", 5)
g69_015 = get_value(unidad7_CUMPLIMIENTO, "Salvamentos liquidados y realizados", 5)
g70_020 = get_value(unidad7_CUMPLIMIENTO, "Recobros", 5)
g71_025 = get_value(unidad7_CUMPLIMIENTO, "Gastos Salvamentos parte reasegurada interior (resta)", 5)
g72_030 = get_value(unidad7_CUMPLIMIENTO, "Gastos Salvamentos parte reasegurada exterior (resta)", 5)

g73_999 = (g67_005 + g68_010 + g69_015 + g70_020 + g71_025 + g72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=7, value=g73_999)

<Cell 'Hoja1'.G73>

#  6   RESPONSABILIDAD CIVIL UNIDAD 7

In [184]:
unidad7_RESPONSABILIDADCI= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_RESPONSABILIDADCI['TOTAL'] = unidad7_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 6, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 6, 68),
    ("Salvamentos liquidados y realizados", 6, 69),
    ("Recobros", 6, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 6, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 6, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h67_005 = get_value(unidad7_RESPONSABILIDADCI, "Salvamentos de aceptaciones reaseguro interior", 6)
h68_010 = get_value(unidad7_RESPONSABILIDADCI, "Salvamentos de aceptaciones reaseguro exterior", 6)
h69_015 = get_value(unidad7_RESPONSABILIDADCI, "Salvamentos liquidados y realizados", 6)
h70_020 = get_value(unidad7_RESPONSABILIDADCI, "Recobros", 6)
h71_025 = get_value(unidad7_RESPONSABILIDADCI, "Gastos Salvamentos parte reasegurada interior (resta)", 6)
h72_030 = get_value(unidad7_RESPONSABILIDADCI, "Gastos Salvamentos parte reasegurada exterior (resta)", 6)

h73_999 = (h67_005 + h68_010 + h69_015 + h70_020 + h71_025 + h72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=8, value=h73_999)

<Cell 'Hoja1'.H73>

#  7   INCENDIO UNIDAD 7

In [185]:
unidad7_INCENDIO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_INCENDIO['TOTAL'] = unidad7_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 7, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 7, 68),
    ("Salvamentos liquidados y realizados", 7, 69),
    ("Recobros", 7, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 7, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 7, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i67_005 = get_value(unidad7_INCENDIO, "Salvamentos de aceptaciones reaseguro interior", 7)
i68_010 = get_value(unidad7_INCENDIO, "Salvamentos de aceptaciones reaseguro exterior", 7)
i69_015 = get_value(unidad7_INCENDIO, "Salvamentos liquidados y realizados", 7)
i70_020 = get_value(unidad7_INCENDIO, "Recobros", 7)
i71_025 = get_value(unidad7_INCENDIO, "Gastos Salvamentos parte reasegurada interior (resta)", 7)
i72_030 = get_value(unidad7_INCENDIO, "Gastos Salvamentos parte reasegurada exterior (resta)", 7)

i73_999 = (i67_005 + i68_010 + i69_015 + i70_020 + i71_025 + i72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=9, value=i73_999)

<Cell 'Hoja1'.I73>

# 8   TERREMOTO UNIDAD 7

In [186]:
unidad7_TERREMOTO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_TERREMOTO['TOTAL'] = unidad7_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 8, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 8, 68),
    ("Salvamentos liquidados y realizados", 8, 69),
    ("Recobros", 8, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 8, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 8, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j67_005 = get_value(unidad7_TERREMOTO, "Salvamentos de aceptaciones reaseguro interior", 8)
j68_010 = get_value(unidad7_TERREMOTO, "Salvamentos de aceptaciones reaseguro exterior", 8)
j69_015 = get_value(unidad7_TERREMOTO, "Salvamentos liquidados y realizados", 8)
j70_020 = get_value(unidad7_TERREMOTO, "Recobros", 8)
j71_025 = get_value(unidad7_TERREMOTO, "Gastos Salvamentos parte reasegurada interior (resta)", 8)
j72_030 = get_value(unidad7_TERREMOTO, "Gastos Salvamentos parte reasegurada exterior (resta)", 8)

j73_999 = (j67_005 + j68_010 + j69_015 + j70_020 + j71_025 + j72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=10, value=j73_999)

<Cell 'Hoja1'.J73>

#  9   SUSTRACCION UNIDAD 7

In [187]:
unidad7_SUSTRACCION= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_SUSTRACCION['TOTAL'] = unidad7_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 9, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 9, 68),
    ("Salvamentos liquidados y realizados", 9, 69),
    ("Recobros", 9, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 9, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 9, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k67_005 = get_value(unidad7_SUSTRACCION, "Salvamentos de aceptaciones reaseguro interior", 9)
k68_010 = get_value(unidad7_SUSTRACCION, "Salvamentos de aceptaciones reaseguro exterior", 9)
k69_015 = get_value(unidad7_SUSTRACCION, "Salvamentos liquidados y realizados", 9)
k70_020 = get_value(unidad7_SUSTRACCION, "Recobros", 9)
k71_025 = get_value(unidad7_SUSTRACCION, "Gastos Salvamentos parte reasegurada interior (resta)", 9)
k72_030 = get_value(unidad7_SUSTRACCION, "Gastos Salvamentos parte reasegurada exterior (resta)", 9)

k73_999 = (k67_005 + k68_010 + k69_015 + k70_020 + k71_025 + k72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=11, value=k73_999)

<Cell 'Hoja1'.K73>

#  10  TRANSPORTE UNIDAD 7

In [188]:
unidad7_TRANSPORTE= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_TRANSPORTE['TOTAL'] = unidad7_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 10, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 10, 68),
    ("Salvamentos liquidados y realizados", 10, 69),
    ("Recobros", 10, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 10, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 10, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l67_005 = get_value(unidad7_TRANSPORTE, "Salvamentos de aceptaciones reaseguro interior", 10)
l68_010 = get_value(unidad7_TRANSPORTE, "Salvamentos de aceptaciones reaseguro exterior", 10)
l69_015 = get_value(unidad7_TRANSPORTE, "Salvamentos liquidados y realizados", 10)
l70_020 = get_value(unidad7_TRANSPORTE, "Recobros", 10)
l71_025 = get_value(unidad7_TRANSPORTE, "Gastos Salvamentos parte reasegurada interior (resta)", 10)
l72_030 = get_value(unidad7_TRANSPORTE, "Gastos Salvamentos parte reasegurada exterior (resta)", 10)

l73_999 = (l67_005 + l68_010 + l69_015 + l70_020 + l71_025 + l72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=12, value=l73_999)

<Cell 'Hoja1'.L73>

#  11  CORRIENTE DÉBIL UNIDAD 7

In [189]:
unidad7_CORRIENDEBIL= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_CORRIENDEBIL['TOTAL'] = unidad7_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 11, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 11, 68),
    ("Salvamentos liquidados y realizados", 11, 69),
    ("Recobros", 11, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 11, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 11, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m67_005 = get_value(unidad7_CORRIENDEBIL, "Salvamentos de aceptaciones reaseguro interior", 11)
m68_010 = get_value(unidad7_CORRIENDEBIL, "Salvamentos de aceptaciones reaseguro exterior", 11)
m69_015 = get_value(unidad7_CORRIENDEBIL, "Salvamentos liquidados y realizados", 11)
m70_020 = get_value(unidad7_CORRIENDEBIL, "Recobros", 11)
m71_025 = get_value(unidad7_CORRIENDEBIL, "Gastos Salvamentos parte reasegurada interior (resta)", 11)
m72_030 = get_value(unidad7_CORRIENDEBIL, "Gastos Salvamentos parte reasegurada exterior (resta)", 11)

m73_999 = (m67_005 + m68_010 + m69_015 + m70_020 + m71_025 + m72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=13, value=m73_999)

<Cell 'Hoja1'.M73>

#  12  TODO RIESGO CONTRATISTA UNIDAD 7

In [190]:
unidad7_TODORIESGOCONTRATISTA= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_TODORIESGOCONTRATISTA['TOTAL'] = unidad7_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 12, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 12, 68),
    ("Salvamentos liquidados y realizados", 12, 69),
    ("Recobros", 12, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 12, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 12, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n67_005 = get_value(unidad7_TODORIESGOCONTRATISTA, "Salvamentos de aceptaciones reaseguro interior", 12)
n68_010 = get_value(unidad7_TODORIESGOCONTRATISTA, "Salvamentos de aceptaciones reaseguro exterior", 12)
n69_015 = get_value(unidad7_TODORIESGOCONTRATISTA, "Salvamentos liquidados y realizados", 12)
n70_020 = get_value(unidad7_TODORIESGOCONTRATISTA, "Recobros", 12)
n71_025 = get_value(unidad7_TODORIESGOCONTRATISTA, "Gastos Salvamentos parte reasegurada interior (resta)", 12)
n72_030 = get_value(unidad7_TODORIESGOCONTRATISTA, "Gastos Salvamentos parte reasegurada exterior (resta)", 12)

n73_999 = (n67_005 + n68_010 + n69_015 + n70_020 + n71_025 + n72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=14, value=n73_999)

<Cell 'Hoja1'.N73>

#  13  MANEJO UNIDAD 7

In [191]:
unidad7_MANEJO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_MANEJO['TOTAL'] = unidad7_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 13, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 13, 68),
    ("Salvamentos liquidados y realizados", 13, 69),
    ("Recobros", 13, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 13, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 13, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o67_005 = get_value(unidad7_MANEJO, "Salvamentos de aceptaciones reaseguro interior", 13)
o68_010 = get_value(unidad7_MANEJO, "Salvamentos de aceptaciones reaseguro exterior", 13)
o69_015 = get_value(unidad7_MANEJO, "Salvamentos liquidados y realizados", 13)
o70_020 = get_value(unidad7_MANEJO, "Recobros", 13)
o71_025 = get_value(unidad7_MANEJO, "Gastos Salvamentos parte reasegurada interior (resta)", 13)
o72_030 = get_value(unidad7_MANEJO, "Gastos Salvamentos parte reasegurada exterior (resta)", 13)

o73_999 = (o67_005 + o68_010 + o69_015 + o70_020 + o71_025 + o72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=15, value=o73_999)

<Cell 'Hoja1'.O73>

#  14  LUCRO CESANTE UNIDAD 7

In [192]:
unidad7_LUCROCESANTE= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_LUCROCESANTE['TOTAL'] = unidad7_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 14, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 14, 68),
    ("Salvamentos liquidados y realizados", 14, 69),
    ("Recobros", 14, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 14, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 14, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p67_005 = get_value(unidad7_LUCROCESANTE, "Salvamentos de aceptaciones reaseguro interior", 14)
p68_010 = get_value(unidad7_LUCROCESANTE, "Salvamentos de aceptaciones reaseguro exterior", 14)
p69_015 = get_value(unidad7_LUCROCESANTE, "Salvamentos liquidados y realizados", 14)
p70_020 = get_value(unidad7_LUCROCESANTE, "Recobros", 14)
p71_025 = get_value(unidad7_LUCROCESANTE, "Gastos Salvamentos parte reasegurada interior (resta)", 14)
p72_030 = get_value(unidad7_LUCROCESANTE, "Gastos Salvamentos parte reasegurada exterior (resta)", 14)

p73_999 = (p67_005 + p68_010 + p69_015 + p70_020 + p71_025 + p72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=16, value=p73_999)

<Cell 'Hoja1'.P73>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 7

In [193]:
unidad7_MONTAJEYROTURA= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_MONTAJEYROTURA['TOTAL'] = unidad7_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 15, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 15, 68),
    ("Salvamentos liquidados y realizados", 15, 69),
    ("Recobros", 15, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 15, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 15, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q67_005 = get_value(unidad7_MONTAJEYROTURA, "Salvamentos de aceptaciones reaseguro interior", 15)
q68_010 = get_value(unidad7_MONTAJEYROTURA, "Salvamentos de aceptaciones reaseguro exterior", 15)
q69_015 = get_value(unidad7_MONTAJEYROTURA, "Salvamentos liquidados y realizados", 15)
q70_020 = get_value(unidad7_MONTAJEYROTURA, "Recobros", 15)
q71_025 = get_value(unidad7_MONTAJEYROTURA, "Gastos Salvamentos parte reasegurada interior (resta)", 15)
q72_030 = get_value(unidad7_MONTAJEYROTURA, "Gastos Salvamentos parte reasegurada exterior (resta)", 15)

q73_999 = (q67_005 + q68_010 + q69_015 + q70_020 + q71_025 + q72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=17, value=q73_999)

<Cell 'Hoja1'.Q73>

#  16  AVIACIÓN UNIDAD 7

In [194]:
unidad7_AVIACION= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_AVIACION['TOTAL'] = unidad7_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 16, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 16, 68),
    ("Salvamentos liquidados y realizados", 16, 69),
    ("Recobros", 16, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 16, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 16, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r67_005 = get_value(unidad7_AVIACION, "Salvamentos de aceptaciones reaseguro interior", 16)
r68_010 = get_value(unidad7_AVIACION, "Salvamentos de aceptaciones reaseguro exterior", 16)
r69_015 = get_value(unidad7_AVIACION, "Salvamentos liquidados y realizados", 16)
r70_020 = get_value(unidad7_AVIACION, "Recobros", 16)
r71_025 = get_value(unidad7_AVIACION, "Gastos Salvamentos parte reasegurada interior (resta)", 16)
r72_030 = get_value(unidad7_AVIACION, "Gastos Salvamentos parte reasegurada exterior (resta)", 16)

r73_999 = (r67_005 + r68_010 + r69_015 + r70_020 + r71_025 + r72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=18, value=r73_999)

<Cell 'Hoja1'.R73>

#  17  NAVEGACIÓN Y CASCO UNIDAD 7

In [195]:
unidad7_NAVEGACION= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_NAVEGACION['TOTAL'] = unidad7_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 17, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 17, 68),
    ("Salvamentos liquidados y realizados", 17, 69),
    ("Recobros", 17, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 17, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 17, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s67_005 = get_value(unidad7_NAVEGACION, "Salvamentos de aceptaciones reaseguro interior", 17)
s68_010 = get_value(unidad7_NAVEGACION, "Salvamentos de aceptaciones reaseguro exterior", 17)
s69_015 = get_value(unidad7_NAVEGACION, "Salvamentos liquidados y realizados", 17)
s70_020 = get_value(unidad7_NAVEGACION, "Recobros", 17)
s71_025 = get_value(unidad7_NAVEGACION, "Gastos Salvamentos parte reasegurada interior (resta)", 17)
s72_030 = get_value(unidad7_NAVEGACION, "Gastos Salvamentos parte reasegurada exterior (resta)", 17)

s73_999 = (s67_005 + s68_010 + s69_015 + s70_020 + s71_025 + s72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=19, value=s73_999)

<Cell 'Hoja1'.S73>

#  18  MINAS Y PETRÓLEOS UNIDAD 7

In [196]:
unidad7_MINAS= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_MINAS['TOTAL'] = unidad7_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 18, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 18, 68),
    ("Salvamentos liquidados y realizados", 18, 69),
    ("Recobros", 18, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 18, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 18, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t67_005 = get_value(unidad7_MINAS, "Salvamentos de aceptaciones reaseguro interior", 18)
t68_010 = get_value(unidad7_MINAS, "Salvamentos de aceptaciones reaseguro exterior", 18)
t69_015 = get_value(unidad7_MINAS, "Salvamentos liquidados y realizados", 18)
t70_020 = get_value(unidad7_MINAS, "Recobros", 18)
t71_025 = get_value(unidad7_MINAS, "Gastos Salvamentos parte reasegurada interior (resta)", 18)
t72_030 = get_value(unidad7_MINAS, "Gastos Salvamentos parte reasegurada exterior (resta)", 18)

t73_999 = (t67_005 + t68_010 + t69_015 + t70_020 + t71_025 + t72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=20, value=t73_999)

<Cell 'Hoja1'.T73>

#  19  VIDRIOS UNIDAD 7

In [197]:
unidad7_VIDRIOS= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_VIDRIOS['TOTAL'] = unidad7_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 19, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 19, 68),
    ("Salvamentos liquidados y realizados", 19, 69),
    ("Recobros", 19, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 19, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 19, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u67_005 = get_value(unidad7_VIDRIOS, "Salvamentos de aceptaciones reaseguro interior", 19)
u68_010 = get_value(unidad7_VIDRIOS, "Salvamentos de aceptaciones reaseguro exterior", 19)
u69_015 = get_value(unidad7_VIDRIOS, "Salvamentos liquidados y realizados", 19)
u70_020 = get_value(unidad7_VIDRIOS, "Recobros", 19)
u71_025 = get_value(unidad7_VIDRIOS, "Gastos Salvamentos parte reasegurada interior (resta)", 19)
u72_030 = get_value(unidad7_VIDRIOS, "Gastos Salvamentos parte reasegurada exterior (resta)", 19)

u73_999 = (u67_005 + u68_010 + u69_015 + u70_020 + u71_025 + u72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=21, value=u73_999)

<Cell 'Hoja1'.U73>

#  20  CRÉDITO COMERCIAL UNIDAD 7

In [198]:
unidad7_CREDITOCOMERCIAL= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_CREDITOCOMERCIAL['TOTAL'] = unidad7_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 20, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 20, 68),
    ("Salvamentos liquidados y realizados", 20, 69),
    ("Recobros", 20, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 20, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 20, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v67_005 = get_value(unidad7_CREDITOCOMERCIAL, "Salvamentos de aceptaciones reaseguro interior", 20)
v68_010 = get_value(unidad7_CREDITOCOMERCIAL, "Salvamentos de aceptaciones reaseguro exterior", 20)
v69_015 = get_value(unidad7_CREDITOCOMERCIAL, "Salvamentos liquidados y realizados", 20)
v70_020 = get_value(unidad7_CREDITOCOMERCIAL, "Recobros", 20)
v71_025 = get_value(unidad7_CREDITOCOMERCIAL, "Gastos Salvamentos parte reasegurada interior (resta)", 20)
v72_030 = get_value(unidad7_CREDITOCOMERCIAL, "Gastos Salvamentos parte reasegurada exterior (resta)", 20)

v73_999 = (v67_005 + v68_010 + v69_015 + v70_020 + v71_025 + v72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=22, value=v73_999)

<Cell 'Hoja1'.V73>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 7

In [199]:
unidad7_CREDITOEXPORTACION= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_CREDITOEXPORTACION['TOTAL'] = unidad7_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 21, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 21, 68),
    ("Salvamentos liquidados y realizados", 21, 69),
    ("Recobros", 21, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 21, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 21, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w67_005 = get_value(unidad7_CREDITOEXPORTACION, "Salvamentos de aceptaciones reaseguro interior", 21)
w68_010 = get_value(unidad7_CREDITOEXPORTACION, "Salvamentos de aceptaciones reaseguro exterior", 21)
w69_015 = get_value(unidad7_CREDITOEXPORTACION, "Salvamentos liquidados y realizados", 21)
w70_020 = get_value(unidad7_CREDITOEXPORTACION, "Recobros", 21)
w71_025 = get_value(unidad7_CREDITOEXPORTACION, "Gastos Salvamentos parte reasegurada interior (resta)", 21)
w72_030 = get_value(unidad7_CREDITOEXPORTACION, "Gastos Salvamentos parte reasegurada exterior (resta)", 21)

w73_999 = (w67_005 + w68_010 + w69_015 + w70_020 + w71_025 + w72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=23, value=w73_999)

<Cell 'Hoja1'.W73>

#  24  DESEMPLEO UNIDAD 7

In [200]:
unidad7_DESEMPLEO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_DESEMPLEO['TOTAL'] = unidad7_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 24, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 24, 68),
    ("Salvamentos liquidados y realizados", 24, 69),
    ("Recobros", 24, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 24, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 24, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z67_005 = get_value(unidad7_DESEMPLEO, "Salvamentos de aceptaciones reaseguro interior", 24)
z68_010 = get_value(unidad7_DESEMPLEO, "Salvamentos de aceptaciones reaseguro exterior", 24)
z69_015 = get_value(unidad7_DESEMPLEO, "Salvamentos liquidados y realizados", 24)
z70_020 = get_value(unidad7_DESEMPLEO, "Recobros", 24)
z71_025 = get_value(unidad7_DESEMPLEO, "Gastos Salvamentos parte reasegurada interior (resta)", 24)
z72_030 = get_value(unidad7_DESEMPLEO, "Gastos Salvamentos parte reasegurada exterior (resta)", 24)

z73_999 = (z67_005 + z68_010 + z69_015 + z70_020 + z71_025 + z72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=26, value=z73_999)

<Cell 'Hoja1'.Z73>

#  25  HOGAR UNIDAD 7

In [201]:
unidad7_HOGAR= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_HOGAR['TOTAL'] = unidad7_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 25, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 25, 68),
    ("Salvamentos liquidados y realizados", 25, 69),
    ("Recobros", 25, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 25, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 25, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa67_005 = get_value(unidad7_HOGAR, "Salvamentos de aceptaciones reaseguro interior", 25)
aa68_010 = get_value(unidad7_HOGAR, "Salvamentos de aceptaciones reaseguro exterior", 25)
aa69_015 = get_value(unidad7_HOGAR, "Salvamentos liquidados y realizados", 25)
aa70_020 = get_value(unidad7_HOGAR, "Recobros", 25)
aa71_025 = get_value(unidad7_HOGAR, "Gastos Salvamentos parte reasegurada interior (resta)", 25)
aa72_030 = get_value(unidad7_HOGAR, "Gastos Salvamentos parte reasegurada exterior (resta)", 25)

aa73_999 = (aa67_005 + aa68_010 + aa69_015 + aa70_020 + aa71_025 + aa72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=27, value=aa73_999)

<Cell 'Hoja1'.AA73>

# 42 AGROPECUARIO UNIDAD 7

In [202]:
unidad7_AGROPECUARIO= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_AGROPECUARIO['TOTAL'] = unidad7_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 42, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 42, 68),
    ("Salvamentos liquidados y realizados", 42, 69),
    ("Recobros", 42, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 42, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 42, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar67_005 = get_value(unidad7_AGROPECUARIO, "Salvamentos de aceptaciones reaseguro interior", 42)
ar68_010 = get_value(unidad7_AGROPECUARIO, "Salvamentos de aceptaciones reaseguro exterior", 42)
ar69_015 = get_value(unidad7_AGROPECUARIO, "Salvamentos liquidados y realizados", 42)
ar70_020 = get_value(unidad7_AGROPECUARIO, "Recobros", 42)
ar71_025 = get_value(unidad7_AGROPECUARIO, "Gastos Salvamentos parte reasegurada interior (resta)", 42)
ar72_030 = get_value(unidad7_AGROPECUARIO, "Gastos Salvamentos parte reasegurada exterior (resta)", 42)

ar73_999 = (ar67_005 + ar68_010 + ar69_015 + ar70_020 + ar71_025 + ar72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=44, value=ar73_999)

<Cell 'Hoja1'.AR73>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 7

In [203]:
unidad7_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_OPERNOASIGNABLES['TOTAL'] = unidad7_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 43, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 43, 68),
    ("Salvamentos liquidados y realizados", 43, 69),
    ("Recobros", 43, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 43, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 43, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as67_005 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos de aceptaciones reaseguro interior", 43)
as68_010 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos de aceptaciones reaseguro exterior", 43)
as69_015 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos liquidados y realizados", 43)
as70_020 = get_value(unidad7_OPERNOASIGNABLES, "Recobros", 43)
as71_025 = get_value(unidad7_OPERNOASIGNABLES, "Gastos Salvamentos parte reasegurada interior (resta)", 43)
as72_030 = get_value(unidad7_OPERNOASIGNABLES, "Gastos Salvamentos parte reasegurada exterior (resta)", 43)

as73_999 = (as67_005 + as68_010 + as69_015 + as70_020 + as71_025 + as72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=45, value=as73_999)

<Cell 'Hoja1'.AS73>

# 44 DECENAL UNIDAD 7

In [204]:
unidad7_DECENAL= bd[(bd['UNIDAD'] == 7) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_DECENAL['TOTAL'] = unidad7_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 44, 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 44, 68),
    ("Salvamentos liquidados y realizados", 44, 69),
    ("Recobros", 44, 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 44, 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 44, 72),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad7_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at67_005 = get_value(unidad7_DECENAL, "Salvamentos de aceptaciones reaseguro interior", 44)
at68_010 = get_value(unidad7_DECENAL, "Salvamentos de aceptaciones reaseguro exterior", 44)
at69_015 = get_value(unidad7_DECENAL, "Salvamentos liquidados y realizados", 44)
at70_020 = get_value(unidad7_DECENAL, "Recobros", 44)
at71_025 = get_value(unidad7_DECENAL, "Gastos Salvamentos parte reasegurada interior (resta)", 44)
at72_030 = get_value(unidad7_DECENAL, "Gastos Salvamentos parte reasegurada exterior (resta)", 44)

at73_999 = (at67_005 + at68_010 + at69_015 + at70_020 + at71_025 + at72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=46, value=at73_999)

<Cell 'Hoja1'.AT73>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 7

In [205]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au67_005 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au67_005, 47, 67)

au68_010 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au68_010, 47, 68)

au69_015 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au69_015, 47, 69)

au70_020 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au70_020, 47, 70)

au71_025 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au71_025, 47, 71)

au72_030 = unidad6_CUMPLIMIENTOESTATALES[(unidad6_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au72_030, 47, 72)


# Calcular ak32_999
au73_999 = au67_005['TOTAL'].sum() + au68_010['TOTAL'].sum() + au69_015['TOTAL'].sum() + au70_020['TOTAL'].sum() + au71_025['TOTAL'].sum() + \
           au72_030['TOTAL'].sum() 

au73_999 = abs(au73_999)
# Crear DataFrame para escribir en Excel
df_au73_999 = pd.DataFrame({'total': [au73_999]})
write_to_excel(df_au73_999, 47, 73, 'total')

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 7

In [206]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av67_005 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av67_005, 48, 67)

av68_010 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av68_010, 48, 68)

av69_015 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av69_015, 48, 69)

av70_020 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av70_020, 48, 70)

av71_025 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av71_025, 48, 71)

av72_030 = unidad6_CUMPLIMIENTOEMPRESASSERV[(unidad6_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av72_030, 48, 72)


# Calcular ak32_999
av73_999 = av67_005['TOTAL'].sum() + av68_010['TOTAL'].sum() + av69_015['TOTAL'].sum() + av70_020['TOTAL'].sum() + av71_025['TOTAL'].sum() + \
           av72_030['TOTAL'].sum() 

av73_999 = abs(av73_999)
# Crear DataFrame para escribir en Excel
df_av73_999 = pd.DataFrame({'total': [av73_999]})
write_to_excel(df_av73_999, 48, 73, 'total')


# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 7

In [207]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


aw67_005 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw67_005, 49, 67)

aw68_010 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw68_010, 49, 68)

aw69_015 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw69_015, 49, 69)

aw70_020 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw70_020, 49, 70)

aw71_025 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw71_025, 49, 71)

aw72_030 = unidad6_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw72_030, 49, 72)


# Calcular ak32_999
aw73_999 = aw67_005['TOTAL'].sum() + aw68_010['TOTAL'].sum() + aw69_015['TOTAL'].sum() + aw70_020['TOTAL'].sum() + aw71_025['TOTAL'].sum() + \
           aw72_030['TOTAL'].sum() 

aw73_999 = abs(aw73_999)
# Crear DataFrame para escribir en Excel
df_aw73_999 = pd.DataFrame({'total': [aw73_999]})
write_to_excel(df_aw73_999, 49, 73, 'total')

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 7

In [208]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax67_005 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax67_005, 50, 67)

ax68_010 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax68_010, 50, 68)

ax69_015 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax69_015, 50, 69)

ax70_020 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax70_020, 50, 70)

ax71_025 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax71_025, 50, 71)

ax72_030 = unidad6_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax72_030, 50, 72)


# Calcular ak32_999
ax73_999 = ax67_005['TOTAL'].sum() + ax68_010['TOTAL'].sum() + ax69_015['TOTAL'].sum() + ax70_020['TOTAL'].sum() + ax71_025['TOTAL'].sum() + \
           ax72_030['TOTAL'].sum() 

ax73_999 = abs(ax73_999)
# Crear DataFrame para escribir en Excel
df_ax73_999 = pd.DataFrame({'total': [ax73_999]})
write_to_excel(df_ax73_999, 50, 73, 'total')

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 7

In [209]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ay67_005 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay67_005, 51, 67)

ay68_010 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay68_010, 51, 68)

ay69_015 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay69_015, 51, 69)

ay70_020 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay70_020, 51, 70)

ay71_025 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay71_025, 51, 71)

ay72_030 = unidad6_CUMPLIMIENTOCAUCIONES[(unidad6_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay72_030, 51, 72)


# Calcular ak32_999
ay73_999 = ay67_005['TOTAL'].sum() + ay68_010['TOTAL'].sum() + ay69_015['TOTAL'].sum() + ay70_020['TOTAL'].sum() + ay71_025['TOTAL'].sum() + \
           ay72_030['TOTAL'].sum() 

ay73_999 = abs(ay73_999)
# Crear DataFrame para escribir en Excel
df_ay73_999 = pd.DataFrame({'total': [ay73_999]})
write_to_excel(df_ay73_999, 51, 73, 'total')

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 7

In [210]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


az67_005 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az67_005, 52, 67)

az68_010 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az68_010, 52, 68)

az69_015 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az69_015, 52, 69)

az70_020 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Recobros") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az70_020, 52, 70)

az71_025 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az71_025, 52, 71)

az72_030 = unidad6_CUMPLIMIENTOARRENDAMIENTO[(unidad6_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az72_030, 52, 72)


# Calcular ak32_999
az73_999 = az67_005['TOTAL'].sum() + az68_010['TOTAL'].sum() + az69_015['TOTAL'].sum() + az70_020['TOTAL'].sum() + az71_025['TOTAL'].sum() + \
           az72_030['TOTAL'].sum() 

az73_999 = abs(az73_999)
# Crear DataFrame para escribir en Excel
df_az73_999 = pd.DataFrame({'total': [az73_999]})
write_to_excel(df_az73_999, 52, 73, 'total')

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 7

In [211]:
# Agrupar los datos según el concepto y el ramo
unidad6_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad6_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba67_005 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro interior") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba67_005, 53, 67)

ba68_010 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Salvamentos de aceptaciones reaseguro exterior") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba68_010, 53, 68)

ba69_015 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Salvamentos liquidados y realizados") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba69_015, 53, 69)

ba70_020 = unidad6_CUMPLIMIENTOPARTICULARES_a[(unidad6_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "6111204091") & (unidad6_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba70_020, 53, 70, 'TOTAL')

ba71_025 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada interior (resta)") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba71_025, 53, 71)

ba72_030 = unidad6_CUMPLIMIENTOPARTICULARES[(unidad6_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Gastos Salvamentos parte reasegurada exterior (resta)") & (unidad6_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba72_030, 53, 72)


# Calcular ak32_999
ba73_999 = ba67_005['TOTAL'].sum() + ba68_010['TOTAL'].sum() + ba69_015['TOTAL'].sum() + ba70_020['TOTAL'].sum() + ba71_025['TOTAL'].sum() + \
           ba72_030['TOTAL'].sum() 

ba73_999 = abs(ba73_999)
# Crear DataFrame para escribir en Excel
df_ba73_999 = pd.DataFrame({'total': [ba73_999]})
write_to_excel(df_ba73_999, 53, 73, 'total')


##  1   TOTAL UNIDAD 7

In [212]:
c67_005 = sum([ws.cell(row=67, column=col).value if ws.cell(row=67, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=67, column=3).value = c67_005  

c68_010 = sum([ws.cell(row=68, column=col).value if ws.cell(row=68, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=68, column=3).value = c68_010 

c69_015 = sum([ws.cell(row=69, column=col).value if ws.cell(row=69, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=69, column=3).value = c69_015 

c70_020 = sum([ws.cell(row=70, column=col).value if ws.cell(row=70, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=70, column=3).value = c70_020  

c71_025 = sum([ws.cell(row=71, column=col).value if ws.cell(row=71, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=71, column=3).value = c71_025  

c72_030 = sum([ws.cell(row=72, column=col).value if ws.cell(row=72, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=72, column=3).value = c72_030  

# Calcular el total general según la fórmula dada
c73_999 = (c67_005 + c68_010 + c69_015 + c70_020 + c71_025 + c72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=3).value = c73_999

#  2   SUBTOTAL - RAMOS UNIDAD 7

In [213]:
d67_005 = sum([ws.cell(row=67, column=col).value if ws.cell(row=67, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=67, column=4).value = d67_005  

d68_010 = sum([ws.cell(row=68, column=col).value if ws.cell(row=68, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=68, column=4).value = d68_010 

d69_015 = sum([ws.cell(row=69, column=col).value if ws.cell(row=69, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=69, column=4).value = d69_015 

d70_020 = sum([ws.cell(row=70, column=col).value if ws.cell(row=70, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=70, column=4).value = d70_020  

d71_025 = sum([ws.cell(row=71, column=col).value if ws.cell(row=71, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=71, column=4).value = d71_025  

d72_030 = sum([ws.cell(row=72, column=col).value if ws.cell(row=72, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=72, column=4).value = d72_030  

# Calcular el total general según la fórmula dada
d73_999 = (d67_005 + d68_010 + d69_015 + d70_020 + d71_025 + d72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=4).value = d73_999

## UNIDAD DE CAPTURA 8

# CONSULTA DE TOTALES SINIESTROS CUENTA COMPAÑÍA UNIDAD 8


In [214]:
# Sumar c50_999, c57_999, c65_999 y c73_999, y escribir en la celda correspondiente

c75_999 = c50_999 - c57_999 + c65_999 + c73_999
df_c75_999 = pd.DataFrame({'total': [c75_999]})
if df_c75_999.empty or c75_999 == 0:
    df_c75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=3).value = df_c75_999['total'].iloc[0]

# Repetir el mismo proceso para las otras variables
e75_999 = e50_999 - e57_999 + e65_999 + e73_999
df_e75_999 = pd.DataFrame({'total': [e75_999]})
if df_e75_999.empty or e75_999 == 0:
    df_e75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=5).value = df_e75_999['total'].iloc[0]

f75_999 = f50_999 - f57_999 + f65_999 + f73_999
df_f75_999 = pd.DataFrame({'total': [f75_999]})
if df_f75_999.empty or f75_999 == 0:
    df_f75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=6).value = df_f75_999['total'].iloc[0]

g75_999 = g50_999 - g57_999 + g65_999 + g73_999
df_g75_999 = pd.DataFrame({'total': [g75_999]})
if df_g75_999.empty or g75_999 == 0:
    df_g75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=7).value = df_g75_999['total'].iloc[0]

h75_999 = h50_999 - h57_999 + h65_999 + h73_999
df_h75_999 = pd.DataFrame({'total': [h75_999]})
if df_h75_999.empty or h75_999 == 0:
    df_h75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=8).value = df_h75_999['total'].iloc[0]

i75_999 = i50_999 - i57_999 + i65_999 + i73_999
df_i75_999 = pd.DataFrame({'total': [i75_999]})
if df_i75_999.empty or i75_999 == 0:
    df_i75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=9).value = df_i75_999['total'].iloc[0]

j75_999 = j50_999 - j57_999 + j65_999 + j73_999
df_j75_999 = pd.DataFrame({'total': [j75_999]})
if df_j75_999.empty or j75_999 == 0:
    df_j75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=10).value = df_j75_999['total'].iloc[0]

k75_999 = k50_999 - k57_999 + k65_999 + k73_999
df_k75_999 = pd.DataFrame({'total': [k75_999]})
if df_k75_999.empty or k75_999 == 0:
    df_k75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=11).value = df_k75_999['total'].iloc[0]

l75_999 = l50_999 - l57_999 + l65_999 + l73_999
df_l75_999= pd.DataFrame({'total': [l75_999]})
if df_l75_999.empty or l75_999 == 0:
    df_l75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=12).value = df_l75_999['total'].iloc[0]

m75_999 = m50_999 - m57_999 + m65_999 + m73_999
df_m75_999 = pd.DataFrame({'total': [m75_999]})
if df_m75_999.empty or m75_999 == 0:
    df_m75_999= pd.DataFrame({'total': [0]})
ws.cell(row=75, column=13).value = df_m75_999['total'].iloc[0]

n75_999 = n50_999 - n57_999 + n65_999 + n73_999
df_n75_999 = pd.DataFrame({'total': [n75_999]})
if df_n75_999.empty or n75_999 == 0:
    df_n75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=14).value = df_n75_999['total'].iloc[0]

o75_999 = o50_999 - o57_999 + o65_999 + o73_999
df_o75_999 = pd.DataFrame({'total': [o75_999]})
if df_o75_999.empty or o75_999 == 0:
    df_o75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=15).value = df_o75_999['total'].iloc[0]

p75_999 = p50_999 - p57_999 + p65_999 + p73_999
df_p75_999 = pd.DataFrame({'total': [p75_999]})
if df_p75_999.empty or p75_999 == 0:
    df_p75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=16).value = df_p75_999['total'].iloc[0]

q75_999 = q50_999 - q57_999 + q65_999 + q73_999
df_q75_999 = pd.DataFrame({'total': [q75_999]})
if df_q75_999.empty or q75_999 == 0:
    df_q75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=17).value = df_q75_999['total'].iloc[0]

r75_999 = r50_999 - r57_999 + r65_999 + r73_999
df_r75_999 = pd.DataFrame({'total': [r75_999]})
if df_r75_999.empty or r75_999 == 0:
    df_r75_999= pd.DataFrame({'total': [0]})
ws.cell(row=75, column=18).value = df_r75_999['total'].iloc[0]

s75_999 = s50_999 - s57_999 + s65_999 + s73_999
df_s75_999 = pd.DataFrame({'total': [s75_999]})
if df_s75_999.empty or s75_999 == 0:
    df_s75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=19).value = df_s75_999['total'].iloc[0]

t75_999 = t50_999 - t57_999 + t65_999 + t73_999
df_t75_999 = pd.DataFrame({'total': [t75_999]})
if df_t75_999.empty or t75_999 == 0:
    df_t75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=20).value = df_t75_999['total'].iloc[0]

u75_999 = u50_999 - u57_999 + u65_999 + u73_999
df_u75_999 = pd.DataFrame({'total': [u75_999]})
if df_u75_999.empty or u75_999 == 0:
    df_u75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=21).value = df_u75_999['total'].iloc[0]

v75_999 = v50_999 - v57_999 + v65_999 + v73_999
df_v75_999 = pd.DataFrame({'total': [v75_999]})
if df_v75_999.empty or v75_999 == 0:
    df_v75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=22).value = df_v75_999['total'].iloc[0]

w75_999 = w50_999 - w57_999 + w65_999 + w73_999
df_w75_999 = pd.DataFrame({'total': [w75_999]})
if df_w75_999.empty or w75_999 == 0:
    df_w75_999= pd.DataFrame({'total': [0]})
ws.cell(row=75, column=23).value = df_w75_999['total'].iloc[0]

z75_999 = z50_999 - z57_999 + z65_999 + z73_999
df_z75_999 = pd.DataFrame({'total': [z75_999]})
if df_z75_999.empty or z75_999 == 0:
    df_z75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=26).value = df_z75_999['total'].iloc[0]

aa75_999 = aa50_999 - aa57_999 + aa65_999 + aa73_999
df_aa75_999 = pd.DataFrame({'total': [aa75_999]})
if df_aa75_999.empty or aa75_999 == 0:
    df_aa75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=27).value = df_aa75_999['total'].iloc[0]

ar75_999 = ar50_999 - ar57_999 + ar65_999 + ar73_999
df_ar75_999= pd.DataFrame({'total': [ar75_999]})
if df_ar75_999.empty or ar75_999 == 0:
    df_ar75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=44).value = df_ar75_999['total'].iloc[0]

as75_999 = as50_999 - as57_999 + as65_999 + as73_999
df_as75_999 = pd.DataFrame({'total': [as75_999]})
if df_as75_999.empty or as75_999 == 0:
    df_as75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=45).value = df_as75_999['total'].iloc[0]

at75_999 = at50_999 - at57_999 + at65_999 + at73_999
df_at75_999 = pd.DataFrame({'total': [at75_999]})
if df_at75_999.empty or at75_999 == 0:
    df_at75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=46).value = df_at75_999['total'].iloc[0]

au75_999 = au50_999 - au57_999 + au65_999 + au73_999
df_au75_999 = pd.DataFrame({'total': [au75_999]})
if df_au75_999.empty or au75_999 == 0:
    df_au75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=47).value = df_au75_999['total'].iloc[0]

av75_999 = av50_999 - av57_999 + av65_999 + av73_999
df_av75_999 = pd.DataFrame({'total': [av75_999]})
if df_av75_999.empty or av75_999 == 0:
    df_av75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=48).value = df_av75_999['total'].iloc[0]

aw75_999 = aw50_999 - aw57_999 + aw65_999 + aw73_999
df_aw75_999 = pd.DataFrame({'total': [aw75_999]})
if df_aw75_999.empty or aw75_999 == 0:
    df_aw75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=49).value = df_aw75_999['total'].iloc[0]

ax75_999 = ax50_999 - ax57_999 + ax65_999 + ax73_999
df_ax75_999 = pd.DataFrame({'total': [ax75_999]})
if df_ax75_999.empty or ax75_999 == 0:
    df_ax75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=50).value = df_ax75_999['total'].iloc[0]

ay75_999 = ay50_999 - ay57_999 + ay65_999 + ay73_999
df_ay75_999 = pd.DataFrame({'total': [ay75_999]})
if df_ay75_999.empty or ay75_999 == 0:
    df_ay75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=51).value = df_ay75_999['total'].iloc[0]

az75_999 = az50_999 - az57_999 + az65_999 + az73_999
df_az75_999 = pd.DataFrame({'total': [az75_999]})
if df_az75_999.empty or az75_999 == 0:
    df_az75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=52).value = df_az75_999['total'].iloc[0]

ba75_999 = ba50_999 - ba57_999 + ba65_999 + ba73_999
df_ba75_999 = pd.DataFrame({'total': [ba75_999]})
if df_ba75_999.empty or ba75_999 == 0:
    df_ba75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=53).value = df_ba75_999['total'].iloc[0]


## CONSULTA DE SUBTOTALES SINIESTROS CUENTA COMPAÑÍA UNIDAD 8


In [215]:
# Realiza la suma para d75_999
d75_999 = d50_999 - d57_999 + d65_999 + d73_999
df_d75_999 = pd.DataFrame({'total': [d75_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d75_999.empty or d75_999 == 0:
    df_d75_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=75, column=4).value = df_d75_999['total'].iloc[0]

## UNIDAD DE CAPTURA 9

# 3   AUTOMOVILES UNIDAD 9

In [216]:
unidad9_AUTOMOVILES = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_AUTOMOVILES['TOTAL'] = unidad9_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 3, 77),
    ("Ingresos Exceso de Pérdida", 3, 78),
    ("Participación Utilidades Reaseguro Cedido", 3, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 3, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 3, 81),
    ("Participación de Utilidades y Otros (resta)", 3, 82),
    ("Costo Exceso de Pérdida (resta)", 3, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
e77_005 = get_value(unidad9_AUTOMOVILES, "Comisiones de Reaseguro Cedido", 3)
e78_010 = get_value(unidad9_AUTOMOVILES, "Ingresos Exceso de Pérdida", 3)
e79_015 = get_value(unidad9_AUTOMOVILES, "Participación Utilidades Reaseguro Cedido", 3)
e80_020 = get_value(unidad9_AUTOMOVILES, "Intereses y Gastos Reconocidos por Reaseguradores", 3)
e81_025 = get_value(unidad9_AUTOMOVILES, "Intereses Reconocidos a Reaseguradores (resta)", 3)
e82_030 = get_value(unidad9_AUTOMOVILES, "Participación de Utilidades y Otros (resta)", 3)
e83_035 = get_value(unidad9_AUTOMOVILES, "Costo Exceso de Pérdida (resta)", 3)

e84_999 = (e77_005 + e78_010 + e79_015 + e80_020 - e81_025 - e82_030 - e83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=5, value=e84_999)

<Cell 'Hoja1'.E84>

# 4   SOAT UNIDAD 9


In [217]:
unidad9_SOAT = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_SOAT['TOTAL'] = unidad9_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 4, 77),
    ("Ingresos Exceso de Pérdida", 4, 78),
    ("Participación Utilidades Reaseguro Cedido", 4, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 4, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 4, 81),
    ("Participación de Utilidades y Otros (resta)", 4, 82),
    ("Costo Exceso de Pérdida (resta)", 4, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
f77_005 = get_value(unidad9_SOAT, "Comisiones de Reaseguro Cedido", 4)
f78_010 = get_value(unidad9_SOAT, "Ingresos Exceso de Pérdida", 4)
f79_015 = get_value(unidad9_SOAT, "Participación Utilidades Reaseguro Cedido", 4)
f80_020 = get_value(unidad9_SOAT, "Intereses y Gastos Reconocidos por Reaseguradores", 4)
f81_025 = get_value(unidad9_SOAT, "Intereses Reconocidos a Reaseguradores (resta)", 4)
f82_030 = get_value(unidad9_SOAT, "Participación de Utilidades y Otros (resta)", 4)
f83_035 = get_value(unidad9_SOAT, "Costo Exceso de Pérdida (resta)", 4)

f84_999 = (f77_005 + f78_010 + f79_015 + f80_020 - f81_025 - f82_030 - f83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=6, value=f84_999)

<Cell 'Hoja1'.F84>

# 5   CUMPLIMIENTO UNIDAD 9


In [218]:
unidad9_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_CUMPLIMIENTO['TOTAL'] = unidad9_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 5, 77),
    ("Ingresos Exceso de Pérdida", 5, 78),
    ("Participación Utilidades Reaseguro Cedido", 5, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 5, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 5, 81),
    ("Participación de Utilidades y Otros (resta)", 5, 82),
    ("Costo Exceso de Pérdida (resta)", 5, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
g77_005 = get_value(unidad9_CUMPLIMIENTO, "Comisiones de Reaseguro Cedido", 5)
g78_010 = get_value(unidad9_CUMPLIMIENTO, "Ingresos Exceso de Pérdida", 5)
g79_015 = get_value(unidad9_CUMPLIMIENTO, "Participación Utilidades Reaseguro Cedido", 5)
g80_020 = get_value(unidad9_CUMPLIMIENTO, "Intereses y Gastos Reconocidos por Reaseguradores", 5)
g81_025 = get_value(unidad9_CUMPLIMIENTO, "Intereses Reconocidos a Reaseguradores (resta)", 5)
g82_030 = get_value(unidad9_CUMPLIMIENTO, "Participación de Utilidades y Otros (resta)", 5)
g83_035 = get_value(unidad9_CUMPLIMIENTO, "Costo Exceso de Pérdida (resta)", 5)

g84_999 = (g77_005 + g78_010 + g79_015 + g80_020 - g81_025 - g82_030 - g83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=7, value=g84_999)

<Cell 'Hoja1'.G84>

#  6   RESPONSABILIDAD CIVIL UNIDAD 9

In [219]:
unidad9_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_RESPONSABILIDADCI['TOTAL'] = unidad9_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 6, 77),
    ("Ingresos Exceso de Pérdida", 6, 78),
    ("Participación Utilidades Reaseguro Cedido", 6, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 6, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 6, 81),
    ("Participación de Utilidades y Otros (resta)", 6, 82),
    ("Costo Exceso de Pérdida (resta)", 6, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
h77_005 = get_value(unidad9_RESPONSABILIDADCI, "Comisiones de Reaseguro Cedido", 6)
h78_010 = get_value(unidad9_RESPONSABILIDADCI, "Ingresos Exceso de Pérdida", 6)
h79_015 = get_value(unidad9_RESPONSABILIDADCI, "Participación Utilidades Reaseguro Cedido", 6)
h80_020 = get_value(unidad9_RESPONSABILIDADCI, "Intereses y Gastos Reconocidos por Reaseguradores", 6)
h81_025 = get_value(unidad9_RESPONSABILIDADCI, "Intereses Reconocidos a Reaseguradores (resta)", 6)
h82_030 = get_value(unidad9_RESPONSABILIDADCI, "Participación de Utilidades y Otros (resta)", 6)
h83_035 = get_value(unidad9_RESPONSABILIDADCI, "Costo Exceso de Pérdida (resta)", 6)

h84_999 = (h77_005 + h78_010 + h79_015 + h80_020 - h81_025 - h82_030 - h83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=8, value=h84_999)

<Cell 'Hoja1'.H84>

#  7   INCENDIO UNIDAD 9

In [220]:
unidad9_INCENDIO = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_INCENDIO['TOTAL'] = unidad9_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 7, 77),
    ("Ingresos Exceso de Pérdida", 7, 78),
    ("Participación Utilidades Reaseguro Cedido", 7, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 7, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 7, 81),
    ("Participación de Utilidades y Otros (resta)", 7, 82),
    ("Costo Exceso de Pérdida (resta)", 7, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
i77_005 = get_value(unidad9_INCENDIO, "Comisiones de Reaseguro Cedido", 7)
i78_010 = get_value(unidad9_INCENDIO, "Ingresos Exceso de Pérdida", 7)
i79_015 = get_value(unidad9_INCENDIO, "Participación Utilidades Reaseguro Cedido", 7)
i80_020 = get_value(unidad9_INCENDIO, "Intereses y Gastos Reconocidos por Reaseguradores", 7)
i81_025 = get_value(unidad9_INCENDIO, "Intereses Reconocidos a Reaseguradores (resta)", 7)
i82_030 = get_value(unidad9_INCENDIO, "Participación de Utilidades y Otros (resta)", 7)
i83_035 = get_value(unidad9_INCENDIO, "Costo Exceso de Pérdida (resta)", 7)

i84_999 = (i77_005 + i78_010 + i79_015 + i80_020 - i81_025 - i82_030 - i83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=9, value=i84_999)

<Cell 'Hoja1'.I84>

#  8   TERREMOTO UNIDAD 9

In [221]:
unidad9_TERREMOTO = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_TERREMOTO['TOTAL'] = unidad9_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 8, 77),
    ("Ingresos Exceso de Pérdida", 8, 78),
    ("Participación Utilidades Reaseguro Cedido", 8, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 8, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 8, 81),
    ("Participación de Utilidades y Otros (resta)", 8, 82),
    ("Costo Exceso de Pérdida (resta)", 8, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
j77_005 = get_value(unidad9_TERREMOTO, "Comisiones de Reaseguro Cedido", 8)
j78_010 = get_value(unidad9_TERREMOTO, "Ingresos Exceso de Pérdida", 8)
j79_015 = get_value(unidad9_TERREMOTO, "Participación Utilidades Reaseguro Cedido", 8)
j80_020 = get_value(unidad9_TERREMOTO, "Intereses y Gastos Reconocidos por Reaseguradores", 8)
j81_025 = get_value(unidad9_TERREMOTO, "Intereses Reconocidos a Reaseguradores (resta)", 8)
j82_030 = get_value(unidad9_TERREMOTO, "Participación de Utilidades y Otros (resta)", 8)
j83_035 = get_value(unidad9_TERREMOTO, "Costo Exceso de Pérdida (resta)", 8)

j84_999 = (j77_005 + j78_010 + j79_015 + j80_020 - j81_025 - j82_030 - j83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=10, value=j84_999)

<Cell 'Hoja1'.J84>

#  9   SUSTRACCION UNIDAD 9

In [222]:
unidad9_SUSTRACCION = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_SUSTRACCION['TOTAL'] = unidad9_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 9, 77),
    ("Ingresos Exceso de Pérdida", 9, 78),
    ("Participación Utilidades Reaseguro Cedido", 9, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 9, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 9, 81),
    ("Participación de Utilidades y Otros (resta)", 9, 82),
    ("Costo Exceso de Pérdida (resta)", 9, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
k77_005 = get_value(unidad9_SUSTRACCION, "Comisiones de Reaseguro Cedido", 9)
k78_010 = get_value(unidad9_SUSTRACCION, "Ingresos Exceso de Pérdida", 9)
k79_015 = get_value(unidad9_SUSTRACCION, "Participación Utilidades Reaseguro Cedido", 9)
k80_020 = get_value(unidad9_SUSTRACCION, "Intereses y Gastos Reconocidos por Reaseguradores", 9)
k81_025 = get_value(unidad9_SUSTRACCION, "Intereses Reconocidos a Reaseguradores (resta)", 9)
k82_030 = get_value(unidad9_SUSTRACCION, "Participación de Utilidades y Otros (resta)", 9)
k83_035 = get_value(unidad9_SUSTRACCION, "Costo Exceso de Pérdida (resta)", 9)

k84_999 = (k77_005 + k78_010 + k79_015 + k80_020 - k81_025 - k82_030 - k83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=11, value=k84_999)

<Cell 'Hoja1'.K84>

#  10  TRANSPORTE UNIDAD 9

In [223]:
unidad9_TRANSPORTE = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_TRANSPORTE['TOTAL'] = unidad9_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 10, 77),
    ("Ingresos Exceso de Pérdida", 10, 78),
    ("Participación Utilidades Reaseguro Cedido", 10, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 10, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 10, 81),
    ("Participación de Utilidades y Otros (resta)", 10, 82),
    ("Costo Exceso de Pérdida (resta)", 10, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
l77_005 = get_value(unidad9_TRANSPORTE, "Comisiones de Reaseguro Cedido", 10)
l78_010 = get_value(unidad9_TRANSPORTE, "Ingresos Exceso de Pérdida", 10)
l79_015 = get_value(unidad9_TRANSPORTE, "Participación Utilidades Reaseguro Cedido", 10)
l80_020 = get_value(unidad9_TRANSPORTE, "Intereses y Gastos Reconocidos por Reaseguradores", 10)
l81_025 = get_value(unidad9_TRANSPORTE, "Intereses Reconocidos a Reaseguradores (resta)", 10)
l82_030 = get_value(unidad9_TRANSPORTE, "Participación de Utilidades y Otros (resta)", 10)
l83_035 = get_value(unidad9_TRANSPORTE, "Costo Exceso de Pérdida (resta)", 10)

l84_999 = (l77_005 + l78_010 + l79_015 + l80_020 - l81_025 - l82_030 - l83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=12, value=l84_999)

<Cell 'Hoja1'.L84>

#  11  CORRIENTE DÉBIL UNIDAD 9

In [224]:
unidad9_CORRIENDEBIL = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_CORRIENDEBIL['TOTAL'] = unidad9_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 11, 77),
    ("Ingresos Exceso de Pérdida", 11, 78),
    ("Participación Utilidades Reaseguro Cedido", 11, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 11, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 11, 81),
    ("Participación de Utilidades y Otros (resta)", 11, 82),
    ("Costo Exceso de Pérdida (resta)", 11, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_CORRIENDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
m77_005 = get_value(unidad9_CORRIENDEBIL, "Comisiones de Reaseguro Cedido", 11)
m78_010 = get_value(unidad9_CORRIENDEBIL, "Ingresos Exceso de Pérdida", 11)
m79_015 = get_value(unidad9_CORRIENDEBIL, "Participación Utilidades Reaseguro Cedido", 11)
m80_020 = get_value(unidad9_CORRIENDEBIL, "Intereses y Gastos Reconocidos por Reaseguradores", 11)
m81_025 = get_value(unidad9_CORRIENDEBIL, "Intereses Reconocidos a Reaseguradores (resta)", 11)
m82_030 = get_value(unidad9_CORRIENDEBIL, "Participación de Utilidades y Otros (resta)", 11)
m83_035 = get_value(unidad9_CORRIENDEBIL, "Costo Exceso de Pérdida (resta)", 11)

m84_999 = (m77_005 + m78_010 + m79_015 + m80_020 - m81_025 - m82_030 - m83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=13, value=m84_999)

<Cell 'Hoja1'.M84>

#  12  TODO RIESGO CONTRATISTA UNIDAD 9

In [225]:
unidad9_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_TODORIESGOCONTRATISTA['TOTAL'] = unidad9_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 12, 77),
    ("Ingresos Exceso de Pérdida", 12, 78),
    ("Participación Utilidades Reaseguro Cedido", 12, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 12, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 12, 81),
    ("Participación de Utilidades y Otros (resta)", 12, 82),
    ("Costo Exceso de Pérdida (resta)", 12, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
n77_005 = get_value(unidad9_TODORIESGOCONTRATISTA, "Comisiones de Reaseguro Cedido", 12)
n78_010 = get_value(unidad9_TODORIESGOCONTRATISTA, "Ingresos Exceso de Pérdida", 12)
n79_015 = get_value(unidad9_TODORIESGOCONTRATISTA, "Participación Utilidades Reaseguro Cedido", 12)
n80_020 = get_value(unidad9_TODORIESGOCONTRATISTA, "Intereses y Gastos Reconocidos por Reaseguradores", 12)
n81_025 = get_value(unidad9_TODORIESGOCONTRATISTA, "Intereses Reconocidos a Reaseguradores (resta)", 12)
n82_030 = get_value(unidad9_TODORIESGOCONTRATISTA, "Participación de Utilidades y Otros (resta)", 12)
n83_035 = get_value(unidad9_TODORIESGOCONTRATISTA, "Costo Exceso de Pérdida (resta)", 12)

n84_999 = (n77_005 + n78_010 + n79_015 + n80_020 - n81_025 - n82_030 - n83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=14, value=n84_999)

<Cell 'Hoja1'.N84>

#  13  MANEJO UNIDAD 9

In [226]:
unidad9_MANEJO = bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_MANEJO['TOTAL'] = unidad9_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 13, 77),
    ("Ingresos Exceso de Pérdida", 13, 78),
    ("Participación Utilidades Reaseguro Cedido", 13, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 13, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 13, 81),
    ("Participación de Utilidades y Otros (resta)", 13, 82),
    ("Costo Exceso de Pérdida (resta)", 13, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
o77_005 = get_value(unidad9_MANEJO, "Comisiones de Reaseguro Cedido", 13)
o78_010 = get_value(unidad9_MANEJO, "Ingresos Exceso de Pérdida", 13)
o79_015 = get_value(unidad9_MANEJO, "Participación Utilidades Reaseguro Cedido", 13)
o80_020 = get_value(unidad9_MANEJO, "Intereses y Gastos Reconocidos por Reaseguradores", 13)
o81_025 = get_value(unidad9_MANEJO, "Intereses Reconocidos a Reaseguradores (resta)", 13)
o82_030 = get_value(unidad9_MANEJO, "Participación de Utilidades y Otros (resta)", 13)
o83_035 = get_value(unidad9_MANEJO, "Costo Exceso de Pérdida (resta)", 13)

o84_999 = (o77_005 + o78_010 + o79_015 + o80_020 - o81_025 - o82_030 - o83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=15, value=o84_999)

<Cell 'Hoja1'.O84>

#  14  LUCRO CESANTE UNIDAD 9

In [227]:
unidad9_LUCROCESANTE= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_LUCROCESANTE['TOTAL'] = unidad9_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 14, 77),
    ("Ingresos Exceso de Pérdida", 14, 78),
    ("Participación Utilidades Reaseguro Cedido", 14, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 14, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 14, 81),
    ("Participación de Utilidades y Otros (resta)", 14, 82),
    ("Costo Exceso de Pérdida (resta)", 14, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
p77_005 = get_value(unidad9_LUCROCESANTE, "Comisiones de Reaseguro Cedido", 14)
p78_010 = get_value(unidad9_LUCROCESANTE, "Ingresos Exceso de Pérdida", 14)
p79_015 = get_value(unidad9_LUCROCESANTE, "Participación Utilidades Reaseguro Cedido", 14)
p80_020 = get_value(unidad9_LUCROCESANTE, "Intereses y Gastos Reconocidos por Reaseguradores", 14)
p81_025 = get_value(unidad9_LUCROCESANTE, "Intereses Reconocidos a Reaseguradores (resta)", 14)
p82_030 = get_value(unidad9_LUCROCESANTE, "Participación de Utilidades y Otros (resta)", 14)
p83_035 = get_value(unidad9_LUCROCESANTE, "Costo Exceso de Pérdida (resta)", 14)

p84_999 = (p77_005 + p78_010 + p79_015 + p80_020 - p81_025 - p82_030 - p83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=16, value=p84_999)

<Cell 'Hoja1'.P84>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 9

In [228]:
unidad9_MONTAJEYROTURA= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_MONTAJEYROTURA['TOTAL'] = unidad9_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 15, 77),
    ("Ingresos Exceso de Pérdida", 15, 78),
    ("Participación Utilidades Reaseguro Cedido", 15, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 15, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 15, 81),
    ("Participación de Utilidades y Otros (resta)", 15, 82),
    ("Costo Exceso de Pérdida (resta)", 15, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
q77_005 = get_value(unidad9_MONTAJEYROTURA, "Comisiones de Reaseguro Cedido", 15)
q78_010 = get_value(unidad9_MONTAJEYROTURA, "Ingresos Exceso de Pérdida", 15)
q79_015 = get_value(unidad9_MONTAJEYROTURA, "Participación Utilidades Reaseguro Cedido", 15)
q80_020 = get_value(unidad9_MONTAJEYROTURA, "Intereses y Gastos Reconocidos por Reaseguradores", 15)
q81_025 = get_value(unidad9_MONTAJEYROTURA, "Intereses Reconocidos a Reaseguradores (resta)", 15)
q82_030 = get_value(unidad9_MONTAJEYROTURA, "Participación de Utilidades y Otros (resta)", 15)
q83_035 = get_value(unidad9_MONTAJEYROTURA, "Costo Exceso de Pérdida (resta)", 15)

q84_999 = (q77_005 + q78_010 + q79_015 + q80_020 - q81_025 - q82_030 - q83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=17, value=q84_999)

<Cell 'Hoja1'.Q84>

#  16  AVIACIÓN UNIDAD 9

In [229]:
unidad9_AVIACION= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_AVIACION['TOTAL'] = unidad9_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 16, 77),
    ("Ingresos Exceso de Pérdida", 16, 78),
    ("Participación Utilidades Reaseguro Cedido", 16, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 16, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 16, 81),
    ("Participación de Utilidades y Otros (resta)", 16, 82),
    ("Costo Exceso de Pérdida (resta)", 16, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
r77_005 = get_value(unidad9_AVIACION, "Comisiones de Reaseguro Cedido", 16)
r78_010 = get_value(unidad9_AVIACION, "Ingresos Exceso de Pérdida", 16)
r79_015 = get_value(unidad9_AVIACION, "Participación Utilidades Reaseguro Cedido", 16)
r80_020 = get_value(unidad9_AVIACION, "Intereses y Gastos Reconocidos por Reaseguradores", 16)
r81_025 = get_value(unidad9_AVIACION, "Intereses Reconocidos a Reaseguradores (resta)", 16)
r82_030 = get_value(unidad9_AVIACION, "Participación de Utilidades y Otros (resta)", 16)
r83_035 = get_value(unidad9_AVIACION, "Costo Exceso de Pérdida (resta)", 16)

r84_999 = (r77_005 + r78_010 + r79_015 + r80_020 - r81_025 - r82_030 - r83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=18, value=r84_999)

<Cell 'Hoja1'.R84>

#  17  NAVEGACIÓN Y CASCO UNIDAD 9

In [230]:
unidad9_NAVEGACION= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_NAVEGACION['TOTAL'] = unidad9_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 17, 77),
    ("Ingresos Exceso de Pérdida", 17, 78),
    ("Participación Utilidades Reaseguro Cedido", 17, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 17, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 17, 81),
    ("Participación de Utilidades y Otros (resta)", 17, 82),
    ("Costo Exceso de Pérdida (resta)", 17, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
s77_005 = get_value(unidad9_NAVEGACION, "Comisiones de Reaseguro Cedido", 17)
s78_010 = get_value(unidad9_NAVEGACION, "Ingresos Exceso de Pérdida", 17)
s79_015 = get_value(unidad9_NAVEGACION, "Participación Utilidades Reaseguro Cedido", 17)
s80_020 = get_value(unidad9_NAVEGACION, "Intereses y Gastos Reconocidos por Reaseguradores", 17)
s81_025 = get_value(unidad9_NAVEGACION, "Intereses Reconocidos a Reaseguradores (resta)", 17)
s82_030 = get_value(unidad9_NAVEGACION, "Participación de Utilidades y Otros (resta)", 17)
s83_035 = get_value(unidad9_NAVEGACION, "Costo Exceso de Pérdida (resta)", 17)

s84_999 = (s77_005 + s78_010 + s79_015 + s80_020 - s81_025 - s82_030 - s83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=19, value=s84_999)

<Cell 'Hoja1'.S84>

#  18  MINAS Y PETRÓLEOS UNIDAD 9

In [231]:
unidad9_MINAS= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_MINAS['TOTAL'] = unidad9_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 18, 77),
    ("Ingresos Exceso de Pérdida", 18, 78),
    ("Participación Utilidades Reaseguro Cedido", 18, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 18, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 18, 81),
    ("Participación de Utilidades y Otros (resta)", 18, 82),
    ("Costo Exceso de Pérdida (resta)", 18, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
t77_005 = get_value(unidad9_MINAS, "Comisiones de Reaseguro Cedido", 18)
t78_010 = get_value(unidad9_MINAS, "Ingresos Exceso de Pérdida", 18)
t79_015 = get_value(unidad9_MINAS, "Participación Utilidades Reaseguro Cedido", 18)
t80_020 = get_value(unidad9_MINAS, "Intereses y Gastos Reconocidos por Reaseguradores", 18)
t81_025 = get_value(unidad9_MINAS, "Intereses Reconocidos a Reaseguradores (resta)", 18)
t82_030 = get_value(unidad9_MINAS, "Participación de Utilidades y Otros (resta)", 18)
t83_035 = get_value(unidad9_MINAS, "Costo Exceso de Pérdida (resta)", 18)

t84_999 = (t77_005 + t78_010 + t79_015 + t80_020 - t81_025 - t82_030 - t83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=20, value=t84_999)

<Cell 'Hoja1'.T84>

#  19  VIDRIOS UNIDAD 9

In [232]:
unidad9_VIDRIOS= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_VIDRIOS['TOTAL'] = unidad9_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 19, 77),
    ("Ingresos Exceso de Pérdida", 19, 78),
    ("Participación Utilidades Reaseguro Cedido", 19, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 19, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 19, 81),
    ("Participación de Utilidades y Otros (resta)", 19, 82),
    ("Costo Exceso de Pérdida (resta)", 19, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
u77_005 = get_value(unidad9_VIDRIOS, "Comisiones de Reaseguro Cedido", 19)
u78_010 = get_value(unidad9_VIDRIOS, "Ingresos Exceso de Pérdida", 19)
u79_015 = get_value(unidad9_VIDRIOS, "Participación Utilidades Reaseguro Cedido", 19)
u80_020 = get_value(unidad9_VIDRIOS, "Intereses y Gastos Reconocidos por Reaseguradores", 19)
u81_025 = get_value(unidad9_VIDRIOS, "Intereses Reconocidos a Reaseguradores (resta)", 19)
u82_030 = get_value(unidad9_VIDRIOS, "Participación de Utilidades y Otros (resta)", 19)
u83_035 = get_value(unidad9_VIDRIOS, "Costo Exceso de Pérdida (resta)", 19)

u84_999 = (u77_005 + u78_010 + u79_015 + u80_020 - u81_025 - u82_030 - u83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=21, value=u84_999)

<Cell 'Hoja1'.U84>

#  20  CRÉDITO COMERCIAL UNIDAD 9

In [233]:
unidad9_CREDITOCOMERCIAL= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_CREDITOCOMERCIAL['TOTAL'] = unidad9_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 20, 77),
    ("Ingresos Exceso de Pérdida", 20, 78),
    ("Participación Utilidades Reaseguro Cedido", 20, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 20, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 20, 81),
    ("Participación de Utilidades y Otros (resta)", 20, 82),
    ("Costo Exceso de Pérdida (resta)", 20, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
v77_005 = get_value(unidad9_CREDITOCOMERCIAL, "Comisiones de Reaseguro Cedido", 20)
v78_010 = get_value(unidad9_CREDITOCOMERCIAL, "Ingresos Exceso de Pérdida", 20)
v79_015 = get_value(unidad9_CREDITOCOMERCIAL, "Participación Utilidades Reaseguro Cedido", 20)
v80_020 = get_value(unidad9_CREDITOCOMERCIAL, "Intereses y Gastos Reconocidos por Reaseguradores", 20)
v81_025 = get_value(unidad9_CREDITOCOMERCIAL, "Intereses Reconocidos a Reaseguradores (resta)", 20)
v82_030 = get_value(unidad9_CREDITOCOMERCIAL, "Participación de Utilidades y Otros (resta)", 20)
v83_035 = get_value(unidad9_CREDITOCOMERCIAL, "Costo Exceso de Pérdida (resta)", 20)

v84_999 = (v77_005 + v78_010 + v79_015 + v80_020 - v81_025 - v82_030 - v83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=22, value=v84_999)

<Cell 'Hoja1'.V84>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 9

In [234]:
unidad9_CREDITOEXPORTACION= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_CREDITOEXPORTACION['TOTAL'] = unidad9_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 21, 77),
    ("Ingresos Exceso de Pérdida", 21, 78),
    ("Participación Utilidades Reaseguro Cedido", 21, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 21, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 21, 81),
    ("Participación de Utilidades y Otros (resta)", 21, 82),
    ("Costo Exceso de Pérdida (resta)", 21, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
w77_005 = get_value(unidad9_CREDITOEXPORTACION, "Comisiones de Reaseguro Cedido", 21)
w78_010 = get_value(unidad9_CREDITOEXPORTACION, "Ingresos Exceso de Pérdida", 21)
w79_015 = get_value(unidad9_CREDITOEXPORTACION, "Participación Utilidades Reaseguro Cedido", 21)
w80_020 = get_value(unidad9_CREDITOEXPORTACION, "Intereses y Gastos Reconocidos por Reaseguradores", 21)
w81_025 = get_value(unidad9_CREDITOEXPORTACION, "Intereses Reconocidos a Reaseguradores (resta)", 21)
w82_030 = get_value(unidad9_CREDITOEXPORTACION, "Participación de Utilidades y Otros (resta)", 21)
w83_035 = get_value(unidad9_CREDITOEXPORTACION, "Costo Exceso de Pérdida (resta)", 21)

w84_999 = (w77_005 + w78_010 + w79_015 + w80_020 - w81_025 - w82_030 - w83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=23, value=w84_999)

<Cell 'Hoja1'.W84>

#  24  DESEMPLEO UNIDAD 9

In [235]:
unidad9_DESEMPLEO= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_DESEMPLEO['TOTAL'] = unidad9_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 24, 77),
    ("Ingresos Exceso de Pérdida", 24, 78),
    ("Participación Utilidades Reaseguro Cedido", 24, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 24, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 24, 81),
    ("Participación de Utilidades y Otros (resta)", 24, 82),
    ("Costo Exceso de Pérdida (resta)", 24, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
z77_005 = get_value(unidad9_DESEMPLEO, "Comisiones de Reaseguro Cedido", 24)
z78_010 = get_value(unidad9_DESEMPLEO, "Ingresos Exceso de Pérdida", 24)
z79_015 = get_value(unidad9_DESEMPLEO, "Participación Utilidades Reaseguro Cedido", 24)
z80_020 = get_value(unidad9_DESEMPLEO, "Intereses y Gastos Reconocidos por Reaseguradores", 24)
z81_025 = get_value(unidad9_DESEMPLEO, "Intereses Reconocidos a Reaseguradores (resta)", 24)
z82_030 = get_value(unidad9_DESEMPLEO, "Participación de Utilidades y Otros (resta)", 24)
z83_035 = get_value(unidad9_DESEMPLEO, "Costo Exceso de Pérdida (resta)", 24)

z84_999 = (z77_005 + z78_010 + z79_015 + z80_020 - z81_025 - z82_030 - z83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=26, value=z84_999)

<Cell 'Hoja1'.Z84>

#  25  HOGAR UNIDAD 9

In [236]:
unidad9_HOGAR= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_HOGAR['TOTAL'] = unidad9_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 25, 77),
    ("Ingresos Exceso de Pérdida", 25, 78),
    ("Participación Utilidades Reaseguro Cedido", 25, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 25, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 25, 81),
    ("Participación de Utilidades y Otros (resta)", 25, 82),
    ("Costo Exceso de Pérdida (resta)", 25, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aa77_005 = get_value(unidad9_HOGAR, "Comisiones de Reaseguro Cedido", 25)
aa78_010 = get_value(unidad9_HOGAR, "Ingresos Exceso de Pérdida", 25)
aa79_015 = get_value(unidad9_HOGAR, "Participación Utilidades Reaseguro Cedido", 25)
aa80_020 = get_value(unidad9_HOGAR, "Intereses y Gastos Reconocidos por Reaseguradores", 25)
aa81_025 = get_value(unidad9_HOGAR, "Intereses Reconocidos a Reaseguradores (resta)", 25)
aa82_030 = get_value(unidad9_HOGAR, "Participación de Utilidades y Otros (resta)", 25)
aa83_035 = get_value(unidad9_HOGAR, "Costo Exceso de Pérdida (resta)", 25)

aa84_999 = (aa77_005 + aa78_010 + aa79_015 + aa80_020 - aa81_025 - aa82_030 - aa83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=27, value=aa84_999)

<Cell 'Hoja1'.AA84>

# 42 AGROPECUARIO UNIDAD 9

In [237]:
unidad9_AGROPECUARIO= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_AGROPECUARIO['TOTAL'] = unidad9_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 42, 77),
    ("Ingresos Exceso de Pérdida", 42, 78),
    ("Participación Utilidades Reaseguro Cedido", 42, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 42, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 42, 81),
    ("Participación de Utilidades y Otros (resta)", 42, 82),
    ("Costo Exceso de Pérdida (resta)", 42, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ar77_005 = get_value(unidad9_AGROPECUARIO, "Comisiones de Reaseguro Cedido", 42)
ar78_010 = get_value(unidad9_AGROPECUARIO, "Ingresos Exceso de Pérdida", 42)
ar79_015 = get_value(unidad9_AGROPECUARIO, "Participación Utilidades Reaseguro Cedido", 42)
ar80_020 = get_value(unidad9_AGROPECUARIO, "Intereses y Gastos Reconocidos por Reaseguradores", 42)
ar81_025 = get_value(unidad9_AGROPECUARIO, "Intereses Reconocidos a Reaseguradores (resta)", 42)
ar82_030 = get_value(unidad9_AGROPECUARIO, "Participación de Utilidades y Otros (resta)", 42)
ar83_035 = get_value(unidad9_AGROPECUARIO, "Costo Exceso de Pérdida (resta)", 42)

ar84_999 = (ar77_005 + ar78_010 + ar79_015 + ar80_020 - ar81_025 - ar82_030 - ar83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=44, value=ar84_999)

<Cell 'Hoja1'.AR84>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 9

In [238]:
unidad9_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_OPERNOASIGNABLES['TOTAL'] = unidad9_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 43, 77),
    ("Ingresos Exceso de Pérdida", 43, 78),
    ("Participación Utilidades Reaseguro Cedido", 43, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 43, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 43, 81),
    ("Participación de Utilidades y Otros (resta)", 43, 82),
    ("Costo Exceso de Pérdida (resta)", 43, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as77_005 = get_value(unidad9_OPERNOASIGNABLES, "Comisiones de Reaseguro Cedido", 43)
as78_010 = get_value(unidad9_OPERNOASIGNABLES, "Ingresos Exceso de Pérdida", 43)
as79_015 = get_value(unidad9_OPERNOASIGNABLES, "Participación Utilidades Reaseguro Cedido", 43)
as80_020 = get_value(unidad9_OPERNOASIGNABLES, "Intereses y Gastos Reconocidos por Reaseguradores", 43)
as81_025 = get_value(unidad9_OPERNOASIGNABLES, "Intereses Reconocidos a Reaseguradores (resta)", 43)
as82_030 = get_value(unidad9_OPERNOASIGNABLES, "Participación de Utilidades y Otros (resta)", 43)
as83_035 = get_value(unidad9_OPERNOASIGNABLES, "Costo Exceso de Pérdida (resta)", 43)

as84_999 = (as77_005 + as78_010 + as79_015 + as80_020 - as81_025 - as82_030 - as83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=45, value=as84_999)

<Cell 'Hoja1'.AS84>

# 44 DECENAL UNIDAD 9

In [239]:
unidad9_DECENAL= bd[(bd['UNIDAD'] == 9) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_DECENAL['TOTAL'] = unidad9_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 44, 77),
    ("Ingresos Exceso de Pérdida", 44, 78),
    ("Participación Utilidades Reaseguro Cedido", 44, 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 44, 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 44, 81),
    ("Participación de Utilidades y Otros (resta)", 44, 82),
    ("Costo Exceso de Pérdida (resta)", 44, 83),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad9_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
at77_005 = get_value(unidad9_DECENAL, "Comisiones de Reaseguro Cedido", 44)
at78_010 = get_value(unidad9_DECENAL, "Ingresos Exceso de Pérdida", 44)
at79_015 = get_value(unidad9_DECENAL, "Participación Utilidades Reaseguro Cedido", 44)
at80_020 = get_value(unidad9_DECENAL, "Intereses y Gastos Reconocidos por Reaseguradores", 44)
at81_025 = get_value(unidad9_DECENAL, "Intereses Reconocidos a Reaseguradores (resta)", 44)
at82_030 = get_value(unidad9_DECENAL, "Participación de Utilidades y Otros (resta)", 44)
at83_035 = get_value(unidad9_DECENAL, "Costo Exceso de Pérdida (resta)", 44)

at84_999 = (at77_005 + at78_010 + at79_015 + at80_020 - at81_025 - at82_030 - at83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=46, value=at84_999)

<Cell 'Hoja1'.AT84>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 9

In [240]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au77_005 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au77_005, 47, 77)

au78_010 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au78_010, 47, 78)

au79_015 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au79_015, 47, 69)

au80_020 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au80_020, 47, 80)

au81_025 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au81_025, 47, 81)

au82_030 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au82_030, 47, 82)

au83_035 = unidad9_CUMPLIMIENTOESTATALES[(unidad9_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au83_035, 47, 72)


# Calcular ak32_999
au84_999 = au77_005['TOTAL'].sum() + au78_010['TOTAL'].sum() + au79_015['TOTAL'].sum() + au80_020['TOTAL'].sum() + au81_025['TOTAL'].sum() + \
           au82_030['TOTAL'].sum() + au83_035['TOTAL'].sum() 

au84_999 = abs(au84_999)
# Crear DataFrame para escribir en Excel
df_au84_999= pd.DataFrame({'total': [au84_999]})
write_to_excel(df_au84_999, 47, 84, 'total')


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 9

In [241]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av77_005 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av77_005, 48, 77)

av78_010 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av78_010, 48, 78)

av79_015 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av79_015, 48, 69)

av80_020 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av80_020, 48, 80)

av81_025 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av81_025, 48, 81)

av82_030 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av82_030, 48, 82)

av83_035 = unidad9_CUMPLIMIENTOEMPRESASSERV[(unidad9_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av83_035, 48, 72)


# Calcular ak32_999
av84_999 = av77_005['TOTAL'].sum() + av78_010['TOTAL'].sum() + av79_015['TOTAL'].sum() + av80_020['TOTAL'].sum() + av81_025['TOTAL'].sum() + \
           av82_030['TOTAL'].sum() + av83_035['TOTAL'].sum() 

av84_999 = abs(av84_999)
# Crear DataFrame para escribir en Excel
df_av84_999= pd.DataFrame({'total': [av84_999]})
write_to_excel(df_av84_999, 48, 84, 'total')


# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 9

In [242]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


aw77_005 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw77_005, 49, 77)

aw78_010 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw78_010, 49, 78)

aw79_015 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw79_015, 49, 69)

aw80_020 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw80_020, 49, 80)

aw81_025 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw81_025, 49, 81)

aw82_030 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw82_030, 49, 82)

aw83_035 = unidad9_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw83_035, 49, 72)


# Calcular ak32_999
aw84_999 = aw77_005['TOTAL'].sum() + aw78_010['TOTAL'].sum() + aw79_015['TOTAL'].sum() + aw80_020['TOTAL'].sum() + aw81_025['TOTAL'].sum() + \
           aw82_030['TOTAL'].sum() + aw83_035['TOTAL'].sum() 

aw84_999 = abs(aw84_999)
# Crear DataFrame para escribir en Excel
df_aw84_999= pd.DataFrame({'total': [aw84_999]})
write_to_excel(df_aw84_999, 49, 84, 'total')

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 9

In [243]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax77_005 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax77_005, 50, 77)

ax78_010 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax78_010, 50, 78)

ax79_015 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax79_015, 50, 69)

ax80_020 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax80_020, 50, 80)

ax81_025 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax81_025, 50, 81)

ax82_030 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax82_030, 50, 82)

ax83_035 = unidad9_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax83_035, 50, 72)


# Calcular ak32_999
ax84_999 = ax77_005['TOTAL'].sum() + ax78_010['TOTAL'].sum() + ax79_015['TOTAL'].sum() + ax80_020['TOTAL'].sum() + ax81_025['TOTAL'].sum() + \
           ax82_030['TOTAL'].sum() + ax83_035['TOTAL'].sum() 

ax84_999 = abs(ax84_999)
# Crear DataFrame para escribir en Excel
df_ax84_999= pd.DataFrame({'total': [ax84_999]})
write_to_excel(df_ax84_999, 50, 84, 'total')

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 9

In [244]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ay77_005 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay77_005, 51, 77)

ay78_010 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay78_010, 51, 78)

ay79_015 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay79_015, 51, 69)

ay80_020 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay80_020, 51, 80)

ay81_025 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay81_025, 51, 81)

ay82_030 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay82_030, 51, 82)

ay83_035 = unidad9_CUMPLIMIENTOCAUCIONES[(unidad9_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay83_035, 51, 72)


# Calcular ak32_999
ay84_999 = ay77_005['TOTAL'].sum() + ay78_010['TOTAL'].sum() + ay79_015['TOTAL'].sum() + ay80_020['TOTAL'].sum() + ay81_025['TOTAL'].sum() + \
           ay82_030['TOTAL'].sum() + ay83_035['TOTAL'].sum() 

ay84_999 = abs(ay84_999)
# Crear DataFrame para escribir en Excel
df_ay84_999= pd.DataFrame({'total': [ay84_999]})
write_to_excel(df_ay84_999, 51, 84, 'total')

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 9

In [245]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


az77_005 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az77_005, 52, 77)

az78_010 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az78_010, 52, 78)

az79_015 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az79_015, 52, 69)

az80_020 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az80_020, 52, 80)

az81_025 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az81_025, 52, 81)

az82_030 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az82_030, 52, 82)

az83_035 = unidad9_CUMPLIMIENTOARRENDAMIENTO[(unidad9_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az83_035, 52, 72)


# Calcular ak32_999
az84_999 = az77_005['TOTAL'].sum() + az78_010['TOTAL'].sum() + az79_015['TOTAL'].sum() + az80_020['TOTAL'].sum() + az81_025['TOTAL'].sum() + \
           ay82_030['TOTAL'].sum() + az83_035['TOTAL'].sum() 

az84_999 = abs(az84_999)
# Crear DataFrame para escribir en Excel
df_az84_999= pd.DataFrame({'total': [az84_999]})
write_to_excel(df_az84_999, 52, 84, 'total')

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 9

In [246]:
# Agrupar los datos según el concepto y el ramo
unidad9_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad9_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba77_005 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Comisiones de Reaseguro Cedido") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba77_005, 53, 77)

ba78_010 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ingresos Exceso de Pérdida") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba78_010, 53, 78)

ba79_015 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Participación Utilidades Reaseguro Cedido") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba79_015, 53, 69)

ba80_020 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Intereses y Gastos Reconocidos por Reaseguradores") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba80_020, 53, 80)

ba81_025 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Intereses Reconocidos a Reaseguradores (resta)") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba81_025, 53, 81)

ba82_030 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Participación de Utilidades y Otros (resta)") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba82_030, 53, 82)

ba83_035 = unidad9_CUMPLIMIENTOPARTICULARES[(unidad9_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Costo Exceso de Pérdida (resta)") & (unidad9_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba83_035, 53, 72)


# Calcular ak32_999
ba84_999 = ba77_005['TOTAL'].sum() + ba78_010['TOTAL'].sum() + ba79_015['TOTAL'].sum() + ba80_020['TOTAL'].sum() + ba81_025['TOTAL'].sum() + \
           ba82_030['TOTAL'].sum() + ba83_035['TOTAL'].sum() 

ba84_999 = abs(ba84_999)
# Crear DataFrame para escribir en Excel
df_ba84_999= pd.DataFrame({'total': [ba84_999]})
write_to_excel(df_ba84_999, 53, 84, 'total')

#  1   TOTAL UNIDAD 9 
 

In [247]:
c77_005 = sum([ws.cell(row=77, column=col).value if ws.cell(row=77, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=77, column=3).value = c77_005  # Multiplicar por -1 para asegurar que sea negativo

c78_010 = sum([ws.cell(row=78, column=col).value if ws.cell(row=78, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=78, column=3).value = c78_010 

c79_015 = sum([ws.cell(row=79, column=col).value if ws.cell(row=79, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=79, column=3).value = c79_015 

c80_020 = sum([ws.cell(row=80, column=col).value if ws.cell(row=80, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=80, column=3).value = c80_020  

c81_025 = sum([ws.cell(row=81, column=col).value if ws.cell(row=81, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=81, column=3).value = c81_025  

c82_030 = sum([ws.cell(row=82, column=col).value if ws.cell(row=82, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=82, column=3).value = c82_030  

c83_035 = sum([ws.cell(row=83, column=col).value if ws.cell(row=83, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=83, column=3).value = c83_035  


# Calcular el total general según la fórmula dada

c84_999 = (c77_005 + c78_010 + c79_015 + c80_020 - c81_025 - c82_030 - c83_035)


# Escribir el total general en Excel
ws.cell(row=84, column=3).value = c84_999

#  2   SUBTOTAL - RAMOS UNIDAD 9
 

In [248]:
d77_005 = sum([ws.cell(row=77, column=col).value if ws.cell(row=77, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=77, column=4).value = d77_005  

d78_010 = sum([ws.cell(row=78, column=col).value if ws.cell(row=78, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=78, column=4).value = d78_010 

d79_015 = sum([ws.cell(row=79, column=col).value if ws.cell(row=79, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=79, column=4).value = d79_015 

d80_020 = sum([ws.cell(row=80, column=col).value if ws.cell(row=80, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=80, column=4).value = d80_020  

d81_025 = sum([ws.cell(row=81, column=col).value if ws.cell(row=81, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=81, column=4).value = d81_025  

d82_030 = sum([ws.cell(row=82, column=col).value if ws.cell(row=82, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=82, column=4).value = d82_030  

d83_035 = sum([ws.cell(row=83, column=col).value if ws.cell(row=83, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=83, column=4).value = d83_035  


# Calcular el total general según la fórmula dada
d84_999 = (d77_005 + d78_010 + d79_015 + d80_020 - d81_025 - d82_030 - d83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=4).value = d84_999

## UNIDAD DE CAPTURA 10

#  3   AUTOMOVILES UNIDAD 10

In [249]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad10_AUTOMOVILES = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_AUTOMOVILES['TOTAL'] = unidad10_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",3, 86),
    ("Remuneración de administración de coaseguro",3, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",3, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",3, 89),
    ("Costos Riesgos Profesionales (resta)",3, 90),
    ("Contribuciones de Seguros (resta)",3, 91),
    ("Otros Egresos (resta)",3, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",3, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e86_005 = get_value(unidad10_AUTOMOVILES, "Otros productos de seguros", 3)
e87_010 = get_value(unidad10_AUTOMOVILES, "Remuneración de administración de coaseguro", 3)
e88_015 = get_value(unidad10_AUTOMOVILES, "Gastos de Administración de Coaseguro Aceptado (resta)", 3)
e89_020 = get_value(unidad10_AUTOMOVILES, "Provisión cuentas por cobrar actividad aseguradora (resta)", 3)
e90_025 = get_value(unidad10_AUTOMOVILES, "Costos Riesgos Profesionales (resta)", 3)
e91_030 = get_value(unidad10_AUTOMOVILES, "Contribuciones de Seguros (resta)", 3)
e92_035 = get_value(unidad10_AUTOMOVILES, "Otros Egresos (resta)", 3)
e93_040 = get_value(unidad10_AUTOMOVILES, "Recuperación provisión cuentas por cobrar actividad aseguradora", 3)

# Calcular ab94_999 usando la fórmula especificada
e94_999 = (e86_005 + e87_010 - e88_015 - e89_020 - e90_025 - e91_030 - e92_035 + e93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=5, value=e94_999)

<Cell 'Hoja1'.E94>

#  4   SOAT UNIDAD 10

In [250]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_SOAT = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_SOAT['TOTAL'] = unidad10_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",4, 86),
    ("Remuneración de administración de coaseguro",4, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",4, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",4, 89),
    ("Costos Riesgos Profesionales (resta)",4, 90),
    ("Contribuciones de Seguros (resta)",4, 91),
    ("Otros Egresos (resta)",4, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",4, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f86_005 = get_value(unidad10_SOAT, "Otros productos de seguros", 4)
f87_010 = get_value(unidad10_SOAT, "Remuneración de administración de coaseguro", 4)
f88_015 = get_value(unidad10_SOAT, "Gastos de Administración de Coaseguro Aceptado (resta)", 4)
f89_020 = get_value(unidad10_SOAT, "Provisión cuentas por cobrar actividad aseguradora (resta)", 4)
f90_025 = get_value(unidad10_SOAT, "Costos Riesgos Profesionales (resta)", 4)
f91_030 = get_value(unidad10_SOAT, "Contribuciones de Seguros (resta)", 4)
f92_035 = get_value(unidad10_SOAT, "Otros Egresos (resta)", 4)
f93_040 = get_value(unidad10_SOAT, "Recuperación provisión cuentas por cobrar actividad aseguradora", 4)

# Calcular ab94_999 usando la fórmula especificada
f94_999 = (f86_005 + f87_010 - f88_015 - f89_020 - f90_025 - f91_030 - f92_035 + f93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=6, value=f94_999)

<Cell 'Hoja1'.F94>

#  5   CUMPLIMIENTO UNIDAD 10

In [251]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_CUMPLIMIENTO['TOTAL'] = unidad10_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",5, 86),
    ("Remuneración de administración de coaseguro",5, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",5, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",5, 89),
    ("Costos Riesgos Profesionales (resta)",5, 90),
    ("Contribuciones de Seguros (resta)",5, 91),
    ("Otros Egresos (resta)",5, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",5, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
g86_005 = get_value(unidad10_CUMPLIMIENTO, "Otros productos de seguros", 5)
g87_010 = get_value(unidad10_CUMPLIMIENTO, "Remuneración de administración de coaseguro", 5)
g88_015 = get_value(unidad10_CUMPLIMIENTO, "Gastos de Administración de Coaseguro Aceptado (resta)", 5)
g89_020 = get_value(unidad10_CUMPLIMIENTO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 5)
g90_025 = get_value(unidad10_CUMPLIMIENTO, "Costos Riesgos Profesionales (resta)", 5)
g91_030 = get_value(unidad10_CUMPLIMIENTO, "Contribuciones de Seguros (resta)", 5)
g92_035 = get_value(unidad10_CUMPLIMIENTO, "Otros Egresos (resta)", 5)
g93_040 = get_value(unidad10_CUMPLIMIENTO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 5)

# Calcular ab94_999 usando la fórmula especificada
g94_999 = (g86_005 + g87_010 - g88_015 - g89_020 - g90_025 - g91_030 - g92_035 + g93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=7, value=g94_999)

<Cell 'Hoja1'.G94>

#  6   RESPONSABILIDAD CIVIL UNIDAD 10

In [252]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_RESPONSABILIDADCI['TOTAL'] = unidad10_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",6, 86),
    ("Remuneración de administración de coaseguro",6, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",6, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",6, 89),
    ("Costos Riesgos Profesionales (resta)",6, 90),
    ("Contribuciones de Seguros (resta)",6, 91),
    ("Otros Egresos (resta)",6, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",5, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
h86_005 = get_value(unidad10_RESPONSABILIDADCI, "Otros productos de seguros", 6)
h87_010 = get_value(unidad10_RESPONSABILIDADCI, "Remuneración de administración de coaseguro", 6)
h88_015 = get_value(unidad10_RESPONSABILIDADCI, "Gastos de Administración de Coaseguro Aceptado (resta)", 6)
h89_020 = get_value(unidad10_RESPONSABILIDADCI, "Provisión cuentas por cobrar actividad aseguradora (resta)", 6)
h90_025 = get_value(unidad10_RESPONSABILIDADCI, "Costos Riesgos Profesionales (resta)", 6)
h91_030 = get_value(unidad10_RESPONSABILIDADCI, "Contribuciones de Seguros (resta)", 6)
h92_035 = get_value(unidad10_RESPONSABILIDADCI, "Otros Egresos (resta)", 6)
h93_040 = get_value(unidad10_RESPONSABILIDADCI, "Recuperación provisión cuentas por cobrar actividad aseguradora", 6)

# Calcular ab94_999 usando la fórmula especificada
h94_999 = (h86_005 + h87_010 - h88_015 - h89_020 - h90_025 - h91_030 - h92_035 + h93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=8, value=h94_999)

<Cell 'Hoja1'.H94>

#  7   INCENDIO UNIDAD 10

In [253]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_INCENDIO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_INCENDIO['TOTAL'] = unidad10_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",7, 86),
    ("Remuneración de administración de coaseguro",7, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",7, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",7, 89),
    ("Costos Riesgos Profesionales (resta)",7, 90),
    ("Contribuciones de Seguros (resta)",7, 91),
    ("Otros Egresos (resta)",7, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",7, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
i86_005 = get_value(unidad10_INCENDIO, "Otros productos de seguros", 7)
i87_010 = get_value(unidad10_INCENDIO, "Remuneración de administración de coaseguro", 7)
i88_015 = get_value(unidad10_INCENDIO, "Gastos de Administración de Coaseguro Aceptado (resta)", 7)
i89_020 = get_value(unidad10_INCENDIO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 7)
i90_025 = get_value(unidad10_INCENDIO, "Costos Riesgos Profesionales (resta)", 7)
i91_030 = get_value(unidad10_INCENDIO, "Contribuciones de Seguros (resta)", 7)
i92_035 = get_value(unidad10_INCENDIO, "Otros Egresos (resta)", 7)
i93_040 = get_value(unidad10_INCENDIO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 7)

# Calcular ab94_999 usando la fórmula especificada
i94_999 = (i86_005 + i87_010 - i88_015 - i89_020 - i90_025 - i91_030 - i92_035 + i93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=9, value=i94_999)

<Cell 'Hoja1'.I94>

#  8   TERREMOTO UNIDAD 10

In [254]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_TERREMOTO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_TERREMOTO['TOTAL'] = unidad10_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",8, 86),
    ("Remuneración de administración de coaseguro",8, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",8, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",8, 89),
    ("Costos Riesgos Profesionales (resta)",8, 90),
    ("Contribuciones de Seguros (resta)",8, 91),
    ("Otros Egresos (resta)",8, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",8, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
j86_005 = get_value(unidad10_TERREMOTO, "Otros productos de seguros", 8)
j87_010 = get_value(unidad10_TERREMOTO, "Remuneración de administración de coaseguro", 8)
j88_015 = get_value(unidad10_TERREMOTO, "Gastos de Administración de Coaseguro Aceptado (resta)", 8)
j89_020 = get_value(unidad10_TERREMOTO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 8)
j90_025 = get_value(unidad10_TERREMOTO, "Costos Riesgos Profesionales (resta)", 8)
j91_030 = get_value(unidad10_TERREMOTO, "Contribuciones de Seguros (resta)", 8)
j92_035 = get_value(unidad10_TERREMOTO, "Otros Egresos (resta)", 8)
j93_040 = get_value(unidad10_TERREMOTO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 8)

# Calcular ab94_999 usando la fórmula especificada
j94_999 = (j86_005 + j87_010 - j88_015 - j89_020 - j90_025 - j91_030 - j92_035 + j93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=10, value=j94_999)

<Cell 'Hoja1'.J94>

#  9   SUSTRACCION UNIDAD 10

In [255]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_SUSTRACCION = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_SUSTRACCION['TOTAL'] = unidad10_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",9, 86),
    ("Remuneración de administración de coaseguro",9, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",9, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",9, 89),
    ("Costos Riesgos Profesionales (resta)",9, 90),
    ("Contribuciones de Seguros (resta)",9, 91),
    ("Otros Egresos (resta)",9, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",9, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
k86_005 = get_value(unidad10_SUSTRACCION, "Otros productos de seguros", 9)
k87_010 = get_value(unidad10_SUSTRACCION, "Remuneración de administración de coaseguro", 9)
k88_015 = get_value(unidad10_SUSTRACCION, "Gastos de Administración de Coaseguro Aceptado (resta)", 9)
k89_020 = get_value(unidad10_SUSTRACCION, "Provisión cuentas por cobrar actividad aseguradora (resta)", 9)
k90_025 = get_value(unidad10_SUSTRACCION, "Costos Riesgos Profesionales (resta)", 9)
k91_030 = get_value(unidad10_SUSTRACCION, "Contribuciones de Seguros (resta)", 9)
k92_035 = get_value(unidad10_SUSTRACCION, "Otros Egresos (resta)", 9)
k93_040 = get_value(unidad10_SUSTRACCION, "Recuperación provisión cuentas por cobrar actividad aseguradora", 9)

# Calcular ab94_999 usando la fórmula especificada
k94_999 = (k86_005 + k87_010 - k88_015 - k89_020 - k90_025 - k91_030 - k92_035 + k93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=11, value=k94_999)

<Cell 'Hoja1'.K94>

#  10  TRANSPORTE UNIDAD 10

In [256]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_TRANSPORTE = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_TRANSPORTE['TOTAL'] = unidad10_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",10, 86),
    ("Remuneración de administración de coaseguro",10, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",10, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",10, 89),
    ("Costos Riesgos Profesionales (resta)",10, 90),
    ("Contribuciones de Seguros (resta)",10, 91),
    ("Otros Egresos (resta)",10, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",10, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
l86_005 = get_value(unidad10_TRANSPORTE, "Otros productos de seguros", 10)
l87_010 = get_value(unidad10_TRANSPORTE, "Remuneración de administración de coaseguro", 10)
l88_015 = get_value(unidad10_TRANSPORTE, "Gastos de Administración de Coaseguro Aceptado (resta)", 10)
l89_020 = get_value(unidad10_TRANSPORTE, "Provisión cuentas por cobrar actividad aseguradora (resta)", 10)
l90_025 = get_value(unidad10_TRANSPORTE, "Costos Riesgos Profesionales (resta)", 10)
l91_030 = get_value(unidad10_TRANSPORTE, "Contribuciones de Seguros (resta)", 10)
l92_035 = get_value(unidad10_TRANSPORTE, "Otros Egresos (resta)", 10)
l93_040 = get_value(unidad10_TRANSPORTE, "Recuperación provisión cuentas por cobrar actividad aseguradora", 10)

# Calcular ab94_999 usando la fórmula especificada
l94_999 = (l86_005 + l87_010 - l88_015 - l89_020 - l90_025 - l91_030 - l92_035 + l93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=12, value=l94_999)

<Cell 'Hoja1'.L94>

#  11  CORRIENTE DÉBIL UNIDAD 10

In [257]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_CORRIENTEDEBIL = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_CORRIENTEDEBIL['TOTAL'] = unidad10_CORRIENTEDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",11, 86),
    ("Remuneración de administración de coaseguro",11, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",11, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",11, 89),
    ("Costos Riesgos Profesionales (resta)",11, 90),
    ("Contribuciones de Seguros (resta)",11, 91),
    ("Otros Egresos (resta)",11, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",11, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_CORRIENTEDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
m86_005 = get_value(unidad10_CORRIENTEDEBIL, "Otros productos de seguros", 11)
m87_010 = get_value(unidad10_CORRIENTEDEBIL, "Remuneración de administración de coaseguro", 11)
m88_015 = get_value(unidad10_CORRIENTEDEBIL, "Gastos de Administración de Coaseguro Aceptado (resta)", 11)
m89_020 = get_value(unidad10_CORRIENTEDEBIL, "Provisión cuentas por cobrar actividad aseguradora (resta)", 11)
m90_025 = get_value(unidad10_CORRIENTEDEBIL, "Costos Riesgos Profesionales (resta)", 11)
m91_030 = get_value(unidad10_CORRIENTEDEBIL, "Contribuciones de Seguros (resta)", 11)
m92_035 = get_value(unidad10_CORRIENTEDEBIL, "Otros Egresos (resta)", 11)
m93_040 = get_value(unidad10_CORRIENTEDEBIL, "Recuperación provisión cuentas por cobrar actividad aseguradora", 11)

# Calcular ab94_999 usando la fórmula especificada
m94_999 = (m86_005 + m87_010 - m88_015 - m89_020 - m90_025 - m91_030 - m92_035 + m93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=13, value=m94_999)

<Cell 'Hoja1'.M94>

#  12  TODO RIESGO CONTRATISTA UNIDAD 10

In [258]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_TODORIESGOCONTRATISTA['TOTAL'] = unidad10_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",12, 86),
    ("Remuneración de administración de coaseguro",12, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",12, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",12, 89),
    ("Costos Riesgos Profesionales (resta)",12, 90),
    ("Contribuciones de Seguros (resta)",12, 91),
    ("Otros Egresos (resta)",12, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",12, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
n86_005 = get_value(unidad10_TODORIESGOCONTRATISTA, "Otros productos de seguros", 12)
n87_010 = get_value(unidad10_TODORIESGOCONTRATISTA, "Remuneración de administración de coaseguro", 12)
n88_015 = get_value(unidad10_TODORIESGOCONTRATISTA, "Gastos de Administración de Coaseguro Aceptado (resta)", 12)
n89_020 = get_value(unidad10_TODORIESGOCONTRATISTA, "Provisión cuentas por cobrar actividad aseguradora (resta)", 12)
n90_025 = get_value(unidad10_TODORIESGOCONTRATISTA, "Costos Riesgos Profesionales (resta)", 12)
n91_030 = get_value(unidad10_TODORIESGOCONTRATISTA, "Contribuciones de Seguros (resta)", 12)
n92_035 = get_value(unidad10_TODORIESGOCONTRATISTA, "Otros Egresos (resta)", 12)
n93_040 = get_value(unidad10_TODORIESGOCONTRATISTA, "Recuperación provisión cuentas por cobrar actividad aseguradora", 12)

# Calcular ab94_999 usando la fórmula especificada
n94_999 = (n86_005 + n87_010 - n88_015 - n89_020 - n90_025 - n91_030 - n92_035 + n93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=14, value=n94_999)

<Cell 'Hoja1'.N94>

#  13  MANEJO UNIDAD 10

In [259]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_MANEJO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_MANEJO['TOTAL'] = unidad10_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",13, 86),
    ("Remuneración de administración de coaseguro",13, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",13, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",13, 89),
    ("Costos Riesgos Profesionales (resta)",13, 90),
    ("Contribuciones de Seguros (resta)",13, 91),
    ("Otros Egresos (resta)",13, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",13, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
o86_005 = get_value(unidad10_MANEJO, "Otros productos de seguros", 13)
o87_010 = get_value(unidad10_MANEJO, "Remuneración de administración de coaseguro", 13)
o88_015 = get_value(unidad10_MANEJO, "Gastos de Administración de Coaseguro Aceptado (resta)", 13)
o89_020 = get_value(unidad10_MANEJO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 13)
o90_025 = get_value(unidad10_MANEJO, "Costos Riesgos Profesionales (resta)", 13)
o91_030 = get_value(unidad10_MANEJO, "Contribuciones de Seguros (resta)", 13)
o92_035 = get_value(unidad10_MANEJO, "Otros Egresos (resta)", 13)
o93_040 = get_value(unidad10_MANEJO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 13)

# Calcular ab94_999 usando la fórmula especificada
o94_999 = (o86_005 + o87_010 - o88_015 - o89_020 - o90_025 - o91_030 - o92_035 + o93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=15, value=o94_999)

<Cell 'Hoja1'.O94>

#  14  LUCRO CESANTE UNIDAD 10

In [260]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_LUCROCESANTE = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_LUCROCESANTE['TOTAL'] = unidad10_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",14, 86),
    ("Remuneración de administración de coaseguro",14, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",14, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",14, 89),
    ("Costos Riesgos Profesionales (resta)",14, 90),
    ("Contribuciones de Seguros (resta)",14, 91),
    ("Otros Egresos (resta)",14, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",14, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
p86_005 = get_value(unidad10_LUCROCESANTE, "Otros productos de seguros", 14)
p87_010 = get_value(unidad10_LUCROCESANTE, "Remuneración de administración de coaseguro", 14)
p88_015 = get_value(unidad10_LUCROCESANTE, "Gastos de Administración de Coaseguro Aceptado (resta)", 14)
p89_020 = get_value(unidad10_LUCROCESANTE, "Provisión cuentas por cobrar actividad aseguradora (resta)", 14)
p90_025 = get_value(unidad10_LUCROCESANTE, "Costos Riesgos Profesionales (resta)", 14)
p91_030 = get_value(unidad10_LUCROCESANTE, "Contribuciones de Seguros (resta)", 14)
p92_035 = get_value(unidad10_LUCROCESANTE, "Otros Egresos (resta)", 14)
p93_040 = get_value(unidad10_LUCROCESANTE, "Recuperación provisión cuentas por cobrar actividad aseguradora", 14)

# Calcular ab94_999 usando la fórmula especificada
p94_999 = (p86_005 + p87_010 - p88_015 - p89_020 - p90_025 - p91_030 - p92_035 + p93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=16, value=p94_999)

<Cell 'Hoja1'.P94>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 10

In [261]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_MONTAJEYROTURA = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_MONTAJEYROTURA['TOTAL'] = unidad10_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",15, 86),
    ("Remuneración de administración de coaseguro",15, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",15, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",15, 89),
    ("Costos Riesgos Profesionales (resta)",15, 90),
    ("Contribuciones de Seguros (resta)",15, 91),
    ("Otros Egresos (resta)",15, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",15, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
q86_005 = get_value(unidad10_MONTAJEYROTURA, "Otros productos de seguros", 15)
q87_010 = get_value(unidad10_MONTAJEYROTURA, "Remuneración de administración de coaseguro", 15)
q88_015 = get_value(unidad10_MONTAJEYROTURA, "Gastos de Administración de Coaseguro Aceptado (resta)", 15)
q89_020 = get_value(unidad10_MONTAJEYROTURA, "Provisión cuentas por cobrar actividad aseguradora (resta)", 15)
q90_025 = get_value(unidad10_MONTAJEYROTURA, "Costos Riesgos Profesionales (resta)", 15)
q91_030 = get_value(unidad10_MONTAJEYROTURA, "Contribuciones de Seguros (resta)", 15)
q92_035 = get_value(unidad10_MONTAJEYROTURA, "Otros Egresos (resta)", 15)
q93_040 = get_value(unidad10_MONTAJEYROTURA, "Recuperación provisión cuentas por cobrar actividad aseguradora", 15)

# Calcular ab94_999 usando la fórmula especificada
q94_999 = (q86_005 + q87_010 - q88_015 - q89_020 - q90_025 - q91_030 - q92_035 + q93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=17, value=q94_999)

<Cell 'Hoja1'.Q94>

#  16  AVIACIÓN UNIDAD 10

In [262]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_AVIACION = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_AVIACION['TOTAL'] = unidad10_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",16, 86),
    ("Remuneración de administración de coaseguro",16, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",16, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",16, 89),
    ("Costos Riesgos Profesionales (resta)",16, 90),
    ("Contribuciones de Seguros (resta)",16, 91),
    ("Otros Egresos (resta)",16, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",16, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
r86_005 = get_value(unidad10_AVIACION, "Otros productos de seguros", 16)
r87_010 = get_value(unidad10_AVIACION, "Remuneración de administración de coaseguro", 16)
r88_015 = get_value(unidad10_AVIACION, "Gastos de Administración de Coaseguro Aceptado (resta)", 16)
r89_020 = get_value(unidad10_AVIACION, "Provisión cuentas por cobrar actividad aseguradora (resta)", 16)
r90_025 = get_value(unidad10_AVIACION, "Costos Riesgos Profesionales (resta)", 16)
r91_030 = get_value(unidad10_AVIACION, "Contribuciones de Seguros (resta)", 16)
r92_035 = get_value(unidad10_AVIACION, "Otros Egresos (resta)", 16)
r93_040 = get_value(unidad10_AVIACION, "Recuperación provisión cuentas por cobrar actividad aseguradora", 16)

# Calcular ab94_999 usando la fórmula especificada
r94_999 = (r86_005 + r87_010 - r88_015 - r89_020 - r90_025 - r91_030 - r92_035 + r93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=18, value=r94_999)

<Cell 'Hoja1'.R94>

#  17  NAVEGACIÓN Y CASCO UNIDAD 10

In [263]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_NAVEGACION = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_NAVEGACION['TOTAL'] = unidad10_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",17, 86),
    ("Remuneración de administración de coaseguro",17, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",17, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",17, 89),
    ("Costos Riesgos Profesionales (resta)",17, 90),
    ("Contribuciones de Seguros (resta)",17, 91),
    ("Otros Egresos (resta)",17, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",17, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
s86_005 = get_value(unidad10_NAVEGACION, "Otros productos de seguros", 17)
s87_010 = get_value(unidad10_NAVEGACION, "Remuneración de administración de coaseguro", 17)
s88_015 = get_value(unidad10_NAVEGACION, "Gastos de Administración de Coaseguro Aceptado (resta)", 17)
s89_020 = get_value(unidad10_NAVEGACION, "Provisión cuentas por cobrar actividad aseguradora (resta)", 17)
s90_025 = get_value(unidad10_NAVEGACION, "Costos Riesgos Profesionales (resta)", 17)
s91_030 = get_value(unidad10_NAVEGACION, "Contribuciones de Seguros (resta)", 17)
s92_035 = get_value(unidad10_NAVEGACION, "Otros Egresos (resta)", 17)
s93_040 = get_value(unidad10_NAVEGACION, "Recuperación provisión cuentas por cobrar actividad aseguradora", 17)

# Calcular ab94_999 usando la fórmula especificada
s94_999 = (s86_005 + s87_010 - s88_015 - s89_020 - s90_025 - s91_030 - s92_035 + s93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=19, value=s94_999)

<Cell 'Hoja1'.S94>

#  18  MINAS Y PETRÓLEOS UNIDAD 10

In [264]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_MINAS = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_MINAS['TOTAL'] = unidad10_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",18, 86),
    ("Remuneración de administración de coaseguro",18, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",18, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",18, 89),
    ("Costos Riesgos Profesionales (resta)",18, 90),
    ("Contribuciones de Seguros (resta)",18, 91),
    ("Otros Egresos (resta)",18, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",18, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
t86_005 = get_value(unidad10_MINAS, "Otros productos de seguros", 18)
t87_010 = get_value(unidad10_MINAS, "Remuneración de administración de coaseguro", 18)
t88_015 = get_value(unidad10_MINAS, "Gastos de Administración de Coaseguro Aceptado (resta)", 18)
t89_020 = get_value(unidad10_MINAS, "Provisión cuentas por cobrar actividad aseguradora (resta)", 18)
t90_025 = get_value(unidad10_MINAS, "Costos Riesgos Profesionales (resta)", 18)
t91_030 = get_value(unidad10_MINAS, "Contribuciones de Seguros (resta)", 18)
t92_035 = get_value(unidad10_MINAS, "Otros Egresos (resta)", 18)
t93_040 = get_value(unidad10_MINAS, "Recuperación provisión cuentas por cobrar actividad aseguradora", 18)

# Calcular ab94_999 usando la fórmula especificada
t94_999 = (t86_005 + t87_010 - t88_015 - t89_020 - t90_025 - t91_030 - t92_035 + t93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=20, value=t94_999)

<Cell 'Hoja1'.T94>

#  19  VIDRIOS UNIDAD 10

In [265]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_VIDRIOS = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_VIDRIOS['TOTAL'] = unidad10_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",19, 86),
    ("Remuneración de administración de coaseguro",19, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",19, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",19, 89),
    ("Costos Riesgos Profesionales (resta)",19, 90),
    ("Contribuciones de Seguros (resta)",19, 91),
    ("Otros Egresos (resta)",19, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",19, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
u86_005 = get_value(unidad10_VIDRIOS, "Otros productos de seguros", 19)
u87_010 = get_value(unidad10_VIDRIOS, "Remuneración de administración de coaseguro", 19)
u88_015 = get_value(unidad10_VIDRIOS, "Gastos de Administración de Coaseguro Aceptado (resta)", 19)
u89_020 = get_value(unidad10_VIDRIOS, "Provisión cuentas por cobrar actividad aseguradora (resta)", 19)
u90_025 = get_value(unidad10_VIDRIOS, "Costos Riesgos Profesionales (resta)", 19)
u91_030 = get_value(unidad10_VIDRIOS, "Contribuciones de Seguros (resta)", 19)
u92_035 = get_value(unidad10_VIDRIOS, "Otros Egresos (resta)", 19)
u93_040 = get_value(unidad10_VIDRIOS, "Recuperación provisión cuentas por cobrar actividad aseguradora", 19)

# Calcular ab94_999 usando la fórmula especificada
u94_999 = (u86_005 + u87_010 - u88_015 - u89_020 - u90_025 - u91_030 - u92_035 + u93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=21, value=u94_999)

<Cell 'Hoja1'.U94>

#  20  CRÉDITO COMERCIAL UNIDAD 10

In [266]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_CREDITOCOMERCIAL['TOTAL'] = unidad10_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",20, 86),
    ("Remuneración de administración de coaseguro",20, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",20, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",20, 89),
    ("Costos Riesgos Profesionales (resta)",20, 90),
    ("Contribuciones de Seguros (resta)",20, 91),
    ("Otros Egresos (resta)",20, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",20, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
v86_005 = get_value(unidad10_CREDITOCOMERCIAL, "Otros productos de seguros", 20)
v87_010 = get_value(unidad10_CREDITOCOMERCIAL, "Remuneración de administración de coaseguro", 20)
v88_015 = get_value(unidad10_CREDITOCOMERCIAL, "Gastos de Administración de Coaseguro Aceptado (resta)", 20)
v89_020 = get_value(unidad10_CREDITOCOMERCIAL, "Provisión cuentas por cobrar actividad aseguradora (resta)", 20)
v90_025 = get_value(unidad10_CREDITOCOMERCIAL, "Costos Riesgos Profesionales (resta)", 20)
v91_030 = get_value(unidad10_CREDITOCOMERCIAL, "Contribuciones de Seguros (resta)", 20)
v92_035 = get_value(unidad10_CREDITOCOMERCIAL, "Otros Egresos (resta)", 20)
v93_040 = get_value(unidad10_CREDITOCOMERCIAL, "Recuperación provisión cuentas por cobrar actividad aseguradora", 20)

# Calcular ab94_999 usando la fórmula especificada
v94_999 = (v86_005 + v87_010 - v88_015 - v89_020 - v90_025 - v91_030 - v92_035 + v93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=22, value=v94_999)

<Cell 'Hoja1'.V94>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 10

In [267]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_CREDITOEXPORTACION['TOTAL'] = unidad10_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",21, 86),
    ("Remuneración de administración de coaseguro",21, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",21, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",21, 89),
    ("Costos Riesgos Profesionales (resta)",21, 90),
    ("Contribuciones de Seguros (resta)",21, 91),
    ("Otros Egresos (resta)",21, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",21, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
w86_005 = get_value(unidad10_CREDITOEXPORTACION, "Otros productos de seguros", 21)
w87_010 = get_value(unidad10_CREDITOEXPORTACION, "Remuneración de administración de coaseguro", 21)
w88_015 = get_value(unidad10_CREDITOEXPORTACION, "Gastos de Administración de Coaseguro Aceptado (resta)", 21)
w89_020 = get_value(unidad10_CREDITOEXPORTACION, "Provisión cuentas por cobrar actividad aseguradora (resta)", 21)
w90_025 = get_value(unidad10_CREDITOEXPORTACION, "Costos Riesgos Profesionales (resta)", 21)
w91_030 = get_value(unidad10_CREDITOEXPORTACION, "Contribuciones de Seguros (resta)", 21)
w92_035 = get_value(unidad10_CREDITOEXPORTACION, "Otros Egresos (resta)", 21)
w93_040 = get_value(unidad10_CREDITOEXPORTACION, "Recuperación provisión cuentas por cobrar actividad aseguradora", 21)

# Calcular ab94_999 usando la fórmula especificada
w94_999 = (w86_005 + w87_010 - w88_015 - w89_020 - w90_025 - w91_030 - w92_035 + w93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=23, value=w94_999)

<Cell 'Hoja1'.W94>

#  24  DESEMPLEO UNIDAD 10

In [268]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_DESEMPLEO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_DESEMPLEO['TOTAL'] = unidad10_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",24, 86),
    ("Remuneración de administración de coaseguro",24, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",24, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",24, 89),
    ("Costos Riesgos Profesionales (resta)",24, 90),
    ("Contribuciones de Seguros (resta)",24, 91),
    ("Otros Egresos (resta)",24, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",24, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
z86_005 = get_value(unidad10_DESEMPLEO, "Otros productos de seguros", 24)
z87_010 = get_value(unidad10_DESEMPLEO, "Remuneración de administración de coaseguro", 24)
z88_015 = get_value(unidad10_DESEMPLEO, "Gastos de Administración de Coaseguro Aceptado (resta)", 24)
z89_020 = get_value(unidad10_DESEMPLEO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 24)
z90_025 = get_value(unidad10_DESEMPLEO, "Costos Riesgos Profesionales (resta)", 24)
z91_030 = get_value(unidad10_DESEMPLEO, "Contribuciones de Seguros (resta)", 24)
z92_035 = get_value(unidad10_DESEMPLEO, "Otros Egresos (resta)", 24)
z93_040 = get_value(unidad10_DESEMPLEO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 24)

# Calcular ab94_999 usando la fórmula especificada
z94_999 = (z86_005 + z87_010 - z88_015 - z89_020 - z90_025 - z91_030 - z92_035 + z93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=26, value=z94_999)

<Cell 'Hoja1'.Z94>

#  25  HOGAR UNIDAD 10

In [269]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_HOGAR = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_HOGAR['TOTAL'] = unidad10_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",25, 86),
    ("Remuneración de administración de coaseguro",25, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",25, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",25, 89),
    ("Costos Riesgos Profesionales (resta)",25, 90),
    ("Contribuciones de Seguros (resta)",25, 91),
    ("Otros Egresos (resta)",25, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",25, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aa86_005 = get_value(unidad10_HOGAR, "Otros productos de seguros", 25)
aa87_010 = get_value(unidad10_HOGAR, "Remuneración de administración de coaseguro", 25)
aa88_015 = get_value(unidad10_HOGAR, "Gastos de Administración de Coaseguro Aceptado (resta)", 25)
aa89_020 = get_value(unidad10_HOGAR, "Provisión cuentas por cobrar actividad aseguradora (resta)", 25)
aa90_025 = get_value(unidad10_HOGAR, "Costos Riesgos Profesionales (resta)", 25)
aa91_030 = get_value(unidad10_HOGAR, "Contribuciones de Seguros (resta)", 25)
aa92_035 = get_value(unidad10_HOGAR, "Otros Egresos (resta)", 25)
aa93_040 = get_value(unidad10_HOGAR, "Recuperación provisión cuentas por cobrar actividad aseguradora", 25)

# Calcular ab94_999 usando la fórmula especificada
aa94_999 = (aa86_005 + aa87_010 - aa88_015 - aa89_020 - aa90_025 - aa91_030 - aa92_035 + aa93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=27, value=aa94_999)

<Cell 'Hoja1'.AA94>

# 42 AGROPECUARIO UNIDAD 10

In [270]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_AGROPECUARIO = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_AGROPECUARIO['TOTAL'] = unidad10_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",42, 86),
    ("Remuneración de administración de coaseguro",42, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",42, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",42, 89),
    ("Costos Riesgos Profesionales (resta)",42, 90),
    ("Contribuciones de Seguros (resta)",42, 91),
    ("Otros Egresos (resta)",42, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",42, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ar86_005 = get_value(unidad10_AGROPECUARIO, "Otros productos de seguros", 42)
ar87_010 = get_value(unidad10_AGROPECUARIO, "Remuneración de administración de coaseguro", 42)
ar88_015 = get_value(unidad10_AGROPECUARIO, "Gastos de Administración de Coaseguro Aceptado (resta)", 42)
ar89_020 = get_value(unidad10_AGROPECUARIO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 42)
ar90_025 = get_value(unidad10_AGROPECUARIO, "Costos Riesgos Profesionales (resta)", 42)
ar91_030 = get_value(unidad10_AGROPECUARIO, "Contribuciones de Seguros (resta)", 42)
ar92_035 = get_value(unidad10_AGROPECUARIO, "Otros Egresos (resta)", 42)
ar93_040 = get_value(unidad10_AGROPECUARIO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 42)

# Calcular ab94_999 usando la fórmula especificada
ar94_999 = (ar86_005 + ar87_010 - ar88_015 - ar89_020 - ar90_025 - ar91_030 - ar92_035 + ar93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=44, value=ar94_999)

<Cell 'Hoja1'.AR94>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 10

In [271]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_OPERNOASIGNABLES['TOTAL'] = unidad10_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",43, 86),
    ("Remuneración de administración de coaseguro",43, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",43, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",43, 89),
    ("Costos Riesgos Profesionales (resta)",43, 90),
    ("Contribuciones de Seguros (resta)",43, 91),
    ("Otros Egresos (resta)",43, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",43, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as86_005 = get_value(unidad10_OPERNOASIGNABLES, "Otros productos de seguros", 43)
as87_010 = get_value(unidad10_OPERNOASIGNABLES, "Remuneración de administración de coaseguro", 43)
as88_015 = get_value(unidad10_OPERNOASIGNABLES, "Gastos de Administración de Coaseguro Aceptado (resta)", 43)
as89_020 = get_value(unidad10_OPERNOASIGNABLES, "Provisión cuentas por cobrar actividad aseguradora (resta)", 43)
as90_025 = get_value(unidad10_OPERNOASIGNABLES, "Costos Riesgos Profesionales (resta)", 43)
as91_030 = get_value(unidad10_OPERNOASIGNABLES, "Contribuciones de Seguros (resta)", 43)
as92_035 = get_value(unidad10_OPERNOASIGNABLES, "Otros Egresos (resta)", 42)
as93_040 = get_value(unidad10_OPERNOASIGNABLES, "Recuperación provisión cuentas por cobrar actividad aseguradora", 43)

# Calcular ab94_999 usando la fórmula especificada
as94_999 = (as86_005 + as87_010 - as88_015 - as89_020 - as90_025 - as91_030 - as92_035 + as93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=45, value=as94_999)

<Cell 'Hoja1'.AS94>

# 44 DECENAL UNIDAD 10

In [272]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_DECENAL = bd[(bd['UNIDAD'] == 10) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_DECENAL['TOTAL'] = unidad10_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros",44, 86),
    ("Remuneración de administración de coaseguro",44, 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)",44, 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)",44, 89),
    ("Costos Riesgos Profesionales (resta)",44, 90),
    ("Contribuciones de Seguros (resta)",44, 91),
    ("Otros Egresos (resta)",44, 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora",44, 93)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad10_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
at86_005 = get_value(unidad10_DECENAL, "Otros productos de seguros", 44)
at87_010 = get_value(unidad10_DECENAL, "Remuneración de administración de coaseguro", 44)
at88_015 = get_value(unidad10_DECENAL, "Gastos de Administración de Coaseguro Aceptado (resta)", 44)
at89_020 = get_value(unidad10_DECENAL, "Provisión cuentas por cobrar actividad aseguradora (resta)", 44)
at90_025 = get_value(unidad10_DECENAL, "Costos Riesgos Profesionales (resta)", 44)
at91_030 = get_value(unidad10_DECENAL, "Contribuciones de Seguros (resta)", 44)
at92_035 = get_value(unidad10_DECENAL, "Otros Egresos (resta)", 44)
at93_040 = get_value(unidad10_DECENAL, "Recuperación provisión cuentas por cobrar actividad aseguradora", 44)

# Calcular ab94_999 usando la fórmula especificada
at94_999 = (at86_005 + at87_010 - at88_015 - at89_020 - at90_025 - at91_030 - at92_035 + at93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=46, value=at94_999)

<Cell 'Hoja1'.AT94>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 10

In [273]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au86_005 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au86_005, 47, 86)

au87_010 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au87_010, 47, 87)

au88_015 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au88_015, 47, 88)

au89_020 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au89_020, 47, 89)

au90_025 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au90_025, 47, 90)

au91_030 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au91_030, 47, 91)

au92_035 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au92_035, 47, 92)

au93_040 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au93_040, 47, 93)

# Calcular ak32_999
au94_999 = au86_005['TOTAL'].sum() + au87_010['TOTAL'].sum() + au88_015['TOTAL'].sum() + au89_020['TOTAL'].sum() + au90_025['TOTAL'].sum() + \
           au91_030['TOTAL'].sum() + au92_035['TOTAL'].sum() + au93_040['TOTAL'].sum()

au94_999 = abs(au94_999)
# Crear DataFrame para escribir en Excel
df_au94_999= pd.DataFrame({'total': [au94_999]})
write_to_excel(df_au94_999, 47, 94, 'total')


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 10

In [274]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av86_005 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av86_005, 48, 86)

av87_010 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av87_010, 48, 87)

av88_015 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av88_015, 48, 88)

av89_020 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av89_020, 48, 89)

av90_025 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av90_025, 48, 90)

av91_030 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av91_030, 48, 91)

av92_035 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av92_035, 48, 92)

av93_040 = unidad10_CUMPLIMIENTOEMPRESASSERV[(unidad10_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av93_040, 48, 93)

# Calcular ak32_999
av94_999 = av86_005['TOTAL'].sum() + av87_010['TOTAL'].sum() + av88_015['TOTAL'].sum() + av89_020['TOTAL'].sum() + av90_025['TOTAL'].sum() + \
           av91_030['TOTAL'].sum() + av92_035['TOTAL'].sum() + av93_040['TOTAL'].sum()

av94_999 = abs(av94_999)
# Crear DataFrame para escribir en Excel
df_av94_999= pd.DataFrame({'total': [av94_999]})
write_to_excel(df_av94_999, 48, 94, 'total')

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 10

In [275]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


aw86_005 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw86_005, 49, 86)

aw87_010 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw87_010, 49, 87)

aw88_015 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw88_015, 49, 88)

aw89_020 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw89_020, 49, 89)

aw90_025 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw90_025, 49, 90)

aw91_030 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw91_030, 49, 91)

aw92_035 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw92_035, 49, 92)

aw93_040 = unidad10_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw93_040, 49, 93)

# Calcular ak32_999
aw94_999 = aw86_005['TOTAL'].sum() + aw87_010['TOTAL'].sum() + aw88_015['TOTAL'].sum() + aw89_020['TOTAL'].sum() + aw90_025['TOTAL'].sum() + \
           aw91_030['TOTAL'].sum() + aw92_035['TOTAL'].sum() + aw93_040['TOTAL'].sum()

aw94_999 = abs(aw94_999)
# Crear DataFrame para escribir en Excel
df_aw94_999= pd.DataFrame({'total': [aw94_999]})
write_to_excel(df_aw94_999, 49, 94, 'total')

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 10

In [276]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax86_005 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax86_005, 50, 86)

ax87_010 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax87_010, 50, 87)

ax88_015 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax88_015, 50, 88)

ax89_020 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax89_020, 50, 89)

ax90_025 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax90_025, 50, 90)

ax91_030 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax91_030, 50, 91)

ax92_035 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax92_035, 50, 92)

ax93_040 = unidad10_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax93_040, 50, 93)

# Calcular ak32_999
ax94_999 = ax86_005['TOTAL'].sum() + ax87_010['TOTAL'].sum() + ax88_015['TOTAL'].sum() + ax89_020['TOTAL'].sum() + ax90_025['TOTAL'].sum() + \
           ax91_030['TOTAL'].sum() + ax92_035['TOTAL'].sum() + ax93_040['TOTAL'].sum()

ax94_999 = abs(ax94_999)
# Crear DataFrame para escribir en Excel
df_ax94_999= pd.DataFrame({'total': [ax94_999]})
write_to_excel(df_ax94_999, 50, 94, 'total')

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 10

In [277]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOCAUCIONES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ay86_005 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay86_005, 51, 86)

ay87_010 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay87_010, 51, 87)

ay88_015 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay88_015, 51, 88)

ay89_020 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay89_020, 51, 89)

ay90_025 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay90_025, 51, 90)

ay91_030 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay91_030, 51, 91)

ay92_035 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay92_035, 51, 92)

ay93_040 = unidad10_CUMPLIMIENTOCAUCIONES[(unidad10_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay93_040, 51, 93)

# Calcular ak32_999
ay94_999 = ay86_005['TOTAL'].sum() + ay87_010['TOTAL'].sum() + ay88_015['TOTAL'].sum() + ay89_020['TOTAL'].sum() + ay90_025['TOTAL'].sum() + \
           ay91_030['TOTAL'].sum() + ay92_035['TOTAL'].sum() + ay93_040['TOTAL'].sum()

ay94_999 = abs(ay94_999)
# Crear DataFrame para escribir en Excel
df_ay94_999= pd.DataFrame({'total': [ay94_999]})
write_to_excel(df_ay94_999, 51, 94, 'total')

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 10

In [278]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOARRENDAMIENTO= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


az86_005 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az86_005, 52, 86)

az87_010 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az87_010, 52, 87)

az88_015 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az88_015, 52, 88)

az89_020 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az89_020, 52, 89)

az90_025 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az90_025, 52, 90)

az91_030 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az91_030, 52, 91)

az92_035 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az92_035, 52, 92)

az93_040 = unidad10_CUMPLIMIENTOARRENDAMIENTO[(unidad10_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az93_040, 52, 93)

# Calcular ak32_999
az94_999 = az86_005['TOTAL'].sum() + az87_010['TOTAL'].sum() + az88_015['TOTAL'].sum() + az89_020['TOTAL'].sum() + az90_025['TOTAL'].sum() + \
           az91_030['TOTAL'].sum() + az92_035['TOTAL'].sum() + az93_040['TOTAL'].sum()

az94_999 = abs(az94_999)
# Crear DataFrame para escribir en Excel
df_az94_999= pd.DataFrame({'total': [az94_999]})
write_to_excel(df_az94_999, 52, 94, 'total')

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 10

In [279]:
# Agrupar los datos según el concepto y el ramo
unidad10_CUMPLIMIENTOPARTICULARES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad10_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba86_005 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba86_005, 53, 86)

ba7_010 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Remuneración de administración de coaseguro") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba7_010, 53, 87)

ba88_015 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Gastos de Administración de Coaseguro Aceptado (resta)") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba88_015, 53, 88)

ba89_020 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Provisión cuentas por cobrar actividad aseguradora (resta)") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba89_020, 53, 89)

ba90_025 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Costos Riesgos Profesionales (resta)") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba90_025, 53, 90)

ba91_030 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Contribuciones de Seguros (resta)") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba91_030, 53, 91)

ba92_035 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Otros Egresos (resta)") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba92_035, 53, 92)

ba93_040 = unidad10_CUMPLIMIENTOPARTICULARES[(unidad10_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Recuperación provisión cuentas por cobrar actividad aseguradora") & (unidad10_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba93_040, 53, 93)

# Calcular ak32_999
ba94_999 = ba86_005['TOTAL'].sum() + ba7_010['TOTAL'].sum() + ba88_015['TOTAL'].sum() + ba89_020['TOTAL'].sum() + ba90_025['TOTAL'].sum() + \
           ba91_030['TOTAL'].sum() + ba92_035['TOTAL'].sum() + ba93_040['TOTAL'].sum()

ba94_999 = abs(ba94_999)
# Crear DataFrame para escribir en Excel
df_ba94_999= pd.DataFrame({'total': [ba94_999]})
write_to_excel(df_ba94_999, 53, 94, 'total')

#  1   TOTAL UNIDAD 10

In [280]:
c86_005 = sum([ws.cell(row=86, column=col).value if ws.cell(row=86, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=86, column=3).value = c86_005  # Multiplicar por -1 para asegurar que sea negativo

c87_010 = sum([ws.cell(row=87, column=col).value if ws.cell(row=87, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=87, column=3).value = c87_010 

c88_015 = sum([ws.cell(row=88, column=col).value if ws.cell(row=88, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=88, column=3).value = c88_015 

c89_020 = sum([ws.cell(row=89, column=col).value if ws.cell(row=89, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=89, column=3).value = c89_020  

c90_025 = sum([ws.cell(row=90, column=col).value if ws.cell(row=90, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=90, column=3).value = c90_025  

c91_030 = sum([ws.cell(row=91, column=col).value if ws.cell(row=91, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=91, column=3).value = c91_030  

c92_035 = sum([ws.cell(row=92, column=col).value if ws.cell(row=92, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=92, column=3).value = c92_035  

c93_040 = sum([ws.cell(row=93, column=col).value if ws.cell(row=93, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=93, column=3).value = c93_040 

# Calcular el total general según la fórmula dada

c94_999 = c86_005 + c87_010 - c88_015 - c89_020 - c90_025 - c91_030 - c92_035 + c93_040

# Escribir el total general en Excel
ws.cell(row=94, column=3).value = c94_999

#  2   SUBTOTAL - RAMOS UNIDAD 10

In [281]:
d86_005 = sum([ws.cell(row=86, column=col).value if ws.cell(row=86, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=86, column=4).value = d86_005  # Multiplicar por -1 para asegurar que sea negativo

d87_010 = sum([ws.cell(row=87, column=col).value if ws.cell(row=87, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=87, column=4).value = d87_010 

d88_015 = sum([ws.cell(row=88, column=col).value if ws.cell(row=88, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=88, column=4).value = d88_015 

d89_020 = sum([ws.cell(row=89, column=col).value if ws.cell(row=89, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=89, column=4).value = d89_020  

d90_025 = sum([ws.cell(row=90, column=col).value if ws.cell(row=90, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=90, column=4).value = d90_025  

d91_030 = sum([ws.cell(row=91, column=col).value if ws.cell(row=91, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=91, column=4).value = d91_030  

d92_035 = sum([ws.cell(row=92, column=col).value if ws.cell(row=92, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=92, column=4).value = d92_035  

d93_040 = sum([ws.cell(row=93, column=col).value if ws.cell(row=93, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=93, column=4).value = d93_040 

# Calcular el total general según la fórmula dada

d94_999 = d86_005 + d87_010 - d88_015 - d89_020 - d90_025 - d91_030 - d92_035 + d93_040

# Escribir el total general en Excel
ws.cell(row=94, column=4).value = d94_999

## UNIDADES DE CAPTURA 11

# 3   AUTOMOVILES UNIDAD 11

In [282]:
# Función ajustada: obtiene el valor del concepto y lo convierte a número
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Filtrar datos de la unidad GRAL y ramo 3 AUTOMOVILES
unidad11_AUTOMOVILES = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '3 AUTOMOVILES')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores específicos
e98_015 = get_value_super_ramo(unidad11_AUTOMOVILES, "Gastos de administración directos ramo")
e99_020 = get_value_super_ramo(unidad11_AUTOMOVILES, "Gastos de personal directos ramo")
e100_025 = get_value_super_ramo(unidad11_AUTOMOVILES, "Gastos de administración indirectos asignables a cada ramo")
e101_030 = get_value_super_ramo(unidad11_AUTOMOVILES, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en las celdas correspondientes de Excel
ws.cell(row=98, column=5, value=e98_015)   
ws.cell(row=99, column=5, value=e99_020)  
ws.cell(row=100, column=5, value=e100_025) 
ws.cell(row=101, column=5, value=e101_030) 

# Calcular y escribir el total
e102_999 = e98_015 + e99_020 + e100_025 + e101_030
ws.cell(row=102, column=5, value=e102_999)


<Cell 'Hoja1'.E102>

#  4   SOAT UNIDAD 11 

In [283]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_SOAT = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '4 SOAT')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

f98_015 = get_value_super_ramo(unidad11_SOAT, "Gastos de administración directos ramo")
f99_020 = get_value_super_ramo(unidad11_SOAT, "Gastos de personal directos ramo")
f100_025 = get_value_super_ramo(unidad11_SOAT, "Gastos de administración indirectos asignables a cada ramo")
f101_030 = get_value_super_ramo(unidad11_SOAT, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=6, value=f98_015)   
ws.cell(row=99, column=6, value=f99_020)  
ws.cell(row=100, column=6, value=f100_025) 
ws.cell(row=101, column=6, value=f101_030) 

# Calcular e102_999 usando la fórmula especificada
f102_999 = (f98_015 + f99_020 + f100_025 + f101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=6, value=f102_999)

<Cell 'Hoja1'.F102>

#  5   CUMPLIMIENTO UNIDAD 11

In [284]:
# Función ajustada: convierte a número y aplica abs()
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 5 CUMPLIMIENTO de la unidad GRAL
unidad11_CUMPLIMIENTO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '5 CUMPLIMIENTO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores específicos
g98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTO, "Gastos de administración directos ramo")
g99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTO, "Gastos de personal directos ramo")
g100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTO, "Gastos de administración indirectos asignables a cada ramo")
g101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTO, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en celdas de Excel, columna 7 (columna G)
ws.cell(row=98, column=7, value=g98_015)   
ws.cell(row=99, column=7, value=g99_020)  
ws.cell(row=100, column=7, value=g100_025) 
ws.cell(row=101, column=7, value=g101_030) 

# Calcular el total general y escribirlo en la fila 102
g102_999 = g98_015 + g99_020 + g100_025 + g101_030
ws.cell(row=102, column=7, value=g102_999)


<Cell 'Hoja1'.G102>

#  6   RESPONSABILIDAD CIVIL UNIDAD 11

In [285]:
# Función segura para obtener el valor numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores para el ramo 6 RESPONSABILIDAD CIVIL de la unidad GRAL
unidad11_RESPONSABILIDADCI = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '6 RESPONSABILIDAD CIVIL')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
h98_015 = get_value_super_ramo(unidad11_RESPONSABILIDADCI, "Gastos de administración directos ramo")
h99_020 = get_value_super_ramo(unidad11_RESPONSABILIDADCI, "Gastos de personal directos ramo")
h100_025 = get_value_super_ramo(unidad11_RESPONSABILIDADCI, "Gastos de administración indirectos asignables a cada ramo")
h101_030 = get_value_super_ramo(unidad11_RESPONSABILIDADCI, "Gastos de personal indirectos asignables a cada ramo")

# Escribir los valores en las celdas correspondientes (columna 8 = columna H)
ws.cell(row=98, column=8, value=h98_015)   
ws.cell(row=99, column=8, value=h99_020)  
ws.cell(row=100, column=8, value=h100_025) 
ws.cell(row=101, column=8, value=h101_030) 

# Calcular el total y escribirlo
h102_999 = h98_015 + h99_020 + h100_025 + h101_030
ws.cell(row=102, column=8, value=h102_999)


<Cell 'Hoja1'.H102>

#  7   INCENDIO UNIDAD 11

In [286]:
# Función segura para obtener el valor de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores para el ramo 7 INCENDIO de la unidad GRAL
unidad11_INCENDIO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '7 INCENDIO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
i98_015 = get_value_super_ramo(unidad11_INCENDIO, "Gastos de administración directos ramo")
i99_020 = get_value_super_ramo(unidad11_INCENDIO, "Gastos de personal directos ramo")
i100_025 = get_value_super_ramo(unidad11_INCENDIO, "Gastos de administración indirectos asignables a cada ramo")
i101_030 = get_value_super_ramo(unidad11_INCENDIO, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel, columna 9 (columna I)
ws.cell(row=98, column=9, value=i98_015)   
ws.cell(row=99, column=9, value=i99_020)  
ws.cell(row=100, column=9, value=i100_025) 
ws.cell(row=101, column=9, value=i101_030) 

# Calcular el total y escribirlo
i102_999 = i98_015 + i99_020 + i100_025 + i101_030
ws.cell(row=102, column=9, value=i102_999)


<Cell 'Hoja1'.I102>

#  8   TERREMOTO UNIDAD 11

In [287]:
# Función segura para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores por concepto para el ramo 8 TERREMOTO de la unidad GRAL
unidad11_TERREMOTO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '8 TERREMOTO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores para cada concepto
j98_015 = get_value_super_ramo(unidad11_TERREMOTO, "Gastos de administración directos ramo")
j99_020 = get_value_super_ramo(unidad11_TERREMOTO, "Gastos de personal directos ramo")
j100_025 = get_value_super_ramo(unidad11_TERREMOTO, "Gastos de administración indirectos asignables a cada ramo")
j101_030 = get_value_super_ramo(unidad11_TERREMOTO, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 10 (columna J)
ws.cell(row=98, column=10, value=j98_015)   
ws.cell(row=99, column=10, value=j99_020)  
ws.cell(row=100, column=10, value=j100_025) 
ws.cell(row=101, column=10, value=j101_030) 

# Calcular el total y escribirlo
j102_999 = j98_015 + j99_020 + j100_025 + j101_030
ws.cell(row=102, column=10, value=j102_999)


<Cell 'Hoja1'.J102>

#  9   SUSTRACCION UNIDAD 11

In [288]:
# Función segura para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores por concepto para el ramo 9 SUSTRACCION de la unidad GRAL
unidad11_SUSTRACCION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '9 SUSTRACCION')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
k98_015 = get_value_super_ramo(unidad11_SUSTRACCION, "Gastos de administración directos ramo")
k99_020 = get_value_super_ramo(unidad11_SUSTRACCION, "Gastos de personal directos ramo")
k100_025 = get_value_super_ramo(unidad11_SUSTRACCION, "Gastos de administración indirectos asignables a cada ramo")
k101_030 = get_value_super_ramo(unidad11_SUSTRACCION, "Gastos de personal indirectos asignables a cada ramo")

# Escribir los valores en las celdas de Excel, columna 11 (columna K)
ws.cell(row=98, column=11, value=k98_015)   
ws.cell(row=99, column=11, value=k99_020)  
ws.cell(row=100, column=11, value=k100_025) 
ws.cell(row=101, column=11, value=k101_030) 

# Calcular y escribir el total
k102_999 = k98_015 + k99_020 + k100_025 + k101_030
ws.cell(row=102, column=11, value=k102_999)


<Cell 'Hoja1'.K102>

#  10  TRANSPORTE UNIDAD 11

In [289]:
# Función segura para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores por concepto para el ramo 10 TRANSPORTE de la unidad GRAL
unidad11_TRANSPORTE = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '10 TRANSPORTE')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
l98_015 = get_value_super_ramo(unidad11_TRANSPORTE, "Gastos de administración directos ramo")
l99_020 = get_value_super_ramo(unidad11_TRANSPORTE, "Gastos de personal directos ramo")
l100_025 = get_value_super_ramo(unidad11_TRANSPORTE, "Gastos de administración indirectos asignables a cada ramo")
l101_030 = get_value_super_ramo(unidad11_TRANSPORTE, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en las celdas de Excel, columna 12 (columna L)
ws.cell(row=98, column=12, value=l98_015)   
ws.cell(row=99, column=12, value=l99_020)  
ws.cell(row=100, column=12, value=l100_025) 
ws.cell(row=101, column=12, value=l101_030) 

# Calcular y escribir el total
l102_999 = l98_015 + l99_020 + l100_025 + l101_030
ws.cell(row=102, column=12, value=l102_999)


<Cell 'Hoja1'.L102>

#  11  CORRIENTE DÉBIL UNIDAD 11

In [290]:
# Función segura para obtener el valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar valores por concepto para el ramo 11 CORRIENTE DÉBIL de la unidad GRAL
unidad11_CORRIENTEDEBIL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '11 CORRIENTE DÉBIL')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
m98_015 = get_value_super_ramo(unidad11_CORRIENTEDEBIL, "Gastos de administración directos ramo")
m99_020 = get_value_super_ramo(unidad11_CORRIENTEDEBIL, "Gastos de personal directos ramo")
m100_025 = get_value_super_ramo(unidad11_CORRIENTEDEBIL, "Gastos de administración indirectos asignables a cada ramo")
m101_030 = get_value_super_ramo(unidad11_CORRIENTEDEBIL, "Gastos de personal indirectos asignables a cada ramo")

# Escribir los valores en Excel, columna 13 (columna M)
ws.cell(row=98, column=13, value=m98_015)   
ws.cell(row=99, column=13, value=m99_020)  
ws.cell(row=100, column=13, value=m100_025) 
ws.cell(row=101, column=13, value=m101_030) 

# Calcular el total general y escribirlo
m102_999 = m98_015 + m99_020 + m100_025 + m101_030
ws.cell(row=102, column=13, value=m102_999)


<Cell 'Hoja1'.M102>

#  12  TODO RIESGO CONTRATISTA UNIDAD 11

In [291]:
# Función robusta para extraer el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 12 TODO RIESGO CONTRATISTA de la unidad GRAL
unidad11_TODORIESGOCONTRATISTA = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '12 TODO RIESGO CONTRATISTA')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores individuales
n98_015 = get_value_super_ramo(unidad11_TODORIESGOCONTRATISTA, "Gastos de administración directos ramo")
n99_020 = get_value_super_ramo(unidad11_TODORIESGOCONTRATISTA, "Gastos de personal directos ramo")
n100_025 = get_value_super_ramo(unidad11_TODORIESGOCONTRATISTA, "Gastos de administración indirectos asignables a cada ramo")
n101_030 = get_value_super_ramo(unidad11_TODORIESGOCONTRATISTA, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 14 (columna N)
ws.cell(row=98, column=14, value=n98_015)   
ws.cell(row=99, column=14, value=n99_020)  
ws.cell(row=100, column=14, value=n100_025) 
ws.cell(row=101, column=14, value=n101_030) 

# Calcular y escribir el total
n102_999 = n98_015 + n99_020 + n100_025 + n101_030
ws.cell(row=102, column=14, value=n102_999)


<Cell 'Hoja1'.N102>

#  13  MANEJO UNIDAD 11

In [292]:
# Función segura para obtener valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 13 MANEJO de la unidad GRAL
unidad11_MANEJO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '13 MANEJO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores
o98_015 = get_value_super_ramo(unidad11_MANEJO, "Gastos de administración directos ramo")
o99_020 = get_value_super_ramo(unidad11_MANEJO, "Gastos de personal directos ramo")
o100_025 = get_value_super_ramo(unidad11_MANEJO, "Gastos de administración indirectos asignables a cada ramo")
o101_030 = get_value_super_ramo(unidad11_MANEJO, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en columna 15 (columna O)
ws.cell(row=98, column=15, value=o98_015)   
ws.cell(row=99, column=15, value=o99_020)  
ws.cell(row=100, column=15, value=o100_025) 
ws.cell(row=101, column=15, value=o101_030) 

# Calcular y escribir el total
o102_999 = o98_015 + o99_020 + o100_025 + o101_030
ws.cell(row=102, column=15, value=o102_999)


<Cell 'Hoja1'.O102>

#  14  LUCRO CESANTE UNIDAD 11

In [293]:
# Función segura para obtener el valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 14 LUCRO CESANTE de la unidad GRAL
unidad11_LUCROCESANTE = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '14 LUCRO CESANTE')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores por concepto
p98_015 = get_value_super_ramo(unidad11_LUCROCESANTE, "Gastos de administración directos ramo")
p99_020 = get_value_super_ramo(unidad11_LUCROCESANTE, "Gastos de personal directos ramo")
p100_025 = get_value_super_ramo(unidad11_LUCROCESANTE, "Gastos de administración indirectos asignables a cada ramo")
p101_030 = get_value_super_ramo(unidad11_LUCROCESANTE, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel, columna 16 (columna P)
ws.cell(row=98, column=16, value=p98_015)   
ws.cell(row=99, column=16, value=p99_020)  
ws.cell(row=100, column=16, value=p100_025) 
ws.cell(row=101, column=16, value=p101_030) 

# Calcular y escribir el total
p102_999 = p98_015 + p99_020 + p100_025 + p101_030
ws.cell(row=102, column=16, value=p102_999)


<Cell 'Hoja1'.P102>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 11

In [294]:
# Función robusta para extraer valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 15 MONTAJE Y ROTURA DE MAQUINARIA
unidad11_MONTAJEYROTURA = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '15 MONTAJE Y ROTURA DE MAQUINARIA')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
q98_015 = get_value_super_ramo(unidad11_MONTAJEYROTURA, "Gastos de administración directos ramo")
q99_020 = get_value_super_ramo(unidad11_MONTAJEYROTURA, "Gastos de personal directos ramo")
q100_025 = get_value_super_ramo(unidad11_MONTAJEYROTURA, "Gastos de administración indirectos asignables a cada ramo")
q101_030 = get_value_super_ramo(unidad11_MONTAJEYROTURA, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en columna 17 (columna Q)
ws.cell(row=98, column=17, value=q98_015)   
ws.cell(row=99, column=17, value=q99_020)  
ws.cell(row=100, column=17, value=q100_025) 
ws.cell(row=101, column=17, value=q101_030) 

# Calcular y escribir total general
q102_999 = q98_015 + q99_020 + q100_025 + q101_030
ws.cell(row=102, column=17, value=q102_999)


<Cell 'Hoja1'.Q102>

#  16  AVIACIÓN UNIDAD 11

In [295]:
# Función robusta para extraer valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 16 AVIACIÓN de la unidad GRAL
unidad11_AVIACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '16 AVIACIÓN')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
r98_015 = get_value_super_ramo(unidad11_AVIACION, "Gastos de administración directos ramo")
r99_020 = get_value_super_ramo(unidad11_AVIACION, "Gastos de personal directos ramo")
r100_025 = get_value_super_ramo(unidad11_AVIACION, "Gastos de administración indirectos asignables a cada ramo")
r101_030 = get_value_super_ramo(unidad11_AVIACION, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en columna 18 (columna R)
ws.cell(row=98, column=18, value=r98_015)   
ws.cell(row=99, column=18, value=r99_020)  
ws.cell(row=100, column=18, value=r100_025) 
ws.cell(row=101, column=18, value=r101_030) 

# Calcular y escribir total general
r102_999 = r98_015 + r99_020 + r100_025 + r101_030
ws.cell(row=102, column=18, value=r102_999)


<Cell 'Hoja1'.R102>

#  17  NAVEGACIÓN Y CASCO UNIDAD 11

In [296]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 17 NAVEGACIÓN Y CASCO de la unidad GRAL
unidad11_NAVEGACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '17 NAVEGACIÓN Y CASCO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
s98_015 = get_value_super_ramo(unidad11_NAVEGACION, "Gastos de administración directos ramo")
s99_020 = get_value_super_ramo(unidad11_NAVEGACION, "Gastos de personal directos ramo")
s100_025 = get_value_super_ramo(unidad11_NAVEGACION, "Gastos de administración indirectos asignables a cada ramo")
s101_030 = get_value_super_ramo(unidad11_NAVEGACION, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 19 (columna S)
ws.cell(row=98, column=19, value=s98_015)   
ws.cell(row=99, column=19, value=s99_020)  
ws.cell(row=100, column=19, value=s100_025) 
ws.cell(row=101, column=19, value=s101_030) 

# Calcular y escribir el total
s102_999 = s98_015 + s99_020 + s100_025 + s101_030
ws.cell(row=102, column=19, value=s102_999)


<Cell 'Hoja1'.S102>

#  18  MINAS Y PETRÓLEOS UNIDAD 11

In [297]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 18 MINAS Y PETRÓLEOS de la unidad GRAL
unidad11_MINAS = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '18 MINAS Y PETRÓLEOS')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
t98_015 = get_value_super_ramo(unidad11_MINAS, "Gastos de administración directos ramo")
t99_020 = get_value_super_ramo(unidad11_MINAS, "Gastos de personal directos ramo")
t100_025 = get_value_super_ramo(unidad11_MINAS, "Gastos de administración indirectos asignables a cada ramo")
t101_030 = get_value_super_ramo(unidad11_MINAS, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 20 (columna T)
ws.cell(row=98, column=20, value=t98_015)   
ws.cell(row=99, column=20, value=t99_020)  
ws.cell(row=100, column=20, value=t100_025) 
ws.cell(row=101, column=20, value=t101_030) 

# Calcular y escribir el total
t102_999 = t98_015 + t99_020 + t100_025 + t101_030
ws.cell(row=102, column=20, value=t102_999)


<Cell 'Hoja1'.T102>

#  19  VIDRIOS UNIDAD 11

In [298]:
# Función segura para obtener el valor numérico absoluto de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 19 VIDRIOS de la unidad GRAL
unidad11_VIDRIOS = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '19 VIDRIOS')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
u98_015 = get_value_super_ramo(unidad11_VIDRIOS, "Gastos de administración directos ramo")
u99_020 = get_value_super_ramo(unidad11_VIDRIOS, "Gastos de personal directos ramo")
u100_025 = get_value_super_ramo(unidad11_VIDRIOS, "Gastos de administración indirectos asignables a cada ramo")
u101_030 = get_value_super_ramo(unidad11_VIDRIOS, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 21 (columna U)
ws.cell(row=98, column=21, value=u98_015)   
ws.cell(row=99, column=21, value=u99_020)  
ws.cell(row=100, column=21, value=u100_025) 
ws.cell(row=101, column=21, value=u101_030) 

# Calcular y escribir el total
u102_999 = u98_015 + u99_020 + u100_025 + u101_030
ws.cell(row=102, column=21, value=u102_999)


<Cell 'Hoja1'.U102>

#  20  CRÉDITO COMERCIAL UNIAD 11

In [299]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 20 CRÉDITO COMERCIAL de la unidad GRAL
unidad11_CREDITOCOMERCIAL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '20 CRÉDITO COMERCIAL')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
v98_015 = get_value_super_ramo(unidad11_CREDITOCOMERCIAL, "Gastos de administración directos ramo")
v99_020 = get_value_super_ramo(unidad11_CREDITOCOMERCIAL, "Gastos de personal directos ramo")
v100_025 = get_value_super_ramo(unidad11_CREDITOCOMERCIAL, "Gastos de administración indirectos asignables a cada ramo")
v101_030 = get_value_super_ramo(unidad11_CREDITOCOMERCIAL, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 22 (columna V)
ws.cell(row=98, column=22, value=v98_015)   
ws.cell(row=99, column=22, value=v99_020)  
ws.cell(row=100, column=22, value=v100_025) 
ws.cell(row=101, column=22, value=v101_030) 

# Calcular y escribir el total
v102_999 = v98_015 + v99_020 + v100_025 + v101_030
ws.cell(row=102, column=22, value=v102_999)


<Cell 'Hoja1'.V102>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 11

In [300]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 21 CRÉDITO A LA EXPORTACIÓN
unidad11_CREDITOEXPORTACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '21 CRÉDITO A LA EXPORTACIÓN')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
w98_015 = get_value_super_ramo(unidad11_CREDITOEXPORTACION, "Gastos de administración directos ramo")
w99_020 = get_value_super_ramo(unidad11_CREDITOEXPORTACION, "Gastos de personal directos ramo")
w100_025 = get_value_super_ramo(unidad11_CREDITOEXPORTACION, "Gastos de administración indirectos asignables a cada ramo")
w101_030 = get_value_super_ramo(unidad11_CREDITOEXPORTACION, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 23 (columna W)
ws.cell(row=98, column=23, value=w98_015)   
ws.cell(row=99, column=23, value=w99_020)  
ws.cell(row=100, column=23, value=w100_025) 
ws.cell(row=101, column=23, value=w101_030) 

# Calcular y escribir el total
w102_999 = w98_015 + w99_020 + w100_025 + w101_030
ws.cell(row=102, column=23, value=w102_999)


<Cell 'Hoja1'.W102>

#  24  DESEMPLEO UNIDAD 11

In [301]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_DESEMPLEO = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '24 DESEMPLEO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

z98_015 = get_value_super_ramo(unidad11_DESEMPLEO, "Gastos de administración directos ramo")
z99_020 = get_value_super_ramo(unidad11_DESEMPLEO, "Gastos de personal directos ramo")
z100_025 = get_value_super_ramo(unidad11_DESEMPLEO, "Gastos de administración indirectos asignables a cada ramo")
z101_030 = get_value_super_ramo(unidad11_DESEMPLEO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=26, value=z98_015)   
ws.cell(row=99, column=26, value=z99_020)  
ws.cell(row=100, column=26, value=z100_025) 
ws.cell(row=101, column=26, value=z101_030) 

# Calcular e102_999 usando la fórmula especificada
z102_999 = (z98_015 + z99_020 + z100_025 + z101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=26, value=z102_999)

<Cell 'Hoja1'.Z102>

#  25  HOGAR UNIDAD 11

In [302]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 25 HOGAR
unidad11_HOGAR = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '25 HOGAR')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
aa98_015 = get_value_super_ramo(unidad11_HOGAR, "Gastos de administración directos ramo")
aa99_020 = get_value_super_ramo(unidad11_HOGAR, "Gastos de personal directos ramo")
aa100_025 = get_value_super_ramo(unidad11_HOGAR, "Gastos de administración indirectos asignables a cada ramo")
aa101_030 = get_value_super_ramo(unidad11_HOGAR, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 27 (columna AA)
ws.cell(row=98, column=27, value=aa98_015)   
ws.cell(row=99, column=27, value=aa99_020)  
ws.cell(row=100, column=27, value=aa100_025) 
ws.cell(row=101, column=27, value=aa101_030) 

# Calcular y escribir el total
aa102_999 = aa98_015 + aa99_020 + aa100_025 + aa101_030
ws.cell(row=102, column=27, value=aa102_999)


<Cell 'Hoja1'.AA102>

# 42 AGROPECUARIO UNIDAD 11

In [303]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 42 AGROPECUARIO
unidad11_AGROPECUARIO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '42 AGROPECUARIO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
ar98_015 = get_value_super_ramo(unidad11_AGROPECUARIO, "Gastos de administración directos ramo")
ar99_020 = get_value_super_ramo(unidad11_AGROPECUARIO, "Gastos de personal directos ramo")
ar100_025 = get_value_super_ramo(unidad11_AGROPECUARIO, "Gastos de administración indirectos asignables a cada ramo")
ar101_030 = get_value_super_ramo(unidad11_AGROPECUARIO, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 44 (columna AR)
ws.cell(row=98, column=44, value=ar98_015)   
ws.cell(row=99, column=44, value=ar99_020)  
ws.cell(row=100, column=44, value=ar100_025) 
ws.cell(row=101, column=44, value=ar101_030) 

# Calcular y escribir el total
ar102_999 = ar98_015 + ar99_020 + ar100_025 + ar101_030
ws.cell(row=102, column=44, value=ar102_999)


<Cell 'Hoja1'.AR102>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 11

In [304]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 43 OPERACIONES NO ASIGNABLES A RAMOS
unidad11_OPERNOASIGNABLES = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '43 OPERACIONES NO ASIGANBLES A RAMOS')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
as98_015 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de administración directos ramo")
as99_020 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de personal directos ramo")
as100_025 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de administración indirectos asignables a cada ramo")
as101_030 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 45 (columna AS)
ws.cell(row=98, column=45, value=as98_015)   
ws.cell(row=99, column=45, value=as99_020)  
ws.cell(row=100, column=45, value=as100_025) 
ws.cell(row=101, column=45, value=as101_030) 

# Calcular y escribir el total
as102_999 = as98_015 + as99_020 + as100_025 + as101_030
ws.cell(row=102, column=45, value=as102_999)


<Cell 'Hoja1'.AS102>

# 44 DECENAL UNIDAD 11

In [305]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 44 DECENAL
unidad11_DECENAL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '44 DECENAL')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales
at98_015 = get_value_super_ramo(unidad11_DECENAL, "Gastos de administración directos ramo")
at99_020 = get_value_super_ramo(unidad11_DECENAL, "Gastos de personal directos ramo")
at100_025 = get_value_super_ramo(unidad11_DECENAL, "Gastos de administración indirectos asignables a cada ramo")
at101_030 = get_value_super_ramo(unidad11_DECENAL, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en la columna 46 (columna AT)
ws.cell(row=98, column=46, value=at98_015)   
ws.cell(row=99, column=46, value=at99_020)  
ws.cell(row=100, column=46, value=at100_025) 
ws.cell(row=101, column=46, value=at101_030) 

# Calcular y escribir el total
at102_999 = at98_015 + at99_020 + at100_025 + at101_030
ws.cell(row=102, column=46, value=at102_999)


<Cell 'Hoja1'.AT102>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 11

In [306]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOESTATALES = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '45 CUMPLIMIENTO - ENTIDADES ESTATALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

au98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOESTATALES, "Gastos de administración directos ramo")
au99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOESTATALES, "Gastos de personal directos ramo")
au100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOESTATALES, "Gastos de administración indirectos asignables a cada ramo")
au101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOESTATALES, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=47, value=au98_015)   
ws.cell(row=99, column=47, value=au99_020)  
ws.cell(row=100, column=47, value=au100_025) 
ws.cell(row=101, column=47, value=au101_030) 

# Calcular e102_999 usando la fórmula especificada
au102_999 = (au98_015 + au99_020 + au100_025 + au101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=47, value=au102_999)

<Cell 'Hoja1'.AU102>

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 11

In [307]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOEMPRESASSERV = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

av98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASSERV, "Gastos de administración directos ramo")
av99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASSERV, "Gastos de personal directos ramo")
av100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASSERV, "Gastos de administración indirectos asignables a cada ramo")
av101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASSERV, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=48, value=av98_015)   
ws.cell(row=99, column=48, value=av99_020)  
ws.cell(row=100, column=48, value=av100_025) 
ws.cell(row=101, column=48, value=av101_030) 

# Calcular e102_999 usando la fórmula especificada
av102_999 = (av98_015 + av99_020 + av100_025 + av101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=48, value=av102_999)

<Cell 'Hoja1'.AV102>

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 11

In [308]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOEMPRESASINDUSTRI = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aw98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASINDUSTRI, "Gastos de administración directos ramo")
aw99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASINDUSTRI, "Gastos de personal directos ramo")
aw100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASINDUSTRI, "Gastos de administración indirectos asignables a cada ramo")
aw101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASINDUSTRI, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=49, value=aw98_015)   
ws.cell(row=99, column=49, value=aw99_020)  
ws.cell(row=100, column=49, value=aw100_025) 
ws.cell(row=101, column=49, value=aw101_030) 

# Calcular e102_999 usando la fórmula especificada
aw102_999 = (aw98_015 + aw99_020 + aw100_025 + aw101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=49, value=aw102_999)

<Cell 'Hoja1'.AW102>

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 11

In [309]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOEMPRESASDISPOLEG = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '48 CUMPLIMIENTO - DISPOSICIONES LEGALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ax98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASDISPOLEG, "Gastos de administración directos ramo")
ax99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASDISPOLEG, "Gastos de personal directos ramo")
ax100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASDISPOLEG, "Gastos de administración indirectos asignables a cada ramo")
ax101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOEMPRESASDISPOLEG, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=50, value=ax98_015)   
ws.cell(row=99, column=50, value=ax99_020)  
ws.cell(row=100, column=50, value=ax100_025) 
ws.cell(row=101, column=50, value=ax101_030) 

# Calcular e102_999 usando la fórmula especificada
ax102_999 = (ax98_015 + ax99_020 + ax100_025 + ax101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=50, value=ax102_999)

<Cell 'Hoja1'.AX102>

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 11

In [310]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOCAUCIONES = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '49 CUMPLIMIENTO - CAUCIONES JUDICIALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ay98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOCAUCIONES, "Gastos de administración directos ramo")
ay99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOCAUCIONES, "Gastos de personal directos ramo")
ay100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOCAUCIONES, "Gastos de administración indirectos asignables a cada ramo")
ay101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOCAUCIONES, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=51, value=ay98_015)   
ws.cell(row=99, column=51, value=ay99_020)  
ws.cell(row=100, column=51, value=ay100_025) 
ws.cell(row=101, column=51, value=ay101_030) 

# Calcular e102_999 usando la fórmula especificada
ay102_999 = (ay98_015 + ay99_020 + ay100_025 + ay101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=51, value=ay102_999)

<Cell 'Hoja1'.AY102>

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 11

In [311]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOARRENDAMIENTO = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '50 CUMPLIMIENTO - ARRENDAMIENTO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

az98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOARRENDAMIENTO, "Gastos de administración directos ramo")
az99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOARRENDAMIENTO, "Gastos de personal directos ramo")
az100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOARRENDAMIENTO, "Gastos de administración indirectos asignables a cada ramo")
az101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOARRENDAMIENTO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=52, value=az98_015)   
ws.cell(row=99, column=52, value=az99_020)  
ws.cell(row=100, column=52, value=az100_025) 
ws.cell(row=101, column=52, value=az101_030) 

# Calcular e102_999 usando la fórmula especificada
az102_999 = (az98_015 + az99_020 + az100_025 + az101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=52, value=az102_999)

<Cell 'Hoja1'.AZ102>

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 11

In [312]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_CUMPLIMIENTOPARTICULARES = Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ba98_015 = get_value_super_ramo(unidad11_CUMPLIMIENTOPARTICULARES, "Gastos de administración directos ramo")
ba99_020 = get_value_super_ramo(unidad11_CUMPLIMIENTOPARTICULARES, "Gastos de personal directos ramo")
ba100_025 = get_value_super_ramo(unidad11_CUMPLIMIENTOPARTICULARES, "Gastos de administración indirectos asignables a cada ramo")
ba101_030 = get_value_super_ramo(unidad11_CUMPLIMIENTOPARTICULARES, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=53, value=ba98_015)   
ws.cell(row=99, column=53, value=ba99_020)  
ws.cell(row=100, column=53, value=ba100_025) 
ws.cell(row=101, column=53, value=ba101_030) 

# Calcular e102_999 usando la fórmula especificada
ba102_999 = (ba98_015 + ba99_020 + ba100_025 + ba101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=53, value=ba102_999)

<Cell 'Hoja1'.BA102>

#  1   TOTAL UNIDAD 11

In [372]:
# Función robusta para obtener el total absoluto de un concepto
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].sum())
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar por concepto para SOCIEDAD_DEST = 'GRAL'
unidad11_total = Base[Base['SOCIEDAD_DEST'] == 'GRAL'].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad11_total['VALOR'] = pd.to_numeric(unidad11_total['VALOR'], errors='coerce').fillna(0)

# Obtener los valores totales por concepto
c98_015 = get_value_total(unidad11_total, "Gastos de administración directos ramo")
c99_020 = get_value_total(unidad11_total, "Gastos de personal directos ramo")
c100_025 = get_value_total(unidad11_total, "Gastos de administración indirectos asignables a cada ramo")
c101_030 = get_value_total(unidad11_total, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en columna 3 (columna C)
ws.cell(row=98, column=3, value=c98_015)   
ws.cell(row=99, column=3, value=c99_020)  
ws.cell(row=100, column=3, value=c100_025) 
ws.cell(row=101, column=3, value=c101_030)

# Calcular y escribir el total general
c102_999 = c98_015 + c99_020 + c100_025 + c101_030
ws.cell(row=102, column=3, value=c102_999)

<Cell 'Hoja1'.C102>

#  2   SUBTOTAL - RAMOS UNIDAD 11

In [374]:
# Función robusta para obtener el total absoluto de un concepto
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].sum())
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar la base por concepto sin filtrar por RAMO_SUPER, solo SOCIEDAD_DEST = 'GRAL'
unidad11_subtotal = Base[Base['SOCIEDAD_DEST'] == 'GRAL'].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad11_subtotal['VALOR'] = pd.to_numeric(unidad11_subtotal['VALOR'], errors='coerce').fillna(0)

# Obtener los valores totales por concepto
d98_015 = get_value_total(unidad11_subtotal, "Gastos de administración directos ramo")
d99_020 = get_value_total(unidad11_subtotal, "Gastos de personal directos ramo")
d100_025 = get_value_total(unidad11_subtotal, "Gastos de administración indirectos asignables a cada ramo")
d101_030 = get_value_total(unidad11_subtotal, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel en columna 4 (columna D)
ws.cell(row=98, column=4, value=d98_015)   
ws.cell(row=99, column=4, value=d99_020)  
ws.cell(row=100, column=4, value=d100_025) 
ws.cell(row=101, column=4, value=d101_030)

# Calcular y escribir el total general
d102_999 = d98_015 + d99_020 + d100_025 + d101_030
ws.cell(row=102, column=4, value=d102_999)


<Cell 'Hoja1'.D102>

## UNIDAD DE CAPTURA 12

#  3   AUTOMOVILES UNIDAD 12

In [376]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar los datos del ramo 3 AUTOMOVILES para SOCIEDAD_DEST = 'GRAL'
unidad12_AUTOMOVILES = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '3 AUTOMOVILES')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_AUTOMOVILES['VALOR'] = pd.to_numeric(unidad12_AUTOMOVILES['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
e104_005 = get_value_super_ramo(unidad12_AUTOMOVILES, "Remuneración a favor de intermediarios dependientes e independientes")
e105_010 = get_value_super_ramo(unidad12_AUTOMOVILES, "Comisiones AFP")
e107_020 = get_value_super_ramo(unidad12_AUTOMOVILES, "Gastos en tarifación")
e108_025 = get_value_super_ramo(unidad12_AUTOMOVILES, "Remuneraciones por uso de red y a canales de comercialización")
e109_030 = get_value_super_ramo(unidad12_AUTOMOVILES, "Papelería y gastos de entrega")

# Escribir en Excel en columna 5 (columna E)
ws.cell(row=104, column=5, value=e104_005)   
ws.cell(row=105, column=5, value=e105_010)  
ws.cell(row=107, column=5, value=e107_020) 
ws.cell(row=108, column=5, value=e108_025) 
ws.cell(row=109, column=5, value=e109_030)

# Calcular y escribir el total
e110_999 = e104_005 + e105_010 + e107_020 + e108_025 + e109_030
ws.cell(row=110, column=5, value=e110_999)


<Cell 'Hoja1'.E110>

#  4   SOAT UNIDAD 12

In [381]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar los datos del ramo 4 SOAT para SOCIEDAD_DEST = 'GRAL'
unidad12_SOAT = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '4 SOAT')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_SOAT['VALOR'] = pd.to_numeric(unidad12_SOAT['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
f104_005 = get_value_super_ramo(unidad12_SOAT, "Remuneración a favor de intermediarios dependientes e independientes")
f105_010 = get_value_super_ramo(unidad12_SOAT, "Comisiones AFP")
f107_020 = get_value_super_ramo(unidad12_SOAT, "Gastos en tarifación")
f108_025 = get_value_super_ramo(unidad12_SOAT, "Remuneraciones por uso de red y a canales de comercialización")
f109_030 = get_value_super_ramo(unidad12_SOAT, "Papelería y gastos de entrega")

# Escribir en Excel en columna 6 (columna F)
ws.cell(row=104, column=6, value=f104_005)   
ws.cell(row=105, column=6, value=f105_010)  
ws.cell(row=107, column=6, value=f107_020) 
ws.cell(row=108, column=6, value=f108_025) 
ws.cell(row=109, column=6, value=f109_030)

# Calcular y escribir el total
f110_999 = f104_005 + f105_010 + f107_020 + f108_025 + f109_030
ws.cell(row=110, column=6, value=f110_999)

<Cell 'Hoja1'.F110>

#  5   CUMPLIMIENTO UNIDAD 12

In [383]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar los datos del ramo 5 CUMPLIMIENTO para SOCIEDAD_DEST = 'GRAL'
unidad12_CUMPLIMIENTO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '5 CUMPLIMIENTO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_CUMPLIMIENTO['VALOR'] = pd.to_numeric(unidad12_CUMPLIMIENTO['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
g104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTO, "Remuneración a favor de intermediarios dependientes e independientes")
g105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTO, "Comisiones AFP")
g107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTO, "Gastos en tarifación")
g108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTO, "Remuneraciones por uso de red y a canales de comercialización")
g109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTO, "Papelería y gastos de entrega")

# Escribir en Excel en columna 7 (columna G)
ws.cell(row=104, column=7, value=g104_005)   
ws.cell(row=105, column=7, value=g105_010)  
ws.cell(row=107, column=7, value=g107_020) 
ws.cell(row=108, column=7, value=g108_025) 
ws.cell(row=109, column=7, value=g109_030)

# Calcular y escribir el total
g110_999 = g104_005 + g105_010 + g107_020 + g108_025 + g109_030
ws.cell(row=110, column=7, value=g110_999)


<Cell 'Hoja1'.G110>

#  6   RESPONSABILIDAD CIVIL UNIDAD 12

In [385]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 6 RESPONSABILIDAD CIVIL para SOCIEDAD_DEST = 'GRAL'
unidad12_RESPONSABILIDADCI = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '6 RESPONSABILIDAD CIVIL')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_RESPONSABILIDADCI['VALOR'] = pd.to_numeric(unidad12_RESPONSABILIDADCI['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
h104_005 = get_value_super_ramo(unidad12_RESPONSABILIDADCI, "Remuneración a favor de intermediarios dependientes e independientes")
h105_010 = get_value_super_ramo(unidad12_RESPONSABILIDADCI, "Comisiones AFP")
h107_020 = get_value_super_ramo(unidad12_RESPONSABILIDADCI, "Gastos en tarifación")
h108_025 = get_value_super_ramo(unidad12_RESPONSABILIDADCI, "Remuneraciones por uso de red y a canales de comercialización")
h109_030 = get_value_super_ramo(unidad12_RESPONSABILIDADCI, "Papelería y gastos de entrega")

# Escribir en Excel en columna 8 (columna H)
ws.cell(row=104, column=8, value=h104_005)   
ws.cell(row=105, column=8, value=h105_010)  
ws.cell(row=107, column=8, value=h107_020) 
ws.cell(row=108, column=8, value=h108_025) 
ws.cell(row=109, column=8, value=h109_030)

# Calcular y escribir el total
h110_999 = h104_005 + h105_010 + h107_020 + h108_025 + h109_030
ws.cell(row=110, column=8, value=h110_999)


<Cell 'Hoja1'.H110>

#  7   INCENDIO UNIDAD 12

In [387]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 7 INCENDIO para SOCIEDAD_DEST = 'GRAL'
unidad12_INCENDIO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '7 INCENDIO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_INCENDIO['VALOR'] = pd.to_numeric(unidad12_INCENDIO['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
i104_005 = get_value_super_ramo(unidad12_INCENDIO, "Remuneración a favor de intermediarios dependientes e independientes")
i105_010 = get_value_super_ramo(unidad12_INCENDIO, "Comisiones AFP")
i107_020 = get_value_super_ramo(unidad12_INCENDIO, "Gastos en tarifación")
i108_025 = get_value_super_ramo(unidad12_INCENDIO, "Remuneraciones por uso de red y a canales de comercialización")
i109_030 = get_value_super_ramo(unidad12_INCENDIO, "Papelería y gastos de entrega")

# Escribir en Excel en columna 9 (columna I)
ws.cell(row=104, column=9, value=i104_005)   
ws.cell(row=105, column=9, value=i105_010)  
ws.cell(row=107, column=9, value=i107_020) 
ws.cell(row=108, column=9, value=i108_025) 
ws.cell(row=109, column=9, value=i109_030)

# Calcular y escribir el total
i110_999 = i104_005 + i105_010 + i107_020 + i108_025 + i109_030
ws.cell(row=110, column=9, value=i110_999)


<Cell 'Hoja1'.I110>

#  8   TERREMOTO UNIDAD 12

In [390]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 8 TERREMOTO para SOCIEDAD_DEST = 'GRAL'
unidad12_TERREMOTO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '8 TERREMOTO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_TERREMOTO['VALOR'] = pd.to_numeric(unidad12_TERREMOTO['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
j104_005 = get_value_super_ramo(unidad12_TERREMOTO, "Remuneración a favor de intermediarios dependientes e independientes")
j105_010 = get_value_super_ramo(unidad12_TERREMOTO, "Comisiones AFP")
j107_020 = get_value_super_ramo(unidad12_TERREMOTO, "Gastos en tarifación")
j108_025 = get_value_super_ramo(unidad12_TERREMOTO, "Remuneraciones por uso de red y a canales de comercialización")
j109_030 = get_value_super_ramo(unidad12_TERREMOTO, "Papelería y gastos de entrega")

# Escribir en Excel en columna 10 (columna J)
ws.cell(row=104, column=10, value=j104_005)   
ws.cell(row=105, column=10, value=j105_010)  
ws.cell(row=107, column=10, value=j107_020) 
ws.cell(row=108, column=10, value=j108_025) 
ws.cell(row=109, column=10, value=j109_030)  # ← corregido aquí

# Calcular y escribir el total
j110_999 = j104_005 + j105_010 + j107_020 + j108_025 + j109_030
ws.cell(row=110, column=10, value=j110_999)


<Cell 'Hoja1'.J110>

#  9   SUSTRACCION UNIDAD 12

In [392]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 9 SUSTRACCIÓN para SOCIEDAD_DEST = 'GRAL'
unidad12_SUSTRACCION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '9 SUSTRACCION')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_SUSTRACCION['VALOR'] = pd.to_numeric(unidad12_SUSTRACCION['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
k104_005 = get_value_super_ramo(unidad12_SUSTRACCION, "Remuneración a favor de intermediarios dependientes e independientes")
k105_010 = get_value_super_ramo(unidad12_SUSTRACCION, "Comisiones AFP")
k107_020 = get_value_super_ramo(unidad12_SUSTRACCION, "Gastos en tarifación")
k108_025 = get_value_super_ramo(unidad12_SUSTRACCION, "Remuneraciones por uso de red y a canales de comercialización")
k109_030 = get_value_super_ramo(unidad12_SUSTRACCION, "Papelería y gastos de entrega")

# Escribir en Excel en columna 11 (columna K)
ws.cell(row=104, column=11, value=k104_005)   
ws.cell(row=105, column=11, value=k105_010)  
ws.cell(row=107, column=11, value=k107_020) 
ws.cell(row=108, column=11, value=k108_025) 
ws.cell(row=109, column=11, value=k109_030)

# Calcular y escribir el total
k110_999 = k104_005 + k105_010 + k107_020 + k108_025 + k109_030
ws.cell(row=110, column=11, value=k110_999)


<Cell 'Hoja1'.K110>

#  10  TRANSPORTE UNIDAD 12

In [394]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 10 TRANSPORTE para SOCIEDAD_DEST = 'GRAL'
unidad12_TRANSPORTE = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '10 TRANSPORTE')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_TRANSPORTE['VALOR'] = pd.to_numeric(unidad12_TRANSPORTE['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
l104_005 = get_value_super_ramo(unidad12_TRANSPORTE, "Remuneración a favor de intermediarios dependientes e independientes")
l105_010 = get_value_super_ramo(unidad12_TRANSPORTE, "Comisiones AFP")
l107_020 = get_value_super_ramo(unidad12_TRANSPORTE, "Gastos en tarifación")
l108_025 = get_value_super_ramo(unidad12_TRANSPORTE, "Remuneraciones por uso de red y a canales de comercialización")
l109_030 = get_value_super_ramo(unidad12_TRANSPORTE, "Papelería y gastos de entrega")

# Escribir en Excel en columna 12 (columna L)
ws.cell(row=104, column=12, value=l104_005)   
ws.cell(row=105, column=12, value=l105_010)  
ws.cell(row=107, column=12, value=l107_020) 
ws.cell(row=108, column=12, value=l108_025)   # ← corregido aquí
ws.cell(row=109, column=12, value=l109_030)   # ← corregido aquí

# Calcular y escribir el total
l110_999 = l104_005 + l105_010 + l107_020 + l108_025 + l109_030
ws.cell(row=110, column=12, value=l110_999)


<Cell 'Hoja1'.L110>

#  11  CORRIENTE DÉBIL UNIDAD 12

In [396]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 11 CORRIENTE DÉBIL para SOCIEDAD_DEST = 'GRAL'
unidad12_CORRIENTEDEBIL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '11 CORRIENTE DÉBIL')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_CORRIENTEDEBIL['VALOR'] = pd.to_numeric(unidad12_CORRIENTEDEBIL['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
m104_005 = get_value_super_ramo(unidad12_CORRIENTEDEBIL, "Remuneración a favor de intermediarios dependientes e independientes")
m105_010 = get_value_super_ramo(unidad12_CORRIENTEDEBIL, "Comisiones AFP")
m107_020 = get_value_super_ramo(unidad12_CORRIENTEDEBIL, "Gastos en tarifación")
m108_025 = get_value_super_ramo(unidad12_CORRIENTEDEBIL, "Remuneraciones por uso de red y a canales de comercialización")
m109_030 = get_value_super_ramo(unidad12_CORRIENTEDEBIL, "Papelería y gastos de entrega")

# Escribir en Excel en columna 13 (columna M)
ws.cell(row=104, column=13, value=m104_005)   
ws.cell(row=105, column=13, value=m105_010)  
ws.cell(row=107, column=13, value=m107_020) 
ws.cell(row=108, column=13, value=m108_025) 
ws.cell(row=109, column=13, value=m109_030)

# Calcular y escribir el total
m110_999 = m104_005 + m105_010 + m107_020 + m108_025 + m109_030
ws.cell(row=110, column=13, value=m110_999)


<Cell 'Hoja1'.M110>

#  12  TODO RIESGO CONTRATISTA UNIDAD 12

In [398]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 12 TODO RIESGO CONTRATISTA
unidad12_TODORIESGOCONTRATISTAL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '12 TODO RIESGO CONTRATISTA')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_TODORIESGOCONTRATISTAL['VALOR'] = pd.to_numeric(unidad12_TODORIESGOCONTRATISTAL['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
n104_005 = get_value_super_ramo(unidad12_TODORIESGOCONTRATISTAL, "Remuneración a favor de intermediarios dependientes e independientes")
n105_010 = get_value_super_ramo(unidad12_TODORIESGOCONTRATISTAL, "Comisiones AFP")
n107_020 = get_value_super_ramo(unidad12_TODORIESGOCONTRATISTAL, "Gastos en tarifación")
n108_025 = get_value_super_ramo(unidad12_TODORIESGOCONTRATISTAL, "Remuneraciones por uso de red y a canales de comercialización")
n109_030 = get_value_super_ramo(unidad12_TODORIESGOCONTRATISTAL, "Papelería y gastos de entrega")

# Escribir en Excel en columna 14 (columna N)
ws.cell(row=104, column=14, value=n104_005)   
ws.cell(row=105, column=14, value=n105_010)  
ws.cell(row=107, column=14, value=n107_020) 
ws.cell(row=108, column=14, value=n108_025) 
ws.cell(row=109, column=14, value=n109_030)

# Calcular y escribir el total
n110_999 = n104_005 + n105_010 + n107_020 + n108_025 + n109_030
ws.cell(row=110, column=14, value=n110_999)


<Cell 'Hoja1'.N110>

#  13  MANEJO UNIDAD 12

In [400]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 13 MANEJO
unidad12_MANEJO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '13 MANEJO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_MANEJO['VALOR'] = pd.to_numeric(unidad12_MANEJO['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
o104_005 = get_value_super_ramo(unidad12_MANEJO, "Remuneración a favor de intermediarios dependientes e independientes")
o105_010 = get_value_super_ramo(unidad12_MANEJO, "Comisiones AFP")
o107_020 = get_value_super_ramo(unidad12_MANEJO, "Gastos en tarifación")
o108_025 = get_value_super_ramo(unidad12_MANEJO, "Remuneraciones por uso de red y a canales de comercialización")
o109_030 = get_value_super_ramo(unidad12_MANEJO, "Papelería y gastos de entrega")

# Escribir en Excel en columna 15 (columna O)
ws.cell(row=104, column=15, value=o104_005)   
ws.cell(row=105, column=15, value=o105_010)  
ws.cell(row=107, column=15, value=o107_020) 
ws.cell(row=108, column=15, value=o108_025) 
ws.cell(row=109, column=15, value=o109_030)

# Calcular y escribir el total
o110_999 = o104_005 + o105_010 + o107_020 + o108_025 + o109_030
ws.cell(row=110, column=15, value=o110_999)


<Cell 'Hoja1'.O110>

#  14  LUCRO CESANTE UNIDAD 12

In [402]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar datos del ramo 14 LUCRO CESANTE para SOCIEDAD_DEST = 'GRAL'
unidad12_LUCROCESANTE = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '14 LUCRO CESANTE')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_LUCROCESANTE['VALOR'] = pd.to_numeric(unidad12_LUCROCESANTE['VALOR'], errors='coerce').fillna(0)

# Obtener valores por concepto
p104_005 = get_value_super_ramo(unidad12_LUCROCESANTE, "Remuneración a favor de intermediarios dependientes e independientes")
p105_010 = get_value_super_ramo(unidad12_LUCROCESANTE, "Comisiones AFP")
p107_020 = get_value_super_ramo(unidad12_LUCROCESANTE, "Gastos en tarifación")
p108_025 = get_value_super_ramo(unidad12_LUCROCESANTE, "Remuneraciones por uso de red y a canales de comercialización")
p109_030 = get_value_super_ramo(unidad12_LUCROCESANTE, "Papelería y gastos de entrega")

# Escribir en Excel en columna 16 (columna P)
ws.cell(row=104, column=16, value=p104_005)   
ws.cell(row=105, column=16, value=p105_010)  
ws.cell(row=107, column=16, value=p107_020) 
ws.cell(row=108, column=16, value=p108_025) 
ws.cell(row=109, column=16, value=p109_030)

# Calcular y escribir el total
p110_999 = p104_005 + p105_010 + p107_020 + p108_025 + p109_030
ws.cell(row=110, column=16, value=p110_999)


<Cell 'Hoja1'.P110>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 12

In [404]:
# Función robusta para obtener el valor de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Filtrar y agrupar datos del ramo 15 MONTAJE Y ROTURA DE MAQUINARIA
unidad12_MONTAJEYROTURA = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '15 MONTAJE Y ROTURA DE MAQUINARIA')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que VALOR sea numérico
unidad12_MONTAJEYROTURA['VALOR'] = pd.to_numeric(unidad12_MONTAJEYROTURA['VALOR'], errors='coerce').fillna(0)

# Extraer valores
q104_005 = get_value_super_ramo(unidad12_MONTAJEYROTURA, "Remuneración a favor de intermediarios dependientes e independientes")
q105_010 = get_value_super_ramo(unidad12_MONTAJEYROTURA, "Comisiones AFP")
q107_020 = get_value_super_ramo(unidad12_MONTAJEYROTURA, "Gastos en tarifación")
q108_025 = get_value_super_ramo(unidad12_MONTAJEYROTURA, "Remuneraciones por uso de red y a canales de comercialización")
q109_030 = get_value_super_ramo(unidad12_MONTAJEYROTURA, "Papelería y gastos de entrega")

# Escribir en Excel (columna 17 = columna 'Q')
ws.cell(row=104, column=17, value=q104_005)
ws.cell(row=105, column=17, value=q105_010)
ws.cell(row=107, column=17, value=q107_020)
ws.cell(row=108, column=17, value=q108_025)
ws.cell(row=109, column=17, value=q109_030)

# Calcular total
q110_999 = q104_005 + q105_010 + q107_020 + q108_025 + q109_030
ws.cell(row=110, column=17, value=q110_999)


<Cell 'Hoja1'.Q110>

#  16  AVIACIÓN UNIDAD 12

In [405]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y procesar datos del ramo 16 AVIACIÓN
unidad12_AVIACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '16 AVIACIÓN')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que VALOR es numérico
unidad12_AVIACION['VALOR'] = pd.to_numeric(unidad12_AVIACION['VALOR'], errors='coerce').fillna(0)

# Obtener valores individuales
r104_005 = get_value_super_ramo(unidad12_AVIACION, "Remuneración a favor de intermediarios dependientes e independientes")
r105_010 = get_value_super_ramo(unidad12_AVIACION, "Comisiones AFP")
r107_020 = get_value_super_ramo(unidad12_AVIACION, "Gastos en tarifación")
r108_025 = get_value_super_ramo(unidad12_AVIACION, "Remuneraciones por uso de red y a canales de comercialización")
r109_030 = get_value_super_ramo(unidad12_AVIACION, "Papelería y gastos de entrega")

# Escribir valores en Excel (columna 18 = columna R)
ws.cell(row=104, column=18, value=r104_005)   
ws.cell(row=105, column=18, value=r105_010)  
ws.cell(row=107, column=18, value=r107_020) 
ws.cell(row=108, column=18, value=r108_025) 
ws.cell(row=109, column=18, value=r109_030)

# Calcular y escribir total
r110_999 = r104_005 + r105_010 + r107_020 + r108_025 + r109_030
ws.cell(row=110, column=18, value=r110_999)


<Cell 'Hoja1'.R110>

#  17  NAVEGACIÓN Y CASCO UNIDAD 12

In [407]:
# Función robusta para obtener el valor absoluto de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y procesar datos del ramo 17 NAVEGACIÓN Y CASCO
unidad12_NAVEGACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') &
    (Base['RAMO_SUPER'] == '17 NAVEGACIÓN Y CASCO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que VALOR es numérico
unidad12_NAVEGACION['VALOR'] = pd.to_numeric(unidad12_NAVEGACION['VALOR'], errors='coerce').fillna(0)

# Obtener valores
s104_005 = get_value_super_ramo(unidad12_NAVEGACION, "Remuneración a favor de intermediarios dependientes e independientes")
s105_010 = get_value_super_ramo(unidad12_NAVEGACION, "Comisiones AFP")
s107_020 = get_value_super_ramo(unidad12_NAVEGACION, "Gastos en tarifación")
s108_025 = get_value_super_ramo(unidad12_NAVEGACION, "Remuneraciones por uso de red y a canales de comercialización")
s109_030 = get_value_super_ramo(unidad12_NAVEGACION, "Papelería y gastos de entrega")

# Escribir en Excel (columna 19 = columna 'S')
ws.cell(row=104, column=19, value=s104_005)
ws.cell(row=105, column=19, value=s105_010)
ws.cell(row=107, column=19, value=s107_020)
ws.cell(row=108, column=19, value=s108_025)
ws.cell(row=109, column=19, value=s109_030)

# Calcular total
s110_999 = s104_005 + s105_010 + s107_020 + s108_025 + s109_030
ws.cell(row=110, column=19, value=s110_999)


<Cell 'Hoja1'.S110>

#  18  MINAS Y PETRÓLEOS UNIDAD 12

In [408]:
# Función robusta para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y asegurar numéricos
unidad12_MINAS = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '18 MINAS Y PETRÓLEOS')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_MINAS['VALOR'] = pd.to_numeric(unidad12_MINAS['VALOR'], errors='coerce').fillna(0)

# Obtener valores
t104_005 = get_value_super_ramo(unidad12_MINAS, "Remuneración a favor de intermediarios dependientes e independientes")
t105_010 = get_value_super_ramo(unidad12_MINAS, "Comisiones AFP")
t107_020 = get_value_super_ramo(unidad12_MINAS, "Gastos en tarifación")
t108_025 = get_value_super_ramo(unidad12_MINAS, "Remuneraciones por uso de red y a canales de comercialización")
t109_030 = get_value_super_ramo(unidad12_MINAS, "Papelería y gastos de entrega")

# Escribir valores en Excel (columna 20)
ws.cell(row=104, column=20, value=t104_005)
ws.cell(row=105, column=20, value=t105_010)
ws.cell(row=107, column=20, value=t107_020)
ws.cell(row=108, column=20, value=t108_025)
ws.cell(row=109, column=20, value=t109_030)

# Total
t110_999 = t104_005 + t105_010 + t107_020 + t108_025 + t109_030
ws.cell(row=110, column=20, value=t110_999)


<Cell 'Hoja1'.T110>

#  19  VIDRIOS UNIDAD 12

In [409]:
# Función robusta para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y asegurar columna VALOR numérica
unidad12_VIDRIOS = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') &
    (Base['RAMO_SUPER'] == '19 VIDRIOS')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_VIDRIOS['VALOR'] = pd.to_numeric(unidad12_VIDRIOS['VALOR'], errors='coerce').fillna(0)

# Obtener valores
u104_005 = get_value_super_ramo(unidad12_VIDRIOS, "Remuneración a favor de intermediarios dependientes e independientes")
u105_010 = get_value_super_ramo(unidad12_VIDRIOS, "Comisiones AFP")
u107_020 = get_value_super_ramo(unidad12_VIDRIOS, "Gastos en tarifación")
u108_025 = get_value_super_ramo(unidad12_VIDRIOS, "Remuneraciones por uso de red y a canales de comercialización")
u109_030 = get_value_super_ramo(unidad12_VIDRIOS, "Papelería y gastos de entrega")

# Escribir en Excel (columna 21)
ws.cell(row=104, column=21, value=u104_005)
ws.cell(row=105, column=21, value=u105_010)
ws.cell(row=107, column=21, value=u107_020)
ws.cell(row=108, column=21, value=u108_025)
ws.cell(row=109, column=21, value=u109_030)

# Calcular y escribir total
u110_999 = u104_005 + u105_010 + u107_020 + u108_025 + u109_030
ws.cell(row=110, column=21, value=u110_999)


<Cell 'Hoja1'.U110>

#  20  CRÉDITO COMERCIAL UNIDAD 12

In [410]:
# Función robusta para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y limpiar valores numéricos
unidad12_CREDITOCOMERCIAL = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '20 CRÉDITO COMERCIAL')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_CREDITOCOMERCIAL['VALOR'] = pd.to_numeric(unidad12_CREDITOCOMERCIAL['VALOR'], errors='coerce').fillna(0)

# Obtener valores
v104_005 = get_value_super_ramo(unidad12_CREDITOCOMERCIAL, "Remuneración a favor de intermediarios dependientes e independientes")
v105_010 = get_value_super_ramo(unidad12_CREDITOCOMERCIAL, "Comisiones AFP")
v107_020 = get_value_super_ramo(unidad12_CREDITOCOMERCIAL, "Gastos en tarifación")
v108_025 = get_value_super_ramo(unidad12_CREDITOCOMERCIAL, "Remuneraciones por uso de red y a canales de comercialización")
v109_030 = get_value_super_ramo(unidad12_CREDITOCOMERCIAL, "Papelería y gastos de entrega")

# Escribir en Excel (columna 22)
ws.cell(row=104, column=22, value=v104_005)
ws.cell(row=105, column=22, value=v105_010)
ws.cell(row=107, column=22, value=v107_020)
ws.cell(row=108, column=22, value=v108_025)
ws.cell(row=109, column=22, value=v109_030)

# Calcular y escribir total
v110_999 = v104_005 + v105_010 + v107_020 + v108_025 + v109_030
ws.cell(row=110, column=22, value=v110_999)


<Cell 'Hoja1'.V110>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 12

In [411]:
# Función robusta para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y asegurar que los valores sean numéricos
unidad12_CREDITOEXPORTACION = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') &
    (Base['RAMO_SUPER'] == '21 CRÉDITO A LA EXPORTACIÓN')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_CREDITOEXPORTACION['VALOR'] = pd.to_numeric(unidad12_CREDITOEXPORTACION['VALOR'], errors='coerce').fillna(0)

# Obtener valores
w104_005 = get_value_super_ramo(unidad12_CREDITOEXPORTACION, "Remuneración a favor de intermediarios dependientes e independientes")
w105_010 = get_value_super_ramo(unidad12_CREDITOEXPORTACION, "Comisiones AFP")
w107_020 = get_value_super_ramo(unidad12_CREDITOEXPORTACION, "Gastos en tarifación")
w108_025 = get_value_super_ramo(unidad12_CREDITOEXPORTACION, "Remuneraciones por uso de red y a canales de comercialización")
w109_030 = get_value_super_ramo(unidad12_CREDITOEXPORTACION, "Papelería y gastos de entrega")

# Escribir en Excel (columna 23)
ws.cell(row=104, column=23, value=w104_005)
ws.cell(row=105, column=23, value=w105_010)
ws.cell(row=107, column=23, value=w107_020)
ws.cell(row=108, column=23, value=w108_025)
ws.cell(row=109, column=23, value=w109_030)

# Calcular total
w110_999 = w104_005 + w105_010 + w107_020 + w108_025 + w109_030
ws.cell(row=110, column=23, value=w110_999)


<Cell 'Hoja1'.W110>

#  24  DESEMPLEO UNIDAD 12

In [412]:
# Definir la función para obtener el valor de un concepto específico (versión robusta)
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar la base y asegurar tipo numérico
unidad12_DESEMPLEO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '24 DESEMPLEO')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_DESEMPLEO['VALOR'] = pd.to_numeric(unidad12_DESEMPLEO['VALOR'], errors='coerce').fillna(0)

# Obtener los valores requeridos
z104_005 = get_value_super_ramo(unidad12_DESEMPLEO, "Remuneración a favor de intermediarios dependientes e independientes")
z105_010 = get_value_super_ramo(unidad12_DESEMPLEO, "Comisiones AFP")
z107_020 = get_value_super_ramo(unidad12_DESEMPLEO, "Gastos en tarifación")
z108_025 = get_value_super_ramo(unidad12_DESEMPLEO, "Remuneraciones por uso de red y a canales de comercialización")
z109_030 = get_value_super_ramo(unidad12_DESEMPLEO, "Papelería y gastos de entrega")

# Escribir los valores en el archivo Excel (columna 26)
ws.cell(row=104, column=26, value=z104_005)
ws.cell(row=105, column=26, value=z105_010)
ws.cell(row=107, column=26, value=z107_020)
ws.cell(row=108, column=26, value=z108_025)
ws.cell(row=109, column=26, value=z109_030)

# Calcular total y escribir en fila 110
z110_999 = z104_005 + z105_010 + z107_020 + z108_025 + z109_030
ws.cell(row=110, column=26, value=z110_999)


<Cell 'Hoja1'.Z110>

#  25  HOGAR UNIDAD 12

In [413]:
# Función segura para obtener valores numéricos de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].iloc[0])
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    return 0

# Agrupar y convertir a valores numéricos
unidad12_HOGAR = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & 
    (Base['RAMO_SUPER'] == '25 HOGAR')
].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

unidad12_HOGAR['VALOR'] = pd.to_numeric(unidad12_HOGAR['VALOR'], errors='coerce').fillna(0)

# Obtener cada concepto
aa104_005 = get_value_super_ramo(unidad12_HOGAR, "Remuneración a favor de intermediarios dependientes e independientes")
aa105_010 = get_value_super_ramo(unidad12_HOGAR, "Comisiones AFP")
aa107_020 = get_value_super_ramo(unidad12_HOGAR, "Gastos en tarifación")
aa108_025 = get_value_super_ramo(unidad12_HOGAR, "Remuneraciones por uso de red y a canales de comercialización")
aa109_030 = get_value_super_ramo(unidad12_HOGAR, "Papelería y gastos de entrega")

# Escribir en Excel
ws.cell(row=104, column=27, value=aa104_005)
ws.cell(row=105, column=27, value=aa105_010)
ws.cell(row=107, column=27, value=aa107_020)
ws.cell(row=108, column=27, value=aa108_025)
ws.cell(row=109, column=27, value=aa109_030)

# Calcular y escribir el total
aa110_999 = aa104_005 + aa105_010 + aa107_020 + aa108_025 + aa109_030
ws.cell(row=110, column=27, value=aa110_999)


<Cell 'Hoja1'.AA110>

# 42 AGROPECUARIO UNIDAD 12

In [415]:
# Función robusta para obtener el valor absoluto numérico de un concepto
def get_value_super_ramo(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    try:
        return abs(float(row['VALOR'].iloc[0])) if not row.empty else 0
    except (ValueError, TypeError, IndexError):
        return 0

# Agrupar datos por concepto para el ramo 42 AGROPECUARIO (unidad 12)
unidad12_AGROPECUARIO = Base[
    (Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER'] == '42 AGROPECUARIO')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores individuales para los conceptos adicionales
ar104_005 = get_value_super_ramo(unidad12_AGROPECUARIO, "Remuneración a favor de intermediarios dependientes e independientes")
ar105_010 = get_value_super_ramo(unidad12_AGROPECUARIO, "Comisiones AFP")
ar107_020 = get_value_super_ramo(unidad12_AGROPECUARIO, "Gastos en tarifación")
ar108_025 = get_value_super_ramo(unidad12_AGROPECUARIO, "Remuneraciones por uso de red y a canales de comercialización")
ar109_030 = get_value_super_ramo(unidad12_AGROPECUARIO, "Papelería y gastos de entrega")

# Escribir en Excel, columna 44 (AR)
ws.cell(row=104, column=44, value=ar104_005)   
ws.cell(row=105, column=44, value=ar105_010)  
ws.cell(row=107, column=44, value=ar107_020) 
ws.cell(row=108, column=44, value=ar108_025) 
ws.cell(row=109, column=44, value=ar109_030) 

# Calcular y escribir el total
ar110_999 = ar104_005 + ar105_010 + ar107_020 + ar108_025 + ar109_030
ws.cell(row=110, column=44, value=ar110_999)


<Cell 'Hoja1'.AR110>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 12

In [416]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_OPERNOASIGNABLES= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '43 OPERACIONES NO ASIGANBLES A RAMOS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

as104_005 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Remuneración a favor de intermediarios dependientes e independientes")
as105_010 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Comisiones AFP")
as107_020 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Gastos en tarifación")
as108_025 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Remuneraciones por uso de red y a canales de comercialización")
as109_030 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Papelería y gastos de entrega")

ws.cell(row=104, column=45, value=as104_005)   
ws.cell(row=105, column=45, value=as105_010)  
ws.cell(row=107, column=45, value=as107_020) 
ws.cell(row=108, column=45, value=as108_025) 
ws.cell(row=109, column=45, value=as109_030) 

# Calcular e102_999 usando la fórmula especificada
as110_999 = (as104_005 + as105_010 + as107_020 + as108_025 + as109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=45). value = as110_999

# 44 DECENAL UNIDAD 12

In [417]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_DECENAL= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '44 DECENAL')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

at104_005 = get_value_super_ramo(unidad12_DECENAL, "Remuneración a favor de intermediarios dependientes e independientes")
at105_010 = get_value_super_ramo(unidad12_DECENAL, "Comisiones AFP")
at107_020 = get_value_super_ramo(unidad12_DECENAL, "Gastos en tarifación")
at108_025 = get_value_super_ramo(unidad12_DECENAL, "Remuneraciones por uso de red y a canales de comercialización")
at109_030 = get_value_super_ramo(unidad12_DECENAL, "Papelería y gastos de entrega")

ws.cell(row=104, column=46, value=at104_005)   
ws.cell(row=105, column=46, value=at105_010)  
ws.cell(row=107, column=46, value=at107_020) 
ws.cell(row=108, column=46, value=at108_025) 
ws.cell(row=109, column=46, value=at109_030) 

# Calcular e102_999 usando la fórmula especificada
at110_999 = (at104_005 + at105_010 + at107_020 + at108_025 + at109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=46). value = at110_999

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 12

In [418]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOESTATALES= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '45 CUMPLIMIENTO - ENTIDADES ESTATALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

au104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOESTATALES, "Remuneración a favor de intermediarios dependientes e independientes")
au105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOESTATALES, "Comisiones AFP")
au107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOESTATALES, "Gastos en tarifación")
au108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOESTATALES, "Remuneraciones por uso de red y a canales de comercialización")
au109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOESTATALES, "Papelería y gastos de entrega")

ws.cell(row=104, column=47, value=au104_005)   
ws.cell(row=105, column=47, value=au105_010)  
ws.cell(row=107, column=47, value=au107_020) 
ws.cell(row=108, column=47, value=au108_025) 
ws.cell(row=109, column=47, value=au109_030) 

# Calcular e102_999 usando la fórmula especificada
au110_999 = (au104_005 + au105_010 + au107_020 + au108_025 + au109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=47). value = au110_999

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 12

In [419]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOEMPRESASSERV= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

av104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASSERV, "Remuneración a favor de intermediarios dependientes e independientes")
av105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASSERV, "Comisiones AFP")
av107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASSERV, "Gastos en tarifación")
av108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASSERV, "Remuneraciones por uso de red y a canales de comercialización")
av109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASSERV, "Papelería y gastos de entrega")

ws.cell(row=104, column=48, value=av104_005)   
ws.cell(row=105, column=48, value=av105_010)  
ws.cell(row=107, column=48, value=av107_020) 
ws.cell(row=108, column=48, value=av108_025) 
ws.cell(row=109, column=48, value=av109_030) 

# Calcular e102_999 usando la fórmula especificada
av110_999 = (av104_005 + av105_010 + av107_020 + av108_025 + av109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=48). value = av110_999

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 12

In [420]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOEMPRESASINDUSTRI= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aw104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASINDUSTRI, "Remuneración a favor de intermediarios dependientes e independientes")
aw105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASINDUSTRI, "Comisiones AFP")
aw107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASINDUSTRI, "Gastos en tarifación")
aw108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASINDUSTRI, "Remuneraciones por uso de red y a canales de comercialización")
aw109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASINDUSTRI, "Papelería y gastos de entrega")

ws.cell(row=104, column=49, value=aw104_005)   
ws.cell(row=105, column=49, value=aw105_010)  
ws.cell(row=107, column=49, value=aw107_020) 
ws.cell(row=108, column=49, value=aw108_025) 
ws.cell(row=109, column=49, value=aw109_030) 

# Calcular e102_999 usando la fórmula especificada
aw110_999 = (aw104_005 + aw105_010 + aw107_020 + aw108_025 + aw109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=49). value = aw110_999

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 12

In [421]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOEMPRESASDISPOLEG= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '48 CUMPLIMIENTO - DISPOSICIONES LEGALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ax104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASDISPOLEG, "Remuneración a favor de intermediarios dependientes e independientes")
ax105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASDISPOLEG, "Comisiones AFP")
ax107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASDISPOLEG, "Gastos en tarifación")
ax108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASDISPOLEG, "Remuneraciones por uso de red y a canales de comercialización")
ax109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOEMPRESASDISPOLEG, "Papelería y gastos de entrega")

ws.cell(row=104, column=50, value=ax104_005)   
ws.cell(row=105, column=50, value=ax105_010)  
ws.cell(row=107, column=50, value=ax107_020) 
ws.cell(row=108, column=50, value=ax108_025) 
ws.cell(row=109, column=50, value=ax109_030) 

# Calcular e102_999 usando la fórmula especificada
ax110_999 = (ax104_005 + ax105_010 + ax107_020 + ax108_025 + ax109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=50). value = ax110_999

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 12

In [422]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOCAUCIONES= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '49 CUMPLIMIENTO - CAUCIONES JUDICIALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ay104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOCAUCIONES, "Remuneración a favor de intermediarios dependientes e independientes")
ay105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOCAUCIONES, "Comisiones AFP")
ay107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOCAUCIONES, "Gastos en tarifación")
ay108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOCAUCIONES, "Remuneraciones por uso de red y a canales de comercialización")
ay109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOCAUCIONES, "Papelería y gastos de entrega")

ws.cell(row=104, column=51, value=ay104_005)   
ws.cell(row=105, column=51, value=ay105_010)  
ws.cell(row=107, column=51, value=ay107_020) 
ws.cell(row=108, column=51, value=ay108_025) 
ws.cell(row=109, column=51, value=ay109_030) 

# Calcular e102_999 usando la fórmula especificada
ay110_999 = (ay104_005 + ay105_010 + ay107_020 + ay108_025 + ay109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=51). value = ay110_999

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 12

In [423]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOARRENDAMIENTO= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '50 CUMPLIMIENTO - ARRENDAMIENTO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

az104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOARRENDAMIENTO, "Remuneración a favor de intermediarios dependientes e independientes")
az105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOARRENDAMIENTO, "Comisiones AFP")
az107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOARRENDAMIENTO, "Gastos en tarifación")
az108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOARRENDAMIENTO, "Remuneraciones por uso de red y a canales de comercialización")
az109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOARRENDAMIENTO, "Papelería y gastos de entrega")

ws.cell(row=104, column=52, value=az104_005)   
ws.cell(row=105, column=52, value=az105_010)  
ws.cell(row=107, column=52, value=az107_020) 
ws.cell(row=108, column=52, value=az108_025) 
ws.cell(row=109, column=52, value=az109_030) 

# Calcular e102_999 usando la fórmula especificada
az110_999 = (az104_005 + az105_010 + az107_020 + az108_025 + az109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=52). value = az110_999

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 12

In [424]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_CUMPLIMIENTOPARTICULARES= Base[(Base['SOCIEDAD_DEST'] == 'GRAL') & (Base['RAMO_SUPER']  == '51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES)')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ba104_005 = get_value_super_ramo(unidad12_CUMPLIMIENTOPARTICULARES, "Remuneración a favor de intermediarios dependientes e independientes")
ba105_010 = get_value_super_ramo(unidad12_CUMPLIMIENTOPARTICULARES, "Comisiones AFP")
ba107_020 = get_value_super_ramo(unidad12_CUMPLIMIENTOPARTICULARES, "Gastos en tarifación")
ba108_025 = get_value_super_ramo(unidad12_CUMPLIMIENTOPARTICULARES, "Remuneraciones por uso de red y a canales de comercialización")
ba109_030 = get_value_super_ramo(unidad12_CUMPLIMIENTOPARTICULARES, "Papelería y gastos de entrega")

ws.cell(row=104, column=53, value=ba104_005)   
ws.cell(row=105, column=53, value=ba105_010)  
ws.cell(row=107, column=53, value=ba107_020) 
ws.cell(row=108, column=53, value=ba108_025) 
ws.cell(row=109, column=53, value=ba109_030) 

# Calcular e102_999 usando la fórmula especificada
ba110_999 = (ba104_005 + ba105_010 + ba107_020 + ba108_025 + ba109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=53). value = ba110_999

#  1   TOTAL UNIDAD 12

In [425]:
# Función robusta para obtener el total absoluto de un concepto
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].sum())
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar toda la base por concepto, solo para SOCIEDAD_DEST = 'GRAL'
unidad12_total = Base[Base['SOCIEDAD_DEST'] == 'GRAL'].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica en la tabla agregada
unidad12_total['VALOR'] = pd.to_numeric(unidad12_total['VALOR'], errors='coerce').fillna(0)

# Obtener valores agregados para conceptos específicos
c104_005 = get_value_total(unidad12_total, "Remuneración a favor de intermediarios dependientes e independientes")
c105_010 = get_value_total(unidad12_total, "Comisiones AFP")
c107_020 = get_value_total(unidad12_total, "Gastos en tarifación")
c108_025 = get_value_total(unidad12_total, "Remuneraciones por uso de red y a canales de comercialización")
c109_030 = get_value_total(unidad12_total, "Papelería y gastos de entrega")

# Escribir en Excel en columna 3 (columna C)
ws.cell(row=104, column=3, value=c104_005)   
ws.cell(row=105, column=3, value=c105_010)  
ws.cell(row=107, column=3, value=c107_020) 
ws.cell(row=108, column=3, value=c108_025)
ws.cell(row=109, column=3, value=c109_030)

# Calcular y escribir el total general
c110_999 = c104_005 + c105_010 + c107_020 + c108_025 + c109_030
ws.cell(row=110, column=3, value=c110_999)


<Cell 'Hoja1'.C110>

#  2   SUBTOTAL - RAMOS UNIDAD 12

In [426]:
# Función robusta para obtener el total absoluto de un concepto
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept].copy()
    if not row.empty:
        try:
            row['VALOR'] = pd.to_numeric(row['VALOR'], errors='coerce').fillna(0)
            return abs(row['VALOR'].sum())
        except Exception as e:
            print(f"Error en concepto '{concept}': {e}")
            return 0
    else:
        return 0

# Agrupar la base por concepto, solo para SOCIEDAD_DEST = 'GRAL'
unidad12_subtotal = Base[Base['SOCIEDAD_DEST'] == 'GRAL'].groupby('CONCEPTO_290')['VALOR'].sum().reset_index()

# Asegurar que la columna VALOR sea numérica
unidad12_subtotal['VALOR'] = pd.to_numeric(unidad12_subtotal['VALOR'], errors='coerce').fillna(0)

# Obtener los valores de los conceptos deseados
d104_005 = get_value_total(unidad12_subtotal, "Remuneración a favor de intermediarios dependientes e independientes")
d105_010 = get_value_total(unidad12_subtotal, "Comisiones AFP")
d107_020 = get_value_total(unidad12_subtotal, "Gastos en tarifación")
d108_025 = get_value_total(unidad12_subtotal, "Remuneraciones por uso de red y a canales de comercialización")
d109_030 = get_value_total(unidad12_subtotal, "Papelería y gastos de entrega")

# Escribir los valores en Excel en columna 4 (columna D)
ws.cell(row=104, column=4, value=d104_005)   
ws.cell(row=105, column=4, value=d105_010)  
ws.cell(row=107, column=4, value=d107_020) 
ws.cell(row=108, column=4, value=d108_025)
ws.cell(row=109, column=4, value=d109_030)

# Calcular y escribir el total
d110_999 = d104_005 + d105_010 + d107_020 + d108_025 + d109_030
ws.cell(row=110, column=4, value=d110_999)

<Cell 'Hoja1'.D110>

## CONSULTA DE TOTALES UNIDAD DE CAPTURA 13

In [428]:
c113_999 = c84_999 + c94_999 - c102_999 - c110_999
df_c113_999 = pd.DataFrame({'total': [c113_999]})
if df_c113_999.empty or c113_999 == 0:
    df_c113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=3).value = df_c113_999['total'].iloc[0]

e113_999 = e84_999 + e94_999 - e102_999 - e110_999
df_e113_999 = pd.DataFrame({'total': [e113_999]})
if df_e113_999.empty or e113_999 == 0:
    df_e113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=5).value = df_e113_999['total'].iloc[0]

f113_999 = f84_999 + f94_999 - f102_999 - f110_999
df_f113_999 = pd.DataFrame({'total': [f113_999]})
if df_f113_999.empty or f113_999 == 0:
    df_f113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=6).value = df_f113_999['total'].iloc[0]

g113_999 = g84_999 + g94_999 - g102_999 - g110_999
df_g113_999 = pd.DataFrame({'total': [g113_999]})
if df_g113_999.empty or g113_999 == 0:
    df_g113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=7).value = df_g113_999['total'].iloc[0]

h113_999 = h84_999 + h94_999 - h102_999 - h110_999
df_h113_999 = pd.DataFrame({'total': [h113_999]})
if df_h113_999.empty or h113_999 == 0:
    df_h113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=8).value = df_h113_999['total'].iloc[0]

i113_999 = i84_999 + i94_999 - i102_999 - i110_999
df_i113_999 = pd.DataFrame({'total': [i113_999]})
if df_i113_999.empty or i113_999 == 0:
    df_i113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=9).value = df_i113_999['total'].iloc[0]

j113_999 = j84_999 + j94_999 - j102_999 - j110_999
df_j113_999 = pd.DataFrame({'total': [j113_999]})
if df_j113_999.empty or j113_999 == 0:
    df_j113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=10).value = df_j113_999['total'].iloc[0]

k113_999 = k84_999 + k94_999 - k102_999 - k110_999
df_k113_999 = pd.DataFrame({'total': [k113_999]})
if df_k113_999.empty or k113_999 == 0:
    df_k113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=11).value = df_k113_999['total'].iloc[0]

l113_999 = l84_999 + l94_999 - l102_999 - l110_999
df_l113_999 = pd.DataFrame({'total': [l113_999]})
if df_l113_999.empty or l113_999 == 0:
    df_l113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=12).value = df_l113_999['total'].iloc[0]

m113_999 = m84_999 + m94_999 - m102_999 - m110_999
df_m113_999 = pd.DataFrame({'total': [m113_999]})
if df_m113_999.empty or m113_999 == 0:
    df_m113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=13).value = df_m113_999['total'].iloc[0]

n113_999 = n84_999 + n94_999 - n102_999 - n110_999
df_n113_999 = pd.DataFrame({'total': [n113_999]})
if df_n113_999.empty or n113_999 == 0:
    df_n113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=14).value = df_n113_999['total'].iloc[0]

o113_999 = o84_999 + o94_999 - o102_999 - o110_999
df_o113_999 = pd.DataFrame({'total': [o113_999]})
if df_o113_999.empty or o113_999 == 0:
    df_o113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=15).value = df_o113_999['total'].iloc[0]

p113_999 = p84_999 + p94_999 - p102_999 - p110_999
df_p113_999 = pd.DataFrame({'total': [p113_999]})
if df_p113_999.empty or p113_999 == 0:
    df_p113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=16).value = df_p113_999['total'].iloc[0]

q113_999 = q84_999 + q94_999 - q102_999 - q110_999
df_q113_999 = pd.DataFrame({'total': [q113_999]})
if df_q113_999.empty or q113_999 == 0:
    df_q113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=17).value = df_q113_999['total'].iloc[0]

r113_999 = r84_999 + r94_999 - r102_999 - r110_999
df_r113_999 = pd.DataFrame({'total': [r113_999]})
if df_r113_999.empty or r113_999 == 0:
    df_r113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=18).value = df_r113_999['total'].iloc[0]

s113_999 = s84_999 + s94_999 - s102_999 - s110_999
df_s113_999 = pd.DataFrame({'total': [s113_999]})
if df_s113_999.empty or s113_999 == 0:
    df_s113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=19).value = df_s113_999['total'].iloc[0]

t113_999 = t84_999 + t94_999 - t102_999 - t110_999
df_t113_999 = pd.DataFrame({'total': [t113_999]})
if df_t113_999.empty or t113_999 == 0:
    df_t113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=20).value = df_t113_999['total'].iloc[0]

u113_999 = u84_999 + u94_999 - u102_999 - u110_999
df_u113_999 = pd.DataFrame({'total': [u113_999]})
if df_u113_999.empty or u113_999 == 0:
    df_u113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=21).value = df_u113_999['total'].iloc[0]

v113_999 = v84_999 + v94_999 - v102_999 - v110_999
df_v113_999 = pd.DataFrame({'total': [v113_999]})
if df_v113_999.empty or v113_999 == 0:
    df_v113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=22).value = df_v113_999['total'].iloc[0]

w113_999 = w84_999 + w94_999 - w102_999 - w110_999
df_w113_999 = pd.DataFrame({'total': [w113_999]})
if df_w113_999.empty or w113_999 == 0:
    df_w113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=23).value = df_w113_999['total'].iloc[0]

z113_999 = z84_999 + z94_999 - z102_999 - z110_999
df_z113_999 = pd.DataFrame({'total': [z113_999]})
if df_z113_999.empty or z113_999 == 0:
    df_z113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=26).value = df_z113_999['total'].iloc[0]

aa113_999 = aa84_999 + aa94_999 - aa102_999 - aa110_999
df_aa113_999 = pd.DataFrame({'total': [aa113_999]})
if df_aa113_999.empty or aa113_999 == 0:
    df_aa113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=27).value = df_aa113_999['total'].iloc[0]

ar113_999 = ar84_999 + ar94_999 - ar102_999 - ar110_999
df_ar113_999 = pd.DataFrame({'total': [ar113_999]})
if df_ar113_999.empty or ar113_999 == 0:
    df_ar113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=44).value = df_ar113_999['total'].iloc[0]

as113_999 = as84_999 + as94_999 - as102_999 - as110_999
df_as113_999 = pd.DataFrame({'total': [as113_999]})
if df_as113_999.empty or as113_999 == 0:
    df_as113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=45).value = df_as113_999['total'].iloc[0]

at113_999 = at84_999 + at94_999 - at102_999 - at110_999
df_at113_999 = pd.DataFrame({'total': [at113_999]})
if df_at113_999.empty or at113_999 == 0:
    df_at113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=46).value = df_at113_999['total'].iloc[0]

au113_999 = au84_999 + au94_999 - au102_999 - au110_999
df_au113_999 = pd.DataFrame({'total': [au113_999]})
if df_au113_999.empty or au113_999 == 0:
    df_au113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=47).value = df_au113_999['total'].iloc[0]

av113_999 = av84_999 + av94_999 - av102_999 - av110_999
df_av113_999 = pd.DataFrame({'total': [av113_999]})
if df_av113_999.empty or av113_999 == 0:
    df_av113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=48).value = df_av113_999['total'].iloc[0]

aw113_999 = aw84_999 + aw94_999 - aw102_999 - aw110_999
df_aw113_999 = pd.DataFrame({'total': [aw113_999]})
if df_aw113_999.empty or aw113_999 == 0:
    df_aw113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=49).value = df_aw113_999['total'].iloc[0]

ax113_999 = ax84_999 + ax94_999 - ax102_999 - ax110_999
df_ax113_999 = pd.DataFrame({'total': [ax113_999]})
if df_ax113_999.empty or ax113_999 == 0:
    df_ax113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=50).value = df_ax113_999['total'].iloc[0]

ay113_999 = ay84_999 + ay94_999 - ay102_999 - ay110_999
df_ay113_999 = pd.DataFrame({'total': [ay113_999]})
if df_ay113_999.empty or ay113_999 == 0:
    df_ay113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=51).value = df_ay113_999['total'].iloc[0]

az113_999 = az84_999 + az94_999 - az102_999 - az110_999
df_az113_999 = pd.DataFrame({'total': [az113_999]})
if df_az113_999.empty or az113_999 == 0:
    df_az113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=52).value = df_az113_999['total'].iloc[0]

ba113_999 = ba84_999 + ba94_999 - ba102_999 - ba110_999
df_ba113_999 = pd.DataFrame({'total': [ba113_999]})
if df_ba113_999.empty or ba113_999 == 0:
    df_ba113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=53).value = df_ba113_999['total'].iloc[0]


#  CONSULTA DE SUBTOTALES - RAMOS UNIDAD 13

In [429]:
# Realiza la suma para d75_999
d113_999 = d84_999 + d94_999 - d102_999 - d110_999
df_d113_999 = pd.DataFrame({'total': [d113_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d113_999.empty or d113_999 == 0:
    df_d113_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=113, column=4).value = df_d113_999['total'].iloc[0]

## UNIDAD DE CAPTURA 14

#  CONSULTA DE TOTALES UNIDAD 14

In [430]:
c115_999 = c34_999 + c75_999 + c113_999
df_c115_999 = pd.DataFrame({'total': [c115_999]})
if df_c115_999.empty or c115_999 == 0:
    df_c115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=3).value = df_c115_999['total'].iloc[0]

e115_999 = e34_999 + e75_999 + e113_999
df_e115_999 = pd.DataFrame({'total': [e115_999]})
if df_e115_999.empty or e115_999 == 0:
    df_e115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=5).value = df_e115_999['total'].iloc[0]

f115_999 = f34_999 + f75_999 + f113_999
df_f115_999 = pd.DataFrame({'total': [f115_999]})
if df_f115_999.empty or f115_999 == 0:
    df_f115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=6).value = df_f115_999['total'].iloc[0]

g115_999 = g34_999 + g75_999 + g113_999
df_g115_999 = pd.DataFrame({'total': [g115_999]})
if df_g115_999.empty or g115_999 == 0:
    df_g115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=7).value = df_g115_999['total'].iloc[0]

h115_999 = h34_999 + h75_999 + h113_999
df_h115_999 = pd.DataFrame({'total': [h115_999]})
if df_h115_999.empty or h115_999 == 0:
    df_h115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=8).value = df_h115_999['total'].iloc[0]

i115_999 = i34_999 + i75_999 + i113_999
df_i115_999 = pd.DataFrame({'total': [i115_999]})
if df_i115_999.empty or i115_999 == 0:
    df_i115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=9).value = df_i115_999['total'].iloc[0]

j115_999 = j34_999 + j75_999 + j113_999
df_j115_999 = pd.DataFrame({'total': [j115_999]})
if df_j115_999.empty or j115_999 == 0:
    df_j115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=10).value = df_j115_999['total'].iloc[0]

k115_999 = k34_999 + k75_999 + k113_999
df_k115_999 = pd.DataFrame({'total': [k115_999]})
if df_k115_999.empty or k115_999 == 0:
    df_k115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=11).value = df_k115_999['total'].iloc[0]

l115_999 = l34_999 + l75_999 + l113_999
df_l115_999 = pd.DataFrame({'total': [l115_999]})
if df_l115_999.empty or l115_999 == 0:
    df_l115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=12).value = df_l115_999['total'].iloc[0]

m115_999 = m34_999 + m75_999 + m113_999
df_m115_999 = pd.DataFrame({'total': [m115_999]})
if df_m115_999.empty or m115_999 == 0:
    df_m115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=13).value = df_m115_999['total'].iloc[0]

n115_999 = n34_999 + n75_999 + n113_999
df_n115_999 = pd.DataFrame({'total': [n115_999]})
if df_n115_999.empty or n115_999 == 0:
    df_n115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=14).value = df_n115_999['total'].iloc[0]

o115_999 = o34_999 + o75_999 + o113_999
df_o115_999 = pd.DataFrame({'total': [o115_999]})
if df_o115_999.empty or o115_999 == 0:
    df_o115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=15).value = df_o115_999['total'].iloc[0]

p115_999 = p34_999 + p75_999 + p113_999
df_p115_999 = pd.DataFrame({'total': [p115_999]})
if df_p115_999.empty or p115_999 == 0:
    df_p115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=16).value = df_p115_999['total'].iloc[0]

q115_999 = q34_999 + q75_999 + q113_999
df_q115_999 = pd.DataFrame({'total': [q115_999]})
if df_q115_999.empty or q115_999 == 0:
    df_q115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=17).value = df_q115_999['total'].iloc[0]

r115_999 = r34_999 + r75_999 + r113_999
df_r115_999 = pd.DataFrame({'total': [r115_999]})
if df_r115_999.empty or r115_999 == 0:
    df_r115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=18).value = df_r115_999['total'].iloc[0]

s115_999 = s34_999 + s75_999 + s113_999
df_s115_999 = pd.DataFrame({'total': [s115_999]})
if df_s115_999.empty or s115_999 == 0:
    df_s115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=19).value = df_s115_999['total'].iloc[0]

t115_999 = t34_999 + t75_999 + t113_999
df_t115_999 = pd.DataFrame({'total': [t115_999]})
if df_t115_999.empty or t115_999 == 0:
    df_t115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=20).value = df_t115_999['total'].iloc[0]

u115_999 = u34_999 + u75_999 + u113_999
df_u115_999 = pd.DataFrame({'total': [u115_999]})
if df_u115_999.empty or u115_999 == 0:
    df_u115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=21).value = df_u115_999['total'].iloc[0]

v115_999 = v34_999 + v75_999 + v113_999
df_v115_999 = pd.DataFrame({'total': [v115_999]})
if df_v115_999.empty or v115_999 == 0:
    df_v115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=22).value = df_v115_999['total'].iloc[0]

w115_999 = w34_999 + w75_999 + w113_999
df_w115_999 = pd.DataFrame({'total': [w115_999]})
if df_w115_999.empty or w115_999 == 0:
    df_w115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=23).value = df_w115_999['total'].iloc[0]

z115_999 = z34_999 + z75_999 + z113_999
df_z115_999 = pd.DataFrame({'total': [z115_999]})
if df_z115_999.empty or z115_999 == 0:
    df_z115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=26).value = df_z115_999['total'].iloc[0]

aa115_999 = aa34_999 + aa75_999 + aa113_999
df_aa115_999 = pd.DataFrame({'total': [aa115_999]})
if df_aa115_999.empty or aa115_999 == 0:
    df_aa115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=27).value = df_aa115_999['total'].iloc[0]

ar115_999 = ar34_999 + ar75_999 + ar113_999
df_ar115_999 = pd.DataFrame({'total': [ar115_999]})
if df_ar115_999.empty or ar115_999 == 0:
    df_ar115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=44).value = df_ar115_999['total'].iloc[0]

as115_999 = as34_999 + as75_999 + as113_999
df_as115_999 = pd.DataFrame({'total': [as115_999]})
if df_as115_999.empty or as115_999 == 0:
    df_as115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=45).value = df_as115_999['total'].iloc[0]

at115_999 = at34_999 + at75_999 + at113_999
df_at115_999 = pd.DataFrame({'total': [at115_999]})
if df_at115_999.empty or at115_999 == 0:
    df_at115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=46).value = df_at115_999['total'].iloc[0]

au115_999 = au34_999 + au75_999 + au113_999
df_au115_999 = pd.DataFrame({'total': [au115_999]})
if df_au115_999.empty or au115_999 == 0:
    df_au115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=47).value = df_au115_999['total'].iloc[0]

av115_999 = av34_999 + av75_999 + av113_999
df_av115_999 = pd.DataFrame({'total': [av115_999]})
if df_av115_999.empty or av115_999 == 0:
    df_av115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=48).value = df_av115_999['total'].iloc[0]

aw115_999 = aw34_999 + aw75_999 + aw113_999
df_aw115_999 = pd.DataFrame({'total': [aw115_999]})
if df_aw115_999.empty or aw115_999 == 0:
    df_aw115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=49).value = df_aw115_999['total'].iloc[0]

ax115_999 = ax34_999 + ax75_999 + ax113_999
df_ax115_999 = pd.DataFrame({'total': [ax115_999]})
if df_ax115_999.empty or ax115_999 == 0:
    df_ax115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=50).value = df_ax115_999['total'].iloc[0]

ay115_999 = ay34_999 + ay75_999 + ay113_999
df_ay115_999 = pd.DataFrame({'total': [ay115_999]})
if df_ay115_999.empty or ay115_999 == 0:
    df_ay115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=51).value = df_ay115_999['total'].iloc[0]

az115_999 = az34_999 + az75_999 + az113_999
df_az115_999 = pd.DataFrame({'total': [az115_999]})
if df_az115_999.empty or az115_999 == 0:
    df_az115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=52).value = df_az115_999['total'].iloc[0]

ba115_999 = ba34_999 + ba75_999 + ba113_999
df_ba115_999 = pd.DataFrame({'total': [ba115_999]})
if df_ba115_999.empty or ba115_999 == 0:
    df_ba115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=53).value = df_ba115_999['total'].iloc[0]


# CONSULTA DE SUBTOTALES - RAMOS UNIDAD 14

In [431]:
# Realiza la suma para d75_999
d115_999 = d34_999 + d75_999 + d113_999
df_d115_999 = pd.DataFrame({'total': [d115_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d115_999.empty or d115_999 == 0:
    df_d115_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=115, column=4).value = df_d115_999['total'].iloc[0]

## UNIDAD DE CAPTURA 15

#  3   AUTOMOVILES UNIDAD 15

In [432]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_AUTOMOVILES
unidad15_AUTOMOVILES = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_AUTOMOVILES['TOTAL'] = unidad15_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 3, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 3, 119),
    ("Otros ingresos y Egresos", 3, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 3, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 3, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 3, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 3, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 3, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_AUTOMOVILES, concept, ramo, 5, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
e118_010 = get_value(unidad15_AUTOMOVILES, "Otras Provisiones (resta)", 3, ignore_subaccounts)
e119_015 = get_value(unidad15_AUTOMOVILES, "Ingresos y/o Egresos reportes programas especiales", 3, ignore_subaccounts)
e120_020 = get_value(unidad15_AUTOMOVILES, "Otros ingresos y Egresos", 3, ignore_subaccounts)
e121_025 = get_value(unidad15_AUTOMOVILES, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 3, ignore_subaccounts)
e122_030 = get_value(unidad15_AUTOMOVILES, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 3, ignore_subaccounts)
e123_035 = get_value(unidad15_AUTOMOVILES, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 3, ignore_subaccounts)
e124_040 = get_value(unidad15_AUTOMOVILES, "Egresos inversiones de las reservas técnicas (resta)", 3, ignore_subaccounts)
e125_045 = get_value(unidad15_AUTOMOVILES, "Otros gastos de administración y personal no asignables a ramos", 3, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
e126_999 = abs (e118_010 + e119_015 - e120_020 - e121_025 - e122_030 - e123_035 - e124_040 + e125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=5, value=e126_999)

<Cell 'Hoja1'.E126>

#  4   SOAT UNIDAD 15

In [433]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_SOAT
unidad15_SOAT = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_SOAT['TOTAL'] = unidad15_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 4, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 4, 119),
    ("Otros ingresos y Egresos", 4, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 4, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 4, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 4, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 4, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 4, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_SOAT, concept, ramo, 6, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
f118_010 = get_value(unidad15_SOAT, "Otras Provisiones (resta)", 4, ignore_subaccounts)
f119_015 = get_value(unidad15_SOAT, "Ingresos y/o Egresos reportes programas especiales", 4, ignore_subaccounts)
f120_020 = get_value(unidad15_SOAT, "Otros ingresos y Egresos", 4, ignore_subaccounts)
f121_025 = get_value(unidad15_SOAT, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 4, ignore_subaccounts)
f122_030 = get_value(unidad15_SOAT, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 4, ignore_subaccounts)
f123_035 = get_value(unidad15_SOAT, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 4, ignore_subaccounts)
f124_040 = get_value(unidad15_SOAT, "Egresos inversiones de las reservas técnicas (resta)", 4, ignore_subaccounts)
f125_045 = get_value(unidad15_SOAT, "Otros gastos de administración y personal no asignables a ramos", 4, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
f126_999 = abs(f118_010 + f119_015 - f120_020 - f121_025 - f122_030 - f123_035 - f124_040 + f125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=6, value=f126_999)

<Cell 'Hoja1'.F126>

#  5   CUMPLIMIENTO UNIDAD 15

In [434]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_CUMPLIMIENTO
unidad15_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_CUMPLIMIENTO['TOTAL'] = unidad15_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 5, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 5, 119),
    ("Otros ingresos y Egresos", 5, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 5, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 5, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 5, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 5, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 5, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_CUMPLIMIENTO, concept, ramo, 7, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
g118_010 = get_value(unidad15_CUMPLIMIENTO, "Otras Provisiones (resta)", 5, ignore_subaccounts)
g119_015 = get_value(unidad15_CUMPLIMIENTO, "Ingresos y/o Egresos reportes programas especiales", 5, ignore_subaccounts)
g120_020 = get_value(unidad15_CUMPLIMIENTO, "Otros ingresos y Egresos", 5, ignore_subaccounts)
g121_025 = get_value(unidad15_CUMPLIMIENTO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 5, ignore_subaccounts)
g122_030 = get_value(unidad15_CUMPLIMIENTO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 5, ignore_subaccounts)
g123_035 = get_value(unidad15_CUMPLIMIENTO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 5, ignore_subaccounts)
g124_040 = get_value(unidad15_CUMPLIMIENTO, "Egresos inversiones de las reservas técnicas (resta)", 5, ignore_subaccounts)
g125_045 = get_value(unidad15_CUMPLIMIENTO, "Otros gastos de administración y personal no asignables a ramos", 5, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
g126_999 = abs(g118_010 + g119_015 - g120_020 - g121_025 - g122_030 - g123_035 - g124_040 + g125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=7, value=g126_999)

<Cell 'Hoja1'.G126>

#  6   RESPONSABILIDAD CIVIL UNIDAD 15

In [435]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_RESPONSABILIDADCI
unidad15_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_RESPONSABILIDADCI['TOTAL'] = unidad15_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 6, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 6, 119),
    ("Otros ingresos y Egresos", 6, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 6, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 6, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 6, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 6, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 6, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_RESPONSABILIDADCI, concept, ramo, 8, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
h118_010 = get_value(unidad15_RESPONSABILIDADCI, "Otras Provisiones (resta)", 6, ignore_subaccounts)
h119_015 = get_value(unidad15_RESPONSABILIDADCI, "Ingresos y/o Egresos reportes programas especiales", 6, ignore_subaccounts)
h120_020 = get_value(unidad15_RESPONSABILIDADCI, "Otros ingresos y Egresos", 6, ignore_subaccounts)
h121_025 = get_value(unidad15_RESPONSABILIDADCI, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 6, ignore_subaccounts)
h122_030 = get_value(unidad15_RESPONSABILIDADCI, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 6, ignore_subaccounts)
h123_035 = get_value(unidad15_RESPONSABILIDADCI, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 6, ignore_subaccounts)
h124_040 = get_value(unidad15_RESPONSABILIDADCI, "Egresos inversiones de las reservas técnicas (resta)", 6, ignore_subaccounts)
h125_045 = get_value(unidad15_RESPONSABILIDADCI, "Otros gastos de administración y personal no asignables a ramos", 6, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
h126_999 = abs(h118_010 + h119_015 - h120_020 - h121_025 - h122_030 - h123_035 - h124_040 + h125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=8, value=h126_999)

<Cell 'Hoja1'.H126>

#  7   INCENDIO UNIDAD 15

In [436]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_RESPONSABILIDADCI
unidad15_INCENDIO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_INCENDIO['TOTAL'] = unidad15_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 7, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 7, 119),
    ("Otros ingresos y Egresos", 7, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 7, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 7, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 7, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 7, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 7, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_INCENDIO, concept, ramo, 9, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
i118_010 = get_value(unidad15_INCENDIO, "Otras Provisiones (resta)", 7, ignore_subaccounts)
i119_015 = get_value(unidad15_INCENDIO, "Ingresos y/o Egresos reportes programas especiales", 7, ignore_subaccounts)
i120_020 = get_value(unidad15_INCENDIO, "Otros ingresos y Egresos", 7, ignore_subaccounts)
i121_025 = get_value(unidad15_INCENDIO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 7, ignore_subaccounts)
i122_030 = get_value(unidad15_INCENDIO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 7, ignore_subaccounts)
i123_035 = get_value(unidad15_INCENDIO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 7, ignore_subaccounts)
i124_040 = get_value(unidad15_INCENDIO, "Egresos inversiones de las reservas técnicas (resta)", 7, ignore_subaccounts)
i125_045 = get_value(unidad15_INCENDIO, "Otros gastos de administración y personal no asignables a ramos", 7, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
i126_999 = abs(i118_010 + i119_015 - i120_020 - i121_025 - i122_030 - i123_035 - i124_040 + i125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=9, value=i126_999)

<Cell 'Hoja1'.I126>

#  8   TERREMOTO UNIDAD 15

In [437]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_TERREMOTO
unidad15_TERREMOTO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_TERREMOTO['TOTAL'] = unidad15_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 8, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 8, 119),
    ("Otros ingresos y Egresos", 8, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 8, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 8, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 8, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 8, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 8, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_TERREMOTO, concept, ramo, 10, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
j118_010 = get_value(unidad15_TERREMOTO, "Otras Provisiones (resta)", 8, ignore_subaccounts)
j119_015 = get_value(unidad15_TERREMOTO, "Ingresos y/o Egresos reportes programas especiales", 8, ignore_subaccounts)
j120_020 = get_value(unidad15_TERREMOTO, "Otros ingresos y Egresos", 8, ignore_subaccounts)
j121_025 = get_value(unidad15_TERREMOTO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 8, ignore_subaccounts)
j122_030 = get_value(unidad15_TERREMOTO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 8, ignore_subaccounts)
j123_035 = get_value(unidad15_TERREMOTO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 8, ignore_subaccounts)
j124_040 = get_value(unidad15_TERREMOTO, "Egresos inversiones de las reservas técnicas (resta)", 8, ignore_subaccounts)
j125_045 = get_value(unidad15_TERREMOTO, "Otros gastos de administración y personal no asignables a ramos", 8, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
j126_999 = abs(j118_010 + j119_015 - j120_020 - j121_025 - j122_030 - j123_035 - j124_040 + j125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=10, value=j126_999)

<Cell 'Hoja1'.J126>

#  9   SUSTRACCION UNIDAD 15

In [438]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_TERREMOTO
unidad15_SUSTRACCION = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_SUSTRACCION['TOTAL'] = unidad15_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 9, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 9, 119),
    ("Otros ingresos y Egresos", 9, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 9, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 9, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 9, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 9, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 9, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_SUSTRACCION, concept, ramo, 11, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
k118_010 = get_value(unidad15_SUSTRACCION, "Otras Provisiones (resta)", 9, ignore_subaccounts)
k119_015 = get_value(unidad15_SUSTRACCION, "Ingresos y/o Egresos reportes programas especiales", 9, ignore_subaccounts)
k120_020 = get_value(unidad15_SUSTRACCION, "Otros ingresos y Egresos", 9, ignore_subaccounts)
k121_025 = get_value(unidad15_SUSTRACCION, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 9, ignore_subaccounts)
k122_030 = get_value(unidad15_SUSTRACCION, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 9, ignore_subaccounts)
k123_035 = get_value(unidad15_SUSTRACCION, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 9, ignore_subaccounts)
k124_040 = get_value(unidad15_SUSTRACCION, "Egresos inversiones de las reservas técnicas (resta)", 9, ignore_subaccounts)
k125_045 = get_value(unidad15_SUSTRACCION, "Otros gastos de administración y personal no asignables a ramos", 9, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
k126_999 = abs(k118_010 + k119_015 - k120_020 - k121_025 - k122_030 - k123_035 - k124_040 + k125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=11, value=k126_999)

<Cell 'Hoja1'.K126>

#  10  TRANSPORTE UNIDAD 15

In [439]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_TERREMOTO
unidad15_TRANSPORTE = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_TRANSPORTE['TOTAL'] = unidad15_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 10, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 10, 119),
    ("Otros ingresos y Egresos", 10, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 10, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 10, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 10, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 10, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 10, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_TRANSPORTE, concept, ramo, 12, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
l118_010 = get_value(unidad15_TRANSPORTE, "Otras Provisiones (resta)", 10, ignore_subaccounts)
l119_015 = get_value(unidad15_TRANSPORTE, "Ingresos y/o Egresos reportes programas especiales", 10, ignore_subaccounts)
l120_020 = get_value(unidad15_TRANSPORTE, "Otros ingresos y Egresos", 10, ignore_subaccounts)
l121_025 = get_value(unidad15_TRANSPORTE, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 10, ignore_subaccounts)
l122_030 = get_value(unidad15_TRANSPORTE, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 10, ignore_subaccounts)
l123_035 = get_value(unidad15_TRANSPORTE, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 10, ignore_subaccounts)
l124_040 = get_value(unidad15_TRANSPORTE, "Egresos inversiones de las reservas técnicas (resta)", 10, ignore_subaccounts)
l125_045 = get_value(unidad15_TRANSPORTE, "Otros gastos de administración y personal no asignables a ramos", 10, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
l126_999 = abs(l118_010 + l119_015 - l120_020 - l121_025 - l122_030 - l123_035 - l124_040 + l125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=12, value=l126_999)

<Cell 'Hoja1'.L126>

#  11  CORRIENTE DÉBIL UNIDAD 15

In [440]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_CORRIENDEBIL
unidad15_CORRIENDEBIL = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_CORRIENDEBIL['TOTAL'] = unidad15_CORRIENDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 11, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 11, 119),
    ("Otros ingresos y Egresos", 11, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 11, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 11, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 11, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 11, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 11, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_CORRIENDEBIL, concept, ramo, 13, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
m118_010 = get_value(unidad15_CORRIENDEBIL, "Otras Provisiones (resta)", 11, ignore_subaccounts)
m119_015 = get_value(unidad15_CORRIENDEBIL, "Ingresos y/o Egresos reportes programas especiales", 11, ignore_subaccounts)
m120_020 = get_value(unidad15_CORRIENDEBIL, "Otros ingresos y Egresos", 11, ignore_subaccounts)
m121_025 = get_value(unidad15_CORRIENDEBIL, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 11, ignore_subaccounts)
m122_030 = get_value(unidad15_CORRIENDEBIL, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 11, ignore_subaccounts)
m123_035 = get_value(unidad15_CORRIENDEBIL, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 11, ignore_subaccounts)
m124_040 = get_value(unidad15_CORRIENDEBIL, "Egresos inversiones de las reservas técnicas (resta)", 11, ignore_subaccounts)
m125_045 = get_value(unidad15_CORRIENDEBIL, "Otros gastos de administración y personal no asignables a ramos", 11, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
m126_999 = abs(m118_010 + m119_015 - m120_020 - m121_025 - m122_030 - m123_035 - m124_040 + m125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=13, value=m126_999)

<Cell 'Hoja1'.M126>

#  12  TODO RIESGO CONTRATISTA UNIDAD 15

In [441]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_TODORIESGOCONTRATISTA
unidad15_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_TODORIESGOCONTRATISTA['TOTAL'] = unidad15_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 12, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 12, 119),
    ("Otros ingresos y Egresos", 12, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 12, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 12, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 12, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 12, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 12, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_TODORIESGOCONTRATISTA, concept, ramo, 14, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
n118_010 = get_value(unidad15_TODORIESGOCONTRATISTA, "Otras Provisiones (resta)", 12, ignore_subaccounts)
n119_015 = get_value(unidad15_TODORIESGOCONTRATISTA, "Ingresos y/o Egresos reportes programas especiales", 12, ignore_subaccounts)
n120_020 = get_value(unidad15_TODORIESGOCONTRATISTA, "Otros ingresos y Egresos", 12, ignore_subaccounts)
n121_025 = get_value(unidad15_TODORIESGOCONTRATISTA, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 12, ignore_subaccounts)
n122_030 = get_value(unidad15_TODORIESGOCONTRATISTA, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 12, ignore_subaccounts)
n123_035 = get_value(unidad15_TODORIESGOCONTRATISTA, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 12, ignore_subaccounts)
n124_040 = get_value(unidad15_TODORIESGOCONTRATISTA, "Egresos inversiones de las reservas técnicas (resta)", 12, ignore_subaccounts)
n125_045 = get_value(unidad15_TODORIESGOCONTRATISTA, "Otros gastos de administración y personal no asignables a ramos", 12, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
n126_999 = abs(n118_010 + n119_015 - n120_020 - n121_025 - n122_030 - n123_035 - n124_040 + n125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=14, value=n126_999)

<Cell 'Hoja1'.N126>

#  13  MANEJO UNIDAD 15

In [442]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_MANEJO
unidad15_MANEJO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_MANEJO['TOTAL'] = unidad15_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 13, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 13, 119),
    ("Otros ingresos y Egresos", 13, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 13, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 13, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 13, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 13, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 13, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_MANEJO, concept, ramo, 15, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
o118_010 = get_value(unidad15_MANEJO, "Otras Provisiones (resta)", 13, ignore_subaccounts)
o119_015 = get_value(unidad15_MANEJO, "Ingresos y/o Egresos reportes programas especiales", 13, ignore_subaccounts)
o120_020 = get_value(unidad15_MANEJO, "Otros ingresos y Egresos", 13, ignore_subaccounts)
o121_025 = get_value(unidad15_MANEJO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 13, ignore_subaccounts)
o122_030 = get_value(unidad15_MANEJO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 13, ignore_subaccounts)
o123_035 = get_value(unidad15_MANEJO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 13, ignore_subaccounts)
o124_040 = get_value(unidad15_MANEJO, "Egresos inversiones de las reservas técnicas (resta)", 13, ignore_subaccounts)
o125_045 = get_value(unidad15_MANEJO, "Otros gastos de administración y personal no asignables a ramos", 13, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
o126_999 = abs(o118_010 + o119_015 - o120_020 - o121_025 - o122_030 - o123_035 - o124_040 + o125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=15, value=o126_999)

<Cell 'Hoja1'.O126>

#  14  LUCRO CESANTE UNIDAD 15

In [443]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_LUCROCESANTE
unidad15_LUCROCESANTE = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_LUCROCESANTE['TOTAL'] = unidad15_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 14, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 14, 119),
    ("Otros ingresos y Egresos", 14, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 14, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 14, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 14, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 14, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 14, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_LUCROCESANTE, concept, ramo, 16, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
p118_010 = get_value(unidad15_LUCROCESANTE, "Otras Provisiones (resta)", 14, ignore_subaccounts)
p119_015 = get_value(unidad15_LUCROCESANTE, "Ingresos y/o Egresos reportes programas especiales", 14, ignore_subaccounts)
p120_020 = get_value(unidad15_LUCROCESANTE, "Otros ingresos y Egresos", 14, ignore_subaccounts)
p121_025 = get_value(unidad15_LUCROCESANTE, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 14, ignore_subaccounts)
p122_030 = get_value(unidad15_LUCROCESANTE, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 14, ignore_subaccounts)
p123_035 = get_value(unidad15_LUCROCESANTE, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 14, ignore_subaccounts)
p124_040 = get_value(unidad15_LUCROCESANTE, "Egresos inversiones de las reservas técnicas (resta)", 14, ignore_subaccounts)
p125_045 = get_value(unidad15_LUCROCESANTE, "Otros gastos de administración y personal no asignables a ramos", 14, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
p126_999 = abs(p118_010 + p119_015 - p120_020 - p121_025 - p122_030 - p123_035 - p124_040 + p125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=16, value=p126_999)

<Cell 'Hoja1'.P126>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 15

In [444]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_MONTAJEYROTURA
unidad15_MONTAJEYROTURA = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_MONTAJEYROTURA['TOTAL'] = unidad15_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 15, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 15, 119),
    ("Otros ingresos y Egresos", 15, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 15, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 15, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 15, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 15, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 15, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_MONTAJEYROTURA, concept, ramo, 17, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
q118_010 = get_value(unidad15_MONTAJEYROTURA, "Otras Provisiones (resta)", 15, ignore_subaccounts)
q119_015 = get_value(unidad15_MONTAJEYROTURA, "Ingresos y/o Egresos reportes programas especiales", 15, ignore_subaccounts)
q120_020 = get_value(unidad15_MONTAJEYROTURA, "Otros ingresos y Egresos", 15, ignore_subaccounts)
q121_025 = get_value(unidad15_MONTAJEYROTURA, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 15, ignore_subaccounts)
q122_030 = get_value(unidad15_MONTAJEYROTURA, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 15, ignore_subaccounts)
q123_035 = get_value(unidad15_MONTAJEYROTURA, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 15, ignore_subaccounts)
q124_040 = get_value(unidad15_MONTAJEYROTURA, "Egresos inversiones de las reservas técnicas (resta)", 15, ignore_subaccounts)
q125_045 = get_value(unidad15_MONTAJEYROTURA, "Otros gastos de administración y personal no asignables a ramos", 15, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
q126_999 = abs(q118_010 + q119_015 - q120_020 - q121_025 - q122_030 - q123_035 - q124_040 + q125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=17, value=q126_999)

<Cell 'Hoja1'.Q126>

#  16  AVIACIÓN UNIDAD 15

In [445]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_AVIACION
unidad15_AVIACION = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_AVIACION['TOTAL'] = unidad15_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 16, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 16, 119),
    ("Otros ingresos y Egresos", 16, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 16, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 16, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 16, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 16, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 16, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_AVIACION, concept, ramo, 18, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
r118_010 = get_value(unidad15_AVIACION, "Otras Provisiones (resta)", 16, ignore_subaccounts)
r119_015 = get_value(unidad15_AVIACION, "Ingresos y/o Egresos reportes programas especiales", 16, ignore_subaccounts)
r120_020 = get_value(unidad15_AVIACION, "Otros ingresos y Egresos", 16, ignore_subaccounts)
r121_025 = get_value(unidad15_AVIACION, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 16, ignore_subaccounts)
r122_030 = get_value(unidad15_AVIACION, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 16, ignore_subaccounts)
r123_035 = get_value(unidad15_AVIACION, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 16, ignore_subaccounts)
r124_040 = get_value(unidad15_AVIACION, "Egresos inversiones de las reservas técnicas (resta)", 16, ignore_subaccounts)
r125_045 = get_value(unidad15_AVIACION, "Otros gastos de administración y personal no asignables a ramos", 16, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
r126_999 = abs(r118_010 + r119_015 - r120_020 - r121_025 - r122_030 - r123_035 - r124_040 + r125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=18, value=r126_999)

<Cell 'Hoja1'.R126>

#  17  NAVEGACIÓN Y CASCO UNIDAD 15

In [446]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_NAVEGACION
unidad15_NAVEGACION = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_NAVEGACION['TOTAL'] = unidad15_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 17, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 17, 119),
    ("Otros ingresos y Egresos", 17, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 17, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 17, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 17, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 17, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 17, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_NAVEGACION, concept, ramo, 19, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
s118_010 = get_value(unidad15_NAVEGACION, "Otras Provisiones (resta)", 17, ignore_subaccounts)
s119_015 = get_value(unidad15_NAVEGACION, "Ingresos y/o Egresos reportes programas especiales", 17, ignore_subaccounts)
s120_020 = get_value(unidad15_NAVEGACION, "Otros ingresos y Egresos", 17, ignore_subaccounts)
s121_025 = get_value(unidad15_NAVEGACION, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 17, ignore_subaccounts)
s122_030 = get_value(unidad15_NAVEGACION, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 17, ignore_subaccounts)
s123_035 = get_value(unidad15_NAVEGACION, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 17, ignore_subaccounts)
s124_040 = get_value(unidad15_NAVEGACION, "Egresos inversiones de las reservas técnicas (resta)", 17, ignore_subaccounts)
s125_045 = get_value(unidad15_NAVEGACION, "Otros gastos de administración y personal no asignables a ramos", 17, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
s126_999 = abs(s118_010 + s119_015 - s120_020 - s121_025 - s122_030 - s123_035 - s124_040 + s125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=19, value=s126_999)

<Cell 'Hoja1'.S126>

#  18  MINAS Y PETRÓLEOS UNIDAD 15

In [447]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_MINAS
unidad15_MINAS = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_MINAS['TOTAL'] = unidad15_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 18, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 18, 119),
    ("Otros ingresos y Egresos", 18, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 18, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 18, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 18, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 18, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 18, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_MINAS, concept, ramo, 20, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
t118_010 = get_value(unidad15_MINAS, "Otras Provisiones (resta)", 18, ignore_subaccounts)
t119_015 = get_value(unidad15_MINAS, "Ingresos y/o Egresos reportes programas especiales", 18, ignore_subaccounts)
t120_020 = get_value(unidad15_MINAS, "Otros ingresos y Egresos", 18, ignore_subaccounts)
t121_025 = get_value(unidad15_MINAS, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 18, ignore_subaccounts)
t122_030 = get_value(unidad15_MINAS, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 18, ignore_subaccounts)
t123_035 = get_value(unidad15_MINAS, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 18, ignore_subaccounts)
t124_040 = get_value(unidad15_MINAS, "Egresos inversiones de las reservas técnicas (resta)", 18, ignore_subaccounts)
t125_045 = get_value(unidad15_MINAS, "Otros gastos de administración y personal no asignables a ramos", 18, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
t126_999 = abs(t118_010 + t119_015 - t120_020 - t121_025 - t122_030 - t123_035 - t124_040 + t125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=20, value=t126_999)

<Cell 'Hoja1'.T126>

#  19  VIDRIOS UNIDAD 15

In [448]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_VIDRIOS
unidad15_VIDRIOS = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_VIDRIOS['TOTAL'] = unidad15_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 19, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 19, 119),
    ("Otros ingresos y Egresos", 19, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 19, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 19, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 19, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 19, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 19, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_VIDRIOS, concept, ramo, 21, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
u118_010 = get_value(unidad15_VIDRIOS, "Otras Provisiones (resta)", 19, ignore_subaccounts)
u119_015 = get_value(unidad15_VIDRIOS, "Ingresos y/o Egresos reportes programas especiales", 19, ignore_subaccounts)
u120_020 = get_value(unidad15_VIDRIOS, "Otros ingresos y Egresos", 19, ignore_subaccounts)
u121_025 = get_value(unidad15_VIDRIOS, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 19, ignore_subaccounts)
u122_030 = get_value(unidad15_VIDRIOS, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 19, ignore_subaccounts)
u123_035 = get_value(unidad15_VIDRIOS, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 19, ignore_subaccounts)
u124_040 = get_value(unidad15_VIDRIOS, "Egresos inversiones de las reservas técnicas (resta)", 19, ignore_subaccounts)
u125_045 = get_value(unidad15_VIDRIOS, "Otros gastos de administración y personal no asignables a ramos", 19, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
u126_999 = abs(u118_010 + u119_015 - u120_020 - u121_025 - u122_030 - u123_035 - u124_040 + u125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=21, value=u126_999)

<Cell 'Hoja1'.U126>

#  20  CRÉDITO COMERCIAL UNIDAD 15

In [449]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_CREDITOCOMERCIAL
unidad15_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_CREDITOCOMERCIAL['TOTAL'] = unidad15_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 20, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 20, 119),
    ("Otros ingresos y Egresos", 20, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 20, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 20, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 20, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 20, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 20, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_CREDITOCOMERCIAL, concept, ramo, 22, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
v118_010 = get_value(unidad15_CREDITOCOMERCIAL, "Otras Provisiones (resta)", 20, ignore_subaccounts)
v119_015 = get_value(unidad15_CREDITOCOMERCIAL, "Ingresos y/o Egresos reportes programas especiales", 20, ignore_subaccounts)
v120_020 = get_value(unidad15_CREDITOCOMERCIAL, "Otros ingresos y Egresos", 20, ignore_subaccounts)
v121_025 = get_value(unidad15_CREDITOCOMERCIAL, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 20, ignore_subaccounts)
v122_030 = get_value(unidad15_CREDITOCOMERCIAL, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 20, ignore_subaccounts)
v123_035 = get_value(unidad15_CREDITOCOMERCIAL, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 20, ignore_subaccounts)
v124_040 = get_value(unidad15_CREDITOCOMERCIAL, "Egresos inversiones de las reservas técnicas (resta)", 20, ignore_subaccounts)
v125_045 = get_value(unidad15_CREDITOCOMERCIAL, "Otros gastos de administración y personal no asignables a ramos", 20, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
v126_999 = abs(v118_010 + v119_015 - v120_020 - v121_025 - v122_030 - v123_035 - v124_040 + v125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=22, value=v126_999)

<Cell 'Hoja1'.V126>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 15

In [450]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_CREDITOEXPORTACION
unidad15_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_CREDITOEXPORTACION['TOTAL'] = unidad15_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 21, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 21, 119),
    ("Otros ingresos y Egresos", 21, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 21, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 21, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 21, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 21, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 21, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_CREDITOEXPORTACION, concept, ramo, 23, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
w118_010 = get_value(unidad15_CREDITOEXPORTACION, "Otras Provisiones (resta)", 21, ignore_subaccounts)
w119_015 = get_value(unidad15_CREDITOEXPORTACION, "Ingresos y/o Egresos reportes programas especiales", 21, ignore_subaccounts)
w120_020 = get_value(unidad15_CREDITOEXPORTACION, "Otros ingresos y Egresos", 21, ignore_subaccounts)
w121_025 = get_value(unidad15_CREDITOEXPORTACION, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 21, ignore_subaccounts)
w122_030 = get_value(unidad15_CREDITOEXPORTACION, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 21, ignore_subaccounts)
w123_035 = get_value(unidad15_CREDITOEXPORTACION, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 21, ignore_subaccounts)
w124_040 = get_value(unidad15_CREDITOEXPORTACION, "Egresos inversiones de las reservas técnicas (resta)", 21, ignore_subaccounts)
w125_045 = get_value(unidad15_CREDITOEXPORTACION, "Otros gastos de administración y personal no asignables a ramos", 21, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
w126_999 = abs(w118_010 + w119_015 - w120_020 - w121_025 - w122_030 - w123_035 - w124_040 + w125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=23, value=w126_999)

<Cell 'Hoja1'.W126>

#  24  DESEMPLEO UNIDAD 15

In [451]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_DESEMPLEO
unidad15_DESEMPLEO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_DESEMPLEO['TOTAL'] = unidad15_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 24, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 24, 119),
    ("Otros ingresos y Egresos", 24, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 24, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 24, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 24, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 24, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 24, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_DESEMPLEO, concept, ramo, 26, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
z118_010 = get_value(unidad15_DESEMPLEO, "Otras Provisiones (resta)", 24, ignore_subaccounts)
z119_015 = get_value(unidad15_DESEMPLEO, "Ingresos y/o Egresos reportes programas especiales", 24, ignore_subaccounts)
z120_020 = get_value(unidad15_DESEMPLEO, "Otros ingresos y Egresos", 24, ignore_subaccounts)
z121_025 = get_value(unidad15_DESEMPLEO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 24, ignore_subaccounts)
z122_030 = get_value(unidad15_DESEMPLEO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 24, ignore_subaccounts)
z123_035 = get_value(unidad15_DESEMPLEO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 24, ignore_subaccounts)
z124_040 = get_value(unidad15_DESEMPLEO, "Egresos inversiones de las reservas técnicas (resta)", 24, ignore_subaccounts)
z125_045 = get_value(unidad15_DESEMPLEO, "Otros gastos de administración y personal no asignables a ramos", 24, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
z126_999 = abs(z118_010 + z119_015 - z120_020 - z121_025 - z122_030 - z123_035 - z124_040 + z125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=26, value=z126_999)

<Cell 'Hoja1'.Z126>

#  25  HOGAR UNIDAD 15

In [452]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_HOGAR
unidad15_HOGAR = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_HOGAR['TOTAL'] = unidad15_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 25, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 27, 119),
    ("Otros ingresos y Egresos", 25, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 25, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 25, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 25, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 25, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 25, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_HOGAR, concept, ramo, 27, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
aa118_010 = get_value(unidad15_HOGAR, "Otras Provisiones (resta)", 25, ignore_subaccounts)
aa119_015 = get_value(unidad15_HOGAR, "Ingresos y/o Egresos reportes programas especiales", 25, ignore_subaccounts)
aa120_020 = get_value(unidad15_HOGAR, "Otros ingresos y Egresos", 25, ignore_subaccounts)
aa121_025 = get_value(unidad15_HOGAR, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 25, ignore_subaccounts)
aa122_030 = get_value(unidad15_HOGAR, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 25, ignore_subaccounts)
aa123_035 = get_value(unidad15_HOGAR, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 25, ignore_subaccounts)
aa124_040 = get_value(unidad15_HOGAR, "Egresos inversiones de las reservas técnicas (resta)", 25, ignore_subaccounts)
aa125_045 = get_value(unidad15_HOGAR, "Otros gastos de administración y personal no asignables a ramos", 25, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
aa126_999 = abs(aa118_010 + aa119_015 - aa120_020 - aa121_025 - aa122_030 - aa123_035 - aa124_040 + aa125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=27, value=aa126_999)

<Cell 'Hoja1'.AA126>

# 42 AGROPECUARIO UNIDAD 15

In [453]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_AGROPECUARIO
unidad15_AGROPECUARIO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_AGROPECUARIO['TOTAL'] = unidad15_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 42, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 42, 119),
    ("Otros ingresos y Egresos", 42, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 42, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 42, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 42, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 42, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 42, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_AGROPECUARIO, concept, ramo, 44, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ar118_010 = get_value(unidad15_AGROPECUARIO, "Otras Provisiones (resta)", 42, ignore_subaccounts)
ar119_015 = get_value(unidad15_AGROPECUARIO, "Ingresos y/o Egresos reportes programas especiales", 42, ignore_subaccounts)
ar120_020 = get_value(unidad15_AGROPECUARIO, "Otros ingresos y Egresos", 42, ignore_subaccounts)
ar121_025 = get_value(unidad15_AGROPECUARIO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 42, ignore_subaccounts)
ar122_030 = get_value(unidad15_AGROPECUARIO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 42, ignore_subaccounts)
ar123_035 = get_value(unidad15_AGROPECUARIO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 42, ignore_subaccounts)
ar124_040 = get_value(unidad15_AGROPECUARIO, "Egresos inversiones de las reservas técnicas (resta)", 42, ignore_subaccounts)
ar125_045 = get_value(unidad15_AGROPECUARIO, "Otros gastos de administración y personal no asignables a ramos", 42, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ar126_999 = abs(ar118_010 + ar119_015 - ar120_020 - ar121_025 - ar122_030 - ar123_035 - ar124_040 + ar125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=44, value=ar126_999)

<Cell 'Hoja1'.AR126>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 15

In [454]:
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otras Provisiones (resta)") ]
as118_010 = cons['TOTAL'].sum()
ws.cell(row=118, column=45, value=as118_010)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") ]
as119_015 = cons['TOTAL'].sum()
ws.cell(row=119, column=45, value=as119_015)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15)].groupby(['CONCEPTO', 'COD_RAMO', 'LEDGER'])['TOTAL'].sum().reset_index()
cons = unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otros ingresos y Egresos")]
as120_020 = cons['TOTAL'].sum() * -1  # Cambiar el signo del valor
ws.cell(row=120, column=45, value=as120_020)

unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") ]
as121_025 = cons['TOTAL'].sum()
as121_025 = 0
ws.cell(row=121, column=45, value=as121_025)


unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") ]
as122_030 = cons['TOTAL'].sum()
as122_030 = 0
ws.cell(row=122, column=45, value=as122_030)
 
as121_025_030 = bd[(bd['UNIDAD'] == 15) & (bd['LINE OF BUSS'] == '999') &
                   (bd['CONCEPTO'].isin([
                       "Ingresos realizados de activos a valor razonable respaldando reservas técnicas",
                       "Ingresos causados de activos a costo amortizado respaldando reservas técnicas"
                   ]))]['TOTAL'].sum()
 
 
as123_035_1 = bd[(bd['UNIDAD'] == 15) &
                 (bd['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas")]['TOTAL'].sum()
as123_035 = as123_035_1 + as121_025_030
# Invertir el signo
as123_035 *= -1  
ws.cell(row=123, column=45, value=as123_035)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") ]
as124_040 = cons['TOTAL'].sum()
ws.cell(row=124, column=45, value=as124_040)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") ]
as125_045 = cons['TOTAL'].sum()
ws.cell(row=125, column=45, value=as125_045)
 
as126_999 = abs (as118_010 + as119_015 - as120_020 - as121_025 - as122_030 - as123_035 - as124_040 + as125_045)
ws.cell(row=126, column=45, value=as126_999)


<Cell 'Hoja1'.AS126>

# 44 DECENAL UNIDAD 15

In [455]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad15_AGROPECUARIO
unidad15_DECENAL = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_DECENAL['TOTAL'] = unidad15_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 44, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 44, 119),
    ("Otros ingresos y Egresos", 44, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 44, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 44, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 44, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 44, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 44, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_DECENAL, concept, ramo, 46, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
at118_010 = get_value(unidad15_DECENAL, "Otras Provisiones (resta)", 44, ignore_subaccounts)
at119_015 = get_value(unidad15_DECENAL, "Ingresos y/o Egresos reportes programas especiales", 44, ignore_subaccounts)
at120_020 = get_value(unidad15_DECENAL, "Otros ingresos y Egresos", 44, ignore_subaccounts)
at121_025 = get_value(unidad15_DECENAL, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 44, ignore_subaccounts)
at122_030 = get_value(unidad15_DECENAL, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 44, ignore_subaccounts)
at123_035 = get_value(unidad15_DECENAL, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 44, ignore_subaccounts)
at124_040 = get_value(unidad15_DECENAL, "Egresos inversiones de las reservas técnicas (resta)", 44, ignore_subaccounts)
at125_045 = get_value(unidad15_DECENAL, "Otros gastos de administración y personal no asignables a ramos", 44, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
at126_999 = abs(at118_010 + at119_015 - at120_020 - at121_025 - at122_030 - at123_035 - at124_040 + at125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=46, value=at126_999)

<Cell 'Hoja1'.AT126>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 15

In [456]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total
    

au118_010 = unidad10_CUMPLIMIENTOESTATALES[(unidad10_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otros productos de seguros") & (unidad10_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au86_005, 47, 86)

au118_010 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au118_010, 47, 118)

au119_015 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au119_015, 47, 119)

au120_020 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au120_020, 47, 120)

au121_025 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au121_025, 47, 121)

au122_030 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au122_030, 47, 122)

au123_035 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au123_035, 47, 123)

au124_040 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au124_040, 47, 124)

au125_045 = unidad15_CUMPLIMIENTOESTATALES[(unidad15_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au125_045, 47, 125)

# Calcular au32_999
au126_999 = au118_010['TOTAL'].sum() + au119_015['TOTAL'].sum() + au120_020['TOTAL'].sum() + au121_025['TOTAL'].sum() + au122_030['TOTAL'].sum() + \
            au123_035['TOTAL'].sum() + au124_040['TOTAL'].sum() + au125_045['TOTAL'].sum()

au126_999 = abs(au126_999)
# Crear DataFrame para escribir en Excel
df_au126_999= pd.DataFrame({'total': [au126_999]})
write_to_excel(df_au126_999, 47, 126, 'total')

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 15

In [457]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av118_010 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av118_010, 48, 118)

av119_015 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av119_015, 48, 119)

av120_020 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av120_020, 48, 120)

av121_025 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av121_025, 48, 121)

av122_030 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av122_030, 48, 122)

av123_035 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av123_035, 48, 123)

av124_040 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av124_040, 48, 124)

av125_045 = unidad15_CUMPLIMIENTOEMPRESASSERV[(unidad15_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av125_045, 48, 125)

# Calcular ak32_999
av126_999 = av118_010['TOTAL'].sum() + av119_015['TOTAL'].sum() + av120_020['TOTAL'].sum() + av121_025['TOTAL'].sum() + av122_030['TOTAL'].sum() + \
            av123_035['TOTAL'].sum() + av124_040['TOTAL'].sum() + av125_045['TOTAL'].sum()

av126_999 = abs(av126_999)
# Crear DataFrame para escribir en Excel
df_av126_999= pd.DataFrame({'total': [av126_999]})
write_to_excel(df_av126_999, 48, 126, 'total')

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 15

In [458]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


aw118_010 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw118_010, 49, 118)

aw119_015 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw119_015, 49, 119)

aw120_020 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw120_020, 49, 120)

aw121_025 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw121_025, 49, 121)

aw122_030 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw122_030, 49, 122)

aw123_035 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw123_035, 49, 123)

aw124_040 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw124_040, 49, 124)

aw125_045 = unidad15_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw125_045, 49, 125)

# Calcular ak32_999
aw126_999 = aw118_010['TOTAL'].sum() + aw119_015['TOTAL'].sum() + aw120_020['TOTAL'].sum() + aw121_025['TOTAL'].sum() + aw122_030['TOTAL'].sum() + \
            aw123_035['TOTAL'].sum() + aw124_040['TOTAL'].sum() + aw125_045['TOTAL'].sum()

aw126_999 = abs(aw126_999)
# Crear DataFrame para escribir en Excel
df_aw126_999= pd.DataFrame({'total': [aw126_999]})
write_to_excel(df_aw126_999, 49, 126, 'total')

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 15

In [459]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ax118_010 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax118_010, 50, 118)

ax119_015 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax119_015, 50, 119)

ax120_020 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax120_020, 50, 120)

ax121_025 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax121_025, 50, 121)

ax122_030 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax122_030, 50, 122)

ax123_035 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax123_035, 50, 123)

ax124_040 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax124_040, 50, 124)

ax125_045 = unidad15_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax125_045, 50, 125)

# Calcular ak32_999
ax126_999 = ax118_010['TOTAL'].sum() + ax119_015['TOTAL'].sum() + ax120_020['TOTAL'].sum() + ax121_025['TOTAL'].sum() + ax122_030['TOTAL'].sum() + \
            ax123_035['TOTAL'].sum() + ax124_040['TOTAL'].sum() + ax125_045['TOTAL'].sum()

ax126_999 = abs(ax126_999)
# Crear DataFrame para escribir en Excel
df_ax126_999= pd.DataFrame({'total': [ax126_999]})
write_to_excel(df_ax126_999, 50, 126, 'total')

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 15

In [460]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ay118_010 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay118_010, 51, 118)

ay119_015 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay119_015, 51, 119)

ay120_020 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay120_020, 51, 120)

ay121_025 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay121_025, 51, 121)

ay122_030 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay122_030, 51, 122)

ay123_035 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay123_035, 51, 123)

ay124_040 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay124_040, 51, 124)

ay125_045 = unidad15_CUMPLIMIENTOCAUCIONES[(unidad15_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay125_045, 51, 125)

# Calcular ak32_999
ay126_999 = ay118_010['TOTAL'].sum() + ay119_015['TOTAL'].sum() + ay120_020['TOTAL'].sum() + ay121_025['TOTAL'].sum() + ay122_030['TOTAL'].sum() + \
            ay123_035['TOTAL'].sum() + ay124_040['TOTAL'].sum() + ay125_045['TOTAL'].sum()

ay126_999 = abs(ay126_999)
# Crear DataFrame para escribir en Excel
df_ay126_999= pd.DataFrame({'total': [ay126_999]})
write_to_excel(df_ay126_999, 51, 126, 'total')

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 15

In [461]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


az118_010 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az118_010, 52, 118)

az119_015 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az119_015, 52, 119)

az120_020 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az120_020, 52, 120)

az121_025 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az121_025, 52, 121)

az122_030 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az122_030, 52, 122)

az123_035 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az123_035, 52, 123)

az124_040 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az124_040, 52, 124)

az125_045 = unidad15_CUMPLIMIENTOARRENDAMIENTO[(unidad15_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az125_045, 52, 125)

# Calcular ak32_999
az126_999 = az118_010['TOTAL'].sum() + az119_015['TOTAL'].sum() + az120_020['TOTAL'].sum() + az121_025['TOTAL'].sum() + az122_030['TOTAL'].sum() + \
            az123_035['TOTAL'].sum() + az124_040['TOTAL'].sum() + az125_045['TOTAL'].sum()

az126_999 = abs(az126_999)
# Crear DataFrame para escribir en Excel
df_az126_999= pd.DataFrame({'total': [az126_999]})
write_to_excel(df_az126_999, 52, 126, 'total')

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 15

In [462]:
# Agrupar los datos según el concepto y el ramo
unidad15_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad15_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba118_010 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Otras Provisiones (resta)") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba118_010, 53, 118)

ba119_015 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba119_015, 53, 119)

ba120_020 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Otros ingresos y Egresos") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba120_020, 53, 120)

ba121_025 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba121_025, 53, 121)

ba122_030 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba122_030, 53, 122)

ba123_035 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba123_035, 53, 123)

ba124_040 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba124_040, 53, 124)

ba125_045 = unidad15_CUMPLIMIENTOPARTICULARES[(unidad15_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") & (unidad15_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba125_045, 53, 125)

# Calcular ak32_999
ba126_999 = ba118_010['TOTAL'].sum() + ba119_015['TOTAL'].sum() + ba120_020['TOTAL'].sum() + ba121_025['TOTAL'].sum() + ba122_030['TOTAL'].sum() + \
            ba123_035['TOTAL'].sum() + ba124_040['TOTAL'].sum() + ba125_045['TOTAL'].sum()

ba126_999 = abs(ba126_999)
# Crear DataFrame para escribir en Excel
df_ba126_999= pd.DataFrame({'total': [ba126_999]})
write_to_excel(df_ba126_999, 53, 126, 'total')


#  1   TOTAL UNIDAD 15

In [463]:
c118_010 = sum([ws.cell(row=118, column=col).value if ws.cell(row=118, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=118, column=3).value = c118_010

c119_015 = sum([ws.cell(row=119, column=col).value if ws.cell(row=119, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=119, column=3).value = c119_015

c120_020 = sum([ws.cell(row=120, column=col).value if ws.cell(row=120, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=120, column=3).value = c120_020

c121_025 = sum([ws.cell(row=121, column=col).value if ws.cell(row=121, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=121, column=3).value = c121_025

c122_030 = sum([ws.cell(row=122, column=col).value if ws.cell(row=122, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=122, column=3).value = c122_030

c123_035 = sum([ws.cell(row=123, column=col).value if ws.cell(row=123, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=123, column=3).value = c123_035

c124_040 = sum([ws.cell(row=124, column=col).value if ws.cell(row=124, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=124, column=3).value = c124_040

c125_045 = sum([ws.cell(row=125, column=col).value if ws.cell(row=125, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=125, column=3).value = c125_045

# Calcular el total general según la fórmula dada

c126_999 = abs(c118_010 + c119_015 - c120_020 - c121_025 - c122_030 - c123_035 - c124_040 + c125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=3).value = c126_999


#  2   SUBTOTAL - RAMOS UNIDAD 15

In [464]:
d118_010 = sum([ws.cell(row=118, column=col).value if ws.cell(row=118, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=118, column=4).value = d118_010

d119_015 = sum([ws.cell(row=119, column=col).value if ws.cell(row=119, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=119, column=4).value = d119_015

d120_020 = sum([ws.cell(row=120, column=col).value if ws.cell(row=120, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=120, column=4).value = d120_020

d121_025 = sum([ws.cell(row=121, column=col).value if ws.cell(row=121, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=121, column=4).value = d121_025

d122_030 = sum([ws.cell(row=122, column=col).value if ws.cell(row=122, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=122, column=4).value = d122_030

d123_035 = sum([ws.cell(row=123, column=col).value if ws.cell(row=123, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=123, column=4).value = d123_035

d124_040 = sum([ws.cell(row=124, column=col).value if ws.cell(row=124, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=124, column=4).value = d124_040

d125_045 = sum([ws.cell(row=125, column=col).value if ws.cell(row=125, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=125, column=4).value = d125_045

# Calcular el total general según la fórmula dada

d126_999 = abs(d118_010 + d119_015 - d120_020 - d121_025 - d122_030 - d123_035 - d124_040 + d125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=4).value = d126_999


# UNIDAD DE CAPTURA 16

## CONSULTA DE TOTALES UNIDAD 16

In [465]:
c128_999 = c115_999 + c126_999
df_c128_999 = pd.DataFrame({'total': [c128_999]})
if df_c128_999.empty or c128_999 == 0:
    df_c128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=3).value = df_c128_999['total'].iloc[0]

e128_999 = e115_999 + e126_999
df_e128_999 = pd.DataFrame({'total': [e128_999]})
if df_e128_999.empty or e128_999 == 0:
    df_e128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=5).value = df_e128_999['total'].iloc[0]

f128_999 = f115_999 + f126_999
df_f128_999 = pd.DataFrame({'total': [f128_999]})
if df_f128_999.empty or f128_999 == 0:
    df_f128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=6).value = df_f128_999['total'].iloc[0]

g128_999 = g115_999 + g126_999
df_g128_999 = pd.DataFrame({'total': [g128_999]})
if df_g128_999.empty or g128_999 == 0:
    df_g128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=7).value = df_g128_999['total'].iloc[0]

h128_999 = h115_999 + h126_999
df_h128_999 = pd.DataFrame({'total': [h128_999]})
if df_h128_999.empty or h128_999 == 0:
    df_h128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=8).value = df_h128_999['total'].iloc[0]

i128_999 = i115_999 + i126_999
df_i128_999 = pd.DataFrame({'total': [i128_999]})
if df_i128_999.empty or i128_999 == 0:
    df_i128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=9).value = df_i128_999['total'].iloc[0]

j128_999 = j115_999 + j126_999
df_j128_999 = pd.DataFrame({'total': [j128_999]})
if df_j128_999.empty or j128_999 == 0:
    df_j128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=10).value = df_j128_999['total'].iloc[0]

k128_999 = k115_999 + k126_999
df_k128_999 = pd.DataFrame({'total': [k128_999]})
if df_k128_999.empty or k128_999 == 0:
    df_k128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=11).value = df_k128_999['total'].iloc[0]

l128_999 = l115_999 + l126_999
df_l128_999 = pd.DataFrame({'total': [l128_999]})
if df_l128_999.empty or l128_999 == 0:
    df_l128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=12).value = df_l128_999['total'].iloc[0]

m128_999 = m115_999 + m126_999
df_m128_999 = pd.DataFrame({'total': [m128_999]})
if df_m128_999.empty or m128_999 == 0:
    df_m128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=13).value = df_m128_999['total'].iloc[0]

n128_999 = n115_999 + n126_999
df_n128_999 = pd.DataFrame({'total': [n128_999]})
if df_n128_999.empty or n128_999 == 0:
    df_n128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=14).value = df_n128_999['total'].iloc[0]

o128_999 = o115_999 + o126_999
df_o128_999 = pd.DataFrame({'total': [o128_999]})
if df_o128_999.empty or o128_999 == 0:
    df_o128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=15).value = df_o128_999['total'].iloc[0]

p128_999 = p115_999 + p126_999
df_p128_999 = pd.DataFrame({'total': [p128_999]})
if df_p128_999.empty or p128_999 == 0:
    df_p128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=16).value = df_p128_999['total'].iloc[0]

q128_999 = q115_999 + q126_999
df_q128_999 = pd.DataFrame({'total': [q128_999]})
if df_q128_999.empty or q128_999 == 0:
    df_q128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=17).value = df_q128_999['total'].iloc[0]

r128_999 = r115_999 + r126_999
df_r128_999 = pd.DataFrame({'total': [r128_999]})
if df_r128_999.empty or r128_999 == 0:
    df_r128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=18).value = df_r128_999['total'].iloc[0]

s128_999 = s115_999 + s126_999
df_s128_999 = pd.DataFrame({'total': [s128_999]})
if df_s128_999.empty or s128_999 == 0:
    df_s128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=19).value = df_s128_999['total'].iloc[0]

t128_999 = t115_999 + t126_999
df_t128_999 = pd.DataFrame({'total': [t128_999]})
if df_t128_999.empty or t128_999 == 0:
    df_t128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=20).value = df_t128_999['total'].iloc[0]

u128_999 = u115_999 + u126_999
df_u128_999 = pd.DataFrame({'total': [u128_999]})
if df_u128_999.empty or u128_999 == 0:
    df_u128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=21).value = df_u128_999['total'].iloc[0]

v128_999 = v115_999 + v126_999
df_v128_999 = pd.DataFrame({'total': [v128_999]})
if df_v128_999.empty or v128_999 == 0:
    df_v128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=22).value = df_v128_999['total'].iloc[0]

w128_999 = w115_999 + w126_999
df_w128_999 = pd.DataFrame({'total': [w128_999]})
if df_w128_999.empty or w128_999 == 0:
    df_w128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=23).value = df_w128_999['total'].iloc[0]

z128_999 = z115_999 + z126_999
df_z128_999 = pd.DataFrame({'total': [z128_999]})
if df_z128_999.empty or z128_999 == 0:
    df_z128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=26).value = df_z128_999['total'].iloc[0]

aa128_999 = aa115_999 + aa126_999
df_aa128_999 = pd.DataFrame({'total': [aa128_999]})
if df_aa128_999.empty or aa128_999 == 0:
    df_aa128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=27).value = df_aa128_999['total'].iloc[0]

ar128_999 = ar115_999 + ar126_999
df_ar128_999 = pd.DataFrame({'total': [ar128_999]})
if df_ar128_999.empty or ar128_999 == 0:
    df_ar128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=44).value = df_ar128_999['total'].iloc[0]

as128_999 = as115_999 + as126_999
df_as128_999 = pd.DataFrame({'total': [as128_999]})
if df_as128_999.empty or as128_999 == 0:
    df_as128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=45).value = df_as128_999['total'].iloc[0]

at128_999 = at115_999 + at126_999
df_at128_999 = pd.DataFrame({'total': [at128_999]})
if df_at128_999.empty or at128_999 == 0:
    df_at128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=46).value = df_at128_999['total'].iloc[0]

au128_999 = au115_999 + au126_999
df_au128_999 = pd.DataFrame({'total': [au128_999]})
if df_au128_999.empty or au128_999 == 0:
    df_au128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=47).value = df_au128_999['total'].iloc[0]

av128_999 = av115_999 + av126_999
df_av128_999 = pd.DataFrame({'total': [av128_999]})
if df_av128_999.empty or av128_999 == 0:
    df_av128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=48).value = df_av128_999['total'].iloc[0]

aw128_999 = aw115_999 + aw126_999
df_aw128_999 = pd.DataFrame({'total': [aw128_999]})
if df_aw128_999.empty or aw128_999 == 0:
    df_aw128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=49).value = df_aw128_999['total'].iloc[0]

ax128_999 = ax115_999 + ax126_999
df_ax128_999 = pd.DataFrame({'total': [ax128_999]})
if df_ax128_999.empty or ax128_999 == 0:
    df_ax128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=50).value = df_ax128_999['total'].iloc[0]

ay128_999 = ay115_999 + ay126_999
df_ay128_999 = pd.DataFrame({'total': [ay128_999]})
if df_ay128_999.empty or ay128_999 == 0:
    df_ay128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=51).value = df_ay128_999['total'].iloc[0]

az128_999 = az115_999 + az126_999
df_az128_999 = pd.DataFrame({'total': [az128_999]})
if df_az128_999.empty or az128_999 == 0:
    df_az128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=52).value = df_az128_999['total'].iloc[0]

ba128_999 = ba115_999 + ba126_999
df_ba128_999 = pd.DataFrame({'total': [ba128_999]})
if df_ba128_999.empty or ba128_999 == 0:
    df_ba128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=53).value = df_ba128_999['total'].iloc[0]


## CONSULTA DE SUBTOTALES UNIDAD 16

In [466]:
# Realiza la suma para d75_999
d128_999 = d115_999 + d126_999
df_d128_999 = pd.DataFrame({'total': [d128_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d128_999.empty or d128_999 == 0:
    df_d128_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=128, column=4).value = df_d128_999['total'].iloc[0]

## UNIDAD DE CAPTURA 17

#  3   AUTOMOVILES UNIDAD 17

In [469]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_AUTOMOVILES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_AUTOMOVILES['TOTAL'] = unidad17_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",3, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e130_999 = get_value(unidad17_AUTOMOVILES, "Impuesto de renta y complementarios", 3)

# Escribir el total general en Excel
ws.cell(row=130, column=5, value=e130_999)

<Cell 'Hoja1'.E130>

#  4   SOAT UNIDAD 17

In [470]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_SOAT = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_SOAT['TOTAL'] = unidad17_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",4, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f130_999 = get_value(unidad17_SOAT, "Impuesto de renta y complementarios", 4)

# Escribir el total general en Excel
ws.cell(row=130, column=6, value=f130_999)

<Cell 'Hoja1'.F130>

#  5   CUMPLIMIENTO UNIDAD 17

In [471]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTO['TOTAL'] = unidad17_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",5, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0])*-1 if not row.empty else 0


# Obtener los valores individuales de cada concepto
g130_999 = get_value(unidad17_CUMPLIMIENTO, "Impuesto de renta y complementarios", 5)

# Escribir el total general en Excel
ws.cell(row=130, column=7, value=g130_999)

<Cell 'Hoja1'.G130>

#  6   RESPONSABILIDAD CIVIL UNIDAD 17

In [472]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_RESPONSABILIDADCI['TOTAL'] = unidad17_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",6, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
h130_999 = get_value(unidad17_RESPONSABILIDADCI, "Impuesto de renta y complementarios", 6)

# Escribir el total general en Excel
ws.cell(row=130, column=8, value=h130_999)

<Cell 'Hoja1'.H130>

#  7   INCENDIO UNIDAD 17

In [473]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_INCENDIO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_INCENDIO['TOTAL'] = unidad17_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",7, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0])*-1 if not row.empty else 0


# Obtener los valores individuales de cada concepto
i130_999 = get_value(unidad17_INCENDIO, "Impuesto de renta y complementarios", 7)

# Escribir el total general en Excel
ws.cell(row=130, column=9, value=i130_999)

<Cell 'Hoja1'.I130>

#  8   TERREMOTO UNIDAD 17

In [474]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_TERREMOTO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_TERREMOTO['TOTAL'] = unidad17_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",8, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
j130_999 = get_value(unidad17_TERREMOTO, "Impuesto de renta y complementarios", 8)

# Escribir el total general en Excel
ws.cell(row=130, column=10, value=j130_999)

<Cell 'Hoja1'.J130>

#  9   SUSTRACCION UNIDAD 17

In [475]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_SUSTRACCION = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_SUSTRACCION['TOTAL'] = unidad17_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",9, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
k130_999 = get_value(unidad17_SUSTRACCION, "Impuesto de renta y complementarios", 9)

# Escribir el total general en Excel
ws.cell(row=130, column=11, value=k130_999)

<Cell 'Hoja1'.K130>

#  10  TRANSPORTE UNIDAD 17

In [476]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_TRANSPORTE = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_TRANSPORTE['TOTAL'] = unidad17_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",10, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
l130_999 = get_value(unidad17_TRANSPORTE, "Impuesto de renta y complementarios", 10)

# Escribir el total general en Excel
ws.cell(row=130, column=12, value=l130_999)

<Cell 'Hoja1'.L130>

#  11  CORRIENTE DÉBIL UNIDAD 17

In [477]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CORRIENTEDEBIL = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CORRIENTEDEBIL['TOTAL'] = unidad17_CORRIENTEDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",11, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CORRIENTEDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0])*-1 if not row.empty else 0


# Obtener los valores individuales de cada concepto
m130_999 = get_value(unidad17_CORRIENTEDEBIL, "Impuesto de renta y complementarios", 11)

# Escribir el total general en Excel
ws.cell(row=130, column=13, value=m130_999)

<Cell 'Hoja1'.M130>

#  12  TODO RIESGO CONTRATISTA UNIDAD 17

In [478]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_TODORIESGOCONTRATISTA['TOTAL'] = unidad17_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",12, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0])*-1 if not row.empty else 0


# Obtener los valores individuales de cada concepto
n130_999 = get_value(unidad17_TODORIESGOCONTRATISTA, "Impuesto de renta y complementarios", 12)

# Escribir el total general en Excel
ws.cell(row=130, column=14, value=n130_999)

<Cell 'Hoja1'.N130>

#  13  MANEJO UNIDAD 17

In [479]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_MANEJO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_MANEJO['TOTAL'] = unidad17_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",13, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
o130_999 = get_value(unidad17_MANEJO, "Impuesto de renta y complementarios", 13)

# Escribir el total general en Excel
ws.cell(row=130, column=15, value=o130_999)

<Cell 'Hoja1'.O130>

#  14  LUCRO CESANTE UNIDAD 17

In [480]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_LUCROCESANTE = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_LUCROCESANTE['TOTAL'] = unidad17_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",14, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
p130_999 = get_value(unidad17_LUCROCESANTE, "Impuesto de renta y complementarios", 14)

# Escribir el total general en Excel
ws.cell(row=130, column=16, value=p130_999)

<Cell 'Hoja1'.P130>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 17

In [482]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_MONTAJEYROTURA = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_MONTAJEYROTURA['TOTAL'] = unidad17_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",15, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
q130_999 = get_value(unidad17_MONTAJEYROTURA, "Impuesto de renta y complementarios", 15)

# Escribir el total general en Excel
ws.cell(row=130, column=17, value=q130_999)

<Cell 'Hoja1'.Q130>

#  16  AVIACIÓN UNIDAD 17

In [483]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_AVIACION = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_AVIACION['TOTAL'] = unidad17_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",16, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0])*-1 if not row.empty else 0


# Obtener los valores individuales de cada concepto
r130_999 = get_value(unidad17_AVIACION, "Impuesto de renta y complementarios", 16)

# Escribir el total general en Excel
ws.cell(row=130, column=18, value=r130_999)

<Cell 'Hoja1'.R130>

#  17  NAVEGACIÓN Y CASCO UNIDAD 17

In [484]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_NAVEGACION = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_NAVEGACION['TOTAL'] = unidad17_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",17, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
s130_999 = get_value(unidad17_NAVEGACION, "Impuesto de renta y complementarios", 17)

# Escribir el total general en Excel
ws.cell(row=130, column=19, value=s130_999)

<Cell 'Hoja1'.S130>

#  18  MINAS Y PETRÓLEOS UNIDAD 17

In [485]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_MINAS = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_MINAS['TOTAL'] = unidad17_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",18, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
t130_999 = get_value(unidad17_MINAS, "Impuesto de renta y complementarios", 18)

# Escribir el total general en Excel
ws.cell(row=130, column=20, value=t130_999)

<Cell 'Hoja1'.T130>

#  19  VIDRIOS UNIDAD 17

In [486]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_VIDRIOS = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_VIDRIOS['TOTAL'] = unidad17_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",19, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
u130_999 = get_value(unidad17_VIDRIOS, "Impuesto de renta y complementarios", 19)

# Escribir el total general en Excel
ws.cell(row=130, column=21, value=u130_999)

<Cell 'Hoja1'.U130>

#  20  CRÉDITO COMERCIAL UNIDAD 17

In [487]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CREDITOCOMERCIAL['TOTAL'] = unidad17_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",20, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
v130_999 = get_value(unidad17_CREDITOCOMERCIAL, "Impuesto de renta y complementarios", 20)

# Escribir el total general en Excel
ws.cell(row=130, column=22, value=v130_999)

<Cell 'Hoja1'.V130>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 17

In [488]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CREDITOEXPORTACION['TOTAL'] = unidad17_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",21, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
w130_999 = get_value(unidad17_CREDITOEXPORTACION, "Impuesto de renta y complementarios", 21)

# Escribir el total general en Excel
ws.cell(row=130, column=23, value=w130_999)

<Cell 'Hoja1'.W130>

#  24  DESEMPLEO UNIDAD 17

In [489]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_DESEMPLEO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_DESEMPLEO['TOTAL'] = unidad17_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",24, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
z130_999 = get_value(unidad17_DESEMPLEO, "Impuesto de renta y complementarios", 24)

# Escribir el total general en Excel
ws.cell(row=130, column=26, value=z130_999)

<Cell 'Hoja1'.Z130>

#  25  HOGAR UNIDAD 17

In [490]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_HOGAR = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_HOGAR['TOTAL'] = unidad17_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",25, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aa130_999 = get_value(unidad17_HOGAR, "Impuesto de renta y complementarios", 25)

# Escribir el total general en Excel
ws.cell(row=130, column=27, value=aa130_999)

<Cell 'Hoja1'.AA130>

# 42 AGROPECUARIO UNIDAD 17

In [491]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_AGROPECUARIO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_AGROPECUARIO['TOTAL'] = unidad17_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",42, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ar130_999 = get_value(unidad17_AGROPECUARIO, "Impuesto de renta y complementarios", 42)

# Escribir el total general en Excel
ws.cell(row=130, column=44, value=ar130_999)

<Cell 'Hoja1'.AR130>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 17

In [492]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_OPERNOASIGNABLES['TOTAL'] = unidad17_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",43, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as130_999 = get_value(unidad17_OPERNOASIGNABLES, "Impuesto de renta y complementarios", 43)

# Escribir el total general en Excel
ws.cell(row=130, column=45, value=as130_999)

<Cell 'Hoja1'.AS130>

# 44 DECENAL UNIDAD 17

In [493]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_DECENAL = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_DECENAL['TOTAL'] = unidad17_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",44, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
at130_999 = get_value(unidad17_DECENAL, "Impuesto de renta y complementarios", 44)

# Escribir el total general en Excel
ws.cell(row=130, column=46, value=at130_999)

<Cell 'Hoja1'.AT130>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 17

In [494]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOESTATALES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 45)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOESTATALES['TOTAL'] = unidad17_CUMPLIMIENTOESTATALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",45, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOESTATALES, concept, ramo, 47, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
au130_999 = get_value(unidad17_CUMPLIMIENTOESTATALES, "Impuesto de renta y complementarios", 45)

# Escribir el total general en Excel
ws.cell(row=130, column=47, value=au130_999)

<Cell 'Hoja1'.AU130>

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 17

In [495]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOEMPRESASSERV = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 46)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOEMPRESASSERV['TOTAL'] = unidad17_CUMPLIMIENTOEMPRESASSERV['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",46, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOEMPRESASSERV, concept, ramo, 48, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
av130_999 = get_value(unidad17_CUMPLIMIENTOEMPRESASSERV, "Impuesto de renta y complementarios", 46)

# Escribir el total general en Excel
ws.cell(row=130, column=48, value=av130_999)

<Cell 'Hoja1'.AV130>

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 17

In [496]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOEMPRESASINDUSTRI = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 47)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOEMPRESASINDUSTRI['TOTAL'] = unidad17_CUMPLIMIENTOEMPRESASINDUSTRI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",47, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOEMPRESASINDUSTRI, concept, ramo, 49, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aw130_999 = get_value(unidad17_CUMPLIMIENTOEMPRESASINDUSTRI, "Impuesto de renta y complementarios", 47)

# Escribir el total general en Excel
ws.cell(row=130, column=49, value=aw130_999)

<Cell 'Hoja1'.AW130>

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 17

In [497]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOEMPRESASDISPOLEG = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 48)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOEMPRESASDISPOLEG['TOTAL'] = unidad17_CUMPLIMIENTOEMPRESASDISPOLEG['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",48, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOEMPRESASDISPOLEG, concept, ramo, 50, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ax130_999 = get_value(unidad17_CUMPLIMIENTOEMPRESASDISPOLEG, "Impuesto de renta y complementarios", 48)

# Escribir el total general en Excel
ws.cell(row=130, column=50, value=ax130_999)

<Cell 'Hoja1'.AX130>

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 17

In [498]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOCAUCIONES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 49)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOCAUCIONES['TOTAL'] = unidad17_CUMPLIMIENTOCAUCIONES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",49, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOCAUCIONES, concept, ramo, 51, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ay130_999 = get_value(unidad17_CUMPLIMIENTOCAUCIONES, "Impuesto de renta y complementarios", 49)

# Escribir el total general en Excel
ws.cell(row=130, column=51, value=ay130_999)

<Cell 'Hoja1'.AY130>

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 17

In [499]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOARRENDAMIENTO = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 50)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOARRENDAMIENTO['TOTAL'] = unidad17_CUMPLIMIENTOARRENDAMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",50, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOARRENDAMIENTO, concept, ramo, 52, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
az130_999 = get_value(unidad17_CUMPLIMIENTOARRENDAMIENTO, "Impuesto de renta y complementarios", 50)

# Escribir el total general en Excel
ws.cell(row=130, column=52, value=az130_999)

<Cell 'Hoja1'.AZ130>

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 17

In [501]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_CUMPLIMIENTOPARTICULARES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 51)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_CUMPLIMIENTOPARTICULARES['TOTAL'] = unidad17_CUMPLIMIENTOPARTICULARES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",51, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_CUMPLIMIENTOPARTICULARES, concept, ramo, 53, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ba130_999 = get_value(unidad17_CUMPLIMIENTOPARTICULARES, "Impuesto de renta y complementarios", 51)

# Escribir el total general en Excel
ws.cell(row=130, column=53, value=ba130_999)

<Cell 'Hoja1'.BA130>

#  1   TOTAL UNIDAD 17

In [502]:
c130_999 = sum([ws.cell(row=130, column=col).value if ws.cell(row=130, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=130, column=3).value = c130_999

#  2   SUBTOTAL - RAMOS UNIDAD 17

In [503]:
d130_999 = sum([ws.cell(row=130, column=col).value if ws.cell(row=130, column=col).value is not None else 0 for col in range(5, 45)])
ws.cell(row=130, column=4).value = d130_999

## UNIDAD DE CAPTURA 18

#  CONSULTA DE TOTALES UNIDAD 18

In [504]:
c132_999 = c128_999 - c130_999
df_c132_999 = pd.DataFrame({'total': [c132_999]})
if df_c132_999.empty or c132_999 == 0:
    df_c132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=3).value = df_c132_999['total'].iloc[0]

e132_999 = e128_999 - e130_999
df_e132_999 = pd.DataFrame({'total': [e132_999]})
if df_e132_999.empty or e132_999 == 0:
    df_e132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=5).value = df_e132_999['total'].iloc[0]

f132_999 = f128_999 - f130_999
df_f132_999 = pd.DataFrame({'total': [f132_999]})
if df_f132_999.empty or f132_999 == 0:
    df_f132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=6).value = df_f132_999['total'].iloc[0]

g132_999 = g128_999 - g130_999
df_g132_999 = pd.DataFrame({'total': [g132_999]})
if df_g132_999.empty or g132_999 == 0:
    df_g132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=7).value = df_g132_999['total'].iloc[0]

h132_999 = h128_999 - h130_999
df_h132_999 = pd.DataFrame({'total': [h132_999]})
if df_h132_999.empty or h132_999 == 0:
    df_h132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=8).value = df_h132_999['total'].iloc[0]

i132_999 = i128_999 - i130_999
df_i132_999 = pd.DataFrame({'total': [i132_999]})
if df_i132_999.empty or i132_999 == 0:
    df_i132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=9).value = df_i132_999['total'].iloc[0]

j132_999 = j128_999 - j130_999
df_j132_999 = pd.DataFrame({'total': [j132_999]})
if df_j132_999.empty or j132_999 == 0:
    df_j132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=10).value = df_j132_999['total'].iloc[0]

k132_999 = k128_999 - k130_999
df_k132_999 = pd.DataFrame({'total': [k132_999]})
if df_k132_999.empty or k132_999 == 0:
    df_k132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=11).value = df_k132_999['total'].iloc[0]

l132_999 = l128_999 - l130_999
df_l132_999 = pd.DataFrame({'total': [l132_999]})
if df_l132_999.empty or l132_999 == 0:
    df_l132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=12).value = df_l132_999['total'].iloc[0]

m132_999 = m128_999 - m130_999
df_m132_999 = pd.DataFrame({'total': [m132_999]})
if df_m132_999.empty or m132_999 == 0:
    df_m132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=13).value = df_m132_999['total'].iloc[0]

n132_999 = n128_999 - n130_999
df_n132_999 = pd.DataFrame({'total': [n132_999]})
if df_n132_999.empty or n132_999 == 0:
    df_n132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=14).value = df_n132_999['total'].iloc[0]

o132_999 = o128_999 - o130_999
df_o132_999 = pd.DataFrame({'total': [o132_999]})
if df_o132_999.empty or o132_999 == 0:
    df_o132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=15).value = df_o132_999['total'].iloc[0]

p132_999 = p128_999 - p130_999
df_p132_999 = pd.DataFrame({'total': [p132_999]})
if df_p132_999.empty or p132_999 == 0:
    df_p132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=16).value = df_p132_999['total'].iloc[0]

q132_999 = q128_999 - q130_999
df_q132_999 = pd.DataFrame({'total': [q132_999]})
if df_q132_999.empty or q132_999 == 0:
    df_q132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=17).value = df_q132_999['total'].iloc[0]

r132_999 = r128_999 - r130_999
df_r132_999 = pd.DataFrame({'total': [r132_999]})
if df_r132_999.empty or r132_999 == 0:
    df_r132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=18).value = df_r132_999['total'].iloc[0]

s132_999 = s128_999 - s130_999
df_s132_999 = pd.DataFrame({'total': [s132_999]})
if df_s132_999.empty or s132_999 == 0:
    df_s132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=19).value = df_s132_999['total'].iloc[0]

t132_999 = t128_999 - t130_999
df_t132_999 = pd.DataFrame({'total': [t132_999]})
if df_t132_999.empty or t132_999 == 0:
    df_t132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=20).value = df_t132_999['total'].iloc[0]

u132_999 = u128_999 - u130_999
df_u132_999 = pd.DataFrame({'total': [u132_999]})
if df_u132_999.empty or u132_999 == 0:
    df_u132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=21).value = df_u132_999['total'].iloc[0]

v132_999 = v128_999 - v130_999
df_v132_999 = pd.DataFrame({'total': [v132_999]})
if df_v132_999.empty or v132_999 == 0:
    df_v132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=22).value = df_v132_999['total'].iloc[0]

w132_999 = w128_999 - w130_999
df_w132_999 = pd.DataFrame({'total': [w132_999]})
if df_w132_999.empty or w132_999 == 0:
    df_w132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=23).value = df_w132_999['total'].iloc[0]

z132_999 = z128_999 - z130_999
df_z132_999 = pd.DataFrame({'total': [z132_999]})
if df_z132_999.empty or z132_999 == 0:
    df_z132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=26).value = df_z132_999['total'].iloc[0]

aa132_999 = aa128_999 - aa130_999
df_aa132_999 = pd.DataFrame({'total': [aa132_999]})
if df_aa132_999.empty or aa132_999 == 0:
    df_aa132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=27).value = df_aa132_999['total'].iloc[0]

ar132_999 = ar128_999 - ar130_999
df_ar132_999 = pd.DataFrame({'total': [ar132_999]})
if df_ar132_999.empty or ar132_999 == 0:
    df_ar132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=44).value = df_ar132_999['total'].iloc[0]

as132_999 = as128_999 - as130_999
df_as132_999 = pd.DataFrame({'total': [as132_999]})
if df_as132_999.empty or as132_999 == 0:
    df_as132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=45).value = df_as132_999['total'].iloc[0]

at132_999 = at128_999 - at130_999
df_at132_999 = pd.DataFrame({'total': [at132_999]})
if df_at132_999.empty or at132_999 == 0:
    df_at132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=46).value = df_at132_999['total'].iloc[0]

au132_999 = au128_999 - au130_999
df_au132_999 = pd.DataFrame({'total': [au132_999]})
if df_au132_999.empty or au132_999 == 0:
    df_au132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=47).value = df_au132_999['total'].iloc[0]

av132_999 = av128_999 - av130_999
df_av132_999 = pd.DataFrame({'total': [av132_999]})
if df_av132_999.empty or av132_999 == 0:
    df_av132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=48).value = df_av132_999['total'].iloc[0]

aw132_999 = aw128_999 - aw130_999
df_aw132_999 = pd.DataFrame({'total': [aw132_999]})
if df_aw132_999.empty or aw132_999 == 0:
    df_aw132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=49).value = df_aw132_999['total'].iloc[0]

ax132_999 = ax128_999 - ax130_999
df_ax132_999 = pd.DataFrame({'total': [ax132_999]})
if df_ax132_999.empty or ax132_999 == 0:
    df_ax132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=50).value = df_ax132_999['total'].iloc[0]

ay132_999 = ay128_999 - ay130_999
df_ay132_999 = pd.DataFrame({'total': [ay132_999]})
if df_ay132_999.empty or ay132_999 == 0:
    df_ay132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=51).value = df_ay132_999['total'].iloc[0]

az132_999 = az128_999 - az130_999
df_az132_999 = pd.DataFrame({'total': [az132_999]})
if df_az132_999.empty or az132_999 == 0:
    df_az132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=52).value = df_az132_999['total'].iloc[0]

ba132_999 = ba128_999 - ba130_999
df_ba132_999 = pd.DataFrame({'total': [ba132_999]})
if df_ba132_999.empty or ba132_999 == 0:
    df_ba132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=53).value = df_ba132_999['total'].iloc[0]


# CONSULTA DE SUBTOTALES - RAMOS UNIDAD 18

In [505]:
# Realiza la suma para d75_999
d132_999 = d128_999 - d130_999
df_d132_999 = pd.DataFrame({'total': [d132_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d132_999.empty or d132_999 == 0:
    df_d132_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=132, column=4).value = df_d132_999['total'].iloc[0]

## UNIDAD DE CAPTURA 19

# 3   AUTOMOVILES UNIDAD 19

In [506]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_AUTOMOVILES = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_AUTOMOVILES['TOTAL'] = unidad19_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",3, 134),
    ("Aceptados en reaseguro",3, 135),
    ("Parte reaseguro al interior (resta)",3, 136),
    ("Parte reaseguro al exterior (resta)",3, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e134_005 = get_value(unidad19_AUTOMOVILES, "Directos", 3)
e135_010 = get_value(unidad19_AUTOMOVILES, "Aceptados en reaseguro", 3)
e136_015 = get_value(unidad19_AUTOMOVILES, "Parte reaseguro al interior (resta)", 3)
e137_020 = get_value(unidad19_AUTOMOVILES, "Parte reaseguro al exterior (resta)", 3)


# Calcular ab94_999 usando la fórmula especificada
e138_999 = (e134_005 + e135_010 - e136_015 - e137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=5, value=e138_999)

<Cell 'Hoja1'.E138>

#  4   SOAT UNIDAD 19

In [507]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_SOAT = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_SOAT['TOTAL'] = unidad19_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",4, 134),
    ("Aceptados en reaseguro",4, 135),
    ("Parte reaseguro al interior (resta)",4, 136),
    ("Parte reaseguro al exterior (resta)",4, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f134_005 = get_value(unidad19_SOAT, "Directos", 4)
f135_010 = get_value(unidad19_SOAT, "Aceptados en reaseguro", 4)
f136_015 = get_value(unidad19_SOAT, "Parte reaseguro al interior (resta)", 4)
f137_020 = get_value(unidad19_SOAT, "Parte reaseguro al exterior (resta)", 4)


# Calcular ab94_999 usando la fórmula especificada
f138_999 = (f134_005 + f135_010 - f136_015 - f137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=6, value=f138_999)

<Cell 'Hoja1'.F138>

#  5   CUMPLIMIENTO UNIDAD 19

In [509]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_CUMPLIMIENTO['TOTAL'] = unidad19_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",5, 134),
    ("Aceptados en reaseguro",5, 135),
    ("Parte reaseguro al interior (resta)",5, 136),
    ("Parte reaseguro al exterior (resta)",5, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
g134_005 = get_value(unidad19_CUMPLIMIENTO, "Directos", 5)
g135_010 = get_value(unidad19_CUMPLIMIENTO, "Aceptados en reaseguro", 5)
g136_015 = get_value(unidad19_CUMPLIMIENTO, "Parte reaseguro al interior (resta)", 5)
g137_020 = get_value(unidad19_CUMPLIMIENTO, "Parte reaseguro al exterior (resta)", 5)


# Calcular ab94_999 usando la fórmula especificada
g138_999 = (g134_005 + g135_010 - g136_015 - g137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=7, value=g138_999)

<Cell 'Hoja1'.G138>

#  6   RESPONSABILIDAD CIVIL UNIDAD 19

In [510]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_RESPONSABILIDADCI['TOTAL'] = unidad19_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",6, 134),
    ("Aceptados en reaseguro",6, 135),
    ("Parte reaseguro al interior (resta)",6, 136),
    ("Parte reaseguro al exterior (resta)",6, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
h134_005 = get_value(unidad19_RESPONSABILIDADCI, "Directos", 6)
h135_010 = get_value(unidad19_RESPONSABILIDADCI, "Aceptados en reaseguro", 6)
h136_015 = get_value(unidad19_RESPONSABILIDADCI, "Parte reaseguro al interior (resta)", 6)
h137_020 = get_value(unidad19_RESPONSABILIDADCI, "Parte reaseguro al exterior (resta)", 6)


# Calcular ab94_999 usando la fórmula especificada
h138_999 = (h134_005 + h135_010 - h136_015 - h137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=8, value=h138_999)

<Cell 'Hoja1'.H138>

#  7   INCENDIO UNIDAD 19

In [511]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_INCENDIO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_INCENDIO['TOTAL'] = unidad19_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",7, 134),
    ("Aceptados en reaseguro",7, 135),
    ("Parte reaseguro al interior (resta)",7, 136),
    ("Parte reaseguro al exterior (resta)",7, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
i134_005 = get_value(unidad19_INCENDIO, "Directos", 7)
i135_010 = get_value(unidad19_INCENDIO, "Aceptados en reaseguro", 7)
i136_015 = get_value(unidad19_INCENDIO, "Parte reaseguro al interior (resta)", 7)
i137_020 = get_value(unidad19_INCENDIO, "Parte reaseguro al exterior (resta)", 7)


# Calcular ab94_999 usando la fórmula especificada
i138_999 = (i134_005 + i135_010 - i136_015 - i137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=9, value=i138_999)

<Cell 'Hoja1'.I138>

#  8   TERREMOTO UNIDAD 19

In [512]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_TERREMOTO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 8)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_TERREMOTO['TOTAL'] = unidad19_TERREMOTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",8, 134),
    ("Aceptados en reaseguro",8, 135),
    ("Parte reaseguro al interior (resta)",8, 136),
    ("Parte reaseguro al exterior (resta)",8, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_TERREMOTO, concept, ramo, 10, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
j134_005 = get_value(unidad19_TERREMOTO, "Directos", 8)
j135_010 = get_value(unidad19_TERREMOTO, "Aceptados en reaseguro", 8)
j136_015 = get_value(unidad19_TERREMOTO, "Parte reaseguro al interior (resta)", 8)
j137_020 = get_value(unidad19_TERREMOTO, "Parte reaseguro al exterior (resta)", 8)


# Calcular ab94_999 usando la fórmula especificada
j138_999 = (j134_005 + j135_010 - j136_015 - j137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=10, value=j138_999)

<Cell 'Hoja1'.J138>

#  9   SUSTRACCION UNIDAD 19

In [513]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_SUSTRACCION = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_SUSTRACCION['TOTAL'] = unidad19_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",9, 134),
    ("Aceptados en reaseguro",9, 135),
    ("Parte reaseguro al interior (resta)",9, 136),
    ("Parte reaseguro al exterior (resta)",9, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
k134_005 = get_value(unidad19_SUSTRACCION, "Directos", 9)
k135_010 = get_value(unidad19_SUSTRACCION, "Aceptados en reaseguro", 9)
k136_015 = get_value(unidad19_SUSTRACCION, "Parte reaseguro al interior (resta)", 9)
k137_020 = get_value(unidad19_SUSTRACCION, "Parte reaseguro al exterior (resta)", 9)


# Calcular ab94_999 usando la fórmula especificada
k138_999 = (k134_005 + k135_010 - k136_015 - k137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=11, value=k138_999)

<Cell 'Hoja1'.K138>

#  10  TRANSPORTE UNIDAD 19

In [514]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_TRANSPORTE = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_TRANSPORTE['TOTAL'] = unidad19_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",10, 134),
    ("Aceptados en reaseguro",10, 135),
    ("Parte reaseguro al interior (resta)",10, 136),
    ("Parte reaseguro al exterior (resta)",10, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
l134_005 = get_value(unidad19_TRANSPORTE, "Directos", 10)
l135_010 = get_value(unidad19_TRANSPORTE, "Aceptados en reaseguro", 10)
l136_015 = get_value(unidad19_TRANSPORTE, "Parte reaseguro al interior (resta)", 10)
l137_020 = get_value(unidad19_TRANSPORTE, "Parte reaseguro al exterior (resta)", 10)


# Calcular ab94_999 usando la fórmula especificada
l138_999 = (l134_005 + l135_010 - l136_015 - l137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=12, value=l138_999)

<Cell 'Hoja1'.L138>

#  11  CORRIENTE DÉBIL UNIDAD 19

In [515]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_CORRIENTEDEBIL = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_CORRIENTEDEBIL['TOTAL'] = unidad19_CORRIENTEDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",11, 134),
    ("Aceptados en reaseguro",11, 135),
    ("Parte reaseguro al interior (resta)",11, 136),
    ("Parte reaseguro al exterior (resta)",11, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_CORRIENTEDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
m134_005 = get_value(unidad19_CORRIENTEDEBIL, "Directos", 11)
m135_010 = get_value(unidad19_CORRIENTEDEBIL, "Aceptados en reaseguro", 11)
m136_015 = get_value(unidad19_CORRIENTEDEBIL, "Parte reaseguro al interior (resta)", 11)
m137_020 = get_value(unidad19_CORRIENTEDEBIL, "Parte reaseguro al exterior (resta)", 11)


# Calcular ab94_999 usando la fórmula especificada
m138_999 = (m134_005 + m135_010 - m136_015 - m137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=13, value=m138_999)

<Cell 'Hoja1'.M138>

#  12  TODO RIESGO CONTRATISTA UNIDAD 19

In [516]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_TODORIESGOCONTRATISTA['TOTAL'] = unidad19_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",12, 134),
    ("Aceptados en reaseguro",12, 135),
    ("Parte reaseguro al interior (resta)",12, 136),
    ("Parte reaseguro al exterior (resta)",12, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
n134_005 = get_value(unidad19_TODORIESGOCONTRATISTA, "Directos", 12)
n135_010 = get_value(unidad19_TODORIESGOCONTRATISTA, "Aceptados en reaseguro", 12)
n136_015 = get_value(unidad19_TODORIESGOCONTRATISTA, "Parte reaseguro al interior (resta)", 12)
n137_020 = get_value(unidad19_TODORIESGOCONTRATISTA, "Parte reaseguro al exterior (resta)", 12)


# Calcular ab94_999 usando la fórmula especificada
n138_999 = (n134_005 + n135_010 - n136_015 - n137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=14, value=n138_999)

<Cell 'Hoja1'.N138>

#  13  MANEJO UNIDAD 19

In [517]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_MANEJO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_MANEJO['TOTAL'] = unidad19_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",13, 134),
    ("Aceptados en reaseguro",13, 135),
    ("Parte reaseguro al interior (resta)",13, 136),
    ("Parte reaseguro al exterior (resta)",13, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
o134_005 = get_value(unidad19_MANEJO, "Directos", 13)
o135_010 = get_value(unidad19_MANEJO, "Aceptados en reaseguro", 13)
o136_015 = get_value(unidad19_MANEJO, "Parte reaseguro al interior (resta)", 13)
o137_020 = get_value(unidad19_MANEJO, "Parte reaseguro al exterior (resta)", 13)


# Calcular ab94_999 usando la fórmula especificada
o138_999 = (o134_005 + o135_010 - o136_015 - o137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=15, value=o138_999)

<Cell 'Hoja1'.O138>

#  14  LUCRO CESANTE UNIDAD 19

In [518]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_LUCROCESANTE = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_LUCROCESANTE['TOTAL'] = unidad19_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",14, 134),
    ("Aceptados en reaseguro",14, 135),
    ("Parte reaseguro al interior (resta)",14, 136),
    ("Parte reaseguro al exterior (resta)",14, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
p134_005 = get_value(unidad19_LUCROCESANTE, "Directos", 14)
p135_010 = get_value(unidad19_LUCROCESANTE, "Aceptados en reaseguro", 14)
p136_015 = get_value(unidad19_LUCROCESANTE, "Parte reaseguro al interior (resta)", 14)
p137_020 = get_value(unidad19_LUCROCESANTE, "Parte reaseguro al exterior (resta)", 14)


# Calcular ab94_999 usando la fórmula especificada
p138_999 = (p134_005 + p135_010 - p136_015 - p137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=16, value=p138_999)

<Cell 'Hoja1'.P138>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 19

In [519]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_MONTAJEYROTURA = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_MONTAJEYROTURA['TOTAL'] = unidad19_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",15, 134),
    ("Aceptados en reaseguro",15, 135),
    ("Parte reaseguro al interior (resta)",15, 136),
    ("Parte reaseguro al exterior (resta)",15, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
q134_005 = get_value(unidad19_MONTAJEYROTURA, "Directos", 15)
q135_010 = get_value(unidad19_MONTAJEYROTURA, "Aceptados en reaseguro", 15)
q136_015 = get_value(unidad19_MONTAJEYROTURA, "Parte reaseguro al interior (resta)", 15)
q137_020 = get_value(unidad19_MONTAJEYROTURA, "Parte reaseguro al exterior (resta)", 15)


# Calcular ab94_999 usando la fórmula especificada
q138_999 = (q134_005 + q135_010 - q136_015 - q137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=17, value=q138_999)

<Cell 'Hoja1'.Q138>

#  16  AVIACIÓN UNIDAD 19

In [520]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_AVIACION = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_AVIACION['TOTAL'] = unidad19_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",16, 134),
    ("Aceptados en reaseguro",16, 135),
    ("Parte reaseguro al interior (resta)",16, 136),
    ("Parte reaseguro al exterior (resta)",16, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
r134_005 = get_value(unidad19_AVIACION, "Directos", 16)
r135_010 = get_value(unidad19_AVIACION, "Aceptados en reaseguro", 16)
r136_015 = get_value(unidad19_AVIACION, "Parte reaseguro al interior (resta)", 16)
r137_020 = get_value(unidad19_AVIACION, "Parte reaseguro al exterior (resta)", 16)


# Calcular ab94_999 usando la fórmula especificada
r138_999 = (r134_005 + r135_010 - r136_015 - r137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=18, value=r138_999)

<Cell 'Hoja1'.R138>

#  17  NAVEGACIÓN Y CASCO UNIDAD 19

In [521]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_NAVEGACION = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_NAVEGACION['TOTAL'] = unidad19_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",17, 134),
    ("Aceptados en reaseguro",17, 135),
    ("Parte reaseguro al interior (resta)",17, 136),
    ("Parte reaseguro al exterior (resta)",17, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
s134_005 = get_value(unidad19_NAVEGACION, "Directos", 17)
s135_010 = get_value(unidad19_NAVEGACION, "Aceptados en reaseguro", 17)
s136_015 = get_value(unidad19_NAVEGACION, "Parte reaseguro al interior (resta)", 17)
s137_020 = get_value(unidad19_NAVEGACION, "Parte reaseguro al exterior (resta)", 17)


# Calcular ab94_999 usando la fórmula especificada
s138_999 = (s134_005 + s135_010 - s136_015 - s137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=19, value=s138_999)

<Cell 'Hoja1'.S138>

#  18  MINAS Y PETRÓLEOS UNIDAD 19

In [522]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_MINAS = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_MINAS['TOTAL'] = unidad19_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",18, 134),
    ("Aceptados en reaseguro",18, 135),
    ("Parte reaseguro al interior (resta)",18, 136),
    ("Parte reaseguro al exterior (resta)",18, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
t134_005 = get_value(unidad19_MINAS, "Directos", 18)
t135_010 = get_value(unidad19_MINAS, "Aceptados en reaseguro", 18)
t136_015 = get_value(unidad19_MINAS, "Parte reaseguro al interior (resta)", 18)
t137_020 = get_value(unidad19_MINAS, "Parte reaseguro al exterior (resta)", 18)


# Calcular ab94_999 usando la fórmula especificada
t138_999 = (t134_005 + t135_010 - t136_015 - t137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=20, value=t138_999)

<Cell 'Hoja1'.T138>

#  19  VIDRIOS UNIDAD 19

In [523]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_VIDRIOS = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_VIDRIOS['TOTAL'] = unidad19_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",19, 134),
    ("Aceptados en reaseguro",19, 135),
    ("Parte reaseguro al interior (resta)",19, 136),
    ("Parte reaseguro al exterior (resta)",19, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
u134_005 = get_value(unidad19_VIDRIOS, "Directos", 19)
u135_010 = get_value(unidad19_VIDRIOS, "Aceptados en reaseguro", 19)
u136_015 = get_value(unidad19_VIDRIOS, "Parte reaseguro al interior (resta)", 19)
u137_020 = get_value(unidad19_VIDRIOS, "Parte reaseguro al exterior (resta)", 19)


# Calcular ab94_999 usando la fórmula especificada
u138_999 = (u134_005 + u135_010 - u136_015 - u137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=21, value=u138_999)

<Cell 'Hoja1'.U138>

#  20  CRÉDITO COMERCIAL UNIDAD 19

In [524]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_CREDITOCOMERCIAL['TOTAL'] = unidad19_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",20, 134),
    ("Aceptados en reaseguro",20, 135),
    ("Parte reaseguro al interior (resta)",20, 136),
    ("Parte reaseguro al exterior (resta)",20, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
v134_005 = get_value(unidad19_CREDITOCOMERCIAL, "Directos", 20)
v135_010 = get_value(unidad19_CREDITOCOMERCIAL, "Aceptados en reaseguro", 20)
v136_015 = get_value(unidad19_CREDITOCOMERCIAL, "Parte reaseguro al interior (resta)", 20)
v137_020 = get_value(unidad19_CREDITOCOMERCIAL, "Parte reaseguro al exterior (resta)", 20)


# Calcular ab94_999 usando la fórmula especificada
v138_999 = (v134_005 + v135_010 - v136_015 - v137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=22, value=v138_999)

<Cell 'Hoja1'.V138>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 19

In [525]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_CREDITOEXPORTACION['TOTAL'] = unidad19_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",21, 134),
    ("Aceptados en reaseguro",21, 135),
    ("Parte reaseguro al interior (resta)",21, 136),
    ("Parte reaseguro al exterior (resta)",21, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
w134_005 = get_value(unidad19_CREDITOEXPORTACION, "Directos", 21)
w135_010 = get_value(unidad19_CREDITOEXPORTACION, "Aceptados en reaseguro", 21)
w136_015 = get_value(unidad19_CREDITOEXPORTACION, "Parte reaseguro al interior (resta)", 21)
w137_020 = get_value(unidad19_CREDITOEXPORTACION, "Parte reaseguro al exterior (resta)", 21)


# Calcular ab94_999 usando la fórmula especificada
w138_999 = (w134_005 + w135_010 - w136_015 - w137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=23, value=w138_999)

<Cell 'Hoja1'.W138>

#  24  DESEMPLEO UNIDAD 19

In [526]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_DESEMPLEO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_DESEMPLEO['TOTAL'] = unidad19_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",24, 134),
    ("Aceptados en reaseguro",24, 135),
    ("Parte reaseguro al interior (resta)",24, 136),
    ("Parte reaseguro al exterior (resta)",24, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
z134_005 = get_value(unidad19_DESEMPLEO, "Directos", 24)
z135_010 = get_value(unidad19_DESEMPLEO, "Aceptados en reaseguro", 24)
z136_015 = get_value(unidad19_DESEMPLEO, "Parte reaseguro al interior (resta)", 24)
z137_020 = get_value(unidad19_DESEMPLEO, "Parte reaseguro al exterior (resta)", 24)


# Calcular ab94_999 usando la fórmula especificada
z138_999 = (z134_005 + z135_010 - z136_015 - z137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=26, value=z138_999)

<Cell 'Hoja1'.Z138>

#  25  HOGAR UNIDAD 19

In [527]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_HOGAR = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_HOGAR['TOTAL'] = unidad19_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",25, 134),
    ("Aceptados en reaseguro",25, 135),
    ("Parte reaseguro al interior (resta)",25, 136),
    ("Parte reaseguro al exterior (resta)",25, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aa134_005 = get_value(unidad19_HOGAR, "Directos", 25)
aa135_010 = get_value(unidad19_HOGAR, "Aceptados en reaseguro", 25)
aa136_015 = get_value(unidad19_HOGAR, "Parte reaseguro al interior (resta)", 25)
aa137_020 = get_value(unidad19_HOGAR, "Parte reaseguro al exterior (resta)", 25)


# Calcular ab94_999 usando la fórmula especificada
aa138_999 = (aa134_005 + aa135_010 - aa136_015 - aa137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=27, value=aa138_999)

<Cell 'Hoja1'.AA138>

# 42 AGROPECUARIO UNIDAD 19

In [528]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_AGROPECUARIO = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_AGROPECUARIO['TOTAL'] = unidad19_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",42, 134),
    ("Aceptados en reaseguro",42, 135),
    ("Parte reaseguro al interior (resta)",42, 136),
    ("Parte reaseguro al exterior (resta)",42, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ar134_005 = get_value(unidad19_AGROPECUARIO, "Directos", 42)
ar135_010 = get_value(unidad19_AGROPECUARIO, "Aceptados en reaseguro", 42)
ar136_015 = get_value(unidad19_AGROPECUARIO, "Parte reaseguro al interior (resta)", 42)
ar137_020 = get_value(unidad19_AGROPECUARIO, "Parte reaseguro al exterior (resta)", 42)


# Calcular ab94_999 usando la fórmula especificada
ar138_999 = (ar134_005 + ar135_010 - ar136_015 - ar137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=44, value=ar138_999)

<Cell 'Hoja1'.AR138>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 19

In [529]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_OPERNOASIGNABLES['TOTAL'] = unidad19_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",43, 134),
    ("Aceptados en reaseguro",43, 135),
    ("Parte reaseguro al interior (resta)",43, 136),
    ("Parte reaseguro al exterior (resta)",43, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as134_005 = get_value(unidad19_OPERNOASIGNABLES, "Directos", 43)
as135_010 = get_value(unidad19_OPERNOASIGNABLES, "Aceptados en reaseguro", 43)
as136_015 = get_value(unidad19_OPERNOASIGNABLES, "Parte reaseguro al interior (resta)", 43)
as137_020 = get_value(unidad19_OPERNOASIGNABLES, "Parte reaseguro al exterior (resta)", 43)


# Calcular ab94_999 usando la fórmula especificada
as138_999 = (as134_005 + as135_010 - as136_015 - as137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=45, value=as138_999)

<Cell 'Hoja1'.AS138>

# 44 DECENAL UNIDAD 19


In [530]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad19_DECENAL = bd[(bd['UNIDAD'] == 19) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_DECENAL['TOTAL'] = unidad19_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos",44, 134),
    ("Aceptados en reaseguro",44, 135),
    ("Parte reaseguro al interior (resta)",44, 136),
    ("Parte reaseguro al exterior (resta)",44, 137),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad19_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
at134_005 = get_value(unidad19_DECENAL, "Directos", 44)
at135_010 = get_value(unidad19_DECENAL, "Aceptados en reaseguro", 44)
at136_015 = get_value(unidad19_DECENAL, "Parte reaseguro al interior (resta)", 44)
at137_020 = get_value(unidad19_DECENAL, "Parte reaseguro al exterior (resta)", 44)


# Calcular ab94_999 usando la fórmula especificada
at138_999 = (at134_005 + at135_010 - at136_015 - at137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=46, value=at138_999)

<Cell 'Hoja1'.AT138>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 19

In [531]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


au134_005 = unidad19_CUMPLIMIENTOESTATALES_a[(unidad19_CUMPLIMIENTOESTATALES_a['CUENTA'] == "9620750021") & (unidad19_CUMPLIMIENTOESTATALES_a['COD_RAMO'] == 45)]
write_to_excel(au134_005, 47, 134, 'TOTAL')

au135_010 = unidad19_CUMPLIMIENTOESTATALES[(unidad19_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au135_010, 47, 135)

au136_015 = unidad19_CUMPLIMIENTOESTATALES[(unidad19_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au136_015, 47, 136)

au137_020 = unidad19_CUMPLIMIENTOESTATALES[(unidad19_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]
write_to_excel(au137_020, 47, 137)


au138_999 = au134_005['TOTAL'].sum() + au135_010['TOTAL'].sum() + au136_015['TOTAL'].sum() + au137_020['TOTAL'].sum() 

au138_999 = abs(au138_999)
# Crear DataFrame para escribir en Excel
df_au138_999= pd.DataFrame({'total': [au138_999]})
write_to_excel(df_au94_999, 47, 138, 'total')


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 19

In [532]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


av134_005 = unidad19_CUMPLIMIENTOEMPRESASSERV_a[(unidad19_CUMPLIMIENTOEMPRESASSERV_a['CUENTA'] == "9620750021") & (unidad19_CUMPLIMIENTOEMPRESASSERV_a['COD_RAMO'] == 46)]
write_to_excel(av134_005, 48, 134, 'TOTAL')

av135_010 = unidad19_CUMPLIMIENTOEMPRESASSERV[(unidad19_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av135_010, 48, 135)

av136_015 = unidad19_CUMPLIMIENTOEMPRESASSERV[(unidad19_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av136_015, 48, 136)

av137_020 = unidad19_CUMPLIMIENTOEMPRESASSERV[(unidad19_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]
write_to_excel(av137_020, 48, 137)


av138_999 = av134_005['TOTAL'].sum() + av135_010['TOTAL'].sum() + av136_015['TOTAL'].sum() + av137_020['TOTAL'].sum() 

av138_999 = abs(av138_999)
# Crear DataFrame para escribir en Excel
df_av138_999= pd.DataFrame({'total': [av138_999]})
write_to_excel(df_av138_999, 48, 138, 'total')


# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 19

In [533]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

aw134_005 = unidad19_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw134_005, 49, 134)

aw135_010 = unidad19_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw135_010, 49, 135)

aw136_015 = unidad19_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw136_015, 49, 136)

aw137_020 = unidad19_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]
write_to_excel(aw137_020, 49, 137)


aw138_999 = aw134_005['TOTAL'].sum() + aw135_010['TOTAL'].sum() + aw136_015['TOTAL'].sum() + aw137_020['TOTAL'].sum() 

aw138_999 = abs(aw138_999)
# Crear DataFrame para escribir en Excel
df_aw138_999= pd.DataFrame({'total': [aw138_999]})
write_to_excel(df_aw138_999, 49, 138, 'total')


# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 19

In [534]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

ax134_005 = unidad19_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax134_005, 50, 134)

ax135_010 = unidad19_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax135_010, 50, 135)

ax136_015 = unidad19_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax136_015, 50, 136)

ax137_020 = unidad19_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]
write_to_excel(ax137_020, 50, 137)


ax138_999 = ax134_005['TOTAL'].sum() + ax135_010['TOTAL'].sum() + ax136_015['TOTAL'].sum() + ax137_020['TOTAL'].sum() 

ax138_999 = abs(ax138_999)
# Crear DataFrame para escribir en Excel
df_ax138_999= pd.DataFrame({'total': [ax138_999]})
write_to_excel(df_ax138_999, 50, 138, 'total')


# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 19

In [535]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

ay134_005 = unidad19_CUMPLIMIENTOCAUCIONES[(unidad19_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay134_005, 51, 134)

ay135_010 = unidad19_CUMPLIMIENTOCAUCIONES[(unidad19_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay135_010, 51, 135)

ay136_015 = unidad19_CUMPLIMIENTOCAUCIONES[(unidad19_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay136_015, 51, 136)

ay137_020 = unidad19_CUMPLIMIENTOCAUCIONES[(unidad19_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]
write_to_excel(ay137_020, 51, 137)


ay138_999 = ay134_005['TOTAL'].sum() + ay135_010['TOTAL'].sum() + ay136_015['TOTAL'].sum() + ay137_020['TOTAL'].sum() 

ay138_999 = abs(ay138_999)
# Crear DataFrame para escribir en Excel
df_ay138_999= pd.DataFrame({'total': [ay138_999]})
write_to_excel(df_ay138_999, 51, 138, 'total')


# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 19

In [536]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

az134_005 = unidad19_CUMPLIMIENTOARRENDAMIENTO[(unidad19_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az134_005, 52, 134)

az135_010 = unidad19_CUMPLIMIENTOARRENDAMIENTO[(unidad19_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az135_010, 52, 135)

az136_015 = unidad19_CUMPLIMIENTOARRENDAMIENTO[(unidad19_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az136_015, 52, 136)

az137_020 = unidad19_CUMPLIMIENTOARRENDAMIENTO[(unidad19_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]
write_to_excel(az137_020, 52, 137)


az138_999 = az134_005['TOTAL'].sum() + az135_010['TOTAL'].sum() + az136_015['TOTAL'].sum() + az137_020['TOTAL'].sum() 

az138_999 = abs(az138_999)
# Crear DataFrame para escribir en Excel
df_az138_999= pd.DataFrame({'total': [az138_999]})
write_to_excel(df_az138_999, 52, 138, 'total')


# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 19

In [537]:
# Agrupar los datos según el concepto y el ramo
unidad19_CUMPLIMIENTOPARTICULARES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad19_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total


ba134_005 = unidad19_CUMPLIMIENTOPARTICULARES_a[(unidad19_CUMPLIMIENTOPARTICULARES_a['CUENTA'] == "9620750021") & (unidad19_CUMPLIMIENTOPARTICULARES_a['COD_RAMO'] == 51)]
write_to_excel(ba134_005, 53, 134, 'TOTAL')

ba135_010 = unidad19_CUMPLIMIENTOPARTICULARES[(unidad19_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Aceptados en reaseguro") & (unidad19_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba135_010, 53, 135)

ba136_015 = unidad19_CUMPLIMIENTOPARTICULARES[(unidad19_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Parte reaseguro al interior (resta)") & (unidad19_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba136_015, 53, 136)

ba137_020 = unidad19_CUMPLIMIENTOPARTICULARES[(unidad19_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Parte reaseguro al exterior (resta)") & (unidad19_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]
write_to_excel(ba137_020, 53, 137)


ba138_999 = ba134_005['TOTAL'].sum() + ba135_010['TOTAL'].sum() + ba136_015['TOTAL'].sum() + ba137_020['TOTAL'].sum() 

ba138_999 = abs(ba138_999)
# Crear DataFrame para escribir en Excel
df_ba138_999= pd.DataFrame({'total': [ba138_999]})
write_to_excel(df_ba138_999, 53, 138, 'total')

#  1   TOTAL UNIDAD 19

In [538]:
c134_005 = sum([ws.cell(row=134, column=col).value if ws.cell(row=134, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=134, column=3).value = c134_005  

c135_010 = sum([ws.cell(row=135, column=col).value if ws.cell(row=135, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=135, column=3).value = c135_010 

c136_015 = sum([ws.cell(row=136, column=col).value if ws.cell(row=136, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=136, column=3).value = c136_015 

c137_020 = sum([ws.cell(row=137, column=col).value if ws.cell(row=137, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=137, column=3).value = c137_020  

# Calcular el total general según la fórmula dada

c138_999 = c134_005 + c135_010 - c136_015 - c137_020

# Escribir el total general en Excel
ws.cell(row=138, column=3).value = c138_999

#  2   SUBTOTAL - RAMOS UNIDAD 19

In [539]:
d134_005 = sum([ws.cell(row=134, column=col).value if ws.cell(row=134, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=134, column=4).value = d134_005  

d135_010 = sum([ws.cell(row=135, column=col).value if ws.cell(row=135, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=135, column=4).value = d135_010 

d136_015 = sum([ws.cell(row=136, column=col).value if ws.cell(row=136, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=136, column=4).value = d136_015 

d137_020 = sum([ws.cell(row=137, column=col).value if ws.cell(row=137, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=137, column=4).value = d137_020  

# Calcular el total general según la fórmula dada

d138_999 = d134_005 + d135_010 - d136_015 - d137_020

# Escribir el total general en Excel
ws.cell(row=138, column=4).value = d138_999

## UNIDAD DE CAPTURA 20

#  3   AUTOMOVILES UNIDAD 20

In [543]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'Periodo' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición de interés
posicion = POLIZAS[(POLIZAS['RAMOS'] == "3   AUTOMOVILES") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Función auxiliar para extraer y convertir valores
def obtener_valor(posicion, columna):
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], columna]
        try:
            return float(valor)
        except:
            return 0
    return 0

# Obtener y escribir cada valor
e140_005 = obtener_valor(posicion, 2)
ws.cell(row=140, column=5).value = e140_005

e141_010 = obtener_valor(posicion, 3)
ws.cell(row=141, column=5).value = e141_010

e142_015 = obtener_valor(posicion, 4)
ws.cell(row=142, column=5).value = e142_015

e143_020 = obtener_valor(posicion, 5)
ws.cell(row=143, column=5).value = e143_020

# Calcular el total final
e144_999 = e140_005 + e141_010 + e142_015 - e143_020
ws.cell(row=144, column=5).value = e144_999


#  4   SOAT UNIDAD 20

In [545]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'Periodo' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "4   SOAT") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener el valor en la posición (columna 3) y escribir en la hoja de Excel
f140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=6).value = f140_005.values[0]

# Obtener el valor para las pólizas emitidas nuevas (columna 4)
f141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=6).value = f141_010.values[0]

# Obtener el valor para las pólizas rehabilitadas (columna 5)
f142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=6).value = f142_015.values[0]

# Obtener el valor para las pólizas canceladas y/o vencidas (columna 6)
f143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=6).value = f143_020.values[0]

# Calcular el valor final ab144_999 y escribir en la hoja de Excel
f144_999 = f140_005.values[0] + f141_010.values[0] + f142_015.values[0] - f143_020.values[0]
ws.cell(row=144, column=6).value = f144_999


#  5   CUMPLIMIENTO UNIDAD 20

In [546]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas de CUMPLIMIENTO
posicion = POLIZAS[(POLIZAS['RAMOS'] == "5   CUMPLIMIENTO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores por columna y convertir a float
g140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=7).value = g140_005.values[0]

g141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=7).value = g141_010.values[0]

g142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=7).value = g142_015.values[0]

g143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=7).value = g143_020.values[0]

# Calcular el valor final
g144_999 = g140_005.values[0] + g141_010.values[0] + g142_015.values[0] - g143_020.values[0]
ws.cell(row=144, column=7).value = g144_999


#  6   RESPONSABILIDAD CIVIL UNIDAD 20

In [547]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas de RESPONSABILIDAD CIVIL
posicion = POLIZAS[(POLIZAS['RAMOS'] == "6   RESPONSABILIDAD CIVIL") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir los valores a float
h140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=8).value = h140_005.values[0]

h141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=8).value = h141_010.values[0]

h142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=8).value = h142_015.values[0]

h143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=8).value = h143_020.values[0]

# Calcular el valor final
h144_999 = h140_005.values[0] + h141_010.values[0] + h142_015.values[0] - h143_020.values[0]
ws.cell(row=144, column=8).value = h144_999


#  7   INCENDIO UNIDAD 20

In [548]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición para el ramo "7   INCENDIO"
posicion = POLIZAS[(POLIZAS['RAMOS'] == "7   INCENDIO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Extraer y convertir los valores a float para cada columna
i140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=9).value = i140_005.values[0]

i141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=9).value = i141_010.values[0]

i142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=9).value = i142_015.values[0]

i143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=9).value = i143_020.values[0]

# Calcular el total
i144_999 = i140_005.values[0] + i141_010.values[0] + i142_015.values[0] - i143_020.values[0]
ws.cell(row=144, column=9).value = i144_999


#  8   TERREMOTO UNIDAD 20

In [549]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición para el ramo "8   TERREMOTO"
posicion = POLIZAS[(POLIZAS['RAMOS'] == "8   TERREMOTO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Extraer y convertir los valores a float para cada columna
j140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=10).value = j140_005.values[0]

j141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=10).value = j141_010.values[0]

j142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=10).value = j142_015.values[0]

j143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=10).value = j143_020.values[0]

# Calcular el total
j144_999 = j140_005.values[0] + j141_010.values[0] + j142_015.values[0] - j143_020.values[0]
ws.cell(row=144, column=10).value = j144_999


#  9   SUSTRACCION UNIDAD 20

In [550]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición para el ramo "9   SUSTRACCION"
posicion = POLIZAS[(POLIZAS['RAMOS'] == "9   SUSTRACCION") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Extraer y convertir los valores a float para cada columna
k140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=11).value = k140_005.values[0]

k141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=11).value = k141_010.values[0]

k142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=11).value = k142_015.values[0]

k143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=11).value = k143_020.values[0]

# Calcular el total
k144_999 = k140_005.values[0] + k141_010.values[0] + k142_015.values[0] - k143_020.values[0]
ws.cell(row=144, column=11).value = k144_999


#  10  TRANSPORTE UNIDAD 20

In [552]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición para el ramo "10  TRANSPORTE"
posicion = POLIZAS[(POLIZAS['RAMOS'] == "10  TRANSPORTE") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Extraer y convertir los valores a float para cada columna
l140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=12).value = l140_005.values[0]

l141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=12).value = l141_010.values[0]

l142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=12).value = l142_015.values[0]

l143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=12).value = l143_020.values[0]

# Calcular el total
l144_999 = l140_005.values[0] + l141_010.values[0] + l142_015.values[0] - l143_020.values[0]
ws.cell(row=144, column=12).value = l144_999


#  11  CORRIENTE DÉBIL UNIDAD 20

In [553]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas del ramo "11  CORRIENTE DÉBIL"
posicion = POLIZAS[(POLIZAS['RAMOS'] == "11  CORRIENTE DÉBIL") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir a float los valores de las columnas necesarias
m140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=13).value = m140_005.values[0]

m141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=13).value = m141_010.values[0]

m142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=13).value = m142_015.values[0]

m143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=13).value = m143_020.values[0]

# Calcular el total final
m144_999 = m140_005.values[0] + m141_010.values[0] + m142_015.values[0] - m143_020.values[0]
ws.cell(row=144, column=13).value = m144_999


#  12  TODO RIESGO CONTRATISTA UNIDAD 20

In [554]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar posición
posicion = POLIZAS[(POLIZAS['RAMOS'] == "12  TODO RIESGO CONTRATISTA") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir a float los valores requeridos
n140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=14).value = n140_005.values[0]

n141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=14).value = n141_010.values[0]

n142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=14).value = n142_015.values[0]

n143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=14).value = n143_020.values[0]

# Calcular el valor final
n144_999 = n140_005.values[0] + n141_010.values[0] + n142_015.values[0] - n143_020.values[0]
ws.cell(row=144, column=14).value = n144_999


#  13  MANEJO UNIDAD 20

In [555]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "13  MANEJO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir los valores
o140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=15).value = o140_005.values[0]

o141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=15).value = o141_010.values[0]

o142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=15).value = o142_015.values[0]

o143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=15).value = o143_020.values[0]

# Calcular el valor final y escribir en la hoja de Excel
o144_999 = o140_005.values[0] + o141_010.values[0] + o142_015.values[0] - o143_020.values[0]
ws.cell(row=144, column=15).value = o144_999


#  14  LUCRO CESANTE UNIDAD 20

In [556]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "14  LUCRO CESANTE") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir los valores
p140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=16).value = p140_005.values[0]

p141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=16).value = p141_010.values[0]

p142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=16).value = p142_015.values[0]

p143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=16).value = p143_020.values[0]

# Calcular el valor final y escribir en la hoja de Excel
p144_999 = p140_005.values[0] + p141_010.values[0] + p142_015.values[0] - p143_020.values[0]
ws.cell(row=144, column=16).value = p144_999


#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 20

In [557]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "15   MONTAJE Y ROTURA DE MAQUINARIA") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir los valores a float
q140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=17).value = q140_005.values[0]

q141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=17).value = q141_010.values[0]

q142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=17).value = q142_015.values[0]

q143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=17).value = q143_020.values[0]

# Calcular el valor final ab144_999 y escribir en la hoja de Excel
q144_999 = q140_005.values[0] + q141_010.values[0] + q142_015.values[0] - q143_020.values[0]
ws.cell(row=144, column=17).value = q144_999


#  16  AVIACIÓN UNIDAD 20

In [558]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "16  AVIACIÓN") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y convertir los valores a float
r140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=18).value = r140_005.values[0]

r141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=18).value = r141_010.values[0]

r142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=18).value = r142_015.values[0]

r143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=18).value = r143_020.values[0]

# Calcular el valor final ab144_999 y escribir en la hoja de Excel
r144_999 = r140_005.values[0] + r141_010.values[0] + r142_015.values[0] - r143_020.values[0]
ws.cell(row=144, column=18).value = r144_999


#  17  NAVEGACIÓN Y CASCO UNIDAD 20

In [559]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "17  NAVEGACIÓN Y CASCO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores y convertir a float
s140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=19).value = s140_005.values[0]

s141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=19).value = s141_010.values[0]

s142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=19).value = s142_015.values[0]

s143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=19).value = s143_020.values[0]

# Calcular el valor final ab144_999
s144_999 = s140_005.values[0] + s141_010.values[0] + s142_015.values[0] - s143_020.values[0]
ws.cell(row=144, column=19).value = s144_999


#  18  MINAS Y PETRÓLEOS UNIDAD 20

In [560]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "18  MINAS Y PETRÓLEOS") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
t140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=20).value = t140_005.values[0]

t141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=20).value = t141_010.values[0]

t142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=20).value = t142_015.values[0]

t143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=20).value = t143_020.values[0]

# Calcular el valor final ab144_999 y escribir en la hoja de Excel
t144_999 = t140_005.values[0] + t141_010.values[0] + t142_015.values[0] - t143_020.values[0]
ws.cell(row=144, column=20).value = t144_999


#  19  VIDRIOS UNIDAD 20

In [561]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "19  VIDRIOS") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
u140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=21).value = u140_005.values[0]

u141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=21).value = u141_010.values[0]

u142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=21).value = u142_015.values[0]

u143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=21).value = u143_020.values[0]

# Calcular el valor final u144_999 y escribir en la hoja de Excel
u144_999 = u140_005.values[0] + u141_010.values[0] + u142_015.values[0] - u143_020.values[0]
ws.cell(row=144, column=21).value = u144_999


#  20  CRÉDITO COMERCIAL UNIDAD 20

In [562]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "20  CRÉDITO COMERCIAL") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
v140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=22).value = v140_005.values[0]

v141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=22).value = v141_010.values[0]

v142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=22).value = v142_015.values[0]

v143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=22).value = v143_020.values[0]

# Calcular el valor final v144_999 y escribir en la hoja de Excel
v144_999 = v140_005.values[0] + v141_010.values[0] + v142_015.values[0] - v143_020.values[0]
ws.cell(row=144, column=22).value = v144_999


#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 20

In [563]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "21  CRÉDITO A LA EXPORTACIÓN") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
w140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=23).value = w140_005.values[0]

w141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=23).value = w141_010.values[0]

w142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=23).value = w142_015.values[0]

w143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=23).value = w143_020.values[0]

# Calcular el valor final w144_999 y escribir en la hoja de Excel
w144_999 = w140_005.values[0] + w141_010.values[0] + w142_015.values[0] - w143_020.values[0]
ws.cell(row=144, column=23).value = w144_999


#  24  DESEMPLEO UNIDAD 20

In [564]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "24  DESEMPLEO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
z140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=26).value = z140_005.values[0]

z141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=26).value = z141_010.values[0]

z142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=26).value = z142_015.values[0]

z143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=26).value = z143_020.values[0]

# Calcular el valor final z144_999 y escribir en la hoja de Excel
z144_999 = z140_005.values[0] + z141_010.values[0] + z142_015.values[0] - z143_020.values[0]
ws.cell(row=144, column=26).value = z144_999


#  25  HOGAR UNIDAD 20

In [565]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DÍA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "25  HOGAR") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
aa140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=27).value = aa140_005.values[0]

aa141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=27).value = aa141_010.values[0]

aa142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=27).value = aa142_015.values[0]

aa143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=27).value = aa143_020.values[0]

# Calcular el valor final aa144_999 y escribir en la hoja de Excel
aa144_999 = aa140_005.values[0] + aa141_010.values[0] + aa142_015.values[0] - aa143_020.values[0]
ws.cell(row=144, column=27).value = aa144_999


# 42 AGROPECUARIO UNIDAD 20

In [566]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "42 AGROPECUARIO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores, convertir a float y escribir en la hoja de Excel
ar140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=44).value = ar140_005.iloc[0]

ar141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=44).value = ar141_010.iloc[0]

ar142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=44).value = ar142_015.iloc[0]

ar143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=44).value = ar143_020.iloc[0]

# Calcular el valor final ar144_999 y escribir en la hoja de Excel
ar144_999 = ar140_005.iloc[0] + ar141_010.iloc[0] + ar142_015.iloc[0] - ar143_020.iloc[0]
ws.cell(row=144, column=44).value = ar144_999


# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 20

In [567]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "43 OPERACIONES NO ASIGANBLES A RAMOS") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y escribir los valores para cada fila, asegurando tipo float
as140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=45).value = as140_005.iloc[0]

as141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=45).value = as141_010.iloc[0]

as142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=45).value = as142_015.iloc[0]

as143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=45).value = as143_020.iloc[0]

# Calcular el valor final y escribir en la hoja de Excel
as144_999 = as140_005.iloc[0] + as141_010.iloc[0] + as142_015.iloc[0] - as143_020.iloc[0]
ws.cell(row=144, column=45).value = as144_999


# 44 DECENAL UNIDAD 20

In [568]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "44 DECENAL") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores y convertir a float
at140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=46).value = at140_005.iloc[0]

at141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=46).value = at141_010.iloc[0]

at142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=46).value = at142_015.iloc[0]

at143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=46).value = at143_020.iloc[0]

# Calcular el valor final y escribir en Excel
at144_999 = at140_005.iloc[0] + at141_010.iloc[0] + at142_015.iloc[0] - at143_020.iloc[0]
ws.cell(row=144, column=46).value = at144_999


# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 20

In [569]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "45 CUMPLIMIENTO - ENTIDADES ESTATALES") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores y convertirlos a float
au140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=47).value = au140_005.iloc[0]

au141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=47).value = au141_010.iloc[0]

au142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=47).value = au142_015.iloc[0]

au143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=47).value = au143_020.iloc[0]

# Calcular el valor final y escribir en la hoja de Excel
au144_999 = au140_005.iloc[0] + au141_010.iloc[0] + au142_015.iloc[0] - au143_020.iloc[0]
ws.cell(row=144, column=47).value = au144_999


# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 20

In [570]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP)") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener los valores convertidos a float (si hay filas, si no, poner 0)
av140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=48).value = av140_005.iloc[0]

av141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=48).value = av141_010.iloc[0]

av142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=48).value = av142_015.iloc[0]

av143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=48).value = av143_020.iloc[0]

# Calcular el valor final
av144_999 = av140_005.iloc[0] + av141_010.iloc[0] + av142_015.iloc[0] - av143_020.iloc[0]
ws.cell(row=144, column=48).value = av144_999


# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 20

In [571]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE)") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener valores convertidos a float
aw140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=49).value = aw140_005.iloc[0]

aw141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=49).value = aw141_010.iloc[0]

aw142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=49).value = aw142_015.iloc[0]

aw143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=49).value = aw143_020.iloc[0]

# Calcular el valor final
aw144_999 = aw140_005.iloc[0] + aw141_010.iloc[0] + aw142_015.iloc[0] - aw143_020.iloc[0]
ws.cell(row=144, column=49).value = aw144_999


# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 20

In [572]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la posición
posicion = POLIZAS[(POLIZAS['RAMOS'] == "48 CUMPLIMIENTO - DISPOSICIONES LEGALES") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener valores convertidos
ax140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=50).value = ax140_005.iloc[0]

ax141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=50).value = ax141_010.iloc[0]

ax142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=50).value = ax142_015.iloc[0]

ax143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=50).value = ax143_020.iloc[0]

# Calcular total
ax144_999 = ax140_005.iloc[0] + ax141_010.iloc[0] + ax142_015.iloc[0] - ax143_020.iloc[0]
ws.cell(row=144, column=50).value = ax144_999


# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 20

In [573]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "49 CUMPLIMIENTO - CAUCIONES JUDICIALES") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y escribir los valores en Excel
ay140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=51).value = ay140_005.iloc[0]

ay141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=51).value = ay141_010.iloc[0]

ay142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=51).value = ay142_015.iloc[0]

ay143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=51).value = ay143_020.iloc[0]

# Calcular y escribir el valor final
ay144_999 = ay140_005.iloc[0] + ay141_010.iloc[0] + ay142_015.iloc[0] - ay143_020.iloc[0]
ws.cell(row=144, column=51).value = ay144_999


# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 20

In [574]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "50 CUMPLIMIENTO - ARRENDAMIENTO") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y escribir los valores en Excel
az140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=52).value = az140_005.iloc[0]

az141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=52).value = az141_010.iloc[0]

az142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=52).value = az142_015.iloc[0]

az143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=52).value = az143_020.iloc[0]

# Calcular y escribir el valor final
az144_999 = az140_005.iloc[0] + az141_010.iloc[0] + az142_015.iloc[0] - az143_020.iloc[0]
ws.cell(row=144, column=52).value = az144_999


# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 20

In [575]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en AÑO, MES y DIA
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Obtener la posición para las pólizas vigentes al final del año anterior
posicion = POLIZAS[(POLIZAS['RAMOS'] == "51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES)") &
                   (POLIZAS['AÑO'] == max_date[0]) &
                   (POLIZAS['MES'] == max_date[1])].index

# Obtener y escribir los valores en Excel
ba140_005 = POLIZAS.iloc[posicion, 2].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=140, column=53).value = ba140_005.iloc[0]

ba141_010 = POLIZAS.iloc[posicion, 3].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=141, column=53).value = ba141_010.iloc[0]

ba142_015 = POLIZAS.iloc[posicion, 4].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=142, column=53).value = ba142_015.iloc[0]

ba143_020 = POLIZAS.iloc[posicion, 5].astype(float) if len(posicion) > 0 else pd.Series([0.0])
ws.cell(row=143, column=53).value = ba143_020.iloc[0]

# Calcular el valor final y escribir en la hoja de Excel
ba144_999 = ba140_005.iloc[0] + ba141_010.iloc[0] + ba142_015.iloc[0] - ba143_020.iloc[0]
ws.cell(row=144, column=53).value = ba144_999


## UNIDAD DE CAPTURA 21

#  3   AUTOMOVILES UNIDAD 21

In [576]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad21_AUTOMOVILES = bd[(bd['UNIDAD'] == 21) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_AUTOMOVILES['TOTAL'] = unidad21_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior",3, 146),
    ("Primas cedidas contratos facultativos interior",3, 147),
    ("Primas cedidas contratos automáticos exterior",3, 148),
    ("Primas cedidas contratos facultativos exterior",3, 149),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad21_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e146_005 = get_value(unidad21_AUTOMOVILES, "Primas cedidas contratos automáticos interior", 3)
e147_010 = get_value(unidad21_AUTOMOVILES, "Primas cedidas contratos facultativos interior", 3)
e148_015 = get_value(unidad21_AUTOMOVILES, "Primas cedidas contratos automáticos exterior", 3)
e149_020 = get_value(unidad21_AUTOMOVILES, "Primas cedidas contratos facultativos exterior", 3)


# Calcular ab94_999 usando la fórmula especificada
e150_999 = (e146_005 + e147_010 - e148_015 - e149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=5, value=e150_999)

<Cell 'Hoja1'.E150>

#  4   SOAT UNIDAD 21

In [577]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad21_SOAT = bd[(bd['UNIDAD'] == 21) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_SOAT['TOTAL'] = unidad21_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior",4, 146),
    ("Primas cedidas contratos facultativos interior",4, 147),
    ("Primas cedidas contratos automáticos exterior",4, 148),
    ("Primas cedidas contratos facultativos exterior",4, 149),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad21_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f146_005 = get_value(unidad21_SOAT, "Primas cedidas contratos automáticos interior", 4)
f147_010 = get_value(unidad21_SOAT, "Primas cedidas contratos facultativos interior", 4)
f148_015 = get_value(unidad21_SOAT, "Primas cedidas contratos automáticos exterior", 4)
f149_020 = get_value(unidad21_SOAT, "Primas cedidas contratos facultativos exterior", 4)


# Calcular ab94_999 usando la fórmula especificada
f150_999 = (f146_005 + f147_010 - f148_015 - f149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=6, value=f150_999)

<Cell 'Hoja1'.F150>

#  5   CUMPLIMIENTO UNIDAD 21

In [578]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad21_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 21) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_CUMPLIMIENTO['TOTAL'] = unidad21_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior",5, 146),
    ("Primas cedidas contratos facultativos interior",5, 147),
    ("Primas cedidas contratos automáticos exterior",5, 148),
    ("Primas cedidas contratos facultativos exterior",5, 149),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad21_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
g146_005 = get_value(unidad21_CUMPLIMIENTO, "Primas cedidas contratos automáticos interior", 5)
g147_010 = get_value(unidad21_CUMPLIMIENTO, "Primas cedidas contratos facultativos interior", 5)
g148_015 = get_value(unidad21_CUMPLIMIENTO, "Primas cedidas contratos automáticos exterior", 5)
g149_020 = get_value(unidad21_CUMPLIMIENTO, "Primas cedidas contratos facultativos exterior", 5)


# Calcular ab94_999 usando la fórmula especificada
g150_999 = (g146_005 + g147_010 - g148_015 - g149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=7, value=g150_999)

<Cell 'Hoja1'.G150>

#  6   RESPONSABILIDAD CIVIL UNIDAD 21

In [579]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_RESPONSABILIDADCI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_RESPONSABILIDADCI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
h146_005 = abs(unidad21_RESPONSABILIDADCI[(unidad21_RESPONSABILIDADCI['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_RESPONSABILIDADCI['COD_RAMO'] == 6)]['TOTAL']).sum()
h146_005 = h146_005 if h146_005 != 0 else 0
ws.cell(row=146, column=8, value=float(h146_005))

# Primas cedidas contratos facultativos interior
h147_010 = abs(unidad21_RESPONSABILIDADCI[(unidad21_RESPONSABILIDADCI['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_RESPONSABILIDADCI['COD_RAMO'] == 6)]['TOTAL']).sum()
h147_010 = h147_010 if h147_010 != 0 else 0
ws.cell(row=147, column=8, value=float(h147_010))

# Parte reaseguro al interior
h148_015 = abs(unidad21_RESPONSABILIDADCI[(unidad21_RESPONSABILIDADCI['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_RESPONSABILIDADCI['COD_RAMO'] == 6)]['TOTAL']).sum()
h148_015 = h148_015 if h148_015 != 0 else 0
ws.cell(row=148, column=8, value=float(h148_015))

cuentas = ["6232101061", "6232101061", "6232101121", "6232101221", "6232101251"]
h149_020 = unidad21_RESPONSABILIDADCI_a[(unidad21_RESPONSABILIDADCI_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_RESPONSABILIDADCI_a['COD_RAMO'] == 6)]['TOTAL'].sum()
ws.cell(row=149, column=8, value=float(h149_020))

# Calcular ae150_999
h150_999 = abs(h146_005 + h147_010 - h148_015 - h149_020)
ws.cell(row=150, column=8, value=float(h150_999))

<Cell 'Hoja1'.H150>

#  7   INCENDIO UNIDAD 21

In [580]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_INCENDIO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_INCENDIO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
i146_005 = abs(unidad21_INCENDIO[(unidad21_INCENDIO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_INCENDIO['COD_RAMO'] == 7)]['TOTAL']).sum()
i146_005 = i146_005 if i146_005 != 0 else 0
ws.cell(row=146, column=9, value=float(i146_005))

# Primas cedidas contratos facultativos interior
i147_010 = abs(unidad21_INCENDIO[(unidad21_INCENDIO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_INCENDIO['COD_RAMO'] == 7)]['TOTAL']).sum()
i147_010 = i147_010 if i147_010 != 0 else 0
ws.cell(row=147, column=9, value=float(i147_010))

# Parte reaseguro al interior
i148_015 = abs(unidad21_INCENDIO[(unidad21_INCENDIO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_INCENDIO['COD_RAMO'] == 7)]['TOTAL']).sum()
i148_015 = i148_015 if i148_015 != 0 else 0
ws.cell(row=148, column=9, value=float(i148_015))

cuentas = ["6232101061", "6232101061", "6232101121", "6232101221", "6232101251"]
i149_020 = unidad21_INCENDIO_a[(unidad21_INCENDIO_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_INCENDIO_a['COD_RAMO'] == 7)]['TOTAL'].sum()
ws.cell(row=149, column=9, value=float(i149_020))

# Calcular ae150_999
i150_999 = abs(i146_005 + i147_010 - i148_015 - i149_020)
ws.cell(row=150, column=9, value=float(i150_999))

<Cell 'Hoja1'.I150>

#  8   TERREMOTO UNIDAD 21

In [581]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_TERREMOTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_TERREMOTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
j146_005 = abs(unidad21_TERREMOTO[(unidad21_TERREMOTO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_TERREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j146_005 = j146_005 if j146_005 != 0 else 0
ws.cell(row=146, column=10, value=float(j146_005))

# Primas cedidas contratos facultativos interior
j147_010 = abs(unidad21_TERREMOTO[(unidad21_TERREMOTO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_TERREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j147_010 = j147_010 if j147_010 != 0 else 0
ws.cell(row=147, column=10, value=float(j147_010))

# Parte reaseguro al interior
j148_015 = abs(unidad21_TERREMOTO[(unidad21_TERREMOTO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_TERREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j148_015 = j148_015 if j148_015 != 0 else 0
ws.cell(row=148, column=10, value=float(j148_015))

cuentas = ["6232101061", "6232101061", "6232101121", "6232101221", "6232101251"]
j149_020 = unidad21_TERREMOTO_a[(unidad21_TERREMOTO_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_TERREMOTO_a['COD_RAMO'] == 8)]['TOTAL'].sum()
ws.cell(row=149, column=10, value=float(j149_020))

# Calcular ae150_999
j150_999 = abs(j146_005 + j147_010 - j148_015 - j149_020)
ws.cell(row=150, column=10, value=float(j150_999))

<Cell 'Hoja1'.J150>

#  9   SUSTRACCION UNIDAD 21

In [582]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_SUSTRACCION = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_SUSTRACCION_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
k146_005 = abs(unidad21_SUSTRACCION[(unidad21_SUSTRACCION['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_SUSTRACCION['COD_RAMO'] == 9)]['TOTAL']).sum()
k146_005 = k146_005 if k146_005 != 0 else 0
ws.cell(row=146, column=11, value=float(k146_005))

# Primas cedidas contratos facultativos interior
k147_010 = abs(unidad21_SUSTRACCION[(unidad21_SUSTRACCION['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_SUSTRACCION['COD_RAMO'] == 9)]['TOTAL']).sum()
k147_010 = k147_010 if k147_010 != 0 else 0
ws.cell(row=147, column=11, value=float(k147_010))

# Parte reaseguro al interior
k148_015 = abs(unidad21_SUSTRACCION[(unidad21_SUSTRACCION['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_SUSTRACCION['COD_RAMO'] == 9)]['TOTAL']).sum()
k148_015 = k148_015 if k148_015 != 0 else 0
ws.cell(row=148, column=11, value=float(k148_015))

cuentas = ["6232101121", "6232101251"]
k149_020 = unidad21_SUSTRACCION_a[(unidad21_SUSTRACCION_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_SUSTRACCION_a['COD_RAMO'] == 9)]['TOTAL'].sum()
ws.cell(row=149, column=11, value=float(k149_020))

# Calcular ae150_999
k150_999 = abs(k146_005 + k147_010 - k148_015 - k149_020)
ws.cell(row=150, column=11, value=float(k150_999))

<Cell 'Hoja1'.K150>

#  10  TRANSPORTE UNIDAD 21

In [583]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_TRANSPORTE = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_TRANSPORTE_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
l146_005 = abs(unidad21_TRANSPORTE[(unidad21_TRANSPORTE['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_TRANSPORTE['COD_RAMO'] == 10)]['TOTAL']).sum()
l146_005 = l146_005 if l146_005 != 0 else 0
ws.cell(row=146, column=12, value=float(l146_005))

# Primas cedidas contratos facultativos interior
l147_010 = abs(unidad21_TRANSPORTE[(unidad21_TRANSPORTE['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_TRANSPORTE['COD_RAMO'] == 10)]['TOTAL']).sum()
l147_010 = l147_010 if l147_010 != 0 else 0
ws.cell(row=147, column=12, value=float(l147_010))

# Parte reaseguro al interior
l148_015 = abs(unidad21_TRANSPORTE[(unidad21_TRANSPORTE['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_TRANSPORTE['COD_RAMO'] == 10)]['TOTAL']).sum()
l148_015 = l148_015 if l148_015 != 0 else 0
ws.cell(row=148, column=12, value=float(l148_015))

cuentas = ["6232101221", "6232101221", "6232101251"]
l149_020 = unidad21_TRANSPORTE_a[(unidad21_TRANSPORTE_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_TRANSPORTE_a['COD_RAMO'] == 10)]['TOTAL'].sum()
ws.cell(row=149, column=12, value=float(l149_020))

# Calcular ae150_999
l150_999 = abs(l146_005 + l147_010 - l148_015 - l149_020)
ws.cell(row=150, column=12, value=float(l150_999))

<Cell 'Hoja1'.L150>

#  11  CORRIENTE DÉBIL UNIDAD 21

In [584]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CORRIENTEDEBIL = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CORRIENTEDEBIL_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
m146_005 = abs(unidad21_CORRIENTEDEBIL[(unidad21_CORRIENTEDEBIL['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CORRIENTEDEBIL['COD_RAMO'] == 11)]['TOTAL']).sum()
m146_005 = m146_005 if m146_005 != 0 else 0
ws.cell(row=146, column=13, value=float(m146_005))

# Primas cedidas contratos facultativos interior
m147_010 = abs(unidad21_CORRIENTEDEBIL[(unidad21_CORRIENTEDEBIL['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CORRIENTEDEBIL['COD_RAMO'] == 11)]['TOTAL']).sum()
m147_010 = m147_010 if m147_010 != 0 else 0
ws.cell(row=147, column=13, value=float(m147_010))

# Parte reaseguro al interior
m148_015 = abs(unidad21_CORRIENTEDEBIL[(unidad21_CORRIENTEDEBIL['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CORRIENTEDEBIL['COD_RAMO'] == 11)]['TOTAL']).sum()
m148_015 = m148_015 if m148_015 != 0 else 0
ws.cell(row=148, column=13, value=float(m148_015))

cuentas = ["6232101121", "6232101121", "6232101251"]
m149_020 = unidad21_CORRIENTEDEBIL_a[(unidad21_CORRIENTEDEBIL_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_CORRIENTEDEBIL_a['COD_RAMO'] == 11)]['TOTAL'].sum()
ws.cell(row=149, column=13, value=float(m149_020))

# Calcular ae150_999
m150_999 = abs(m146_005 + m147_010 - m148_015 - m149_020)
ws.cell(row=150, column=13, value=float(m150_999))

<Cell 'Hoja1'.M150>

#  12  TODO RIESGO CONTRATISTA UNIDAD 21

In [585]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_TODORIESGOCONTRATISTA = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_TODORIESGOCONTRATISTA_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
n146_005 = abs(unidad21_TODORIESGOCONTRATISTA[(unidad21_TODORIESGOCONTRATISTA['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_TODORIESGOCONTRATISTA['COD_RAMO'] == 12)]['TOTAL']).sum()
n146_005 = n146_005 if n146_005 != 0 else 0
ws.cell(row=146, column=14, value=float(n146_005))

# Primas cedidas contratos facultativos interior
n147_010 = abs(unidad21_TODORIESGOCONTRATISTA[(unidad21_TODORIESGOCONTRATISTA['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_TODORIESGOCONTRATISTA['COD_RAMO'] == 12)]['TOTAL']).sum()
n147_010 = n147_010 if n147_010 != 0 else 0
ws.cell(row=147, column=14, value=float(n147_010))

# Parte reaseguro al interior
n148_015 = abs(unidad21_TODORIESGOCONTRATISTA[(unidad21_TODORIESGOCONTRATISTA['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_TODORIESGOCONTRATISTA['COD_RAMO'] == 12)]['TOTAL']).sum()
n148_015 = n148_015 if n148_015 != 0 else 0
ws.cell(row=148, column=14, value=float(n148_015))

cuentas = ["6232101061", "6232101121", "6232101221","6232101251"]
n149_020 = unidad21_TODORIESGOCONTRATISTA_a[(unidad21_TODORIESGOCONTRATISTA_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_TODORIESGOCONTRATISTA_a['COD_RAMO'] == 12)]['TOTAL'].sum()
ws.cell(row=149, column=14, value=float(n149_020))

# Calcular ae150_999
n150_999 = abs(n146_005 + n147_010 - n148_015 - n149_020)
ws.cell(row=150, column=14, value=float(n150_999))

<Cell 'Hoja1'.N150>

#  13  MANEJO UNIDAD 21

In [586]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_MANEJO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_MANEJO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
o146_005 = abs(unidad21_MANEJO[(unidad21_MANEJO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_MANEJO['COD_RAMO'] == 13)]['TOTAL']).sum()
o146_005 = o146_005 if o146_005 != 0 else 0
ws.cell(row=146, column=15, value=float(o146_005))

# Primas cedidas contratos facultativos interior
o147_010 = abs(unidad21_MANEJO[(unidad21_MANEJO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_MANEJO['COD_RAMO'] == 13)]['TOTAL']).sum()
o147_010 = o147_010 if o147_010 != 0 else 0
ws.cell(row=147, column=15, value=float(o147_010))

# Parte reaseguro al interior
o148_015 = abs(unidad21_MANEJO[(unidad21_MANEJO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_MANEJO['COD_RAMO'] == 13)]['TOTAL']).sum()
o148_015 = o148_015 if o148_015 != 0 else 0
ws.cell(row=148, column=15, value=float(o148_015))

cuentas = ["6232101061", "6232101121", "6232101221","6232101251"]
o149_020 = unidad21_MANEJO_a[(unidad21_MANEJO_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_MANEJO_a['COD_RAMO'] == 13)]['TOTAL'].sum()
ws.cell(row=149, column=15, value=float(o149_020))

# Calcular ae150_999
o150_999 = abs(o146_005 + o147_010 - o148_015 - o149_020)
ws.cell(row=150, column=15, value=float(o150_999))

<Cell 'Hoja1'.O150>

#  14  LUCRO CESANTE UNIDAD 21

In [587]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_LUCROCESANTE = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_LUCROCESANTE_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
p146_005 = abs(unidad21_LUCROCESANTE[(unidad21_LUCROCESANTE['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_LUCROCESANTE['COD_RAMO'] == 14)]['TOTAL']).sum()
p146_005 = p146_005 if p146_005 != 0 else 0
ws.cell(row=146, column=16, value=float(p146_005))

# Primas cedidas contratos facultativos interior
p147_010 = abs(unidad21_LUCROCESANTE[(unidad21_LUCROCESANTE['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_LUCROCESANTE['COD_RAMO'] == 14)]['TOTAL']).sum()
p147_010 = p147_010 if p147_010 != 0 else 0
ws.cell(row=147, column=16, value=float(p147_010))

# Parte reaseguro al interior
p148_015 = abs(unidad21_LUCROCESANTE[(unidad21_LUCROCESANTE['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_LUCROCESANTE['COD_RAMO'] == 14)]['TOTAL']).sum()
p148_015 = p148_015 if p148_015 != 0 else 0
ws.cell(row=148, column=16, value=float(p148_015))

cuentas = ["6232101121", "6232101121", "6232101251"]
p149_020 = unidad21_LUCROCESANTE_a[(unidad21_LUCROCESANTE_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_LUCROCESANTE_a['COD_RAMO'] == 14)]['TOTAL'].sum()
ws.cell(row=149, column=16, value=float(p149_020))

# Calcular ae150_999
p150_999 = abs(p146_005 + p147_010 - p148_015 - p149_020)
ws.cell(row=150, column=16, value=float(p150_999))

<Cell 'Hoja1'.P150>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 21

In [588]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_MONTAJEYROTURA = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_MONTAJEYROTURA_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
q146_005 = abs(unidad21_MONTAJEYROTURA[(unidad21_MONTAJEYROTURA['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_MONTAJEYROTURA['COD_RAMO'] == 15)]['TOTAL']).sum()
q146_005 = q146_005 if q146_005 != 0 else 0
ws.cell(row=146, column=17, value=float(q146_005))

# Primas cedidas contratos facultativos interior
q147_010 = abs(unidad21_MONTAJEYROTURA[(unidad21_MONTAJEYROTURA['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_MONTAJEYROTURA['COD_RAMO'] == 15)]['TOTAL']).sum()
q147_010 = q147_010 if q147_010 != 0 else 0
ws.cell(row=147, column=17, value=float(q147_010))

# Parte reaseguro al interior
q148_015 = abs(unidad21_MONTAJEYROTURA[(unidad21_MONTAJEYROTURA['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_MONTAJEYROTURA['COD_RAMO'] == 15)]['TOTAL']).sum()
q148_015 = q148_015 if q148_015 != 0 else 0
ws.cell(row=148, column=17, value=float(q148_015))

cuentas = ["6232101061", "6232101121", "6232101251"]
q149_020 = unidad21_MONTAJEYROTURA_a[(unidad21_MONTAJEYROTURA_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_MONTAJEYROTURA_a['COD_RAMO'] == 15)]['TOTAL'].sum()
ws.cell(row=149, column=17, value=float(q149_020))

# Calcular ae150_999
q150_999 = abs(q146_005 + q147_010 - q148_015 - q149_020)
ws.cell(row=150, column=17, value=float(q150_999))

<Cell 'Hoja1'.Q150>

#  16  AVIACIÓN UNIDAD 21

In [589]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_AVIACION = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_AVIACION_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
r146_005 = abs(unidad21_AVIACION[(unidad21_AVIACION['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_AVIACION['COD_RAMO'] == 16)]['TOTAL']).sum()
r146_005 = r146_005 if r146_005 != 0 else 0
ws.cell(row=146, column=18, value=float(r146_005))

# Primas cedidas contratos facultativos interior
r147_010 = abs(unidad21_AVIACION[(unidad21_AVIACION['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_AVIACION['COD_RAMO'] == 16)]['TOTAL']).sum()
r147_010 = r147_010 if r147_010 != 0 else 0
ws.cell(row=147, column=18, value=float(r147_010))

# Parte reaseguro al interior
r148_015 = abs(unidad21_AVIACION[(unidad21_AVIACION['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_AVIACION['COD_RAMO'] == 16)]['TOTAL']).sum()
r148_015 = r148_015 if r148_015 != 0 else 0
ws.cell(row=148, column=18, value=float(r148_015))

cuentas = ["6232101061", "6232101121", "6232101221","6232101251"]
r149_020 = unidad21_AVIACION_a[(unidad21_AVIACION_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_AVIACION_a['COD_RAMO'] == 16)]['TOTAL'].sum()
ws.cell(row=149, column=18, value=float(r149_020))

# Calcular ae150_999
r150_999 = abs(r146_005 + r147_010 - r148_015 - r149_020)
ws.cell(row=150, column=18, value=float(r150_999))

<Cell 'Hoja1'.R150>

#  17  NAVEGACIÓN Y CASCO UNIDAD 21

In [590]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_NAVEGACION = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_NAVEGACION_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
s146_005 = abs(unidad21_NAVEGACION[(unidad21_NAVEGACION['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_NAVEGACION['COD_RAMO'] == 17)]['TOTAL']).sum()
s146_005 = s146_005 if s146_005 != 0 else 0
ws.cell(row=146, column=19, value=float(s146_005))

# Primas cedidas contratos facultativos interior
s147_010 = abs(unidad21_NAVEGACION[(unidad21_NAVEGACION['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_NAVEGACION['COD_RAMO'] == 17)]['TOTAL']).sum()
s147_010 = s147_010 if s147_010 != 0 else 0
ws.cell(row=147, column=19, value=float(s147_010))

# Parte reaseguro al interior
s148_015 = abs(unidad21_NAVEGACION[(unidad21_NAVEGACION['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_NAVEGACION['COD_RAMO'] == 17)]['TOTAL']).sum()
s148_015 = s148_015 if s148_015 != 0 else 0
ws.cell(row=148, column=19, value=float(s148_015))

s149_020 = abs(unidad21_NAVEGACION[(unidad21_NAVEGACION['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_NAVEGACION['COD_RAMO'] == 17)]['TOTAL']).sum()
s149_020 = s149_020 if s149_020 != 0 else 0
ws.cell(row=149, column=19, value=float(s149_020))

# Calcular ae150_999
s150_999 = abs(s146_005 + s147_010 - s148_015 - s149_020)
ws.cell(row=150, column=19, value=float(s150_999))

<Cell 'Hoja1'.S150>

#  18  MINAS Y PETRÓLEOS UNIDAD 21

In [591]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_MINAS = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_MINAS_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
t146_005 = abs(unidad21_MINAS[(unidad21_MINAS['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_MINAS['COD_RAMO'] == 18)]['TOTAL']).sum()
t146_005 = t146_005 if t146_005 != 0 else 0
ws.cell(row=146, column=20, value=float(t146_005))

# Primas cedidas contratos facultativos interior
t147_010 = abs(unidad21_MINAS[(unidad21_MINAS['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_MINAS['COD_RAMO'] == 18)]['TOTAL']).sum()
t147_010 = t147_010 if t147_010 != 0 else 0
ws.cell(row=147, column=20, value=float(t147_010))

# Parte reaseguro al interior
t148_015 = abs(unidad21_MINAS[(unidad21_MINAS['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_MINAS['COD_RAMO'] == 18)]['TOTAL']).sum()
t148_015 = t148_015 if t148_015 != 0 else 0
ws.cell(row=148, column=20, value=float(t148_015))

t149_020 = abs(unidad21_MINAS[(unidad21_MINAS['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_MINAS['COD_RAMO'] == 18)]['TOTAL']).sum()
t149_020 = t149_020 if t149_020 != 0 else 0
ws.cell(row=149, column=20, value=float(t149_020))

# Calcular ae150_999
t150_999 = abs(t146_005 + t147_010 - t148_015 - t149_020)
ws.cell(row=150, column=20, value=float(t150_999))

<Cell 'Hoja1'.T150>

#  19  VIDRIOS UNIDAD 21

In [592]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_VIDRIOS = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_VIDRIOS_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
u146_005 = abs(unidad21_VIDRIOS[(unidad21_VIDRIOS['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_VIDRIOS['COD_RAMO'] == 19)]['TOTAL']).sum()
u146_005 = u146_005 if u146_005 != 0 else 0
ws.cell(row=146, column=21, value=float(u146_005))

# Primas cedidas contratos facultativos interior
u147_010 = abs(unidad21_VIDRIOS[(unidad21_VIDRIOS['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_VIDRIOS['COD_RAMO'] == 19)]['TOTAL']).sum()
u147_010 = u147_010 if u147_010 != 0 else 0
ws.cell(row=147, column=21, value=float(u147_010))

# Parte reaseguro al interior
u148_015 = abs(unidad21_VIDRIOS[(unidad21_VIDRIOS['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_VIDRIOS['COD_RAMO'] == 19)]['TOTAL']).sum()
u148_015 = u148_015 if u148_015 != 0 else 0
ws.cell(row=148, column=21, value=float(u148_015))

u149_020 = abs(unidad21_VIDRIOS[(unidad21_VIDRIOS['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_VIDRIOS['COD_RAMO'] == 19)]['TOTAL']).sum()
u149_020 = u149_020 if u149_020 != 0 else 0
ws.cell(row=149, column=21, value=float(u149_020))

# Calcular ae150_999
u150_999 = abs(u146_005 + u147_010 - u148_015 - u149_020)
ws.cell(row=150, column=21, value=float(u150_999))

<Cell 'Hoja1'.U150>

#  20  CRÉDITO COMERCIAL UNIDAD 21

In [593]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CREDITOCOMERCIAL = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CREDITOCOMERCIAL_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
v146_005 = abs(unidad21_CREDITOCOMERCIAL[(unidad21_CREDITOCOMERCIAL['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CREDITOCOMERCIAL['COD_RAMO'] == 20)]['TOTAL']).sum()
v146_005 = v146_005 if v146_005 != 0 else 0
ws.cell(row=146, column=22, value=float(v146_005))

# Primas cedidas contratos facultativos interior
v147_010 = abs(unidad21_CREDITOCOMERCIAL[(unidad21_CREDITOCOMERCIAL['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CREDITOCOMERCIAL['COD_RAMO'] == 20)]['TOTAL']).sum()
v147_010 = v147_010 if v147_010 != 0 else 0
ws.cell(row=147, column=22, value=float(v147_010))

# Parte reaseguro al interior
v148_015 = abs(unidad21_CREDITOCOMERCIAL[(unidad21_CREDITOCOMERCIAL['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CREDITOCOMERCIAL['COD_RAMO'] == 20)]['TOTAL']).sum()
v148_015 = v148_015 if v148_015 != 0 else 0
ws.cell(row=148, column=22, value=float(v148_015))

v149_020 = abs(unidad21_CREDITOCOMERCIAL[(unidad21_CREDITOCOMERCIAL['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CREDITOCOMERCIAL['COD_RAMO'] == 20)]['TOTAL']).sum()
v149_020 = v149_020 if v149_020 != 0 else 0
ws.cell(row=149, column=22, value=float(v149_020))

# Calcular ae150_999
v150_999 = abs(v146_005 + v147_010 - v148_015 - v149_020)
ws.cell(row=150, column=22, value=float(v150_999))

<Cell 'Hoja1'.V150>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 21

In [594]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CREDITOEXPORTACION = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CREDITOEXPORTACION_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
w146_005 = abs(unidad21_CREDITOEXPORTACION[(unidad21_CREDITOEXPORTACION['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CREDITOEXPORTACION['COD_RAMO'] == 21)]['TOTAL']).sum()
w146_005 = w146_005 if w146_005 != 0 else 0
ws.cell(row=146, column=23, value=float(w146_005))

# Primas cedidas contratos facultativos interior
w147_010 = abs(unidad21_CREDITOEXPORTACION[(unidad21_CREDITOEXPORTACION['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CREDITOEXPORTACION['COD_RAMO'] == 21)]['TOTAL']).sum()
w147_010 = w147_010 if w147_010 != 0 else 0
ws.cell(row=147, column=23, value=float(w147_010))

# Parte reaseguro al interior
w148_015 = abs(unidad21_CREDITOEXPORTACION[(unidad21_CREDITOEXPORTACION['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CREDITOEXPORTACION['COD_RAMO'] == 21)]['TOTAL']).sum()
w148_015 = w148_015 if w148_015 != 0 else 0
ws.cell(row=148, column=23, value=float(w148_015))

w149_020 = abs(unidad21_CREDITOEXPORTACION[(unidad21_CREDITOEXPORTACION['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CREDITOEXPORTACION['COD_RAMO'] == 21)]['TOTAL']).sum()
w149_020 = w149_020 if w149_020 != 0 else 0
ws.cell(row=149, column=23, value=float(w149_020))

# Calcular ae150_999
w150_999 = abs(w146_005 + w147_010 - w148_015 - w149_020)
ws.cell(row=150, column=23, value=float(w150_999))

<Cell 'Hoja1'.W150>

#  24  DESEMPLEO UNIDAD 21

In [595]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_DESEMPLEO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_DESEMPLEO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
z146_005 = abs(unidad21_DESEMPLEO[(unidad21_DESEMPLEO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_DESEMPLEO['COD_RAMO'] == 24)]['TOTAL']).sum()
z146_005 = z146_005 if z146_005 != 0 else 0
ws.cell(row=146, column=26, value=float(z146_005))

# Primas cedidas contratos facultativos interior
z147_010 = abs(unidad21_DESEMPLEO[(unidad21_DESEMPLEO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_DESEMPLEO['COD_RAMO'] == 24)]['TOTAL']).sum()
z147_010 = z147_010 if z147_010 != 0 else 0
ws.cell(row=147, column=26, value=float(z147_010))

# Parte reaseguro al interior
z148_015 = abs(unidad21_DESEMPLEO[(unidad21_DESEMPLEO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_DESEMPLEO['COD_RAMO'] == 24)]['TOTAL']).sum()
z148_015 = z148_015 if z148_015 != 0 else 0
ws.cell(row=148, column=26, value=float(z148_015))

z149_020 = abs(unidad21_DESEMPLEO[(unidad21_DESEMPLEO['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_DESEMPLEO['COD_RAMO'] == 24)]['TOTAL']).sum()
z149_020 = z149_020 if z149_020 != 0 else 0
ws.cell(row=149, column=26, value=float(z149_020))

# Calcular ae150_999
z150_999 = abs(z146_005 + z147_010 - z148_015 - w149_020)
ws.cell(row=150, column=26, value=float(z150_999))

<Cell 'Hoja1'.Z150>

#  25  HOGAR UNIDAD 21

In [596]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_HOGAR = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_HOGAR_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
aa146_005 = abs(unidad21_HOGAR[(unidad21_HOGAR['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_HOGAR['COD_RAMO'] == 25)]['TOTAL']).sum()
aa146_005 = aa146_005 if aa146_005 != 0 else 0
ws.cell(row=146, column=27, value=float(aa146_005))

# Primas cedidas contratos facultativos interior
aa147_010 = abs(unidad21_HOGAR[(unidad21_HOGAR['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_HOGAR['COD_RAMO'] == 25)]['TOTAL']).sum()
aa147_010 = aa147_010 if aa147_010 != 0 else 0
ws.cell(row=147, column=27, value=float(aa147_010))

# Parte reaseguro al interior
aa148_015 = abs(unidad21_HOGAR[(unidad21_HOGAR['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_HOGAR['COD_RAMO'] == 25)]['TOTAL']).sum()
aa148_015 = aa148_015 if aa148_015 != 0 else 0
ws.cell(row=148, column=27, value=float(aa148_015))

cuentas = ["6232101061", "6232101121"]
aa149_020 = unidad21_HOGAR_a[(unidad21_HOGAR_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_HOGAR_a['COD_RAMO'] == 25)]['TOTAL'].sum()
ws.cell(row=149, column=27, value=float(aa149_020))

# Calcular ae150_999
aa150_999 = abs(aa146_005 + aa147_010 - aa148_015 - aa149_020)
ws.cell(row=150, column=27, value=float(aa150_999))

<Cell 'Hoja1'.AA150>

# 42 AGROPECUARIO UNIDAD 21

In [597]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_AGROPECUARIO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_AGROPECUARIO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ar146_005 = abs(unidad21_AGROPECUARIO[(unidad21_AGROPECUARIO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_AGROPECUARIO['COD_RAMO'] == 42)]['TOTAL']).sum()
ar146_005 = ar146_005 if ar146_005 != 0 else 0
ws.cell(row=146, column=44, value=float(ar146_005))

# Primas cedidas contratos facultativos interior
ar147_010 = abs(unidad21_AGROPECUARIO[(unidad21_AGROPECUARIO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_AGROPECUARIO['COD_RAMO'] == 42)]['TOTAL']).sum()
ar147_010 = ar147_010 if ar147_010 != 0 else 0
ws.cell(row=147, column=44, value=float(ar147_010))

# Parte reaseguro al interior
ar148_015 = abs(unidad21_AGROPECUARIO[(unidad21_AGROPECUARIO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_AGROPECUARIO['COD_RAMO'] == 42)]['TOTAL']).sum()
ar148_015 = ar148_015 if ar148_015 != 0 else 0
ws.cell(row=148, column=44, value=float(ar148_015))

cuentas = ["6232101061", "6232101121","6232101221","6232101251"]
ar149_020 = unidad21_AGROPECUARIO_a[(unidad21_AGROPECUARIO_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_AGROPECUARIO_a['COD_RAMO'] == 42)]['TOTAL'].sum()
ws.cell(row=149, column=44, value=float(ar149_020))

# Calcular ae150_999
ar150_999 = abs(ar146_005 + ar147_010 - ar148_015 - ar149_020)
ws.cell(row=150, column=44, value=float(ar150_999))

<Cell 'Hoja1'.AR150>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 21

In [598]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_OPERNOASIGNABLES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
as146_005 = abs(unidad21_OPERNOASIGNABLES[(unidad21_OPERNOASIGNABLES['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_OPERNOASIGNABLES['COD_RAMO'] == 43)]['TOTAL']).sum()
as146_005 = as146_005 if as146_005 != 0 else 0
ws.cell(row=146, column=45, value=float(as146_005))

# Primas cedidas contratos facultativos interior
as147_010 = abs(unidad21_OPERNOASIGNABLES[(unidad21_OPERNOASIGNABLES['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_OPERNOASIGNABLES['COD_RAMO'] == 43)]['TOTAL']).sum()
as147_010 = as147_010 if as147_010 != 0 else 0
ws.cell(row=147, column=45, value=float(as147_010))

# Parte reaseguro al interior
as148_015 = abs(unidad21_OPERNOASIGNABLES[(unidad21_OPERNOASIGNABLES['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_OPERNOASIGNABLES['COD_RAMO'] == 43)]['TOTAL']).sum()
as148_015 = as148_015 if as148_015 != 0 else 0
ws.cell(row=148, column=45, value=float(as148_015))

as149_020 = abs(unidad21_OPERNOASIGNABLES[(unidad21_OPERNOASIGNABLES['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_OPERNOASIGNABLES['COD_RAMO'] == 43)]['TOTAL']).sum()
as149_020 = as149_020 if as149_020 != 0 else 0
ws.cell(row=149, column=45, value=float(as149_020))

# Calcular ae150_999
as150_999 = abs(as146_005 + as147_010 - as148_015 - as149_020)
ws.cell(row=150, column=45, value=float(as150_999))

<Cell 'Hoja1'.AS150>

# 44 DECENAL UNIDAD 21

In [599]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_DECENAL = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_DECENAL_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
at146_005 = abs(unidad21_DECENAL[(unidad21_DECENAL['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_DECENAL['COD_RAMO'] == 44)]['TOTAL']).sum()
at146_005 = at146_005 if at146_005 != 0 else 0
ws.cell(row=146, column=46, value=float(at146_005))

# Primas cedidas contratos facultativos interior
at147_010 = abs(unidad21_DECENAL[(unidad21_DECENAL['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_DECENAL['COD_RAMO'] == 44)]['TOTAL']).sum()
at147_010 = at147_010 if at147_010 != 0 else 0
ws.cell(row=147, column=46, value=float(at147_010))

# Parte reaseguro al interior
at148_015 = abs(unidad21_DECENAL[(unidad21_DECENAL['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_DECENAL['COD_RAMO'] == 44)]['TOTAL']).sum()
at148_015 = at148_015 if at148_015 != 0 else 0
ws.cell(row=148, column=46, value=float(at148_015))

at149_020 = abs(unidad21_DECENAL[(unidad21_DECENAL['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_DECENAL['COD_RAMO'] == 44)]['TOTAL']).sum()
at149_020 = at149_020 if at149_020 != 0 else 0
ws.cell(row=149, column=46, value=float(at149_020))

# Calcular ae150_999
at150_999 = abs(at146_005 + at147_010 - at148_015 - at149_020)
ws.cell(row=150, column=46, value=float(at150_999))

<Cell 'Hoja1'.AT150>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 21

In [600]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
au146_005 = abs(unidad21_CUMPLIMIENTOESTATALES[(unidad21_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au146_005 = au146_005 if au146_005 != 0 else 0
ws.cell(row=146, column=47, value=float(au146_005))

# Primas cedidas contratos facultativos interior
au147_010 = abs(unidad21_CUMPLIMIENTOESTATALES[(unidad21_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au147_010 = au147_010 if au147_010 != 0 else 0
ws.cell(row=147, column=47, value=float(au147_010))

# Parte reaseguro al interior
au148_015 = abs(unidad21_CUMPLIMIENTOESTATALES[(unidad21_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au148_015 = au148_015 if au148_015 != 0 else 0
ws.cell(row=148, column=47, value=float(au148_015))

au149_020 = abs(unidad21_CUMPLIMIENTOESTATALES[(unidad21_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au149_020 = au149_020 if au149_020 != 0 else 0
ws.cell(row=149, column=47, value=float(au149_020))

# Calcular ae150_999
au150_999 = abs(au146_005 + au147_010 - au148_015 - au149_020)
ws.cell(row=150, column=47, value=float(au150_999))

<Cell 'Hoja1'.AU150>

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 21

In [601]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOEMPRESASSERV= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
av146_005 = abs(unidad21_CUMPLIMIENTOEMPRESASSERV[(unidad21_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av146_005 = av146_005 if av146_005 != 0 else 0
ws.cell(row=146, column=48, value=float(av146_005))

# Primas cedidas contratos facultativos interior
av147_010 = abs(unidad21_CUMPLIMIENTOEMPRESASSERV[(unidad21_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av147_010 = av147_010 if av147_010 != 0 else 0
ws.cell(row=147, column=48, value=float(av147_010))

# Parte reaseguro al interior
av148_015 = abs(unidad21_CUMPLIMIENTOEMPRESASSERV[(unidad21_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av148_015 = av148_015 if av148_015 != 0 else 0
ws.cell(row=148, column=48, value=float(av148_015))

av149_020 = abs(unidad21_CUMPLIMIENTOEMPRESASSERV[(unidad21_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av149_020 = av149_020 if av149_020 != 0 else 0
ws.cell(row=149, column=48, value=float(av149_020))

# Calcular ae150_999
av150_999 = abs(av146_005 + av147_010 - av148_015 - av149_020)
ws.cell(row=150, column=48, value=float(av150_999))

<Cell 'Hoja1'.AV150>

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 21

In [602]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOEMPRESASINDUSTRI= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
aw146_005 = abs(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw146_005 = aw146_005 if aw146_005 != 0 else 0
ws.cell(row=146, column=49, value=float(aw146_005))

# Primas cedidas contratos facultativos interior
aw147_010 = abs(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw147_010 = aw147_010 if aw147_010 != 0 else 0
ws.cell(row=147, column=49, value=float(aw147_010))

# Parte reaseguro al interior
aw148_015 = abs(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw148_015 = aw148_015 if aw148_015 != 0 else 0
ws.cell(row=148, column=49, value=float(aw148_015))

aw149_020 = abs(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw149_020 = aw149_020 if aw149_020 != 0 else 0
ws.cell(row=149, column=49, value=float(aw149_020))

# Calcular ae150_999
aw150_999 = abs(aw146_005 + aw147_010 - aw148_015 - aw149_020)
ws.cell(row=150, column=49, value=float(aw150_999))

<Cell 'Hoja1'.AW150>

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 21

In [603]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOEMPRESASDISPOLEG= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ax146_005 = abs(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax146_005 = ax146_005 if ax146_005 != 0 else 0
ws.cell(row=146, column=50, value=float(ax146_005))

# Primas cedidas contratos facultativos interior
ax147_010 = abs(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax147_010 = ax147_010 if ax147_010 != 0 else 0
ws.cell(row=147, column=50, value=float(ax147_010))

# Parte reaseguro al interior
ax148_015 = abs(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax148_015 = ax148_015 if ax148_015 != 0 else 0
ws.cell(row=148, column=50, value=float(ax148_015))

ax149_020 = abs(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax149_020 = ax149_020 if ax149_020 != 0 else 0
ws.cell(row=149, column=50, value=float(ax149_020))

# Calcular ae150_999
ax150_999 = abs(ax146_005 + ax147_010 - ax148_015 - ax149_020)
ws.cell(row=150, column=50, value=float(ax150_999))

<Cell 'Hoja1'.AX150>

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 21

In [604]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOCAUCIONES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ay146_005 = abs(unidad21_CUMPLIMIENTOCAUCIONES[(unidad21_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay146_005 = ay146_005 if ay146_005 != 0 else 0
ws.cell(row=146, column=51, value=float(ay146_005))

# Primas cedidas contratos facultativos interior
ay147_010 = abs(unidad21_CUMPLIMIENTOCAUCIONES[(unidad21_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay147_010 = ay147_010 if ay147_010 != 0 else 0
ws.cell(row=147, column=51, value=float(ay147_010))

# Parte reaseguro al interior
ay148_015 = abs(unidad21_CUMPLIMIENTOCAUCIONES[(unidad21_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay148_015 = ay148_015 if ay148_015 != 0 else 0
ws.cell(row=148, column=51, value=float(ax148_015))

ay149_020 = abs(unidad21_CUMPLIMIENTOCAUCIONES[(unidad21_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay149_020 = ay149_020 if ay149_020 != 0 else 0
ws.cell(row=149, column=51, value=float(ay149_020))

# Calcular ae150_999
ay150_999 = abs(ay146_005 + ay147_010 - ay148_015 - ay149_020)
ws.cell(row=150, column=51, value=float(ay150_999))

<Cell 'Hoja1'.AY150>

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 21

In [605]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOARRENDAMIENTO= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
az146_005 = abs(unidad21_CUMPLIMIENTOARRENDAMIENTO[(unidad21_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az146_005 = az146_005 if az146_005 != 0 else 0
ws.cell(row=146, column=52, value=float(az146_005))

# Primas cedidas contratos facultativos interior
az147_010 = abs(unidad21_CUMPLIMIENTOARRENDAMIENTO[(unidad21_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az147_010 = az147_010 if az147_010 != 0 else 0
ws.cell(row=147, column=52, value=float(az147_010))

# Parte reaseguro al interior
az148_015 = abs(unidad21_CUMPLIMIENTOARRENDAMIENTO[(unidad21_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az148_015 = az148_015 if az148_015 != 0 else 0
ws.cell(row=148, column=52, value=float(az148_015))

az149_020 = abs(unidad21_CUMPLIMIENTOARRENDAMIENTO[(unidad21_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az149_020 = az149_020 if az149_020 != 0 else 0
ws.cell(row=149, column=52, value=float(az149_020))

# Calcular ae150_999
az150_999 = abs(az146_005 + az147_010 - az148_015 - az149_020)
ws.cell(row=150, column=52, value=float(az150_999))

<Cell 'Hoja1'.AZ150>

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 21

In [606]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_CUMPLIMIENTOPARTICULARES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ba146_005 = abs(unidad21_CUMPLIMIENTOPARTICULARES[(unidad21_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba146_005 = ba146_005 if ba146_005 != 0 else 0
ws.cell(row=146, column=53, value=float(ba146_005))

# Primas cedidas contratos facultativos interior
ba147_010 = abs(unidad21_CUMPLIMIENTOPARTICULARES[(unidad21_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba147_010 = ba147_010 if ba147_010 != 0 else 0
ws.cell(row=147, column=53, value=float(ba147_010))

# Parte reaseguro al interior
ba148_015 = abs(unidad21_CUMPLIMIENTOPARTICULARES[(unidad21_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas contratos automáticos exterior)") &
                                   (unidad21_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba148_015 = ba148_015 if ba148_015 != 0 else 0
ws.cell(row=148, column=53, value=float(ba148_015))

ba149_020 = abs(unidad21_CUMPLIMIENTOPARTICULARES[(unidad21_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas contratos facultativos exterior)") &
                                   (unidad21_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba149_020 = ba149_020 if ba149_020 != 0 else 0
ws.cell(row=149, column=53, value=float(ba149_020))

# Calcular ae150_999
ba150_999 = abs(ba146_005 + ba147_010 - ba148_015 - ba149_020)
ws.cell(row=150, column=53, value=float(ba150_999))

<Cell 'Hoja1'.BA150>

#  1   TOTAL UNIDAD 21

In [607]:
c146_005 = sum([ws.cell(row=146, column=col).value if ws.cell(row=146, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=146, column=3).value = c146_005  

c147_010 = sum([ws.cell(row=147, column=col).value if ws.cell(row=147, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=147, column=3).value = c147_010 

c148_015 = sum([ws.cell(row=148, column=col).value if ws.cell(row=148, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=148, column=3).value = c148_015 

c149_020 = sum([ws.cell(row=149, column=col).value if ws.cell(row=149, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=149, column=3).value = c149_020  

# Calcular el total general según la fórmula dada

c150_999 = abs(c146_005 + c147_010 - c148_015 - c149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=3).value = c150_999

#  2   SUBTOTAL - RAMOS UNIDAD 21

In [608]:
d146_005 = sum([ws.cell(row=146, column=col).value if ws.cell(row=146, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=146, column=4).value = d146_005  

d147_010 = sum([ws.cell(row=147, column=col).value if ws.cell(row=147, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=147, column=4).value = d147_010 

d148_015 = sum([ws.cell(row=148, column=col).value if ws.cell(row=148, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=148, column=4).value = d148_015 

d149_020 = sum([ws.cell(row=149, column=col).value if ws.cell(row=149, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=149, column=4).value = d149_020  

# Calcular el total general según la fórmula dada

d150_999 = abs(d146_005 + d147_010 - d148_015 - d149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=4).value = d150_999

## UNIDAD DE CAPTURA 23

#  3   AUTOMOVILES UNIDAD 23

In [610]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_AUTOMOVILES = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_AUTOMOVILES['TOTAL'] = unidad23_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",3, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",3, 155),
    ("Primas cedidas interior y exterior terremoto",3, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",3, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e154_005 = get_value(unidad23_AUTOMOVILES, "Primas emitidas ramo de terremoto", 3)
e155_010 = get_value(unidad23_AUTOMOVILES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 3)
e156_015 = get_value(unidad23_AUTOMOVILES, "Primas cedidas interior y exterior terremoto", 3)
e157_020 = get_value(unidad23_AUTOMOVILES, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 3)


# Calcular ab94_999 usando la fórmula especificada
e158_999 = (e154_005 + e155_010 - e156_015 - e157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=5, value=e158_999)

<Cell 'Hoja1'.E158>

#  4   SOAT UNIDAD 23

In [611]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_SOAT = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_SOAT['TOTAL'] = unidad23_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",4, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",4, 155),
    ("Primas cedidas interior y exterior terremoto",4, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",4, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f154_005 = get_value(unidad23_SOAT, "Primas emitidas ramo de terremoto", 4)
f155_010 = get_value(unidad23_SOAT, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 4)
f156_015 = get_value(unidad23_SOAT, "Primas cedidas interior y exterior terremoto", 4)
f157_020 = get_value(unidad23_SOAT, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 4)


# Calcular ab94_999 usando la fórmula especificada
f158_999 = (f154_005 + f155_010 - f156_015 - f157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=6, value=f158_999)

<Cell 'Hoja1'.F158>

#  5   CUMPLIMIENTO UNIDAD 23

In [612]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_CUMPLIMIENTO['TOTAL'] = unidad23_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",5, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",5, 155),
    ("Primas cedidas interior y exterior terremoto",5, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",5, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
g154_005 = get_value(unidad23_CUMPLIMIENTO, "Primas emitidas ramo de terremoto", 5)
g155_010 = get_value(unidad23_CUMPLIMIENTO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 5)
g156_015 = get_value(unidad23_CUMPLIMIENTO, "Primas cedidas interior y exterior terremoto", 5)
g157_020 = get_value(unidad23_CUMPLIMIENTO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 5)


# Calcular ab94_999 usando la fórmula especificada
g158_999 = (g154_005 + g155_010 - g156_015 - g157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=7, value=g158_999)

<Cell 'Hoja1'.G158>

#  6   RESPONSABILIDAD CIVIL UNIDAD 23

In [613]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_RESPONSABILIDADCI['TOTAL'] = unidad23_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",6, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",6, 155),
    ("Primas cedidas interior y exterior terremoto",6, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",6, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
h154_005 = get_value(unidad23_RESPONSABILIDADCI, "Primas emitidas ramo de terremoto", 6)
h155_010 = get_value(unidad23_RESPONSABILIDADCI, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 6)
h156_015 = get_value(unidad23_RESPONSABILIDADCI, "Primas cedidas interior y exterior terremoto", 6)
h157_020 = get_value(unidad23_RESPONSABILIDADCI, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 6)


# Calcular ab94_999 usando la fórmula especificada
h158_999 = (h154_005 + h155_010 - h156_015 - h157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=8, value=h158_999)

<Cell 'Hoja1'.H158>

#  7   INCENDIO UNIDAD 23

In [614]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_INCENDIO = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 7)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_INCENDIO['TOTAL'] = unidad23_INCENDIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",7, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",7, 155),
    ("Primas cedidas interior y exterior terremoto",7, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",7, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_INCENDIO, concept, ramo, 9, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
i154_005 = get_value(unidad23_INCENDIO, "Primas emitidas ramo de terremoto", 7)
i155_010 = get_value(unidad23_INCENDIO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 7)
i156_015 = get_value(unidad23_INCENDIO, "Primas cedidas interior y exterior terremoto", 7)
i157_020 = get_value(unidad23_INCENDIO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 7)


# Calcular ab94_999 usando la fórmula especificada
i158_999 = (i154_005 + i155_010 - i156_015 - i157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=9, value=i158_999)

<Cell 'Hoja1'.I158>

#  8   TERREMOTO UNIDAD 23

In [615]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_TERREMOTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_TERREMOTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior

cuentas = ["5114001011", "5114001021", "5114001061", "5114001071", "5114001081", "5114001091"]
j154_005 = unidad23_TERREMOTO_a[(unidad23_TERREMOTO_a['CUENTA'].isin(cuentas)) & 
                                (unidad23_TERREMOTO_a['COD_RAMO'] == 8)]['TOTAL'].sum()
j154_005 = abs(float(j154_005))
ws.cell(row=154, column=10, value=j154_005)

# Primas cedidas contratos facultativos interior
j155_010 = abs(unidad23_TERREMOTO[(unidad23_TERREMOTO['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_TERREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j155_010 = j155_010 if j155_010 != 0 else 0
ws.cell(row=155, column=10, value=float(j155_010))

cuentas = ["6232101061", "6232101121"]
j156_015 = unidad23_TERREMOTO_a[(unidad23_TERREMOTO_a['CUENTA'].isin(cuentas)) &
                                     (unidad23_TERREMOTO_a['COD_RAMO'] == 8)]['TOTAL'].sum()
ws.cell(row=157, column=10, value=float(j156_015))

# Parte reaseguro al interior
j157_020 = abs(unidad23_TERREMOTO[(unidad23_TERREMOTO['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_TERREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j157_020 = j157_020 if j157_020 != 0 else 0
ws.cell(row=156, column=10, value=float(j157_020))

# Calcular 
j158_999 = abs(j154_005 + j155_010 - j156_015 + j157_020)
ws.cell(row=158, column=10, value=float(j158_999))

<Cell 'Hoja1'.J158>

#  9   SUSTRACCION UNIDAD 23

In [617]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_SUSTRACCION = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_SUSTRACCION['TOTAL'] = unidad23_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",9, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",9, 155),
    ("Primas cedidas interior y exterior terremoto",9, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",9, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
k154_005 = get_value(unidad23_SUSTRACCION, "Primas emitidas ramo de terremoto", 9)
k155_010 = get_value(unidad23_SUSTRACCION, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 9)
k156_015 = get_value(unidad23_SUSTRACCION, "Primas cedidas interior y exterior terremoto", 9)
k157_020 = get_value(unidad23_SUSTRACCION, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 9)


# Calcular ab94_999 usando la fórmula especificada
k158_999 = (k154_005 + k155_010 - k156_015 - k157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=11, value=k158_999)

<Cell 'Hoja1'.K158>

#  10  TRANSPORTE UNIDAD 23

In [618]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_TRANSPORTE = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_TRANSPORTE['TOTAL'] = unidad23_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",10, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",10, 155),
    ("Primas cedidas interior y exterior terremoto",10, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",10, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
l154_005 = get_value(unidad23_TRANSPORTE, "Primas emitidas ramo de terremoto",10)
l155_010 = get_value(unidad23_TRANSPORTE, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 10)
l156_015 = get_value(unidad23_TRANSPORTE, "Primas cedidas interior y exterior terremoto", 10)
l157_020 = get_value(unidad23_TRANSPORTE, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 10)


# Calcular ab94_999 usando la fórmula especificada
l158_999 = (l154_005 + l155_010 - l156_015 - l157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=12, value=l158_999)

<Cell 'Hoja1'.L158>

#  11  CORRIENTE DÉBIL UNIDAD 23

In [619]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_CORRIENTEDEBIL = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_CORRIENTEDEBIL['TOTAL'] = unidad23_CORRIENTEDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",11, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",11, 155),
    ("Primas cedidas interior y exterior terremoto",11, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",11, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_CORRIENTEDEBIL, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
m154_005 = get_value(unidad23_CORRIENTEDEBIL, "Primas emitidas ramo de terremoto",11)
m155_010 = get_value(unidad23_CORRIENTEDEBIL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 11)
m156_015 = get_value(unidad23_CORRIENTEDEBIL, "Primas cedidas interior y exterior terremoto", 11)
m157_020 = get_value(unidad23_CORRIENTEDEBIL, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 11)


# Calcular ab94_999 usando la fórmula especificada
m158_999 = (m154_005 + m155_010 - m156_015 - m157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=13, value=m158_999)

<Cell 'Hoja1'.M158>

#  12  TODO RIESGO CONTRATISTA UNIDAD 23

In [620]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_TODORIESGOCONTRATISTA['TOTAL'] = unidad23_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",12, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",12, 155),
    ("Primas cedidas interior y exterior terremoto",12, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",12, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
n154_005 = get_value(unidad23_CORRIENTEDEBIL, "Primas emitidas ramo de terremoto",12)
n155_010 = get_value(unidad23_CORRIENTEDEBIL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 12)
n156_015 = get_value(unidad23_CORRIENTEDEBIL, "Primas cedidas interior y exterior terremoto", 12)
n157_020 = get_value(unidad23_CORRIENTEDEBIL, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 12)


# Calcular ab94_999 usando la fórmula especificada
n158_999 = (n154_005 + n155_010 - n156_015 - n157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=14, value=n158_999)

<Cell 'Hoja1'.N158>

#  13  MANEJO UNIDAD 23

In [621]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_MANEJO = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_MANEJO['TOTAL'] = unidad23_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",13, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",13, 155),
    ("Primas cedidas interior y exterior terremoto",13, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",13, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
o154_005 = get_value(unidad23_MANEJO, "Primas emitidas ramo de terremoto",13)
o155_010 = get_value(unidad23_MANEJO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 13)
o156_015 = get_value(unidad23_MANEJO, "Primas cedidas interior y exterior terremoto", 13)
o157_020 = get_value(unidad23_MANEJO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 13)


# Calcular ab94_999 usando la fórmula especificada
o158_999 = (o154_005 + o155_010 - o156_015 - o157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=15, value=o158_999)

<Cell 'Hoja1'.O158>

#  14  LUCRO CESANTE UNIDAD 23

In [622]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_LUCROCESANTE = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_LUCROCESANTE['TOTAL'] = unidad23_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",14, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",14, 155),
    ("Primas cedidas interior y exterior terremoto",14, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",14, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
p154_005 = get_value(unidad23_LUCROCESANTE, "Primas emitidas ramo de terremoto",14)
p155_010 = get_value(unidad23_LUCROCESANTE, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 14)
p156_015 = get_value(unidad23_LUCROCESANTE, "Primas cedidas interior y exterior terremoto", 14)
p157_020 = get_value(unidad23_LUCROCESANTE, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 14)


# Calcular ab94_999 usando la fórmula especificada
p158_999 = (p154_005 + p155_010 - p156_015 - p157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=16, value=p158_999)

<Cell 'Hoja1'.P158>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 23

In [623]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_MONTAJEYROTURA  = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_MONTAJEYROTURA['TOTAL'] = unidad23_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",15, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",15, 155),
    ("Primas cedidas interior y exterior terremoto",15, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",15, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
q154_005 = get_value(unidad23_MONTAJEYROTURA, "Primas emitidas ramo de terremoto",15)
q155_010 = get_value(unidad23_MONTAJEYROTURA, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 15)
q156_015 = get_value(unidad23_MONTAJEYROTURA, "Primas cedidas interior y exterior terremoto", 15)
q157_020 = get_value(unidad23_MONTAJEYROTURA, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 15)


# Calcular ab94_999 usando la fórmula especificada
q158_999 = (q154_005 + q155_010 - q156_015 - q157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=17, value=q158_999)

<Cell 'Hoja1'.Q158>

#  16  AVIACIÓN UNIDAD 23

In [624]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_AVIACION = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_AVIACION['TOTAL'] = unidad23_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",16, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",16, 155),
    ("Primas cedidas interior y exterior terremoto",16, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",16, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
r154_005 = get_value(unidad23_AVIACION, "Primas emitidas ramo de terremoto",16)
r155_010 = get_value(unidad23_AVIACION, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 16)
r156_015 = get_value(unidad23_AVIACION, "Primas cedidas interior y exterior terremoto", 16)
r157_020 = get_value(unidad23_AVIACION, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 16)


# Calcular ab94_999 usando la fórmula especificada
r158_999 = (r154_005 + r155_010 - r156_015 - r157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=18, value=r158_999)

<Cell 'Hoja1'.R158>

#  17  NAVEGACIÓN Y CASCO UNIDAD 23

In [625]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_NAVEGACION = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_NAVEGACION['TOTAL'] = unidad23_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",17, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",17, 155),
    ("Primas cedidas interior y exterior terremoto",17, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",17, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
s154_005 = get_value(unidad23_NAVEGACION, "Primas emitidas ramo de terremoto",17)
s155_010 = get_value(unidad23_NAVEGACION, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 17)
s156_015 = get_value(unidad23_NAVEGACION, "Primas cedidas interior y exterior terremoto", 17)
s157_020 = get_value(unidad23_NAVEGACION, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 17)


# Calcular ab94_999 usando la fórmula especificada
s158_999 = (s154_005 + s155_010 - s156_015 - s157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=19, value=s158_999)

<Cell 'Hoja1'.S158>

#  18  MINAS Y PETRÓLEOS UNIDAD 23

In [626]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_MINAS = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_MINAS['TOTAL'] = unidad23_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",18, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",18, 155),
    ("Primas cedidas interior y exterior terremoto",18, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",18, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
t154_005 = get_value(unidad23_MINAS, "Primas emitidas ramo de terremoto",18)
t155_010 = get_value(unidad23_MINAS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 18)
t156_015 = get_value(unidad23_MINAS, "Primas cedidas interior y exterior terremoto", 18)
t157_020 = get_value(unidad23_MINAS, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 18)


# Calcular ab94_999 usando la fórmula especificada
t158_999 = (t154_005 + t155_010 - t156_015 - t157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=20, value=t158_999)

<Cell 'Hoja1'.T158>

#  19  VIDRIOS UNIDAD 23

In [627]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_VIDRIOS = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_VIDRIOS['TOTAL'] = unidad23_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",19, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",19, 155),
    ("Primas cedidas interior y exterior terremoto",19, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",19, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
u154_005 = get_value(unidad23_VIDRIOS, "Primas emitidas ramo de terremoto",19)
u155_010 = get_value(unidad23_VIDRIOS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 19)
u156_015 = get_value(unidad23_VIDRIOS, "Primas cedidas interior y exterior terremoto", 19)
u157_020 = get_value(unidad23_VIDRIOS, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 19)


# Calcular ab94_999 usando la fórmula especificada
u158_999 = (u154_005 + u155_010 - u156_015 - u157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=21, value=u158_999)

<Cell 'Hoja1'.U158>

#  20  CRÉDITO COMERCIAL UNIDAD 23

In [628]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_CREDITOCOMERCIAL['TOTAL'] = unidad23_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",20, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",20, 155),
    ("Primas cedidas interior y exterior terremoto",20, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",20, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
v154_005 = get_value(unidad23_CREDITOCOMERCIAL, "Primas emitidas ramo de terremoto",20)
v155_010 = get_value(unidad23_CREDITOCOMERCIAL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 20)
v156_015 = get_value(unidad23_CREDITOCOMERCIAL, "Primas cedidas interior y exterior terremoto", 20)
v157_020 = get_value(unidad23_CREDITOCOMERCIAL, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 20)


# Calcular ab94_999 usando la fórmula especificada
v158_999 = (v154_005 + v155_010 - v156_015 - v157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=22, value=v158_999)

<Cell 'Hoja1'.V158>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 23


In [629]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_CREDITOEXPORTACION['TOTAL'] = unidad23_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",21, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",21, 155),
    ("Primas cedidas interior y exterior terremoto",21, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",21, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
w154_005 = get_value(unidad23_CREDITOEXPORTACION, "Primas emitidas ramo de terremoto",21)
w155_010 = get_value(unidad23_CREDITOEXPORTACION, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 21)
w156_015 = get_value(unidad23_CREDITOEXPORTACION, "Primas cedidas interior y exterior terremoto", 21)
w157_020 = get_value(unidad23_CREDITOEXPORTACION, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 21)


# Calcular ab94_999 usando la fórmula especificada
w158_999 = (w154_005 + w155_010 - w156_015 - w157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=23, value=w158_999)

<Cell 'Hoja1'.W158>

#  24  DESEMPLEO UNIDAD 23

In [630]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_DESEMPLEO = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_DESEMPLEO['TOTAL'] = unidad23_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",24, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",24, 155),
    ("Primas cedidas interior y exterior terremoto",24, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",24, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
z154_005 = get_value(unidad23_DESEMPLEO, "Primas emitidas ramo de terremoto",24)
z155_010 = get_value(unidad23_DESEMPLEO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 24)
z156_015 = get_value(unidad23_DESEMPLEO, "Primas cedidas interior y exterior terremoto", 24)
z157_020 = get_value(unidad23_DESEMPLEO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 24)


# Calcular ab94_999 usando la fórmula especificada
z158_999 = (z154_005 + z155_010 - z156_015 - z157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=26, value=z158_999)

<Cell 'Hoja1'.Z158>

#  25  HOGAR UNIDAD 23

In [631]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_HOGAR = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 25)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_HOGAR['TOTAL'] = unidad23_HOGAR['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",25, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",25, 155),
    ("Primas cedidas interior y exterior terremoto",25, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",25, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_HOGAR, concept, ramo, 27, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aa154_005 = get_value(unidad23_HOGAR, "Primas emitidas ramo de terremoto",25)
aa155_010 = get_value(unidad23_HOGAR, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 25)
aa156_015 = get_value(unidad23_HOGAR, "Primas cedidas interior y exterior terremoto", 25)
aa157_020 = get_value(unidad23_HOGAR, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 25)


# Calcular ab94_999 usando la fórmula especificada
aa158_999 = (aa154_005 + aa155_010 - aa156_015 - aa157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=27, value=aa158_999)

<Cell 'Hoja1'.AA158>

# 42 AGROPECUARIO UNIDAD 23

In [632]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_AGROPECUARIO = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_AGROPECUARIO['TOTAL'] = unidad23_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",42, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",42, 155),
    ("Primas cedidas interior y exterior terremoto",42, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",42, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ar154_005 = get_value(unidad23_AGROPECUARIO, "Primas emitidas ramo de terremoto",42)
ar155_010 = get_value(unidad23_AGROPECUARIO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 42)
ar156_015 = get_value(unidad23_AGROPECUARIO, "Primas cedidas interior y exterior terremoto", 42)
ar157_020 = get_value(unidad23_AGROPECUARIO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 42)


# Calcular ab94_999 usando la fórmula especificada
ar158_999 = (ar154_005 + ar155_010 - ar156_015 - ar157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=44, value=ar158_999)

<Cell 'Hoja1'.AR158>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 23

In [633]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_OPERNOASIGNABLES['TOTAL'] = unidad23_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",43, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",43, 155),
    ("Primas cedidas interior y exterior terremoto",43, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",43, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as154_005 = get_value(unidad23_OPERNOASIGNABLES, "Primas emitidas ramo de terremoto",43)
as155_010 = get_value(unidad23_OPERNOASIGNABLES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 43)
as156_015 = get_value(unidad23_OPERNOASIGNABLES, "Primas cedidas interior y exterior terremoto", 43)
as157_020 = get_value(unidad23_OPERNOASIGNABLES, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 43)


# Calcular ab94_999 usando la fórmula especificada
as158_999 = (as154_005 + as155_010 - as156_015 - as157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=45, value=as158_999)

<Cell 'Hoja1'.AS158>

# 44 DECENAL UNIDAD 23

In [634]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_DECENAL = bd[(bd['UNIDAD'] == 23) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_DECENAL['TOTAL'] = unidad23_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto",44, 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos",44, 155),
    ("Primas cedidas interior y exterior terremoto",44, 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos",44, 157),
   
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad23_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
at154_005 = get_value(unidad23_DECENAL, "Primas emitidas ramo de terremoto",44)
at155_010 = get_value(unidad23_DECENAL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 44)
at156_015 = get_value(unidad23_DECENAL, "Primas cedidas interior y exterior terremoto", 44)
at157_020 = get_value(unidad23_DECENAL, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 44)


# Calcular ab94_999 usando la fórmula especificada
at158_999 = (at154_005 + at155_010 - at156_015 - at157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=46, value=at158_999)

<Cell 'Hoja1'.AT158>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 23

In [635]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
au154_005 = abs(unidad23_CUMPLIMIENTOESTATALES[(unidad23_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au154_005 = au154_005 if au154_005 != 0 else 0
ws.cell(row=154, column=47, value=float(au154_005))

# Primas cedidas contratos facultativos interior
au155_010 = abs(unidad23_CUMPLIMIENTOESTATALES[(unidad23_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au155_010 = au155_010 if au155_010 != 0 else 0
ws.cell(row=155, column=47, value=float(au155_010))

# Parte reaseguro al interior
au156_015 = abs(unidad23_CUMPLIMIENTOESTATALES[(unidad23_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au156_015 = au156_015 if au156_015 != 0 else 0
ws.cell(row=156, column=47, value=float(au156_015))

au157_020 = abs(unidad23_CUMPLIMIENTOESTATALES[(unidad23_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au157_020 = au157_020 if au157_020 != 0 else 0
ws.cell(row=157, column=47, value=float(au157_020))

# Calcular ae150_999
au158_999 = abs(au154_005 + au155_010 - au156_015 - au157_020)
ws.cell(row=158, column=47, value=float(au158_999))

<Cell 'Hoja1'.AU158>

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 23

In [636]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
av154_005 = abs(unidad23_CUMPLIMIENTOEMPRESASSERV[(unidad23_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av154_005 = av154_005 if av154_005 != 0 else 0
ws.cell(row=154, column=48, value=float(av154_005))

# Primas cedidas contratos facultativos interior
av155_010 = abs(unidad23_CUMPLIMIENTOEMPRESASSERV[(unidad23_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av155_010 = av155_010 if av155_010 != 0 else 0
ws.cell(row=155, column=48, value=float(av155_010))

# Parte reaseguro al interior
av156_015 = abs(unidad23_CUMPLIMIENTOEMPRESASSERV[(unidad23_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av156_015 = av156_015 if av156_015 != 0 else 0
ws.cell(row=156, column=48, value=float(av156_015))

av157_020 = abs(unidad23_CUMPLIMIENTOEMPRESASSERV[(unidad23_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av157_020 = av157_020 if av157_020 != 0 else 0
ws.cell(row=157, column=48, value=float(av157_020))

# Calcular ae150_999
av158_999 = abs(av154_005 + av155_010 - av156_015 - av157_020)
ws.cell(row=158, column=48, value=float(av158_999))

<Cell 'Hoja1'.AV158>

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 23

In [637]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
aw154_005 = abs(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw154_005 = aw154_005 if aw154_005 != 0 else 0
ws.cell(row=154, column=49, value=float(aw154_005))

# Primas cedidas contratos facultativos interior
aw155_010 = abs(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw155_010 = aw155_010 if aw155_010 != 0 else 0
ws.cell(row=155, column=49, value=float(aw155_010))

# Parte reaseguro al interior
aw156_015 = abs(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw156_015 = aw156_015 if aw156_015 != 0 else 0
ws.cell(row=156, column=49, value=float(aw156_015))

aw157_020 = abs(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw157_020 = aw157_020 if aw157_020 != 0 else 0
ws.cell(row=157, column=49, value=float(aw157_020))

# Calcular ae150_999
av158_999 = abs(aw154_005 + aw155_010 - aw156_015 - aw157_020)
ws.cell(row=158, column=49, value=float(av158_999))

<Cell 'Hoja1'.AW158>

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 23

In [638]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ax154_005 = abs(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax154_005 = ax154_005 if ax154_005 != 0 else 0
ws.cell(row=154, column=50, value=float(ax154_005))

# Primas cedidas contratos facultativos interior
ax155_010 = abs(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax155_010 = ax155_010 if ax155_010 != 0 else 0
ws.cell(row=155, column=50, value=float(ax155_010))

# Parte reaseguro al interior
ax156_015 = abs(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax156_015 = ax156_015 if ax156_015 != 0 else 0
ws.cell(row=156, column=50, value=float(ax156_015))

ax157_020 = abs(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 48)]['TOTAL']).sum()
ax157_020 = ax157_020 if ax157_020 != 0 else 0
ws.cell(row=157, column=50, value=float(ax157_020))

# Calcular ae150_999
ax158_999 = abs(ax154_005 + ax155_010 - ax156_015 - ax157_020)
ws.cell(row=158, column=50, value=float(ax158_999))

<Cell 'Hoja1'.AX158>

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 23

In [639]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOCAUCIONES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOCAUCIONES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ay154_005 = abs(unidad23_CUMPLIMIENTOCAUCIONES[(unidad23_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay154_005 = ay154_005 if ay154_005 != 0 else 0
ws.cell(row=154, column=51, value=float(ay154_005))

# Primas cedidas contratos facultativos interior
ay155_010 = abs(unidad23_CUMPLIMIENTOCAUCIONES[(unidad23_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay155_010 = ay155_010 if ay155_010 != 0 else 0
ws.cell(row=155, column=51, value=float(ay155_010))

# Parte reaseguro al interior
ay156_015 = abs(unidad23_CUMPLIMIENTOCAUCIONES[(unidad23_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay156_015 = ay156_015 if ay156_015 != 0 else 0
ws.cell(row=156, column=51, value=float(ay156_015))

ay157_020 = abs(unidad23_CUMPLIMIENTOCAUCIONES[(unidad23_CUMPLIMIENTOCAUCIONES['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOCAUCIONES['COD_RAMO'] == 49)]['TOTAL']).sum()
ay157_020 = ay157_020 if ay157_020 != 0 else 0
ws.cell(row=157, column=51, value=float(ay157_020))

# Calcular ae150_999
ay158_999 = abs(ay154_005 + ay155_010 - ay156_015 - ay157_020)
ws.cell(row=158, column=51, value=float(ay158_999))

<Cell 'Hoja1'.AY158>

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 23

In [640]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOARRENDAMIENTO= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
az154_005 = abs(unidad23_CUMPLIMIENTOARRENDAMIENTO[(unidad23_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az154_005 = az154_005 if az154_005 != 0 else 0
ws.cell(row=154, column=52, value=float(az154_005))

# Primas cedidas contratos facultativos interior
az155_010 = abs(unidad23_CUMPLIMIENTOARRENDAMIENTO[(unidad23_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az155_010 = az155_010 if az155_010 != 0 else 0
ws.cell(row=155, column=52, value=float(az155_010))

# Parte reaseguro al interior
az156_015 = abs(unidad23_CUMPLIMIENTOARRENDAMIENTO[(unidad23_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az156_015 = az156_015 if az156_015 != 0 else 0
ws.cell(row=156, column=52, value=float(az156_015))

az157_020 = abs(unidad23_CUMPLIMIENTOARRENDAMIENTO[(unidad23_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az157_020 = az157_020 if az157_020 != 0 else 0
ws.cell(row=157, column=52, value=float(az157_020))

# Calcular ae150_999
az158_999 = abs(az154_005 + az155_010 - az156_015 - az157_020)
ws.cell(row=158, column=52, value=float(az158_999))

<Cell 'Hoja1'.AZ158>

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 23

In [641]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad23_CUMPLIMIENTOPARTICULARES= bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad23_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ba154_005 = abs(unidad23_CUMPLIMIENTOPARTICULARES[(unidad23_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas emitidas ramo de terremoto") &
                                   (unidad23_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba154_005 = ba154_005 if ba154_005 != 0 else 0
ws.cell(row=154, column=53, value=float(ba154_005))

# Primas cedidas contratos facultativos interior
ba155_010 = abs(unidad23_CUMPLIMIENTOPARTICULARES[(unidad23_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas emitidas (coberturas) de terremoto incluidas en otros ramos") &
                                   (unidad23_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba155_010 = ba155_010 if ba155_010 != 0 else 0
ws.cell(row=155, column=53, value=float(ba155_010))

# Parte reaseguro al interior
ba156_015 = abs(unidad23_CUMPLIMIENTOPARTICULARES[(unidad23_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas interior y exterior terremoto)") &
                                   (unidad23_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba156_015 = ba156_015 if ba156_015 != 0 else 0
ws.cell(row=156, column=53, value=float(ba156_015))

ba157_020 = abs(unidad23_CUMPLIMIENTOPARTICULARES[(unidad23_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos)") &
                                   (unidad23_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba157_020 = ba157_020 if ba157_020 != 0 else 0
ws.cell(row=157, column=53, value=float(ba157_020))

# Calcular ae150_999
ba158_999 = abs(ba154_005 + ba155_010 - ba156_015 - ba157_020)
ws.cell(row=158, column=53, value=float(ba158_999))

<Cell 'Hoja1'.BA158>

#  1   TOTAL UNIDAD 23

In [642]:
c154_005 = sum([ws.cell(row=154, column=col).value if ws.cell(row=154, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=154, column=3).value = c154_005  

c155_010 = sum([ws.cell(row=155, column=col).value if ws.cell(row=155, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=155, column=3).value = c155_010 

c156_015 = sum([ws.cell(row=156, column=col).value if ws.cell(row=156, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=156, column=3).value = c156_015 

c157_020 = sum([ws.cell(row=157, column=col).value if ws.cell(row=157, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=157, column=3).value = c157_020  

# Calcular el total general según la fórmula dada

c158_999 = abs(c154_005 + c155_010 - c156_015 - c157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=3).value = c158_999

#  2   SUBTOTAL - RAMOS UNIDAD 23

In [643]:
d154_005 = sum([ws.cell(row=154, column=col).value if ws.cell(row=154, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=154, column=4).value = d154_005  

d155_010 = sum([ws.cell(row=155, column=col).value if ws.cell(row=155, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=155, column=4).value = d155_010 

d156_015 = sum([ws.cell(row=156, column=col).value if ws.cell(row=156, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=156, column=4).value = d156_015 

d157_020 = sum([ws.cell(row=157, column=col).value if ws.cell(row=157, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=157, column=4).value = d157_020  

# Calcular el total general según la fórmula dada

d158_999 = abs(d154_005 + d155_010 - d156_015 - d157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=4).value = d158_999

# UNIDAD DE CAPTURA 24

# 3   AUTOMOVILES UNIDAD 24

In [645]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_AUTOMOVILES = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 3)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_AUTOMOVILES['TOTAL'] = unidad24_AUTOMOVILES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",3, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",3, 161),
     
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_AUTOMOVILES, concept, ramo, 5, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
e160_005 = get_value(unidad24_AUTOMOVILES, "Primas emitidas pólizas de seguros sujetas a aporte", 3)
e161_010 = get_value(unidad24_AUTOMOVILES, "Ajustes o compensaciones períodos anteriores (resta)", 3)

# Calcular ab94_999 usando la fórmula especificada
e162_999 = (e160_005 + e161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=5, value=e162_999)

<Cell 'Hoja1'.E162>

#  4   SOAT UNIDAD 24

In [646]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_SOAT = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 4)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_SOAT['TOTAL'] = unidad24_SOAT['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",4, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",4, 161),
     
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_SOAT, concept, ramo, 6, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
f160_005 = get_value(unidad24_SOAT, "Primas emitidas pólizas de seguros sujetas a aporte", 4)
f161_010 = get_value(unidad24_SOAT, "Ajustes o compensaciones períodos anteriores (resta)", 4)

# Calcular ab94_999 usando la fórmula especificada
f162_999 = (f160_005 + f161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=6, value=f162_999)

<Cell 'Hoja1'.F162>

#  5   CUMPLIMIENTO UNIDAD 24

In [647]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_CUMPLIMIENTO = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 5)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_CUMPLIMIENTO['TOTAL'] = unidad24_CUMPLIMIENTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",5, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",5, 161),
     
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_CUMPLIMIENTO, concept, ramo, 7, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
g160_005 = get_value(unidad24_CUMPLIMIENTO, "Primas emitidas pólizas de seguros sujetas a aporte", 5)
g161_010 = get_value(unidad24_CUMPLIMIENTO, "Ajustes o compensaciones períodos anteriores (resta)", 5)

# Calcular ab94_999 usando la fórmula especificada
g162_999 = (g160_005 + g161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=7, value=g162_999)

<Cell 'Hoja1'.G162>

#  6   RESPONSABILIDAD CIVIL UNIDAD 24

In [649]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_RESPONSABILIDADCI = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 6)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_RESPONSABILIDADCI['TOTAL'] = unidad24_RESPONSABILIDADCI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",6, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",6, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_RESPONSABILIDADCI, concept, ramo, 8, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
h160_005 = get_value(unidad24_RESPONSABILIDADCI, "Primas emitidas pólizas de seguros sujetas a aporte", 6)
h161_010 = get_value(unidad24_RESPONSABILIDADCI, "Ajustes o compensaciones períodos anteriores (resta)", 6)

# Calcular ab94_999 usando la fórmula especificada
h162_999 = (h160_005 + h161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=8, value=h162_999)

<Cell 'Hoja1'.H162>

#  7   INCENDIO UNIDAD 24

In [650]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_INCENDIO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_INCENDIO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior

cuentas = ["5114001011", "5114001021", "5114001061", "5114001071", "5114001081", "5114001091"]
i160_005 = unidad24_INCENDIO_a[(unidad24_INCENDIO_a['CUENTA'].isin(cuentas)) & 
                                (unidad24_INCENDIO_a['COD_RAMO'] == 7)]['TOTAL'].sum()
i160_005 = abs(float(i160_005))
ws.cell(row=160, column=9, value=i160_005)

# Primas cedidas contratos facultativos interior
i161_010 = abs(unidad24_INCENDIO[(unidad24_INCENDIO['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_INCENDIO['COD_RAMO'] == 7)]['TOTAL']).sum()
i161_010 = i161_010 if i161_010 != 0 else 0
ws.cell(row=161, column=9, value=float(i161_010))

# Calcular 
i162_999 = abs(i160_005 + i161_010)
ws.cell(row=162, column=9, value=float(i162_999))

<Cell 'Hoja1'.I162>

#  8   TERREMOTO UNIDAD 24

In [651]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_TERRREMOTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_TERRREMOTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior

cuentas = ["5114001011", "5114001021", "5114001061", "5114001071", "5114001081", "5114001091"]
j160_005 = unidad24_TERRREMOTO_a[(unidad24_TERRREMOTO_a['CUENTA'].isin(cuentas)) & 
                                (unidad24_TERRREMOTO_a['COD_RAMO'] == 8)]['TOTAL'].sum()
j160_005 = abs(float(j160_005))
ws.cell(row=160, column=10, value=j160_005)

# Primas cedidas contratos facultativos interior
j161_010 = abs(unidad24_TERRREMOTO[(unidad24_TERRREMOTO['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_TERRREMOTO['COD_RAMO'] == 8)]['TOTAL']).sum()
j161_010 = j161_010 if j161_010 != 0 else 0
ws.cell(row=161, column=10, value=float(j161_010))

# Calcular 
j162_999 = abs(j160_005 + j161_010)
ws.cell(row=162, column=10, value=float(j162_999))

<Cell 'Hoja1'.J162>

#  9   SUSTRACCION UNIDAD 24

In [653]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_SUSTRACCION = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 9)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_SUSTRACCION['TOTAL'] = unidad24_SUSTRACCION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",9, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",9, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_SUSTRACCION, concept, ramo, 11, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
k160_005 = get_value(unidad24_SUSTRACCION, "Primas emitidas pólizas de seguros sujetas a aporte", 9)
k161_010 = get_value(unidad24_SUSTRACCION, "Ajustes o compensaciones períodos anteriores (resta)", 9)

# Calcular ab94_999 usando la fórmula especificada
k162_999 = (k160_005 + k161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=11, value=k162_999)

<Cell 'Hoja1'.K162>

#  10  TRANSPORTE UNIDAD 24

In [654]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_TRANSPORTE = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 10)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_TRANSPORTE['TOTAL'] = unidad24_TRANSPORTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",10, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",10, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_TRANSPORTE, concept, ramo, 12, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
l160_005 = get_value(unidad24_TRANSPORTE, "Primas emitidas pólizas de seguros sujetas a aporte", 10)
l161_010 = get_value(unidad24_TRANSPORTE, "Ajustes o compensaciones períodos anteriores (resta)", 10)

# Calcular ab94_999 usando la fórmula especificada
l162_999 = (l160_005 + l161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=12, value=l162_999)

<Cell 'Hoja1'.L162>

#  11  CORRIENTE DÉBIL UNIDAD 24

In [655]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_CORRIENTEDEBIL = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 11)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_CORRIENTEDEBIL['TOTAL'] = unidad24_CORRIENTEDEBIL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",11, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",11, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_TRANSPORTE, concept, ramo, 13, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
m160_005 = get_value(unidad24_CORRIENTEDEBIL, "Primas emitidas pólizas de seguros sujetas a aporte", 11)
m161_010 = get_value(unidad24_CORRIENTEDEBIL, "Ajustes o compensaciones períodos anteriores (resta)", 11)

# Calcular ab94_999 usando la fórmula especificada
m162_999 = (m160_005 + m161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=13, value=m162_999)

<Cell 'Hoja1'.M162>

#  12  TODO RIESGO CONTRATISTA UNIDAD 24

In [656]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_TODORIESGOCONTRATISTA = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 12)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_TODORIESGOCONTRATISTA['TOTAL'] = unidad24_TODORIESGOCONTRATISTA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",12, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",12, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_TODORIESGOCONTRATISTA, concept, ramo, 14, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
n160_005 = get_value(unidad24_TODORIESGOCONTRATISTA, "Primas emitidas pólizas de seguros sujetas a aporte", 12)
n161_010 = get_value(unidad24_TODORIESGOCONTRATISTA, "Ajustes o compensaciones períodos anteriores (resta)", 12)

# Calcular ab94_999 usando la fórmula especificada
n162_999 = (n160_005 + n161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=14, value=n162_999)

<Cell 'Hoja1'.N162>

#  13  MANEJO UNIDAD 24

In [657]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_MANEJO = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 13)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_MANEJO['TOTAL'] = unidad24_MANEJO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",13, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",13, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_MANEJO, concept, ramo, 15, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
o160_005 = get_value(unidad24_MANEJO, "Primas emitidas pólizas de seguros sujetas a aporte", 13)
o161_010 = get_value(unidad24_MANEJO, "Ajustes o compensaciones períodos anteriores (resta)", 13)

# Calcular ab94_999 usando la fórmula especificada
o162_999 = (o160_005 + o161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=15, value=o162_999)

<Cell 'Hoja1'.O162>

#  14  LUCRO CESANTE UNIDAD 24

In [658]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_LUCROCESANTE = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 14)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_LUCROCESANTE['TOTAL'] = unidad24_LUCROCESANTE['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",14, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",14, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_LUCROCESANTE, concept, ramo, 16, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
p160_005 = get_value(unidad24_LUCROCESANTE, "Primas emitidas pólizas de seguros sujetas a aporte", 14)
p161_010 = get_value(unidad24_LUCROCESANTE, "Ajustes o compensaciones períodos anteriores (resta)", 14)

# Calcular ab94_999 usando la fórmula especificada
p162_999 = (p160_005 + p161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=16, value=p162_999)

<Cell 'Hoja1'.P162>

#  15   MONTAJE Y ROTURA DE MAQUINARIA UNIDAD 24

In [659]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_MONTAJEYROTURA  = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 15)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_MONTAJEYROTURA['TOTAL'] = unidad24_MONTAJEYROTURA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",15, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",15, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_MONTAJEYROTURA, concept, ramo, 17, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
q160_005 = get_value(unidad24_MONTAJEYROTURA, "Primas emitidas pólizas de seguros sujetas a aporte", 15)
q161_010 = get_value(unidad24_MONTAJEYROTURA, "Ajustes o compensaciones períodos anteriores (resta)", 15)

# Calcular ab94_999 usando la fórmula especificada
q162_999 = (q160_005 + q161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=17, value=q162_999)

<Cell 'Hoja1'.Q162>

#  16  AVIACIÓN UNIDAD 24

In [660]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_AVIACION = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 16)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_AVIACION['TOTAL'] = unidad24_AVIACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",16, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",16, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_AVIACION, concept, ramo, 18, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
r160_005 = get_value(unidad24_AVIACION, "Primas emitidas pólizas de seguros sujetas a aporte", 16)
r161_010 = get_value(unidad24_AVIACION, "Ajustes o compensaciones períodos anteriores (resta)", 16)

# Calcular ab94_999 usando la fórmula especificada
r162_999 = (r160_005 + r161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=18, value=r162_999)

<Cell 'Hoja1'.R162>

#  17  NAVEGACIÓN Y CASCO UNIDAD 24

In [661]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_NAVEGACION = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 17)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_NAVEGACION['TOTAL'] = unidad24_NAVEGACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",17, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",17, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_NAVEGACION, concept, ramo, 19, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
s160_005 = get_value(unidad24_NAVEGACION, "Primas emitidas pólizas de seguros sujetas a aporte", 17)
s161_010 = get_value(unidad24_NAVEGACION, "Ajustes o compensaciones períodos anteriores (resta)", 17)

# Calcular ab94_999 usando la fórmula especificada
s162_999 = (s160_005 + s161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=19, value=s162_999)

<Cell 'Hoja1'.S162>

#  18  MINAS Y PETRÓLEOS UNIDAD 24

In [662]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_MINAS = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 18)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_MINAS['TOTAL'] = unidad24_MINAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",18, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",18, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_MINAS, concept, ramo, 20, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
t160_005 = get_value(unidad24_MINAS, "Primas emitidas pólizas de seguros sujetas a aporte", 18)
t161_010 = get_value(unidad24_MINAS, "Ajustes o compensaciones períodos anteriores (resta)", 18)

# Calcular ab94_999 usando la fórmula especificada
t162_999 = (t160_005 + t161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=20, value=t162_999)

<Cell 'Hoja1'.T162>

#  19  VIDRIOS UNIDAD 24

In [663]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_VIDRIOS = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 19)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_VIDRIOS['TOTAL'] = unidad24_VIDRIOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",19, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",19, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_VIDRIOS, concept, ramo, 21, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
u160_005 = get_value(unidad24_VIDRIOS, "Primas emitidas pólizas de seguros sujetas a aporte", 19)
u161_010 = get_value(unidad24_VIDRIOS, "Ajustes o compensaciones períodos anteriores (resta)", 19)

# Calcular ab94_999 usando la fórmula especificada
u162_999 = (u160_005 + u161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=21, value=u162_999)

<Cell 'Hoja1'.U162>

#  20  CRÉDITO COMERCIAL UNIDAD 24

In [664]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_CREDITOCOMERCIAL = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 20)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_CREDITOCOMERCIAL['TOTAL'] = unidad24_CREDITOCOMERCIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",20, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",20, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_CREDITOCOMERCIAL, concept, ramo, 22, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
v160_005 = get_value(unidad24_CREDITOCOMERCIAL, "Primas emitidas pólizas de seguros sujetas a aporte", 20)
v161_010 = get_value(unidad24_CREDITOCOMERCIAL, "Ajustes o compensaciones períodos anteriores (resta)", 20)

# Calcular ab94_999 usando la fórmula especificada
v162_999 = (v160_005 + v161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=22, value=v162_999)

<Cell 'Hoja1'.V162>

#  21  CRÉDITO A LA EXPORTACIÓN UNIDAD 24

In [665]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_CREDITOEXPORTACION = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 21)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_CREDITOEXPORTACION['TOTAL'] = unidad24_CREDITOEXPORTACION['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",21, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",21, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_CREDITOEXPORTACION, concept, ramo, 23, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
w160_005 = get_value(unidad24_CREDITOEXPORTACION, "Primas emitidas pólizas de seguros sujetas a aporte", 21)
w161_010 = get_value(unidad24_CREDITOEXPORTACION, "Ajustes o compensaciones períodos anteriores (resta)", 21)

# Calcular ab94_999 usando la fórmula especificada
w162_999 = (w160_005 + w161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=23, value=w162_999)

<Cell 'Hoja1'.W162>

#  24  DESEMPLEO UNIDAD 24

In [667]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_DESEMPLEO = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 24)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_DESEMPLEO['TOTAL'] = unidad24_DESEMPLEO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",24, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",24, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_DESEMPLEO, concept, ramo, 26, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
z160_005 = get_value(unidad24_DESEMPLEO, "Primas emitidas pólizas de seguros sujetas a aporte", 24)
z161_010 = get_value(unidad24_DESEMPLEO, "Ajustes o compensaciones períodos anteriores (resta)", 24)

# Calcular ab94_999 usando la fórmula especificada
z162_999 = (z160_005 + z161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=26, value=z162_999)

<Cell 'Hoja1'.Z162>

#  25  HOGAR UNIDAD 24

In [668]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_HOGAR = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_HOGAR_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior

cuentas = ["5114001011", "5114001021", "5114001061", "5114001071", "5114001081"]
aa160_005 = unidad24_HOGAR_a[(unidad24_HOGAR_a['CUENTA'].isin(cuentas)) & 
                                (unidad24_HOGAR_a['COD_RAMO'] == 25)]['TOTAL'].sum()
aa160_005 = abs(float(aa160_005))
ws.cell(row=160, column=27, value=aa160_005)

# Primas cedidas contratos facultativos interior
aa161_010 = abs(unidad24_HOGAR[(unidad24_HOGAR['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_HOGAR['COD_RAMO'] == 25)]['TOTAL']).sum()
aa161_010 = aa161_010 if aa161_010 != 0 else 0
ws.cell(row=161, column=27, value=float(aa161_010))

# Calcular 
aa162_999 = abs(aa160_005 + aa161_010)
ws.cell(row=162, column=27, value=float(aa162_999))

<Cell 'Hoja1'.AA162>

# 42 AGROPECUARIO UNIDAD 24

In [669]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_AGROPECUARIO = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 42)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_AGROPECUARIO['TOTAL'] = unidad24_AGROPECUARIO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",42, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",42, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_AGROPECUARIO, concept, ramo, 44, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ar160_005 = get_value(unidad24_AGROPECUARIO, "Primas emitidas pólizas de seguros sujetas a aporte", 42)
ar161_010 = get_value(unidad24_AGROPECUARIO, "Ajustes o compensaciones períodos anteriores (resta)", 42)

# Calcular ab94_999 usando la fórmula especificada
ar162_999 = (ar160_005 + ar161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=44, value=ar162_999)

<Cell 'Hoja1'.AR162>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 24

In [670]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_OPERNOASIGNABLES['TOTAL'] = unidad24_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",43, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",43, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as160_005 = get_value(unidad24_OPERNOASIGNABLES, "Primas emitidas pólizas de seguros sujetas a aporte", 43)
as161_010 = get_value(unidad24_OPERNOASIGNABLES, "Ajustes o compensaciones períodos anteriores (resta)", 43)

# Calcular ab94_999 usando la fórmula especificada
as162_999 = (as160_005 + as161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=45, value=as162_999)

<Cell 'Hoja1'.AS162>

# 44 DECENAL UNIDAD 24

In [672]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_DECENAL = bd[(bd['UNIDAD'] == 24) & (bd['COD_RAMO'] == 44)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_DECENAL['TOTAL'] = unidad24_DECENAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas pólizas de seguros sujetas a aporte",44, 160),
    ("Ajustes o compensaciones períodos anteriores (resta)",44, 161),
]
# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad24_DECENAL, concept, ramo, 46, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
at160_005 = get_value(unidad24_DECENAL, "Primas emitidas pólizas de seguros sujetas a aporte", 44)
at161_010 = get_value(unidad24_DECENAL, "Ajustes o compensaciones períodos anteriores (resta)", 44)

# Calcular ab94_999 usando la fórmula especificada
at162_999 = (at160_005 + at161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=46, value=at162_999)

<Cell 'Hoja1'.AT162>

# 45 CUMPLIMIENTO - ENTIDADES ESTATALES UNIDAD 24

In [673]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOESTATALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOESTATALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
au160_005 = abs(unidad24_CUMPLIMIENTOESTATALES[(unidad24_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au160_005 = au160_005 if au160_005 != 0 else 0
ws.cell(row=160, column=47, value=float(au160_005))

# Primas cedidas contratos facultativos interior
au161_010 = abs(unidad24_CUMPLIMIENTOESTATALES[(unidad24_CUMPLIMIENTOESTATALES['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOESTATALES['COD_RAMO'] == 45)]['TOTAL']).sum()
au161_010 = au161_010 if au161_010 != 0 else 0
ws.cell(row=161, column=47, value=float(au161_010))

# Calcular ae150_999
au162_999 = abs(au160_005 + au161_010)
ws.cell(row=162, column=47, value=float(au162_999))

<Cell 'Hoja1'.AU162>

# 46 CUMPLIMIENTO - EMPRESAS SERVICIOS PUBLICOS (ESP) UNIDAD 24

In [674]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOEMPRESASSERV = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOEMPRESASSERV_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
av160_005 = abs(unidad24_CUMPLIMIENTOEMPRESASSERV[(unidad24_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av160_005 = av160_005 if av160_005 != 0 else 0
ws.cell(row=160, column=48, value=float(av160_005))

# Primas cedidas contratos facultativos interior
av161_010 = abs(unidad24_CUMPLIMIENTOEMPRESASSERV[(unidad24_CUMPLIMIENTOEMPRESASSERV['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOEMPRESASSERV['COD_RAMO'] == 46)]['TOTAL']).sum()
av161_010 = av161_010 if av161_010 != 0 else 0
ws.cell(row=161, column=48, value=float(av161_010))

# Calcular ae150_999
av162_999 = abs(av160_005 + av161_010)
ws.cell(row=162, column=48, value=float(av162_999))

<Cell 'Hoja1'.AV162>

# 47 CUMPLIMIENTO - EMPRESAS INDUSTRIALES Y COMERCIALES DEL ESTADO (EICE) UNIDAD 24

In [675]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
aw160_005 = abs(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw160_005 = aw160_005 if aw160_005 != 0 else 0
ws.cell(row=160, column=49, value=float(aw160_005))

# Primas cedidas contratos facultativos interior
aw161_010 = abs(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 47)]['TOTAL']).sum()
aw161_010 = aw161_010 if aw161_010 != 0 else 0
ws.cell(row=161, column=49, value=float(aw161_010))

# Calcular ae150_999
aw162_999 = abs(aw160_005 + aw161_010)
ws.cell(row=162, column=49, value=float(aw162_999))

<Cell 'Hoja1'.AW162>

# 48 CUMPLIMIENTO - DISPOSICIONES LEGALES UNIDAD 24

In [676]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOEMPRESASINDUSTRI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOEMPRESASINDUSTRI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ax160_005 = abs(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 48)]['TOTAL']).sum()
ax160_005 = ax160_005 if ax160_005 != 0 else 0
ws.cell(row=160, column=50, value=float(ax160_005))

# Primas cedidas contratos facultativos interior
ax161_010 = abs(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI[(unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOEMPRESASINDUSTRI['COD_RAMO'] == 48)]['TOTAL']).sum()
ax161_010 = ax161_010 if ax161_010 != 0 else 0
ws.cell(row=161, column=50, value=float(ax161_010))

# Calcular ae150_999
ax162_999 = abs(ax160_005 + ax161_010)
ws.cell(row=162, column=50, value=float(ax162_999))

<Cell 'Hoja1'.AX162>

# 49 CUMPLIMIENTO - CAUCIONES JUDICIALES UNIDAD 24

In [677]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOEMPRESASDISPOLEG = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOEMPRESASDISPOLEG_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ay160_005 = abs(unidad24_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad24_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 49)]['TOTAL']).sum()
ay160_005 = ay160_005 if ay160_005 != 0 else 0
ws.cell(row=160, column=51, value=float(ay160_005))

# Primas cedidas contratos facultativos interior
ay161_010 = abs(unidad24_CUMPLIMIENTOEMPRESASDISPOLEG[(unidad24_CUMPLIMIENTOEMPRESASDISPOLEG['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOEMPRESASDISPOLEG['COD_RAMO'] == 49)]['TOTAL']).sum()
ay161_010 = ay161_010 if ay161_010 != 0 else 0
ws.cell(row=161, column=51, value=float(ay161_010))

# Calcular ae150_999
ay162_999 = abs(ay160_005 + ay161_010)
ws.cell(row=162, column=51, value=float(ay162_999))

<Cell 'Hoja1'.AY162>

# 50 CUMPLIMIENTO - ARRENDAMIENTO UNIDAD 24

In [678]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOARRENDAMIENTO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOARRENDAMIENTO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
az160_005 = abs(unidad24_CUMPLIMIENTOARRENDAMIENTO[(unidad24_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az160_005 = az160_005 if az160_005 != 0 else 0
ws.cell(row=160, column=52, value=float(az160_005))

# Primas cedidas contratos facultativos interior
az161_010 = abs(unidad24_CUMPLIMIENTOARRENDAMIENTO[(unidad24_CUMPLIMIENTOARRENDAMIENTO['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOARRENDAMIENTO['COD_RAMO'] == 50)]['TOTAL']).sum()
az161_010 = az161_010 if az161_010 != 0 else 0
ws.cell(row=161, column=52, value=float(az161_010))

# Calcular ae150_999
az162_999 = abs(az160_005 + az161_010)
ws.cell(row=162, column=52, value=float(az162_999))

<Cell 'Hoja1'.AZ162>

# 51 CUMPLIMIENTO - PARTICULARES (PERSONAS JURÍDICAS, NATURALES) UNIDAD 24

In [679]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad24_CUMPLIMIENTOPARTICULARES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad24_CUMPLIMIENTOPARTICULARES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ba160_005 = abs(unidad24_CUMPLIMIENTOPARTICULARES[(unidad24_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Primas emitidas pólizas de seguros sujetas a aporte") &
                                   (unidad24_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba160_005 = ba160_005 if ba160_005 != 0 else 0
ws.cell(row=160, column=53, value=float(ba160_005))

# Primas cedidas contratos facultativos interior
ba161_010 = abs(unidad24_CUMPLIMIENTOPARTICULARES[(unidad24_CUMPLIMIENTOPARTICULARES['CONCEPTO'] == "Ajustes o compensaciones períodos anteriores (resta)") &
                                   (unidad24_CUMPLIMIENTOPARTICULARES['COD_RAMO'] == 51)]['TOTAL']).sum()
ba161_010 = ba161_010 if ba161_010 != 0 else 0
ws.cell(row=161, column=53, value=float(ba161_010))

# Calcular ae150_999
ba162_999 = abs(ba160_005 + ba161_010)
ws.cell(row=162, column=53, value=float(ba162_999))

<Cell 'Hoja1'.BA162>

#  1   TOTAL UNIDAD 24

In [680]:
c160_005 = sum([ws.cell(row=160, column=col).value if ws.cell(row=160, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=160, column=3).value = c160_005  

c161_010 = sum([ws.cell(row=161, column=col).value if ws.cell(row=161, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=161, column=3).value = c161_010 

# Calcular el total general según la fórmula dada

c162_999 = abs(c160_005 + c161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=3).value = c162_999

#  2   SUBTOTAL - RAMOS UNIDAD 24

In [681]:
d160_005 = sum([ws.cell(row=160, column=col).value if ws.cell(row=160, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=160, column=4).value = d160_005  

d161_010 = sum([ws.cell(row=161, column=col).value if ws.cell(row=161, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=161, column=4).value = d161_010 

# Calcular el total general según la fórmula dada

d162_999 = abs(d160_005 + d161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=4).value = d162_999

# IMPRIMIR FORMATO 290

In [682]:
wb.save('Bases/Formato_290_COGN.xlsx')
wb.close()

# PROFORMA TXT

In [683]:
# Función para formatear la línea con los valores dados
def formatear_linea(numero_registro, codigo_cuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # Aseguramos 5 dígitos
    tipo_registro = "3"  # Siempre '3' para las primeras tres partes
    codigo_cuenta_str = str(int(codigo_cuenta)).zfill(6)  # 6 dígitos para el código de cuenta
    valor_str = f"{int(valor):020d}"  # Valor sin decimales, formateado a 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_cuenta_str}{signo}{valor_str}"

# Obtener la fecha correcta (mes vencido)
fecha_actual = datetime.now()
fecha_mes_vencido = fecha_actual.replace(day=1) - timedelta(days=1)
fecha_str = fecha_mes_vencido.strftime("%d%m%Y")  # Formato día-mes-año

# Encabezados fijos de la primera y segunda línea
registro_inicial = f"00001113000001{fecha_str}01864SGENCOLASEG0100"
segunda_linea = "000022000000000000"

# Leer la base de datos principal
bd = pd.read_excel(r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COGN.xlsx",
    sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)
bd.columns = ["ALTERNATIVA 6", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]
bd = bd.dropna(subset=['ALTERNATIVA 6'])

# Convertir las columnas de 'VALOR', 'VALOR2', y 'VALOR MONEDA EXTRANJERA' a numérico y manejar valores NaN
bd['VALOR'] = pd.to_numeric(bd['VALOR'], errors='coerce').fillna(0)
bd['VALOR2'] = pd.to_numeric(bd['VALOR2'], errors='coerce').fillna(0)
bd['VALOR MONEDA EXTRANJERA'] = pd.to_numeric(bd['VALOR MONEDA EXTRANJERA'], errors='coerce').fillna(0)

# Lista para almacenar las líneas del archivo TXT
txt_datos = [registro_inicial, segunda_linea]

# Iniciar el número de registro en 3 (para la tercera línea)
numero_registro = 3

# --------- Primera parte: Procesar ALTERNATIVA 6 y VALOR ---------
for _, row in bd.iterrows():
    codigo_cuenta = row['ALTERNATIVA 6']
    valor = row['VALOR']

    # Saltar si el valor es cero
    if valor == 0:
        continue

    # Determinar el signo del valor
    signo = '+' if valor >= 0 else '-'
    valor = abs(valor)  # Usar el valor absoluto para la línea

    # Formatear la línea y agregarla a los datos del TXT
    linea = formatear_linea(numero_registro, codigo_cuenta, signo, valor)
    txt_datos.append(linea)

    numero_registro += 1

# Cierre de la primera parte
txt_datos.append(f"{str(numero_registro).zfill(5)}2000001000000")
numero_registro += 1

# --------- Segunda parte: Procesar ALTERNATIVA 6 y VALOR2 ---------
for _, row in bd.iterrows():
    codigo_cuenta = row['ALTERNATIVA 6']
    valor2 = row['VALOR2']

    # Saltar si el valor es cero
    if valor2 == 0:
        continue

    # Determinar el signo del valor
    signo = '+' if valor2 >= 0 else '-'
    valor2 = abs(valor2)  # Usar el valor absoluto para la línea

    # Formatear la línea y agregarla a los datos del TXT
    linea = formatear_linea(numero_registro, codigo_cuenta, signo, valor2)
    txt_datos.append(linea)

    numero_registro += 1

# Cierre de la segunda parte
txt_datos.append(f"{str(numero_registro).zfill(5)}2000002000000")
numero_registro += 1

# --------- Tercera parte: Procesar ALTERNATIVA 6 y VALOR MONEDA EXTRANJERA ---------
cuentas_vistas = set()  # Para evitar duplicados
for _, row in bd.iterrows():
    codigo_cuenta = row['ALTERNATIVA 6']
    valor_moneda_ext = row['VALOR MONEDA EXTRANJERA']

    # Saltar si el valor es cero o si ya hemos visto esta cuenta
    if valor_moneda_ext == 0 or codigo_cuenta in cuentas_vistas:
        continue

    cuentas_vistas.add(codigo_cuenta)

    # Determinar el signo del valor
    signo = '+' if valor_moneda_ext >= 0 else '-'
    valor_moneda_ext = abs(valor_moneda_ext)  # Usar el valor absoluto para la línea

    # Formatear la línea y agregarla a los datos del TXT
    linea = formatear_linea(numero_registro, codigo_cuenta, signo, valor_moneda_ext)
    txt_datos.append(linea)

    numero_registro += 1

# Cierre de la tercera parte con el formato correcto: "consecutivo + 2000001100000"
txt_datos.append(f"{str(numero_registro).zfill(5)}2000001100000")
numero_registro += 1

# --------- Cuarta parte: Procesar FORMATO 237 (solo si es trimestral) ---------
mes_actual = fecha_actual.month
es_trimestral = mes_actual in [4, 7, 10, 1]  # Abril, Julio, Octubre, Enero

if es_trimestral:
    def formatear_linea_formato_237(numero_registro, codigo_formato, columna_idx, unidad_captura, subcuenta, signo, valor):
        num_registro_str = str(numero_registro).zfill(5)  # Asegurar 5 dígitos
        tipo_registro = "4"  # Tipo de registro fijo
        codigo_formato_str = str(codigo_formato).zfill(3)  # Código del formato con 3 dígitos
        columna_str = str(columna_idx).zfill(2)  # Número de la columna con 2 dígitos
        unidad_captura_str = str(unidad_captura).zfill(2).strip()  # Unidad de captura con 2 dígitos
        subcuenta_str = str(subcuenta).zfill(3).strip()  # Subcuenta con 3 dígitos
        valor_str = f"{int(valor):020d}"  # Valor formateado a 20 dígitos
        return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

     # Nueva ruta y nombre de hoja
    ruta_formato_237 = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COGN.xlsx"
    # Cargar el archivo Excel desde la hoja específica, con encabezados y rango de columnas
    df_formato_237 = pd.read_excel(ruta_formato_237,sheet_name="Formato237_TRIMESTRAL",dtype=str,skiprows=12,usecols="B:O")
    # Eliminar la primera fila después de los encabezados y las dos últimas filas
    df_formato_237 = df_formato_237.iloc[1:-2]
    
    subcuenta_col = 'SUBCUENTA'  # Columna de subcuenta
    unidad_captura_col = 'UNIDAD DE CAPTURA'  # Columna guía para unidad de captura
    codigo_formato = 237  # Código del formato

    # Determinar las columnas relevantes desde el DataFrame
    columnas_valores = df_formato_237.columns[1:-1]  # Desde la segunda columna hasta la penúltima (excluyendo "SUBCUENTA" y "UNIDAD DE CAPTURA")

    # Recorrer las columnas y generar líneas
    for idx, columna in enumerate(columnas_valores, start=0):
        for _, fila in df_formato_237.iterrows():
            try:
                subcuenta = str(fila[subcuenta_col]).zfill(3)  # Extraer la subcuenta y formatear a 3 dígitos
                unidad_captura = str(fila[unidad_captura_col]).zfill(2)  # Extraer la unidad de captura
                valor = float(fila[columna])  # Convertir a número
                if valor == 0 or pd.isna(valor):
                    continue  # Saltar valores cero o NaN

                signo = '+' if valor >= 0 else '-'
                valor_abs = abs(valor)  # Valor absoluto

                # Formatear línea
                linea = formatear_linea_formato_237(
                    numero_registro,
                    codigo_formato,
                    idx,
                    unidad_captura,
                    subcuenta,
                    signo,
                    valor_abs
                )
                txt_datos.append(linea)
                numero_registro += 1
            except ValueError:
                continue

    txt_datos.append(f"{str(numero_registro).zfill(5)}2000001100000")
    numero_registro += 1
else:
    print("No es un mes de cierre trimestral. El formato 237 no será procesado ni incluido.")

# --------- Quinta parte: Procesar FORMATO 290 ---------

# Ruta al archivo Excel
ruta_formato_290 = r"D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_290/Bases/Formato_290_COGN.xlsx"

# Cargar el archivo Excel
df_290 = pd.read_excel(ruta_formato_290, dtype=str)

# Definir las columnas clave
columna_subcuenta = 'SUB CUENTA'
columna_unidad_captura = 'UNIDAD DE CAPTURA'

# Filtrar las filas que no tengan datos válidos en la columna 'SUB CUENTA'
df_290 = df_290.dropna(subset=[columna_subcuenta])

# Tomamos las columnas de datos desde la tercera hasta la columna 51 (inclusive)
columnas_datos = df_290.columns[2:53]  # El índice 52 no se incluye, por lo tanto, este abarca hasta la columna 51

# Función para la quinta parte
def formatear_linea_quinta_parte(numero_registro, codigo_formato, columna_idx, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # Aseguramos 5 dígitos para el número de registro
    tipo_registro = "4"  # Tipo de registro para la quinta parte
    codigo_formato_str = str(codigo_formato).zfill(3)  # 3 dígitos para el código del formato
    columna_str = str(columna_idx).zfill(2)  # 2 dígitos para el número de la columna
    unidad_captura_str = str(unidad_captura).zfill(2).strip()  # Aseguramos 2 dígitos para la unidad de captura, eliminar espacios
    subcuenta_str = str(subcuenta).zfill(3).strip()  # 3 dígitos para la subcuenta, eliminar espacios
    valor_str = f"{int(valor):020d}"  # Valor sin decimales, formateado a 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Recorrer las columnas de datos (por ejemplo, columnas numéricas con valores)
for idx, columna in enumerate(columnas_datos, start=1):
    for _, row in df_290.iterrows():
        subcuenta = str(row[columna_subcuenta]).strip()  # Convertir a texto y eliminar espacios en blanco
        unidad_captura = str(row[columna_unidad_captura]).strip()  # Convertir a texto y eliminar espacios en blanco
        valor = row[columna]

        # Saltar si el valor es cero o NaN
        if pd.isna(valor) or valor == '0':
            continue

        # Determinar el signo del valor
        try:
            valor_float = float(valor)
            valor_redondeado = round(valor_float)  # Redondear el valor
            signo = '+' if valor_redondeado >= 0 else '-'
            valor_abs = abs(valor_redondeado)

            # Formatear la línea específica para el Formato 290 (quinta parte)
            linea = formatear_linea_quinta_parte(numero_registro, 290, idx, unidad_captura, subcuenta, signo, valor_abs)
            txt_datos.append(linea)

            numero_registro += 1

        except ValueError:
            print(f"Valor no válido en la fila {row.name}, columna {columna}: {valor}")

# Agregar la última línea con el conteo de registros y un "5" al final
total_registros = len(txt_datos) + 1  # Contar todas las líneas generadas en las cinco partes
linea_final = f"{str(total_registros).zfill(5)}5"  # Formatear el total con un "5" al final
txt_datos.append(linea_final)

# Guardar el archivo TXT final con todas las partes
with open("PlanoF290_COGN.txt", "w") as f:
    for linea in txt_datos:
        f.write(f"{linea}\n")


No es un mes de cierre trimestral. El formato 237 no será procesado ni incluido.
